In [9]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import List, Optional
from utililty import text_getter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
import serpapi
load_dotenv()

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

def use_component(component_cls, input_dict:dict, llm=llm, human_prompt=None):
    structured_llm = llm.with_structured_output(component_cls)
    if not human_prompt:
        human_prompt = component_cls.get_user_prompt()


    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", component_cls.get_system_prompt()),
            ("human", human_prompt)
        ]
    )
    # system_msg = prompt.messages[0].prompt
    # user_msg = prompt.messages[1].prompt.format(input_dict)
    # print(system_msg)
    # print(user_msg)
    # print(f"\n\n######## PROMPT\nSystem: {system_msg}\nUser: {user_msg}\nTotal len: {len(system_msg)+len(user_msg)}########\n\n")
    generator = prompt | structured_llm
    result = generator.invoke(input_dict)
    return result

In [10]:
from bs4 import BeautifulSoup
from lxml import html
import requests

def get_text_from_html(url, query):
    headers = {
        "User-Agent": os.getenv("USER_AGENT")
    }

    # Step 5: Fetch the webpage content
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, features="html.parser")

        # kill all script and style elements
        for script in soup(["script", "style"]):
            script.extract()    # rip it out

        # get text
        text = soup.get_text()

        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)

        return {"text": text, "query": query, "url": url}
    else:
        print(f"Failed to retrieve {url}. Status code: {response.status_code}")


In [11]:
from serpapi import GoogleSearch

def search_google(query, gl="it", hl="it", search_type=None):
  params = {
    "api_key": os.getenv("SERP_API_KEY"),
    "engine": "google",
    "q": query,
    # "location": "Austin, Texas, United States",
    # "google_domain": "google.com",
    "gl": gl,
    "hl": hl,
    "tbm": search_type #nws: Google News API,
  }

  search = GoogleSearch(params)
  search = search.get_dict()
  return search

# search_results = search_google(para_title)
# search_results

## Get trend

In [12]:
def get_trends():
    params = {
    "api_key": os.getenv("SERP_API_KEY"),
    "engine": "google_trends_trending_now",
    "frequency": "realtime",
    "geo": "IT",
    "hl": "en",
    "cat": "t"
    }

    search = serpapi.search(params)
    return search

In [13]:
sample_day_trends = {
  "search_metadata": {
    "id": "6676863d95bf922e3ba8b41b",
    "status": "Success",
    "json_endpoint": "https://serpapi.com/searches/e089364abf0d11fd/6676863d95bf922e3ba8b41b.json",
    "created_at": "2024-06-22 08:07:25 UTC",
    "processed_at": "2024-06-22 08:07:25 UTC",
    "google_trends_trending_now_url": "https://trends.google.com/trends/api/dailytrends?hl=en&geo=IT",
    "raw_html_file": "https://serpapi.com/searches/e089364abf0d11fd/6676863d95bf922e3ba8b41b.html",
    "prettify_html_file": "https://serpapi.com/searches/e089364abf0d11fd/6676863d95bf922e3ba8b41b.prettify",
    "total_time_taken": 1.51
  },
  "search_parameters": {
    "engine": "google_trends_trending_now",
    "hl": "en",
    "geo": "IT",
    "frequency": "daily"
  },
  "daily_searches": [
    {
      "date": "20240622",
      "searches": [
        {
          "query": "Platini",
          "google_trends_link": "https://trends.google.com/trends/explore?q=Platini&date=now+7-d&geo=IT",
          "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Platini&tz=420",
          "traffic": 10000,
          "related_queries": [
            {
              "query": "michel platini",
              "google_trends_link": "https://trends.google.com/trends/explore?q=michel+platini&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=michel+platini&tz=420"
            }
          ],
          "articles": [
            {
              "title": "Notti Europee, Marco Mazzocchi fa infuriare Michel Platini: “Che ca ...",
              "link": "https://www.libero.it/magazine/news/notti-europee-mazzocchi-michel-platini-puntata-21-giugno-2024-127544",
              "snippet": "Il mitico \"Le Roi\", ospite di Notti Europee su Rai 1, stronca Mazzocchi dopo una domanda poco gradita. Attimi di imbarazzo, poi interviene Paola Ferrari.",
              "source": "Libero Tecnologia",
              "date": "6h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcSM2YDnMBJ2tT2BP2SPw366hc1AUG0uxM-EuyHewXEfn1SWBvt69unE-lVjBsXddgbKk3O2CMc8"
            },
            {
              "title": "Euro2024, Platini a Notti Europee: \"La Spagna è la sorpresa, ha un ...",
              "link": "https://sport.virgilio.it/euro2024-platini-a-notti-europee-i-giocatori-devono-governare-il-calcio-la-battuta-a-mazzocchi-867735",
              "snippet": "Michel Platini, ospite a Notti Europee, parla dell'Europeo e della sua Francia, ma ha anche allargato lo sguardo al futuro del calcio proponendo la sua ...",
              "source": "Virgilio Sport",
              "date": "9h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcSwdBxo2tiuF9zkixbLKVp-uMEMQgjDhjhlcSBkQLp5otaVE96mNwQ_cruEc5maZk-9XgptfzxI"
            }
          ]
        },
        {
          "query": "Pele",
          "google_trends_link": "https://trends.google.com/trends/explore?q=Pele&date=now+7-d&geo=IT",
          "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Pele&tz=420",
          "traffic": 5000,
          "related_queries": [
            {
              "query": "Celeste Arantes",
              "google_trends_link": "https://trends.google.com/trends/explore?q=Celeste+Arantes&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Celeste+Arantes&tz=420"
            }
          ],
          "articles": [
            {
              "title": "È morta Donna Celeste Arantes, la mamma di Pelé: aveva 101 anni ...",
              "link": "https://www.fanpage.it/sport/calcio/e-morta-donna-celeste-arantes-la-mamma-di-pele-aveva-101-anni-era-ricoverata-per-problemi-di-salute/",
              "snippet": "L'anziana madre di Pelé è morta in ospedale all'età di 101 anni.",
              "source": "Napoli Fanpage",
              "date": "10h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcSyLmx4xhQMGeuUqKu1X5o9-uRr9eFtOL8rP5UhpMFcsocKK8yX79cdpm0S2fiwupmIbTNthEDfmA"
            },
            {
              "title": "Pelé, addio alla madre Celeste Antares: è morta all'età di 101 anni",
              "link": "https://www.tuttosport.com/news/calcio/2024/06/21-129349432/pele_addio_alla_madre_celeste_antares_e_morta_all_eta_di_101_anni",
              "snippet": "La Fondazione: \"Un punto di riferimento come madre, che ha insegnato a Edson i valori umanitari che ha diffuso e difeso per tutta la sua carriera\"",
              "source": "Tuttosport",
              "date": "11h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcSpt8FS8Grbd5RjT8R42qOaXrP2_kAGtqP8d7hujPHMheYizELFXg_bWGtfkPXy_Z5pRW7dxlc-"
            },
            {
              "title": "Brasile, morta a 101 anni Celeste Arantes, la mamma di Pelé",
              "link": "https://www.tgcom24.mediaset.it/mondo/pele-morta-mamma-101-anni_83800340-202402k.shtml",
              "snippet": "Celeste Arantes ha trascorso gli ultimi 5 anni in stato vegetativo e non ha mai saputo della morte del figlio il 29 dicembre 2022.",
              "source": "TGCOM",
              "date": "12h ago"
            },
            {
              "title": "Muore a 101 anni Celeste Arantes, la mamma di Pelé",
              "link": "https://www.ansa.it/sito/notizie/sport/calcio/2024/06/21/muore-a-101-anni-celeste-arantes-la-mamma-di-pele_e1878c8c-4fab-4e83-a2b1-a6b6ea568627.html",
              "snippet": "Celeste Arantes, la madre di Pelé, è morta all'età di 101 anni a Santos, sulla costa di San Paolo. Lo conferma il figlio del campione mondiale del calcio, ...",
              "source": "Agenzia ANSA",
              "date": "13h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcR0gZW0rTYH9rxolZGT0BBtV-KEg_SrS0NegwGd551CjQONqR2ndAFaetODy6h84dViDkmcrGJf"
            },
            {
              "title": "Addio a Celeste Arantes, morta all'età di 101 anni la madre di Pelé",
              "link": "https://www.derbyderbyderby.it/calcio-estero/addio-a-celeste-arantes-morta-alleta-di-101-anni-la-madre-di-pele/",
              "snippet": "All'età di 101 anni è venuta a mancare la madre di Pelé, Doña Celeste. La donna era ricoverata in ospedale da otto giorni.",
              "source": "DerbyDerbyDerby",
              "date": "13h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcTAI7qFs4xcj658suoKDS2vXVeAeeeKSfoGrHeRm2zpVeW4L01cTzXJsOFrUFlw2-4NA_eQUkvf"
            },
            {
              "title": "È morta a 101 anni la madre di Pelé, Celeste Arantes - LaPresse",
              "link": "https://www.lapresse.it/calcio/2024/06/21/e-morta-a-101-anni-la-madre-di-pele-celeste-arantes/",
              "snippet": "È morta a 101 anni in Brasile la madre del grande calciatore Pelé, Celeste Arantes. Lo hanno annunciato ad Associated Press l'ex club di Pelé, il Santos, e.",
              "source": "LaPresse",
              "date": "12h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcSeF6HomqQYKlIFCtYZekX-epELZ1SLXpjAeaHFLnN92ZGjAJCwSb3tR8JOTeIltEW38lvVjjzW"
            }
          ]
        },
        {
          "query": "Rabiot",
          "google_trends_link": "https://trends.google.com/trends/explore?q=Rabiot&date=now+7-d&geo=IT",
          "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Rabiot&tz=420",
          "traffic": 5000,
          "articles": [
            {
              "title": "Juventus: Rabiot va via, tutto fatto per il sostituto",
              "link": "https://www.tshot.it/articolo/juventus-rabiot-va-via-tutto-fatto-per-il-sostituto/30367/",
              "snippet": "La Juventus è pronta a salutare Adrien Rabiot, con il quale non c'è intesa sul rinnovo de contratto: arriva un sostituto di lusso.",
              "source": "Tshot",
              "date": "1h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcSGNYhhnPM-ETEvL_0jdItCeFdPQ-ex5uIsGknZORYKw6f0Q-vowC2XqARBKH-Qe2eXNGsyckaB"
            },
            {
              "title": "Gazzetta: “Juve, Rabiot sempre più lontano: ecco perché. Lo tentano ...",
              "link": "https://www.sosfanta.com/news/gazzetta-juve-rabiot-sempre-piu-lontano-ecco-perche-lo-tentano-real-e-premier/",
              "snippet": "Futuro tutto da scrivere per Adrien Rabiot, sempre più lontano dalla Juventus però col passare dei giorni. Ecco il punto della Gazzetta dello Sport sull'ex ...",
              "source": "SOS Fanta",
              "date": "20h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcSzFKnhrByBAVjgcgeDxiMAo9_ifEFsz8E_EU5K6r1nMexjTEeXOak0L-id71AlyuYzouU0Tc05"
            },
            {
              "title": "Rabiot, la Juventus ci spera ancora. Il 'sogno' per il centrocampo",
              "link": "https://www.ilbianconero.com/a/rabiot-la-juventus-ci-spera-ancora-il-sogno-per-il-centrocampo-76422",
              "snippet": "La Juventus è in attesa di una risposta cruciale da parte di Adrien Rabiot. Il contratto del centrocampista francese è in scadenza a breve e la dirigenza.",
              "source": "ilBianconero",
              "date": "15h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcTNmVF55e340nSM_JTnXVttg5EoennGQ6tq29e8jtPI4xyB_KxrMoLiBz6o9MqkQjPq9nghwJbX"
            },
            {
              "title": "Juve, Rabiot nel mirino del Real Il Milan valuta Depay dell'Atletico",
              "link": "https://www.ilcentro.it/sport/juve-rabiot-nel-mirino-del-real-il-milan-valuta-depay-dell-atletico-1.3296769",
              "snippet": "In casa Juve Rabiot per ora non rinnova, anche perché avrebbe ricevuto un'offerta dal Real Madrid e ci sta riflettendo. Le “merengues”, tra l'altro, ...",
              "source": "Il Centro",
              "date": "8h ago"
            },
            {
              "title": "Calciomercato Juventus/ Rabiot non rinnova vuole il Real: visite ...",
              "link": "https://www.ilsussidiario.net/news/calciomercato-juventus-rabiot-non-rinnova-vuole-il-real-visite-mediche-per-douglas-luiz-21-giugno-2024/2721424/",
              "snippet": "Calciomercato Juventus, ultime news 21 giugno 2024: Douglas Luiz sarà un nuovo giocatore bianconero. Rabiot vicino ai saluti, alla porta c'è il Real Madrid.",
              "source": "Il Sussidiario.net",
              "date": "14h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcTACEtdD10DoPIdxHpqdYKvOjqZ8_8Gj9SZ7y3aAC8v17SzYbamwFZlC1_dFdFXHhX7uVkABlq1"
            },
            {
              "title": "Rabiot Juve, c'è una nuova pretendente per il francese! Se non ...",
              "link": "https://www.juventusnews24.com/rabiot-juve-calciomercato-nuova-pretendente-real-madrid-rinnovo-juventus-ultime-notizie-scenario/",
              "snippet": "Mg Verona 17/02/2024 - campionato di calcio serie A / Hellas Verona-Juventus / foto Matteo Gribaudi/Image Sport nella foto: esultanza gol Adrien Rabiot ...",
              "source": "Juventus News 24",
              "date": "11h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcTyDABJojuAfexPfR4JjcU3fMLZIwHKpHz3Cg0ib4LogpLSQueAahkyATRGjJrGbNuGr73gN8h7"
            },
            {
              "title": "Adrien Rabiot verso un 'nuovo ruolo'? L'idea di Deschamps",
              "link": "https://www.ilbianconero.com/a/adrien-rabiot-verso-un-nuovo-ruolo-lidea-di-deschamps-44556",
              "snippet": "La Francia stasera affronterà l'Olanda in un match chiave per le gerarchie del girone D. Nella scorsa gara, vinta 1-0 contro l'Austria i transalpini.",
              "source": "ilBianconero",
              "date": "18h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcRcJNbWz_XVFFOL50p9n8aSJjhK4hHXt242yOoRJBmmzHgG8n7XWZYaZHdFXWDc3i0wAALmqJsr"
            },
            {
              "title": "La Juve aspetta Rabiot ma studia alternative. Lazio-Greenwood, si ...",
              "link": "https://www.repubblica.it/sport/calcio/mercato/2024/06/21/news/juve_aspetta_rabiot_lazio-greenwood_martinez_allinter_calciomercato-423266725/",
              "snippet": "La Juventus aspetta una risposta da Adrien Rabiot. Il contratto del francese scade tra pochi giorni e i bianconeri hanno presentato una proposta di rinnovo ...",
              "source": "La Repubblica",
              "date": "22h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcRU5t5Q7gsf-XTum_V2YLGvnCK4l9kdqkSxeopHqoszurnWmVATXGv6uksZieTZgOrhJjmg1lR2"
            }
          ]
        },
        {
          "query": "Grealish",
          "google_trends_link": "https://trends.google.com/trends/explore?q=Grealish&date=now+7-d&geo=IT",
          "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Grealish&tz=420",
          "traffic": 5000,
          "articles": [
            {
              "title": "Grealish, l'omaggio alla Juventus fa sognare i tifosi bianconeri",
              "link": "https://www.tuttosport.com/news/calcio/calcio-estero/2024/06/21-129347375/grealish_l_omaggio_alla_juventus_fa_sognare_i_tifosi_bianconeri",
              "snippet": "La stella del City ha postato un video mentre si allena su un campo dell'Academy e ha scatenato i commenti social del popolo juventino.",
              "source": "Tuttosport",
              "date": "12h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcQoM2mUcMwur0vDr9LNX4sn_qIojBvOqSyQXysJJyk8CEUC1EV3qslOg-rO52NVux3-KzW7z4Um"
            },
            {
              "title": "Grealish e la Juventus, l'ultimo allenamento fa impazzire i social ...",
              "link": "https://www.calciomercato.it/2024/06/21/grealish-e-la-juventus-lultimo-allenamento-fa-impazzire-i-social-ecco-cosa-e-successo/",
              "snippet": "L'ultima Instagram Story pubblicata dall'attaccante del Manchester City non è affatto passata inosservata. Juventus 'tirata' in ballo.",
              "source": "CalcioMercato.it",
              "date": "12h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcTrjvlOBhN6qjh3sfvfBaJzS_bpqmgQqmjNsaRKkZsZQg7RAJYL99P3GjQzVcVwlLeNk3VfGHGX"
            },
            {
              "title": "Grealish si allena in un centro Juventus Academy",
              "link": "https://www.ilbianconero.com/a/grealish-si-allena-in-un-centro-juventus-academy-52796",
              "snippet": "L'esclusione di Jack Grealish dalla lista dei convocati per Euro2024 ha fatto discutere nelle scorse settimane, ma la stella del Manchester City non si è.",
              "source": "ilBianconero",
              "date": "10h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcSMRRsAaTHMP7Y280Yl0HjoiTELrdbANnmYSKTvLBU0SWTCOAbko7TRtXXNQeBwEXvFKg7WAyD4"
            },
            {
              "title": "Grealish si propone alla Juve? L'inglese si allena in un campo dell ...",
              "link": "https://www.juventusnews24.com/grealish-juve-campo-academy-manchester-city-video/",
              "snippet": "Il fantasista del Manchester City, infatti, ha pubblicato un video in cui si allena in un campo dell'Academy bianconera. Grealish, tra l'altro, ha messo proprio ...",
              "source": "Juventus News 24",
              "date": "12h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcTlmdNlB8zN572cxQFzY7FognP99bNr5FWl4EzRKSUm-s0US5_zIR-5Z2fKbB-jNKBZXhykMd--"
            },
            {
              "title": "La stella del Manchester City Grealish fa parlare i fan dopo il video ...",
              "link": "https://it.football-italia.net/La-star-del-manchester-city-grealish-parla-della-juventus/",
              "snippet": "La stella del Manchester City Jack Grealish ha fatto parlare i fan durante le sue vacanze estive quando ha confermato su Instagram che si stava allenando ...",
              "source": "Football Italia",
              "date": "11h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcQQ-k45z_5koxs3gU-EzYTk4K3F_tXU0C8VFozR0jaT2VAcU6bsCkWZHP3g0nEis3LDQsdmuH3Z"
            },
            {
              "title": "Grealish, allenamento sul campo di un'academy Juventus: il VIDEO ...",
              "link": "https://www.bianconeranews.it/altre-notizie/grealish-allenamento-sul-campo-di-un-academy-juventus-il-video-esalta-i-tifosi-75551",
              "snippet": "La Juventus sta lavorando in questa sessione estiva di calciomercato al fine di rafforzare al meglio la propria rosa. E' in entrata Douglas.",
              "source": "Bianconera News",
              "date": "11h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcRZJFOjo0kP4kNR6Fzlm0BMkfJyrgt_K8H25Inn2ZM4GowGkMUBbR2CLnMW67xwFzhp09SMpT7d"
            },
            {
              "title": "Jack Grealish si mostra mentre si allena da solo all'accademia della ...",
              "link": "https://colornews.it/jack-grealish-si-mostra-mentre-si-allena-da-solo-allaccademia-della-juventus-mentre-la-stella-del-manchester-city-va-in-vacanza-in-italia-dopo-laffronto-dellinghilterra-a-euro-2024/",
              "snippet": "La stella del Manchester City e dell'Inghilterra si è mostrata allenata durante la bassa stagioneJack Grealish è stata un'omissione scioccante dalla.",
              "source": "COLORnews",
              "date": "7h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcTn5Y5O4ldQA2atbHgbkQ9g6tI4l5Mksjj5DTLzrjZeHRwJ3URjuaRdVwLOyzGmd__oZ2T6pg8J"
            }
          ]
        },
        {
          "query": "Taylor Wily",
          "google_trends_link": "https://trends.google.com/trends/explore?q=Taylor+Wily&date=now+7-d&geo=IT",
          "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Taylor+Wily&tz=420",
          "traffic": 2000,
          "articles": [
            {
              "title": "Morto a 56 anni Taylor Wily, star di 'Hawaii Five-O'",
              "link": "https://www.sulpanaro.net/2024/06/morto-a-56-anni-taylor-wily-star-di-hawaii-five-o/",
              "snippet": "(Adnkronos) - E' morto a 56 anni Taylor Wily. L'attore americano aveva raggiunto la fama nel circuito del sumo alla fine degli anni '80 prima di passare ...",
              "source": "SulPanaro",
              "date": "1h ago"
            },
            {
              "title": "Taylor Wily, morto a 56 anni l'attore e campione di sumo, star della ...",
              "link": "https://www.repubblica.it/spettacoli/tv-radio/2024/06/21/news/taylor_wily_morto_a_56_anni_lattore_e_campione_di_sumo_star_della_serie_hawaii_five-0-423269346/",
              "snippet": "Il suo ruolo era quello di Kamekona Tupuola, proprietario di un chiosco di gamberetti sulla spiaggia e informatore della task force.",
              "source": "La Repubblica",
              "date": "13h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcSmjImjx6PDt3DmZvxznbHqQyo0dVbMN9dXcaPQ8tSdW6046tXhI4N-G79PA1a3HYwsE2gSMkFp"
            },
            {
              "title": "Taylor Wily morto a 56 anni: l'attore e campione di sumo e star della ...",
              "link": "https://www.ilmattino.it/spettacoli/televisione/taylor_wily_morto_attore_hawaii_five_0_oggi_21_6_2024-8196142.html",
              "snippet": "La notizia è stata confermata dal produttore televisivo Peter M. Lenkov che ha salutato l'attore con un post accorato su Instagram.",
              "source": "Il Mattino",
              "date": "11h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcQ844GJx6xlRBtjT1GZuMeY5WoiSC5_zmh46eRncyfuUGIZtkj3GKyiU6RwS1BpPPtUzazKK7pJ"
            },
            {
              "title": "È morto Taylor Wily, l'attore di “Hawaii Five-0” e “Magnum P.I.” aveva ...",
              "link": "https://www.fanpage.it/spettacolo/personaggi/e-morto-taylor-wily-lattore-di-hawaii-five-0-e-magnum-p-i-aveva-56-anni/",
              "snippet": "È morto a 56 anni Taylor Wily, attore conosciuto per la partecipazione alle serie tv “Hawaii Five-0” e “Magnum P.",
              "source": "Napoli Fanpage",
              "date": "16h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcRMpX2nIbYWdGkfAMLJA_PRq1HzarwN9WUDsmiwMSS2_sYCyO13oBr4q8oB3vKVuGCxu7pi-J7j"
            },
            {
              "title": "È morto Taylor Wily, star di Hawaii Five-O ed ex lottatore di sumo",
              "link": "https://movieplayer.it/news/morto-taylor-wily-star-hawaii-five-o-ex-lottatore-sumo_143578/",
              "snippet": "È scomparso l'attore e lottatore, famoso per il ruolo nella serie tv in onda dal 2010 al 2020.",
              "source": "Movieplayer.it",
              "date": "16h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcQomnYiWIskSPtdXjfmb46p-hTNg-RbvKqTUYuxY_ajZVYYw8QfSo4HpN-uIeAsJOMfDo-3TzzH"
            },
            {
              "title": "Taylor Wily, morto a 56 l'attore della serie tv Hawaii Five-0",
              "link": "https://dilei.it/vip/taylor-wily-morto/1416611/",
              "snippet": "Imponente, ma dal cuore d'oro. Così Taylor Wily ha interpretato per anni il ruolo di Kamekona Tupuola, proprietario di un chiosco di gamberetti sulla ...",
              "source": "DiLei",
              "date": "2h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcT6Fghb_ZJ2dIXWw4onsawnRgbpsm-D8ScFTh9PS3tHH5uzr-WHp2D0HECa84yQE1yfzf4yI_qE"
            },
            {
              "title": "Hawaii Five-0: È morto l'attore Taylor Wily, aveva 56 anni",
              "link": "https://www.comingsoon.it/serietv/news/hawaii-five-0-e-morto-l-attore-taylor-wily-aveva-56-anni/n183448/",
              "snippet": "Hawaii Five-0 morto attore Taylor Wily 56 anni Kamekona Tupuola personaggio regolare serie tv CBS cause lottatore sumo vita biografia.",
              "source": "ComingSoon.it",
              "date": "16h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcRlkWaH51ZrhJbjpN7XIr9UbWeoOsQ4WLHEhMc_qMpO0ywyGuEjK-oQjtpv5Jzn4OVChYUydR2j"
            }
          ]
        },
        {
          "query": "Boban",
          "google_trends_link": "https://trends.google.com/trends/explore?q=Boban&date=now+7-d&geo=IT",
          "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Boban&tz=420",
          "traffic": 2000,
          "articles": [
            {
              "title": "Boban: \"Sconfitta dell'Italia un po' imbarazzante. Adoro Spalletti, ma ...",
              "link": "https://www.tuttomercatoweb.com/serie-a/boban-sconfitta-dell-italia-un-po-imbarazzante-adoro-spalletti-ma-basta-pellegrini-a-sinistra-1981748",
              "snippet": "Zvonimir Boban, ospite presso gli studi di Sky Sport, ha criticato la prova degli azzurri dopo la sconfitta incassata dall'Italia contro la Spagna:.",
              "source": "TUTTO mercato WEB",
              "date": "10h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcTNPE-UhBIXLG0Jcl0vmy_bkNwgjKYR1QNMHa8ogoOxi1npwFPaDvx9cyAPlOZqwXGpNLpyWwzq"
            },
            {
              "title": "Boban critica l'Italia: \"Dominata dalla Spagna, è mancata la ...",
              "link": "https://www.fcinternews.it/focus/boban-critica-l-italia-dominata-dalla-spagna-e-mancata-la-personalita-su-barella-957634",
              "snippet": "Ospite negli studi di Sky Sport, Zvonimir Boban ha usato parole critiche nei confronti della Nazionale italiana dopo la brutta performance contro la.",
              "source": "Fcinternews.it",
              "date": "11h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcSrNuiMnCN50IdR27R7qaVBE8765gYGsOrf5wJFhY2rXMo2zDeoyj4r53lIG5zbORhkJhhahZwH"
            },
            {
              "title": "Euro 2024, Boban: “Italia-Spagna un brusco e doloroso risveglio ...",
              "link": "https://www.sportface.it/calcio/nazionali/euro-2024-boban-italia-spagna-un-brusco-doloroso-risveglio-con-croazia-partita-aperta/2590667",
              "snippet": "Euro 2024, le dichiarazioni di Zvonimir Boban: \"Italia-Spagna un brusco e doloroso risveglio. Con la Croazia partita aperta\"",
              "source": "Sportface.it",
              "date": "15h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcQhujvoNLzpS9sVNY-AkaimqgV0kJEQ150BA6E3Q5ueWquEv74-zkoio63z0Sm9EIYj4jEnUEEW"
            },
            {
              "title": "Boban: \"Italia-Croazia? Sarà una partita aperta, sono squadre che ...",
              "link": "https://www.sardiniapost.it/sport/boban-italia-croazia-sara-una-partita-aperta-sono-squadre-che-reagiscono-nelle-difficolta/",
              "snippet": "Mi ha sorpreso, però, la mancanza di personalità, ma è evidente che pochi ragazzi a questi livelli hanno la personalità per dimostrare». Così Zvonimir Boban, ex ...",
              "source": "SardiniaPost",
              "date": "10h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcTZlZK8jRWj0finTyhPtbkibUFx1772MJuUyLk3srsBObw6Wb-PwNDdMGLhADCanJlFxAU8D592"
            },
            {
              "title": "Boban duro: \"La sconfitta dell'Italia è stata imbarazzante''",
              "link": "https://www.tuttojuve.com/altre-notizie/boban-duro-la-sconfitta-dell-italia-e-stata-imbarazzante-692157",
              "snippet": "Zvonimir Boban dagli studi di Sky Sport ha criticato aspramente la performance dall'Italia contro la.",
              "source": "Tutto Juve",
              "date": "8h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcTDsf442FEJWWD7xmPO0flgTGb8brOLKEvGRNYAYrnTYpqOnopph42Vx0_qkD52-16BBMT-EJ4V"
            },
            {
              "title": "Boban stronca l'Italia: «E' stata imbarazzante con la Spagna. Adoro ...",
              "link": "https://www.juventusnews24.com/boban-spagna-italia-spalletti-pellegrini-sky/",
              "snippet": "Boban dagli studi di Sky Sport ha commentato in modo critico la sconfitta rimediata dall'Italia contro la Spagna ad Euro 2024 decisa da un autogol di Calafiori, ...",
              "source": "Juventus News 24",
              "date": "12h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcS9zWlJIyeHit7AMHuLl6x4WNu38BuIZ7ntJrzRXkHvjR8QhEFYIpElzpeeOzpYPx07yjulCVtG"
            }
          ]
        },
        {
          "query": "Bolelli",
          "google_trends_link": "https://trends.google.com/trends/explore?q=Bolelli&date=now+7-d&geo=IT",
          "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Bolelli&tz=420",
          "traffic": 2000,
          "articles": [
            {
              "title": "Bolelli e Vavassori in finale di doppio all'ATP Halle: battuti ...",
              "link": "https://sport.sky.it/tennis/2024/06/21/bolelli-vavassori-atp-halle-doppio-risultato",
              "snippet": "Prosegue l'anno d'oro di Simone Bolelli e Andrea Vavassori che tornano in finale ad Halle battendo in due set i tedeschi Hanfmann e Koepfer dopo aver ...",
              "source": "Sky Sport",
              "date": "20h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcR12XzT5qNH2ZnMDafdE8HFT_pbvjuQ1QCix9bHN7rJODs3NfiN10Nus7aJPyZyKer5Q3oUkDJ8"
            },
            {
              "title": "Torneo di Halle, Bolelli e Vavassori conquistano un'altra finale",
              "link": "https://www.corrieredellosport.it/news/tennis/2024/06/21-129324363/torneo_di_halle_bolelli_e_vavassori_conquistano_un_altra_finale",
              "snippet": "La coppia azzurra elimina il duo di casa composto da Koepfer e Hanfmann e si andrà a giocare un titolo per la quarta volta in stagione.",
              "source": "Corriere dello Sport.it",
              "date": "12h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcQY-1pLhSGc_3_oCicko6BRJjbWguBhZr4_UbcN2IKbPcKOX0A57srfUf4H285nvGX9MguBeok6"
            },
            {
              "title": "Atp Halle, Bolelli e Vavassori in finale nel doppio",
              "link": "https://www.sulpanaro.net/2024/06/atp-halle-bolelli-e-vavassori-in-finale-nel-doppio/",
              "snippet": "(Adnkronos) - Simone Bolelli e Andrea Vavassori volano in finale del doppio all'Atp di Halle in corso in Germania. Gli azzurri hanno battuto in due set il ...",
              "source": "SulPanaro",
              "date": "18h ago"
            },
            {
              "title": "Bolelli e Vavassori annullano 9 set ponti! Quarta finale della ...",
              "link": "https://www.eurosport.it/tennis/atp-halle-doppio/2024/bolelli-e-vavassori-annullano-9-set-ponti-quarta-finale-della-stagione-in-doppio-a-halle_sto20012445/story.shtml",
              "snippet": "TENNIS, HALLE - Simone Bolelli e Andrea Vavassori accedono alla quarta finale della stagione superando per 6-3 7-6 la coppia tedesca composta da Yannick ...",
              "source": "Eurosport.it",
              "date": "19h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcQQiQKPRk97yyF82Jg7H3LyKGm5AExlwSdtGj4d1z-_PkiasU83mue1qIwKjQs2i6fRJJ8k5E25"
            },
            {
              "title": "Atp Halle, Bolelli-Vavassori Hanfmann-Koepfer 6-3, 7-6: gli highlights",
              "link": "https://video.gazzetta.it/video-atp-halle-bolelli-vavassori-hanfmann-koepfer-highlights/3cf8f0f5-cc15-40ee-b328-d989f43edxlk",
              "snippet": "Bolelli e Vavassori cambiano superficie, ma non sbagliano: la coppia azzurra conquista la finale del doppio maschile del torneo Atp 500 di Halle battendo la ...",
              "source": "La Gazzetta dello Sport",
              "date": "18h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcSq3QPraSoIPD8VPJ9AFiiqQdBIfBYr03UKujs7VcRInoBjwJQkx_oB9xJn0JbRJHDC3zl9kUyy"
            },
            {
              "title": "Bolelli-Vavassori in finale ad Halle",
              "link": "https://www.ansa.it/sito/notizie/sport/2024/06/21/bolelli-vavassori-in-finale-ad-halle_3a554205-9c78-4741-b95a-d2329ebca67f.html",
              "snippet": "Per il secondo torneo consecutivo, Simone Bolelli e Andrea Vavassori raggiungono la finale in doppio, stavolta sull'erba tedesca di Halle.",
              "source": "Agenzia ANSA",
              "date": "18h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcSDYujZj2pEt_MDoItGyL8wawSg7pQ4Ojl_UICxWSLLPyquEzWrZejrURC2ml92_JoZbwaasdgw"
            }
          ]
        }
      ]
    },
    {
      "date": "20240621",
      "searches": [
        {
          "query": "Olanda-Francia",
          "google_trends_link": "https://trends.google.com/trends/explore?q=Olanda-Francia&date=now+7-d&geo=IT",
          "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Olanda-Francia&tz=420",
          "traffic": 200000,
          "related_queries": [
            {
              "query": "francia olanda",
              "google_trends_link": "https://trends.google.com/trends/explore?q=francia+olanda&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=francia+olanda&tz=420"
            },
            {
              "query": "olanda - francia",
              "google_trends_link": "https://trends.google.com/trends/explore?q=olanda+-+francia&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=olanda+-+francia&tz=420"
            },
            {
              "query": "Francia",
              "google_trends_link": "https://trends.google.com/trends/explore?q=Francia&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Francia&tz=420"
            },
            {
              "query": "Griezmann",
              "google_trends_link": "https://trends.google.com/trends/explore?q=Griezmann&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Griezmann&tz=420"
            },
            {
              "query": "xavi simons",
              "google_trends_link": "https://trends.google.com/trends/explore?q=xavi+simons&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=xavi+simons&tz=420"
            },
            {
              "query": "olanda francia formazioni",
              "google_trends_link": "https://trends.google.com/trends/explore?q=olanda+francia+formazioni&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=olanda+francia+formazioni&tz=420"
            },
            {
              "query": "paesi bassi francia",
              "google_trends_link": "https://trends.google.com/trends/explore?q=paesi+bassi+francia&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=paesi+bassi+francia&tz=420"
            }
          ],
          "articles": [
            {
              "title": "Olanda - Francia pareggio senza lampi",
              "link": "https://www.agi.it/sport/news/2024-06-21/euro-2024-olanda-francia-pareggio-26870256/",
              "snippet": "Non si va oltre lo 0-0. Grande assente Mbappè, con Thuram che conquista l'unica occasione nitida. I blues che nell'ultima gara affronteranno la Polonia, ...",
              "source": "AGI - Agenzia Giornalistica Italia",
              "date": "10h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcTVfk8AhYZFNdjyFYEaYd_su3UkwNwvsGX-EPoxjRMssfE3TNr1cvIW3jb8MtmaMCXNwjy7Asf2"
            },
            {
              "title": "Olanda-Francia 0-0, le pagelle della gara di Euro 2024",
              "link": "https://sport.sky.it/calcio/europei/2024/06/21/olanda-francia-pagelle-europei-2024",
              "snippet": "A Lipsia termina 0-0 la sfida tra Olanda-Francia. Nella squadra di Koeman brilla la difesa, mentre in casa Francia, con Mbappé che non è sceso in campo, ...",
              "source": "Sky Sport",
              "date": "10h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcRCShu4qgtkCQXp7NyiTXb-y05fPt3xPTGIo58Hhs62DuZ7ikJq5sT9Ywy0piYM3zuHhEgEl5fL"
            },
            {
              "title": "Olanda-Francia 0-0: qualche errore in zona gol, sostanziale ...",
              "link": "https://www.eurosport.it/calcio/campionati-europei/2024/olanda-francia-0-0-qualche-errore-in-zona-gol-sostanziale-equilibrio_sto10203427/story.shtml",
              "snippet": "EURO 2024 - Nel gruppo D le due big danno vita a una bella sfida, ma non riescono a segnare. Griezmann spreca, Maignan para.",
              "source": "Eurosport.it",
              "date": "11h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcRofsVpb05TAuwKq-fk1_vchCiP4e0NW8cMigvo4WmMfphSxKBe8bESiGUkc7UsjLSbu5WJ58-b"
            },
            {
              "title": "Olanda-Francia 0-0 al Leipzig Stadium di Lipsia. Tutto rimandato per ...",
              "link": "https://www.rainews.it/articoli/2024/06/calcio-europei-euro2024-fase-a-gironi-girone-d-olanda-francia-redbull-arena-lipsia-germania-dove-quando-vederla-diretta-tv-streaming-cronaca-testuale-80e9f020-79ea-43f7-9859-2157d3b5bad1.html",
              "snippet": "Un pareggio senza reti, il primo di questi campionati, che fa sorridere entrambe le compagini. I Blues che affronteranno la Polonia nella prossima partita.",
              "source": "Rai News",
              "date": "10h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcSEbZWU7bD-oqO9W-cXU659SeZxzfhCJHx1tTLYZxM5pMlM4JXiWMR2JQDXEhTzjod4og9eRMBq"
            },
            {
              "title": "Olanda - Francia (0-0) Europei 2024/2025",
              "link": "https://www.repubblica.it/sport/dirette/calcio/euro-2024/diretta-olanda-francia-g2413158/",
              "snippet": "Senza Mbappé, non segna nessuno: Olanda e Francia si accontentano del pareggio dopo un grande inizio di partita, potando a casa uno 0-0 che permette a ...",
              "source": "La Repubblica",
              "date": "11h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcT5uExKtms0kUnia_nFFw8hQDzyGwoYRByyZmtx-lsasZy7D4S8UUc3juUo4a3cU2uzPxJ7RXE3"
            },
            {
              "title": "FINALE Olanda-Francia 0-0: Maignan para, Griezmann spreca. Un ...",
              "link": "https://www.gazzetta.it/Calcio/Europei/21-06-2024/olanda-francia-live-degli-europei-2024-la-diretta.shtml",
              "snippet": "Olandesi e Bleus restano in testa al gruppo D con 4 punti. Austria a 3 e ancora in corsa. Polonia eliminata.",
              "source": "La Gazzetta dello Sport",
              "date": "11h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcTWUg-_5T6QwhZmh3koo7KYdcH6um_csdhl_3ATYAneXCU7c2DwQF-T37Ie4tAhjqPBAX5b0V2_"
            },
            {
              "title": "Euro 2024, Olanda-Francia 0-0: Maignan para e Griezmann ...",
              "link": "https://www.sportmediaset.mediaset.it/speciale/europei-germania-2024/euro-2024-olanda-francia-0-0-maignan-para-e-griezmann-spreca_83793616-202402k.shtml",
              "snippet": "Al Lipsia Stadium gara tirata ed equilibrata: Orange e Galletti \"vedono\" gli ottavi. Nella ripresa annullato un gol di Simons. Mbappe in panchina per...",
              "source": "Sport Mediaset",
              "date": "11h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcTAHOdYGMCGEZ2Th4oUO_tL5goUr2Ou9ji2vfMN7IgIegINHn7kDb9fpGVQNxKOvouRD1FyVYtJ"
            }
          ]
        },
        {
          "query": "Roberto Baggio",
          "google_trends_link": "https://trends.google.com/trends/explore?q=Roberto+Baggio&date=now+7-d&geo=IT",
          "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Roberto+Baggio&tz=420",
          "traffic": 200000,
          "related_queries": [
            {
              "query": "Baggio",
              "google_trends_link": "https://trends.google.com/trends/explore?q=Baggio&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Baggio&tz=420"
            },
            {
              "query": "Altavilla Vicentina",
              "google_trends_link": "https://trends.google.com/trends/explore?q=Altavilla+Vicentina&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Altavilla+Vicentina&tz=420"
            },
            {
              "query": "baggio roberto",
              "google_trends_link": "https://trends.google.com/trends/explore?q=baggio+roberto&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=baggio+roberto&tz=420"
            },
            {
              "query": "rapina baggio",
              "google_trends_link": "https://trends.google.com/trends/explore?q=rapina+baggio&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=rapina+baggio&tz=420"
            },
            {
              "query": "baggio rapina villa",
              "google_trends_link": "https://trends.google.com/trends/explore?q=baggio+rapina+villa&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=baggio+rapina+villa&tz=420"
            },
            {
              "query": "baggio rapina",
              "google_trends_link": "https://trends.google.com/trends/explore?q=baggio+rapina&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=baggio+rapina&tz=420"
            },
            {
              "query": "rapina villa baggio",
              "google_trends_link": "https://trends.google.com/trends/explore?q=rapina+villa+baggio&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=rapina+villa+baggio&tz=420"
            },
            {
              "query": "roberto baggio oggi",
              "google_trends_link": "https://trends.google.com/trends/explore?q=roberto+baggio+oggi&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=roberto+baggio+oggi&tz=420"
            },
            {
              "query": "baggio oggi",
              "google_trends_link": "https://trends.google.com/trends/explore?q=baggio+oggi&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=baggio+oggi&tz=420"
            }
          ],
          "articles": [
            {
              "title": "\"Ora mi rimane tanta rabbia\", Roberto Baggio il giorno dopo l ...",
              "link": "https://www.rainews.it/articoli/2024/06/ora-mi-rimane-tanta-rabbia-roberto-baggio-il-giorno-dopo-laggressione-lintervista-altavilla-vicentina-138414ca-c368-413e-829c-4455265c7a0e.html",
              "snippet": "Il racconto del campione: \"Quando mi ha dato il colpo mi sono sentito impotente. Per fortuna a livello fisico ho rimediato solamente alcuni punti di sutura\"",
              "source": "Rai News",
              "date": "1h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcQb1ec9rQFLPGTBcC7eVMond4oQlEI1SAUF85AK_SzD3-ggP0adEL5wFbAlNMrJfAg8FrQ8Bx_B"
            },
            {
              "title": "Roberto Baggio rapinato e aggredito in casa durante Spagna-Italia",
              "link": "https://radionorba.it/roberto-baggio-rapinato-e-aggredito-in-casa-durante-spagna-italia/",
              "snippet": "Ad agire, secondo le prime ricostruzioni, cinque persone armate. L'ex campione sarebbe stato colpito alla testa con il calcio di una pistola.",
              "source": "Radionorba",
              "date": "1d ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcSOiEs4KQACgjooAQJ8BftBZgVppil-VmKMjgYzOaj8LOuBJwRq1A23q1scKsXI9Yo4OFRS_qsa"
            },
            {
              "title": "Roberto Baggio, rapina nella sua villa durante Italia-Spagna ...",
              "link": "https://corrieredelveneto.corriere.it/notizie/vicenza/cronaca/24_giugno_21/roberto-baggio-rapina-nella-sua-villa-di-altavilla-durante-italia-spagna-l-ex-calciatore-picchiato-dai-malviventi-f9137905-7ba9-4f83-8ab2-df192d46dxlk.shtml",
              "snippet": "I malviventi hanno chiuso la famiglia Baggio in una stanza e sono stati nell'abitazione di Altavilla Vicentina per 40 minuti: Baggio è stato colpito alla ...",
              "source": "Corriere della Sera",
              "date": "13h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcRXtAsXicR_zrV6B2td90wJNPs2maD-3XxYTjZmETftZnWCNozngXM8B3JqcCobLhsY8cC2vCt1"
            },
            {
              "title": "Roberto Baggio rapinato in casa durante Spagna-Italia. 'Grazie per l ...",
              "link": "https://www.ansa.it/veneto/notizie/2024/06/21/roberto-baggio-rapinato-in-casa-durante-spagna-italia.-grazie-per-laffetto_c9cdb1cf-d6e7-42f7-95fa-8a139978c803.html",
              "snippet": "Quaranta minuti di terrore, ostaggio in casa propria di cinque banditi che non hanno esitato a picchiarlo per arraffare denaro e gioielli.",
              "source": "Agenzia ANSA",
              "date": "18h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcRCVKsNdBqp_j4lLrqY7jeICBSo3IP8VQCqQ99Nv94BdMKTNDwOHbOX-Neq-3E-YUVxBbTGYW1Q"
            },
            {
              "title": "Rapina nella villa di Roberto Baggio, l'ex calciatore sequestrato con ...",
              "link": "https://tg24.sky.it/cronaca/2024/06/21/rapina-roberto-baggio",
              "snippet": "Leggi su Sky TG24 l'articolo Rapina nella villa di Roberto Baggio, l'ex calciatore sequestrato con la famiglia.",
              "source": "Sky Tg24",
              "date": "19h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcSzr809fmLG4KQRV7seH4Vj2iWoMLp8aN6mr9uSkaCcS74J12xD2Z6y2Q82wGOZn18vFxTlvDR5"
            },
            {
              "title": "Roberto Baggio ferito, rapinato e sequestrato in casa: ora ringrazia i ...",
              "link": "https://it.euronews.com/my-europe/2024/06/21/roberto-baggio-ferito-rapinato-e-sequestrato-in-casa-ora-ringrazia-i-fan-per-laffetto-rice",
              "snippet": "Ferito e derubato in casa, Roberto Biaggio e la sua famiglia ringraziano chi li sta sostenendo mentre il sopralluogo nella sua tenuta continua.",
              "source": "euronews Italiano",
              "date": "21h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcRUsZ5eWOovXFkNTQkk-yjd5xAHiZvYcFvGQI7xKjen5MD6MXTnQHBnANX10TqlWtffosB22pUx"
            },
            {
              "title": "Baggio, rapina in villa durante Italia-Spagna. L'ex calciatore colpito ...",
              "link": "https://www.repubblica.it/cronaca/2024/06/21/news/baggio_rapina_villa_italia_spagna-423265669/",
              "snippet": "Baggio colpito in testa con il calcio di una pistola. La banda è entrata nella casa dell'ex Pallone d'oro ad Altavilla Vicentina.",
              "source": "La Repubblica",
              "date": "10h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcR60u9I1msLy_4IaybS8zgabOT3L6jsAQEm2RJNr0Jbuox6hfUh8y63xfZp0WUJwmDKULxEobvy"
            },
            {
              "title": "“Ciccia, adesso basta però”: il primo post di Baggio su Instagram",
              "link": "https://www.rainews.it/video/2024/06/ciccia-adesso-basta-pero-il-primo-post-di-baggio-su-instagram-altavilla-vicentina-vicenza-c8b9400d-febd-40b2-8a2f-8f38d7954d03.html",
              "snippet": "Il primo ingresso di Roberto Baggio su Instagram era avvenuto alla tenera di 57 anni, a bordo della sua Panda 4X4 verde, grazie ad un video postato dalla ...",
              "source": "Rai News",
              "date": "1d ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcSytBynohgge_ywVZj6LwmQVDBcM5SMhQ04SC-BC6PlzxvCn5YEzoZ6mBVjyQ-GyFQMyCrrvooF"
            }
          ]
        },
        {
          "query": "Copa America",
          "google_trends_link": "https://trends.google.com/trends/explore?q=Copa+America&date=now+7-d&geo=IT",
          "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Copa+America&tz=420",
          "traffic": 50000,
          "related_queries": [
            {
              "query": "argentina canada",
              "google_trends_link": "https://trends.google.com/trends/explore?q=argentina+canada&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=argentina+canada&tz=420"
            },
            {
              "query": "argentina - canada",
              "google_trends_link": "https://trends.google.com/trends/explore?q=argentina+-+canada&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=argentina+-+canada&tz=420"
            },
            {
              "query": "Coppa America",
              "google_trends_link": "https://trends.google.com/trends/explore?q=Coppa+America&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Coppa+America&tz=420"
            },
            {
              "query": "Copa America 2024",
              "google_trends_link": "https://trends.google.com/trends/explore?q=Copa+America+2024&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Copa+America+2024&tz=420"
            }
          ],
          "articles": [
            {
              "title": "Argentina-Canada 2-0, i gol della partita inaugurale della Coppa ...",
              "link": "https://sport.sky.it/calcio/2024/06/21/argentina-canada-risultato-gol",
              "snippet": "Leggi su Sky Sport l'articolo Argentina-Canada 2-0, i gol della partita inaugurale della Coppa America.",
              "source": "Sky Sport",
              "date": "1d ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcRYWX83ue5JKlNTIQ7kWO1dXvlTNkX3fknkIrlK29LANyEG2cyTFA8VI3YJIEwe1ilGU5_4GzKn"
            },
            {
              "title": "Copa America, buona la prima per l'Argentina: Lautaro in gol ...",
              "link": "https://www.sportmediaset.mediaset.it/calcio/calcioestero/copa-america-buona-la-prima-per-l-argentina-lautaro-in-gol-e-2-0-sul-canada_83767079-202402k.shtml",
              "snippet": "Il Toro parte dalla panchina e assiste alla rete di Julian Alvarez, poi e decisivo: sua la zampata che chiude i giochi, dopo due errori. Ottima la...",
              "source": "Sport Mediaset",
              "date": "1d ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcQO06MjO-JYCzxQNlpCUB6bdEpBBoid-rmXazsoWuzAYSHGZ9LkBmHd9ntlWU9r3hjSq79Snlc-"
            },
            {
              "title": "Copa America 2024 in TV, dove vederla in chiaro e in streaming: il ...",
              "link": "https://www.fanpage.it/sport/calcio/copa-america-2024-in-tv-dove-vederla-in-chiaro-e-in-streaming-il-calendario-delle-partite/",
              "snippet": "Il calendario completo della Copa America 2024 negli Stati Uniti. Il calendario completo delle partite e la composizione dei gironi.",
              "source": "Napoli Fanpage",
              "date": "21h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcQpEIoD2pwlxSK-lf3q4RmjC1kHM7RzLIadAvkuFIV6amyRIFLBRd0R7IpjHo3kWWKtPhhdvJvB"
            },
            {
              "title": "Copa America, gol di Lautaro e vittoria dell'Argentina",
              "link": "https://www.inter.it/it/notizie/copa-america-2024-inter-gol-lautaro-argentina-canada",
              "snippet": "Esordio vincente per il Toro e l'Albiceleste campione in carica: contro il Canada di Tajon Buchanan finisce 2-0. È iniziata questa notte la Copa America ...",
              "source": "Inter Sito Ufficiale",
              "date": "1d ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcTrmceoGY8UW5z-SEsEDFlQNqispYDNOuek-ZcLRPvfZI1DgJs62-a_FPAcEOtFXXZ_D5EBzPr6"
            },
            {
              "title": "Via alla Copa America 2024: tutte le partite dei bianconeri",
              "link": "https://www.juventus.com/it/news/articoli/via-copa-america-2024-tutte-le-partite-dei-bianconeri",
              "snippet": "La Copa America del 2024 prenderà ufficialmente il via questa notte alle ore 02:00 con il match inaugurale fra Argentina, campione in carica, e...",
              "source": "Juventus.com",
              "date": "1d ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcQ5wgvChcTtX1WWVTfZvwS_7lfMBU7ApmRvthzKxkpZZJTEHVmZEWLDTN8QauNOnKnWYymaVh-k"
            },
            {
              "title": "L'Argentina non sbaglia la gara d'apertura della Copa America - RSI ...",
              "link": "https://www.rsi.ch/sport/calcio/L%E2%80%99Argentina-non-sbaglia-la-gara-d%E2%80%99apertura-della-Copa-America--2183319.html",
              "snippet": "Canada messo ko dalle reti di Julian Alvarez e Lautaro Martinez.",
              "source": "RSI.ch Informazione",
              "date": "1d ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcTtz0gQMSupsXqGsBvGqQuh91OEQGaKu05AHgJc5VikFrnqJJzPjs-kpHozJiWpJC87-Dz5mpHy"
            },
            {
              "title": "Argentina-Canada 2-0: Messi illumina, Alvarez e Lautaro Martinez ...",
              "link": "https://www.eurosport.it/calcio/copa-america/2024/argentina-canada-2-0-messi-illumina-alvarez-e-lautaro-martinez-vanno-in-gol.-esordio-ok-per-lalbiceleste-di-scaloni_sto20012333/story.shtml",
              "snippet": "COPA AMERICA - Nella gara inaugurale della 48esima edizione della Copa America, l'Argentina campione in carica non tradisce le attese nel match d'esordio.",
              "source": "Eurosport.it",
              "date": "1d ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcRaeyd-rVxH7jx5PaPiA94Ye2mP-Kp5dnuBJWs2RemWGnexFf_XzzaUR8C5u2UzBNfcH-w1Y5yf"
            }
          ]
        },
        {
          "query": "Polonia-Austria",
          "google_trends_link": "https://trends.google.com/trends/explore?q=Polonia-Austria&date=now+7-d&geo=IT",
          "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Polonia-Austria&tz=420",
          "traffic": 50000,
          "related_queries": [
            {
              "query": "polonia - austria",
              "google_trends_link": "https://trends.google.com/trends/explore?q=polonia+-+austria&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=polonia+-+austria&tz=420"
            },
            {
              "query": "austria polonia",
              "google_trends_link": "https://trends.google.com/trends/explore?q=austria+polonia&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=austria+polonia&tz=420"
            },
            {
              "query": "Lewandowski",
              "google_trends_link": "https://trends.google.com/trends/explore?q=Lewandowski&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Lewandowski&tz=420"
            },
            {
              "query": "Piatek",
              "google_trends_link": "https://trends.google.com/trends/explore?q=Piatek&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Piatek&tz=420"
            },
            {
              "query": "Polonia",
              "google_trends_link": "https://trends.google.com/trends/explore?q=Polonia&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Polonia&tz=420"
            },
            {
              "query": "polonia austria dove vederla",
              "google_trends_link": "https://trends.google.com/trends/explore?q=polonia+austria+dove+vederla&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=polonia+austria+dove+vederla&tz=420"
            },
            {
              "query": "polonia austria formazioni",
              "google_trends_link": "https://trends.google.com/trends/explore?q=polonia+austria+formazioni&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=polonia+austria+formazioni&tz=420"
            },
            {
              "query": "Austria",
              "google_trends_link": "https://trends.google.com/trends/explore?q=Austria&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Austria&tz=420"
            }
          ],
          "articles": [
            {
              "title": "Polonia - Austria (1-3) Europei 2024/2025",
              "link": "https://www.repubblica.it/sport/dirette/calcio/euro-2024/diretta-polonia-austria-g2413157/",
              "snippet": "Trionfo per la squadra di Ralf Rangnick che ha interpretato al meglio l'incontro, con un maggiore possesso soprattutto nella metà campo offensiva.",
              "source": "La Repubblica",
              "date": "13h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcT5uExKtms0kUnia_nFFw8hQDzyGwoYRByyZmtx-lsasZy7D4S8UUc3juUo4a3cU2uzPxJ7RXE3"
            },
            {
              "title": "Arnautovic rilancia l'Austria: Polonia battuta 3-1",
              "link": "https://sport.sky.it/calcio/europei/partite/2024/giornata-2/polonia-austria/risultato-gol",
              "snippet": "Polonia Austria su Sky Sport: scopri tutti gli aggiornamenti sulla partita di Europei in tempo reale.",
              "source": "Sky Sport",
              "date": "14h ago"
            },
            {
              "title": "Euro 2024, Polonia-Austria 1-3",
              "link": "https://www.sulpanaro.net/2024/06/euro-2024-polonia-austria-1-3/",
              "snippet": "(Adnkronos) - L'Austria batte la Polonia per 3-1 a Berlino nel match valido per il gruppo D di Euro 2024 e rimane in corsa per la qualificazione agli ottavi ...",
              "source": "SulPanaro",
              "date": "13h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcTpfsd-OIbo2jO02mPbDuC61b6lKckPevB7bEjCpihUMDOWi2yLocSLaDfESklbR5HBjnyEfy7B"
            },
            {
              "title": "Euro 2024 - Polonia-Austria 1-3: a segno anche Arnautovic su rigore ...",
              "link": "https://www.eurosport.it/calcio/campionati-europei/2024/polonia-austria-1-3-a-segno-anche-arnautovic-su-rigore-polacchi-gia-eliminati_sto10203146/story.shtml",
              "snippet": "EURO 2024 - L'Austria sconfigge 3-1 la Polonia e rimane in corsa per la qualificazione agli ottavi di finale. Lewandowski e compagni quasi eliminati.",
              "source": "Eurosport.it",
              "date": "11h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcSbXbVTiDejdTNtutbvoeDZFy6XDVu6sVjXc4sorCympuh-56ba_raHHw81T_J_PNxtE3tRc4e-"
            },
            {
              "title": "Euro 24: l'Austria sorride, la Polonia battuta 3-1 CRONACA e FOTO",
              "link": "https://www.ansa.it/europei_2024/notizie/2024/06/21/euro-24-laustria-sorride-la-polonia-battuta-3-1-cronaca-e-foto_393d61e4-584a-4084-a7f5-731bf7d2abef.html",
              "snippet": "Austria batte Polonia 3-1 (1-1) in una partita del gruppo D di Euro 2024. Le reti: nel primo tempo austriaci avanti al 9' con Trauner, al 30' pari di Piatek ...",
              "source": "Agenzia ANSA",
              "date": "13h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcQqsUD4BV0D9s9CtdCAPcZe-5oQG1mIzYuuserb1C1U0v58wrXSxNA3oaL915-FnIG_wbHDN0yz"
            },
            {
              "title": "Euro 2024, Polonia-Austria 1-3: Arnautovic show, a Probierz non ...",
              "link": "https://www.corrieredellosport.it/news/calcio/euro-2024/2024/06/21-129341864/euro_2024_polonia-austria_1-3_arnautovic_trascina_ragnick_probierz_ko",
              "snippet": "A Berlino vittoria pesantissima della nazionale del ct Rangnick che riscatta la sconfitta con la Francia e si rilancia. Deludono Zielinski e il 'romanista' ...",
              "source": "Corriere dello Sport.it",
              "date": "6h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcQCei6TGJyj95fvq_2j9yaNtZKtJA4T1MmV4KSq1VS1tzDik4FHvmm5bxzotN3yvdxQjGqR44EQ"
            },
            {
              "title": "FINALE Polonia-Austria 1-3: Rangnick si rilancia, Lewandowski è ...",
              "link": "https://www.gazzetta.it/Calcio/Europei/21-06-2024/polonia-austria-live-degli-europei-2024-la-diretta.shtml",
              "snippet": "All'Olympiastadion di Berlino i gol di Trauner, Baumgartner e, su rigore, Arnautovic rilanciano la squadra del c.t. tedesco. Secondo stop per i polacchi ...",
              "source": "La Gazzetta dello Sport",
              "date": "14h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcReaNIZKQAxOArvPwt_e7B36gyMU9AjK9Pb1_htLn-a4Ak02SzOuiy7V-RMo4J15o9Blv6k4Iyk"
            }
          ]
        },
        {
          "query": "Slovacchia-Ucraina",
          "google_trends_link": "https://trends.google.com/trends/explore?q=Slovacchia-Ucraina&date=now+7-d&geo=IT",
          "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Slovacchia-Ucraina&tz=420",
          "traffic": 50000,
          "related_queries": [
            {
              "query": "slovacchia - ucraina",
              "google_trends_link": "https://trends.google.com/trends/explore?q=slovacchia+-+ucraina&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=slovacchia+-+ucraina&tz=420"
            },
            {
              "query": "ucraina calcio",
              "google_trends_link": "https://trends.google.com/trends/explore?q=ucraina+calcio&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=ucraina+calcio&tz=420"
            },
            {
              "query": "ucraina slovacchia partita",
              "google_trends_link": "https://trends.google.com/trends/explore?q=ucraina+slovacchia+partita&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=ucraina+slovacchia+partita&tz=420"
            },
            {
              "query": "slovacchia calcio",
              "google_trends_link": "https://trends.google.com/trends/explore?q=slovacchia+calcio&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=slovacchia+calcio&tz=420"
            }
          ],
          "articles": [
            {
              "title": "L'Ucraina vince in rimonta: Slovacchia battuta 2-1",
              "link": "https://sport.sky.it/calcio/europei/partite/2024/giornata-2/slovacchia-ucraina/risultato-gol",
              "snippet": "Slovacchia Ucraina su Sky Sport: scopri tutti gli aggiornamenti sulla partita di Europei in tempo reale.",
              "source": "Sky Sport",
              "date": "17h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcSxVjQOlSAK_KHkq5Xt4W7-iq1F2IObq4qYdRsufzWdXmvoKUC7ZAGjJe8u23WTaXoCMlokD-rU"
            },
            {
              "title": "Slovacchia - Ucraina (1-2) Europei 2024/2025",
              "link": "https://www.repubblica.it/sport/dirette/calcio/euro-2024/diretta-slovacchia-ucraina-g2413163/",
              "snippet": "Grazie per aver seguito la diretta di Slovacchia-Ucraina, partita che si è giocata a Düsseldorf, valida per la seconda giornata del Gruppo E di EURO 2024.",
              "source": "La Repubblica",
              "date": "17h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcT5uExKtms0kUnia_nFFw8hQDzyGwoYRByyZmtx-lsasZy7D4S8UUc3juUo4a3cU2uzPxJ7RXE3"
            },
            {
              "title": "Slovacchia-Ucraina ore 15: dove vederla in tv, streaming e formazioni",
              "link": "https://www.corrieredellosport.it/news/calcio/euro-2024/2024/06/21-129307316/slovacchia-ucraina_ore_15_dove_vederla_in_tv_streaming_e_formazioni",
              "snippet": "Scopri tutto sulla partita di Euro 2024: info, orario e canali per seguirla in tempo reale o in differita. Le possibili scelte dei due ct.",
              "source": "Corriere dello Sport.it",
              "date": "17h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcTYBMbN1MuEwyleCt8gak7ZzG-OAbdxdi7PT1jnDpAoZHZmKI2egAs6A-tZDQmenNXg6SqPe84K"
            },
            {
              "title": "Slovacchia-Ucraina risultato 1-2: Shaparenko e Yaremchuk firmano ...",
              "link": "https://www.corriere.it/sport/calcio/europei/24_giugno_21/slovacchia-ucraina-risultato-123dbe38-6d55-4102-b93a-6b61add26xlk.shtml",
              "snippet": "Shaparenko poi Yaremchuk ribaltano la partita e regalano all'Ucraina una vittoria fondamentale contro la Slovacchia. In questo Gruppo E è ancora tutto da ...",
              "source": "Corriere della Sera",
              "date": "17h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcQWdmfEtwX0ZwfvFodgZqZOVP-2YTeeQZ5iDGfKe1_8u9lYf9eVUjuLlk0KWcJAyTPATT6Ig79c"
            },
            {
              "title": "Slovacchia - Ucraina live: Calcio - Euro 2024 | Eurosport",
              "link": "https://www.eurosport.it/calcio/campionati-europei/2024/live-slovacchia-ucraina_mtc1427024/live-commentary.shtml",
              "snippet": "Ricevi le ultime notizie di Calcio su Eurosport. Accedi al Slovacchia - Ucraina live su 21/06/2024. Qui trovi risultati, statistiche e commenti in tempo ...",
              "source": "Eurosport.it",
              "date": "17h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcRfOdJSIa4mxF6Z0UwYLLrpPgK9RTiZEIkKf0OgmEhwLzlVhPYRxcAm_P7ZTyQPddj6OhAvCkgG"
            },
            {
              "title": "Slovacchia-Ucraina 1-2: gol di Schranz, Shaparenko e Yaremchuk ...",
              "link": "https://www.gazzetta.it/Calcio/Europei/21-06-2024/slovacchia-ucraina-live-degli-europei-2024-la-diretta.shtml",
              "snippet": "I gialloblù vincono riscattandosi dal pessimo esordio contro la Romania, gli avversari mancano la qualificazione aritmetica dopo il colpo con il Belgio.",
              "source": "La Gazzetta dello Sport",
              "date": "17h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcT7mRgOFzGNBIk6pPgci4ptEw9QcDvUSQ09K7nL-M9Fwx37hBMFhucLAIpaSnJnWvgusHCI89PU"
            },
            {
              "title": "Slovacchia-Ucraina 1-2: cronaca, tabellino e voti per il Fantacalcio",
              "link": "https://www.fantacalcio.it/news/calcio-estero/europei/21_06_2024/slovacchia-ucraina-cronaca-tabellino-e-voti-per-il-fantacalcio-462601",
              "snippet": "A sbloccare il match è ancora una volta Schranz, che al 19' buca Trubin con l'assistenza dello scatenato Haraslin. Un vero peccato per il portiere ucraino (oggi ...",
              "source": "Fantacalcio.it",
              "date": "17h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcTFeonqnXClp_qjVQToUVNUWaQFxOfEV1d3e-FIQ5_muwTIZdTi7xw1tJQBkqTd--4k-8jn0gO_bw"
            }
          ]
        },
        {
          "query": "F1",
          "google_trends_link": "https://trends.google.com/trends/explore?q=F1&date=now+7-d&geo=IT",
          "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=F1&tz=420",
          "traffic": 50000,
          "related_queries": [
            {
              "query": "Formula 1",
              "google_trends_link": "https://trends.google.com/trends/explore?q=Formula+1&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Formula+1&tz=420"
            },
            {
              "query": "f1 oggi",
              "google_trends_link": "https://trends.google.com/trends/explore?q=f1+oggi&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=f1+oggi&tz=420"
            }
          ],
          "articles": [
            {
              "title": "F1, orari GP Spagna: dove vederlo su Sky, Now e TV8 in tv e ...",
              "link": "https://sport.sky.it/formula-1/2024/06/21/formula-1-orari-gp-spagna-2024",
              "snippet": "La Formula 1 è Barcellona per il Gran Premio di Spagna. Appuntamento oggi con la terza sessione di prove libere in programma alle 12.30 e alle 16 le qualifiche ...",
              "source": "Sky Sport",
              "date": "15h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcRQPiqvnSOhQkJIrJRHk2R4qtkElkOUnqKcHUA4aZleJ0Cg5c3IcFCsKFzEtYjgN8KVnlTDAOAC"
            },
            {
              "title": "F1: Flavio Briatore torna nel circus, l'Alpine gli affida il futuro",
              "link": "https://www.ansa.it/canale_motori/notizie/protagonisti/2024/06/21/f1-flavio-briatore-torna-nel-circus-lalpine-gli-affida-il-futuro_0555669d-2854-41c1-860f-6e1833997372.html",
              "snippet": "Dai trionfi con Schumi e Alonso, a 74 anni è 'executive advisor' (ANSA)",
              "source": "Agenzia ANSA",
              "date": "19h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcTvfD5uiy8yp8Uyc1tRmiDuEnUdcLMuLWD-D6AdNzjc8sWhInRN5ck8g90KuHjwqGYM3dB-m5Le"
            },
            {
              "title": "F1, Gp Spagna. Norris e Hamilton volano nelle libere. VIDEO",
              "link": "https://tg24.sky.it/sport/2024/06/21/f1-gp-spagna-norris-e-hamilton-volano-nelle-libere-video",
              "snippet": "Miglior tempo per la McLaren di Lando Norris nella prima sessione di prove libere, nelle seconde vola Hamilton. La Ferrari di Sainz insegue.",
              "source": "Sky Tg24",
              "date": "13h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcRg5bZC-g99OCIXhz4kQm3gjYkUPY5v4ctK0S1fReYtTyoi7ilTSOEU1w5td4qM5lQ7T-gvFryc"
            },
            {
              "title": "GP Spagna – Classifica FP2: Hamilton batte Sainz per 22 millesimi ...",
              "link": "https://www.formulapassion.it/f1/f1-risultati/gp-spagna-classifica-fp2-ferrari-hamilton-sainz-ferrari-leclerc-verstappen-norris-prove-libere",
              "snippet": "Nelle FP2 del GP di Spagna, è di Lewis Hamilton il miglior tempo: il britannico precede Carlos Sainz di 22 millesimi. Leclerc 6° dietro Max.",
              "source": "FormulaPassion.it",
              "date": "12h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcR5emC6KZXuIxi5bHVK-cgPmFWaGKaJ3hjtfZI-X8YiKBebH8nNmqRpBKDpGEiHW6M70ntC3PYn"
            },
            {
              "title": "Briatore torna in F1 con Alpine come Executive advisor. Le news",
              "link": "https://sport.sky.it/formula-1/2024/06/21/briatore-f1-alpine-news",
              "snippet": "\"BWT Alpine F1 Team annuncia Flavio Briatore come consigliere esecutivo. Briatore è stato nominato dal CEO del Gruppo Renault Luca de Meo suo consigliere ...",
              "source": "Sky Sport",
              "date": "23h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcSq1HM57Ly71O0ID_Tsl0QkOjjJgRolj0i8VxBaI9vg5czudCIpHB94O1i8opx57vndS-PXZ1QM"
            },
            {
              "title": "F1 GP Spagna LIVE, rivivi tutta la giornata delle Libere da Montmelò",
              "link": "https://www.gazzetta.it/Formula-1/21-06-2024/f1-gp-spagna-live-la-diretta-delle-prove-libere.shtml",
              "snippet": "​Lewis HAMILTON Mercedes 1:13.264; Carlos SAINZ Ferrari +0.022; Lando NORRIS McLaren +0.055; Pierre GASLY Alpine +0.179; Max VERSTAPPEN Red Bull +0.240; Charles ...",
              "source": "La Gazzetta dello Sport",
              "date": "16h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcScI8XvuP24Q5ykfh7JgmNDOWiVPvmJ8IAiZA5LL_ZHuBijZN6JMcpnGNOqGc0lxQnU16GfI9tD"
            },
            {
              "title": "F1, Ferrari e le novità aerodinamiche in Spagna: cosa cambia sulle ...",
              "link": "https://sport.sky.it/formula-1/2024/06/21/f1-gp-spagna-ferrari-2024-analisi-tecnica",
              "snippet": "Continua lo sviluppo della monoposto in casa Ferrari: a Barcellona è stato introdotto il secondo pacchetto di aggiornamento corposo, inizialmente previsto ...",
              "source": "Sky Sport",
              "date": "18h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcQD4TiR3Kc7CzqeCz14i0nXKoBbSWXZ28n85adHfr3ZfiwISwT5n2BzEeluc_O148gVLs_EEif_"
            },
            {
              "title": "F1 Gp Spagna, il risultato delle prove libere: Hamilton davanti a tutti ...",
              "link": "https://www.corriere.it/sport/formula-1/diretta-live/24_giugno_21/f1-gp-spagna-prove-libere-oggi-live.shtml",
              "snippet": "A Barcellona la Ferrari ha portato alcuni sviluppi tecnici. Sorride Sainz, meno Leclerc. In difficoltà anche le Red Bull.",
              "source": "Corriere della Sera",
              "date": "16h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcS17_EpfcQmnxAie8a1R11aWnEM0nMZMmgf9fMk1e3CQeQjtgNS3V53bXXiPY0OEN2Q5szQ5-Wq"
            },
            {
              "title": "F1 | Leclerc: \"Un venerdì difficile, ma so dove tirare fuori il tempo\"",
              "link": "https://it.motorsport.com/f1/news/f1-leclerc-un-venerdi-difficile-ma-so-dove-tirare-fuori-il-tempo/10625783/",
              "snippet": "Stesso discorso che vale anche per il compagno Carlos Sainz, che però pare un po' preoccupato in ottica gara, anche se apprezza gli aggiornamenti della SF-24.",
              "source": "Motorsport.com, Edizione: Italia",
              "date": "13h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcQnytt1uX9Gu6HZODu2joDlGxIwkFY2JU3L4kKOp24z8hLP_V1IXMWRlHV5MBhA7yDJpWITnbak"
            },
            {
              "title": "F1, Ezpeleta ospite al Montmelò: \"Più le gare sono competitive ...",
              "link": "https://sport.sky.it/formula-1/2024/06/21/f1-gp-spagna-ezpeleta-intervista-video",
              "snippet": "Ezepeleta si trova sul circuito di Montmelò per il weekend di gara di Formula 1: \"Qui mi sento a casa\". Intervistato da Sky Sport al termine delle FP2, ...",
              "source": "Sky Sport",
              "date": "15h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcSOkwIPGZm7tHJXsiN2qpsi5cqQ37P31Q_3Jtu86sdpr2SktaF9kjKFBNPUgW-TPvd4kRdijQOr"
            }
          ]
        },
        {
          "query": "Vera Slepoj",
          "google_trends_link": "https://trends.google.com/trends/explore?q=Vera+Slepoj&date=now+7-d&geo=IT",
          "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Vera+Slepoj&tz=420",
          "traffic": 20000,
          "articles": [
            {
              "title": "Addio a Vera Slepoj, pioniera della professione dello psicologo ...",
              "link": "https://www.farodiroma.it/addio-a-vera-slepoj-pioniera-della-professione-dello-psicologo-aveva-70-anni/",
              "snippet": "Impegnata nell'avvio dell'Ordine professionale, era laureata in Psicologia e specializzata in psicoterapia individuale e di gruppo e oggi è psicologa ...",
              "source": "Farodiroma",
              "date": "15h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcSCUII-LqEiwdyXPqqPOsNVQGLakf_GSgEsYZNtILOJjutcZrEQyLxYjPWp_QlgBLzi8adsaBVI"
            },
            {
              "title": "La psicologa Vera Slepoj morta nella notte a Padova",
              "link": "https://www.ilsole24ore.com/art/la-psicologa-vera-slepoj-morta-notte-padova-AGQZFIh",
              "snippet": "Una perdita non solo per la comunità scientifica - ha scritto sui social il sindaco di Padova, Sergio Giordani - ma anche per la città»",
              "source": "Il Sole 24 ORE",
              "date": "19h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcTff2hYgMqaAoROo-GGjhTpTvRSeJTBnaFDPcjHAbTp_YTjuPm-uLwyAKjNv18gA-8okbApDS-4"
            },
            {
              "title": "Vera Slepoj, il cordoglio del Consiglio Superiore del Cinema e dell ...",
              "link": "https://www.beniculturali.it/comunicato/26501",
              "snippet": "Presidente Assumma: “Vestale della trasformazione, suo contributo fondamentale”Il Presidente del Consiglio Superiore del Cinema e...",
              "source": "Ministero della cultura",
              "date": "16h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcTmC_29NjebBIZqJoIzw8ltvd0vd2l7a2A0KRYhaFxgsSIQ1mNXP5NB353qGMCyJmsokAwejITK"
            },
            {
              "title": "Vera Slepoj stroncata da un malore nella sua casa di Padova",
              "link": "https://culture.globalist.it/letture/2024/06/21/vera-slepoj-stroncata-da-un-malore-nella-sua-casa-di-padova/",
              "snippet": "Ci lascia la nota psicologa e psicoterapeuta nata nel 1954. La dimensione affettiva dell'essere umano il tema delle sue pubblicazioni.",
              "source": "Culture",
              "date": "11h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcQTLnJzTUzFHe4KGuQmWkQqlalltwCHxKRmVtwhq43T4wLHUZyRXgRFulTNBIp3RBK9P9-lcaob"
            },
            {
              "title": "Morta Vera Slepoj: la famosa psicologa colta da un malore a 70 anni",
              "link": "https://tg24.sky.it/cronaca/2024/06/21/vera-slepoj-morta",
              "snippet": "Si occupava della dimensione affettiva dell'essere umano, sulla quale aveva scritto diversi libri e partecipato spesso a trasmissioni televisive per la ...",
              "source": "Sky Tg24",
              "date": "20h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcROaaK8lWZlBEbeWPfBiVkWUBNKBcjRwYGYSy3xwhcFxMDOiDrytFNcwpwdFyTdQ4vUNbwLeDFu"
            },
            {
              "title": "Addio a Vera Slepoj, la psicologa dei sentimenti che diventò una ...",
              "link": "https://www.repubblica.it/cultura/2024/06/21/news/vera_slepoj_morta_psicologa_sentimenti-423269316/",
              "snippet": "Trovata morta nella sua casa di Padova l'accademica e divulgatrice veneta. Aveva settant'anni.",
              "source": "La Repubblica",
              "date": "13h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcQvnQ88kVRHJd59eBlVKDRcBd_TdhVsaqwH5KhLnLo_rOXaTwpVOVGkJOiWMaGbrwf5oxwQD2CW"
            },
            {
              "title": "Vera Slepoj, la nota psicologa trovata morta in casa, stroncata da un ...",
              "link": "https://www.rainews.it/articoli/2024/06/vera-slepoj-la-psicologa-trovata-morta-in-casa-stroncata-da-un-malore-aveva-70-anni-e5da1e43-728b-4b7a-ac40-ee62fc46e81d.html",
              "snippet": "Sarebbe stata uccisa da un malore improvviso nel cuore della notte. La scrittrice ha curato per anni rubriche a tema psicologico su varie riviste.",
              "source": "Rai News",
              "date": "22h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcTD7C_KsmXvkMRSsm-6vRQWNKbunHxBMEq9axT3lgMcRPOhBRMotKNZ-22XF-30IEsrL2LmA6TY"
            }
          ]
        },
        {
          "query": "Generale Claudio Graziano",
          "google_trends_link": "https://trends.google.com/trends/explore?q=Generale+Claudio+Graziano&date=now+7-d&geo=IT",
          "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Generale+Claudio+Graziano&tz=420",
          "traffic": 20000,
          "related_queries": [
            {
              "query": "generale graziano",
              "google_trends_link": "https://trends.google.com/trends/explore?q=generale+graziano&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=generale+graziano&tz=420"
            }
          ],
          "articles": [
            {
              "title": "Folla ai funerali del generale Claudio Graziano presidente di ...",
              "link": "https://tg24.sky.it/cronaca/2024/06/21/generale-claudio-graziano-funerali",
              "snippet": "La cerimonia si è svolta nella Chiesa di Santa Maria degli Angeli e dei Maritiri a Roma. Ad accoglierlo per un ultimo saluto le forze dell'ordine e membri ...",
              "source": "Sky Tg24",
              "date": "17h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcQjRE4bGxBznrVMdct6_9KisSTPx9hfGs6wW-G4GMbT2kXrcUfMGEc_mEWzMVWcwI7TymThoRX6"
            },
            {
              "title": "I funerali di Claudio Graziano a Roma: per l'ultimo saluto al generale ...",
              "link": "https://roma.corriere.it/notizie/cronaca/24_giugno_21/funerali-claudio-graziano-roma-79bc1a26-c9b8-4c1a-ab15-c7bd17136xlk.shtml",
              "snippet": "Nella basilica di Santa Maria degli Angeli in piazza Esedra i funerali del presidente di Fincantieri. Tra la folla i ministri Tajani e Crosetto, ...",
              "source": "Corriere Roma",
              "date": "17h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcTBMDeytD_klaMXZwo_Mg01CsEXlwCjPdU1FDFoV2PliGBmzM5lfw7jkevrNzII0KHoJ_mC6tGs"
            },
            {
              "title": "Ultimo saluto al generale Claudio Graziano (VIDEO)",
              "link": "https://www.difesaonline.it/evidenza/eventi/ultimo-saluto-al-generale-claudio-graziano-video",
              "snippet": "Oggi la grande famiglia della Difesa ha dato il suo ultimo abbraccio al generale Claudio Graziano, un grande servitore della repubblica.",
              "source": "Difesa Online",
              "date": "7h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcRMvBVITwkXboU4KiMsQlJ3wra7DqUvnAN0U1C3AjWmIGqQz1-mjvBY0CPLap9AqWnkrxozQu_I"
            },
            {
              "title": "I funerali del Generale Claudio Graziano: il picchetto d'onore e l ...",
              "link": "https://video.corriere.it/politica/i-funerali-del-generale-claudio-graziano-il-picchetto-d-onore-e-l-arrivo-delle-autorita-all-ultimo-saluto/ed3a76a5-fe4b-4ef3-83cf-8ba32e5a2xlk",
              "snippet": "Il presidente di Fincantieri è morto lo scorso 17 giugno. Si è tenuto oggi - venerdì 21 giugno - a Roma, presso la Basilica di Santa Maria degli Angeli e ...",
              "source": "Corriere TV",
              "date": "19h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcS238js5o1mkfVLPxYf2SgMEOrkbBuJ94nCLEyOfs_SLHcfg3jluUaoPKmpeopt8C8kb14uEygq"
            },
            {
              "title": "Funerali del generale Graziano a Roma, tricolore sul feretro - Video",
              "link": "https://www.adnkronos.com/cronaca/graziano-fincantieri-funerali-roma_3LflbqNtOTtqBtJclJRgzH",
              "snippet": "Si celebrano nella basilica di Santa Maria degli Angeli, a Roma, i funerali del generale Claudio Graziano, presidente di Fincantieri e già capo di Stato ...",
              "source": "Adnkronos",
              "date": "20h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcSmbEa73m6kySxGKUaZCDNhjHHgsp2NrDWwPo5NDtC-byq2HiUkB3NPTwxB_epRA8Sx5rJ37utY"
            },
            {
              "title": "Chiesa gremita per i funerali del generale Graziano. Monsignor ...",
              "link": "https://www.agi.it/cronaca/news/2024-06-21/funerali-generale-graziano-26861000/",
              "snippet": "Alle esequie a Roma stanno partecipando le massime autorità dello Stato e gli esponenti delle Forze Armate. Chiusa Piazza della Repubblica per consentire ...",
              "source": "AGI - Agenzia Giornalistica Italia",
              "date": "22h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcRSJo2pg2oNg7go43jSrD-PG9Voh2sOM_sSnPUdUkqAaCTgU3T3FmyObTJww76gx-MgIV9uifIf"
            },
            {
              "title": "Ultimo saluto al Generale Claudio Graziano di Chiara Taigi alla ...",
              "link": "https://fai.informazione.it/49609031-DE3A-4644-A37A-C20D1351EB1D/Ultimo-saluto-al-Generale-Claudio-Graziano-di-Chiara-Taigi-alla-Basilica-Santa-Maria-degli-Angeli-e-dei-Martiri-Roma-21-Giugno-2024",
              "snippet": "Ultimo saluto al Generale Claudio Graziano di Chiara Taigi 21 GIUGNO 2024 – Santa Maria degli Angeli e dei Martiri - Roma Dedicato a Colui che non ha…",
              "source": "Fai Informazione",
              "date": "5h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcR1ufwfZOLpwOXOd-T3hz9yjvaUvonh6Rnl1PS3Q_TFSvWU5x26ljQvYPDptHLIjcjC9050175a"
            }
          ]
        },
        {
          "query": "Vodafone",
          "google_trends_link": "https://trends.google.com/trends/explore?q=Vodafone&date=now+7-d&geo=IT",
          "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Vodafone&tz=420",
          "traffic": 20000,
          "articles": [
            {
              "title": "Vodafone, down app e sito, impossibile ricaricare: cosa sta ...",
              "link": "https://www.hdblog.it/mobile/articoli/n585983/vodafone-down-ricariche-app-sito-chiamate-internet/",
              "snippet": "In compenso, non sono stati registrati malfunzionamenti relativi alla rete mobile né a quella fissa, sia per quanto riguarda le chiamate sia per quanto concerne ...",
              "source": "HDblog",
              "date": "13h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcQwS_HciSBYuS7-E6YkZ0EvO_S49YCLMkzbAQ9x6Gvyuss-CpGfC0HekswDUziEwiMzx-LUJ2BQ"
            },
            {
              "title": "Vodafone, che succede? Area clienti e servizi down per qualche ora ...",
              "link": "https://www.mondo3.com/vodafone/2024-06-21-down-giugno.html",
              "snippet": "Non sarebbe impattato, invece, il normale funzionamento delle utenze di rete fissa e mobile dei clienti del gestore rosso, che quindi non riscontrano problemi ...",
              "source": "Mondo3",
              "date": "14h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcRiEIcxwYolBCU6MKbG3LQEueFO5FpYek7k-3_FqsLpZraM3r_CjO4B7aUf9NjvHBIxEORjUbDi"
            },
            {
              "title": "Disservizi Vodafone Italia: sito, app e gestionale down, ma chiamate ...",
              "link": "https://www.mondomobileweb.it/282705-disservizi-vodafone-italia-sito-app-e-gestionale-down-ma-chiamate-e-internet-funzionano/",
              "snippet": "A partire dalla mattina di oggi, 21 Giugno 2024, i clienti dell'operatore telefonico Vodafone stanno riscontrando alcuni disservizi.",
              "source": "MondoMobileWeb.it",
              "date": "21h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcRhjMXNEpMgH-HCQSmqPTEzUX7EbSjq7iwWRnapNgHzgSIwrI7kws0xKcYT1NqyZqT-LZqK5to9"
            },
            {
              "title": "Evidenti problemi Vodafone e con rete HO oggi: non funziona ...",
              "link": "https://www.bufale.net/evidenti-problemi-vodafone-e-con-rete-ho-oggi-non-funziona-assistenza-ed-internet/",
              "snippet": "Un disservizio che ha conosciuto una vera e propria impennata a partire dalle 9 di oggi 21 giugno, al punto che pochi secondi fa è arrivata la prima risposta ...",
              "source": "Bufale.net",
              "date": "22h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcS0YQVXs4PnhMpMXSH7xOaFHMqmM1XAgQ-QKJofFK5TTWuua6MYp1lCnfTMX4ATCrAs5BNNK7tg"
            }
          ]
        },
        {
          "query": "Marco Nosotti giornalista",
          "google_trends_link": "https://trends.google.com/trends/explore?q=Marco+Nosotti+giornalista&date=now+7-d&geo=IT",
          "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Marco+Nosotti+giornalista&tz=420",
          "traffic": 20000,
          "related_queries": [
            {
              "query": "marco nosotti moglie",
              "google_trends_link": "https://trends.google.com/trends/explore?q=marco+nosotti+moglie&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=marco+nosotti+moglie&tz=420"
            }
          ],
          "articles": [
            {
              "title": "Lutto per il giornalista di SkySport Marco Nosotti, è morta la moglie ...",
              "link": "https://www.ilfattoquotidiano.it/2024/06/21/lutto-per-il-giornalista-di-skysport-marco-nosotti-e-morta-la-moglie-silvia-si-e-spenta-dopo-una-lunga-malattia/7596378/",
              "snippet": "Un grave lutto ha colpito Marco Nosotti, noto giornalista di Sky Sport, nella serata di giovedì: è morta, dopo una lunga malattia, la moglie Silvia.",
              "source": "Il Fatto Quotidiano",
              "date": "18h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcR5Orel4Rx6dNWABzSiRhzWmbJKEjXJjOEPFU39_X7LDdyoKOkj-1OKL5ao_vbvtW86f0VamPNA"
            },
            {
              "title": "Marco Nosotti: è morta Silvia, la moglie del giornalista Sky - RTL 102.5",
              "link": "https://www.rtl.it/notizie/articoli/marco-nosotti-e-morta-silvia-la-moglie-del-giornalista-sky/",
              "snippet": "Marco Nosotti, in Germania per seguire la Nazionale agli Europei di calcio, era rientrato in Italia domenica per stare vicino alla donna sposata 28 anni fa.",
              "source": "RTL 102.5",
              "date": "21h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcSTB7bz3gkC99P90nxN2d70kt9yrZwlV5Z_kYDckjy7P9BVdiFizgN-10ROuXiMg1nKrjoFmaVi"
            },
            {
              "title": "Morta la moglie del giornalista di Sky Marco Nosotti. Lui rientrato da ...",
              "link": "https://www.lastampa.it/cronaca/2024/06/21/news/marco_nosotti_morta_moglie_silvia-14410867/",
              "snippet": "La coppia era sposata da quasi 30 anni. Il dolore del figlio sui social. Il commento del Sassuolo Calcio su X.",
              "source": "La Stampa",
              "date": "1d ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcTi7u03FqeOY_xFkBQI8YRWXXRkfXFdh20qWYWmmgRT4Yw8vc44SYZXztW9_HDhHwRjLwUZkUcU"
            },
            {
              "title": "Lutto per il giornalista Sky Nosotti: la moglie Silvia è morta dopo una ...",
              "link": "https://www.goal.com/it/liste/lutto-giornalista-sky-nosotti-moglie-silvia-e-morta-dopo-una-lunga-malattia/blt939d54f9b44a9e72",
              "snippet": "Grave lutto per Marco Nosotti: la moglie Silvia è scomparsa dopo una lunga malattia. Il giornalista è rientrato in Italia dalla Germania.",
              "source": "Goal.com",
              "date": "23h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcQ2N9wDjqt1OmTqBIZzRPyr579PVJK1zRl_q7eoGzRv8SeOQT2LCGx2tNhzpyiX8fYM5rSzOXnM"
            },
            {
              "title": "Morta la moglie di Marco Nosotti giornalista di Sky Sport, era ...",
              "link": "https://notizie.virgilio.it/morta-la-moglie-di-marco-nosotti-giornalista-di-sky-sport-era-appena-rientrato-dagli-europei-2024-in-germania-1625591",
              "snippet": "Marco Nosotti, giornalista di Sky Sport impegnato in questo periodo a seguire gli Europei 2024 in Germania, è stato colpito da un grave lutto: nelle scorse ...",
              "source": "Virgilio Notizie",
              "date": "22h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcRKDf2axTOxR_F-2V9MOv5PtuVrD4_lkW2qsLTPGyoqJwZJoTLU_yJXvaNUBOID3VVGiBjYVuQp"
            },
            {
              "title": "Un grande abbraccio a Marco Nosotti e ai figli per la prematura ...",
              "link": "https://www.modenavolley.it/news/un-grande-abbraccio-a-marco-nosotti-e-ai-figli-per-la-prematura-scomparsa-della-moglie-silvia/",
              "snippet": "Il Presidente Giulia Gabana e tutta Modena Volley si stringono forte al giornalista e amico Marco Nosotti per la prematura perdita della moglie Silvia, ...",
              "source": "Modena Volley",
              "date": "22h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcT3ME4NOWQ1DIudnobL66E4mW8CxIu7MeYVI2yYYNuflIaQgRzps5WxxiB-3jcKY-LUi3CN-4AA"
            },
            {
              "title": "Lutto a Formigine, è morta la moglie del giornalista Marco Nosotti",
              "link": "https://www.lapressa.it/articoli/societa/lutto-a-formigine-morta-la-moglie-del-giornalista-marco-nosotti",
              "snippet": "Silvia Mantovani, da tempo combatteva con una grave malattia: oltre al marito con cui era sposata da 28 anni, lascia i figli Margherita e Giulio ...",
              "source": "La Pressa",
              "date": "20h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcSr0u8YC6mzvsZV5nno80lbmDRfszrE65P7qYXW8CtgiPnECyW435Ttt5FOeW7k37OkHhWmljTB"
            }
          ]
        },
        {
          "query": "Corea",
          "google_trends_link": "https://trends.google.com/trends/explore?q=Corea&date=now+7-d&geo=IT",
          "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Corea&tz=420",
          "traffic": 10000,
          "related_queries": [
            {
              "query": "Kim Jong Un",
              "google_trends_link": "https://trends.google.com/trends/explore?q=Kim+Jong+Un&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Kim+Jong+Un&tz=420"
            }
          ],
          "articles": [
            {
              "title": "Schock geopolitico: Russia e Corea del Nord firmano patto di difesa ...",
              "link": "https://www.antimafiaduemila.com/home/terzo-millennio/232-crisi/100943-schock-geopolitico-russia-e-corea-del-nord-firmano-patto-di-difesa-reciproca.html",
              "snippet": "John Kirby: Stati Uniti allarmati. La Corea del Sud minaccia supporto letale a Kiev e paventa la possibilità di dotarsi del nucleare.",
              "source": "Antimafia Duemila",
              "date": "17h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcQPDQI7lAKJHeXXRGgg4RMcJnfHFlozgDiQUTRcRjNAeIXrlfWqooCo1GdvcucnIwu9HRrI5ciJ"
            },
            {
              "title": "La Corea del Nord sta costruendo un muro al confine con il Sud",
              "link": "https://tg24.sky.it/mondo/2024/06/21/corea-nord-costruzione-muro-sud",
              "snippet": "È il risultato dell'esame di alcune immagini satellitari che, secondo la BBC, potrebbe costituire una violazione della tregua che pose fine alla Guerra di ...",
              "source": "Sky Tg24",
              "date": "16h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcTnnE2P5fbfAx4mjJ4sgU73xnwLixLN3oLZt2987nIelyjjvbR3MvMt5cPyuWoejf8mVW2ucXh3"
            },
            {
              "title": "La Corea del Sud convoca l'ambasciatore della Russia: \"Negativo\" il ...",
              "link": "https://it.euronews.com/2024/06/21/la-corea-del-sud-convoca-lambasciatore-della-russia-negativo-il-patto-di-difesa-tra-putin-",
              "snippet": "L'ambasciatore russo in Corea del Sud è stato convocato venerdì dal governo di Seoul per chiarimenti. Il ministro degli Esteri sudcoreano si è consultato ...",
              "source": "euronews Italiano",
              "date": "17h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcSzQ4_eHEmNnrL2kqBKDkyANMnnSzWpL2XRDrUaxmfdoXumWYaCZMsmijLBFTW1ncQVdR5f3YEk"
            },
            {
              "title": "La Corea del Sud convoca l'ambasciatore russo: \"subito stop\" alla ...",
              "link": "https://www.agenzianova.com/news/la-corea-del-sud-convoca-lambasciatore-russo-subito-stop-alla-cooperazione-con-la-corea-del-nord/",
              "snippet": "Zinoviev, da parte sua, ha definito “inaccettabile” ogni tentativo “di minacciare o ricattare” la Russia, evidenziando come la cooperazione militare tra Mosca e ...",
              "source": "Agenzia Nova",
              "date": "20h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcSq8D0YoOSb6qSWiiug1unoXPkDMjdp5Z0BWj93vq4c6Ory4cqzfEoAFMU2cyPqY-gPu9m5SrSe"
            },
            {
              "title": "L'accordo di Partenariato strategico tra la Corea del Nord e la ...",
              "link": "https://www.analisidifesa.it/2024/06/laccordo-partenariato-strategico-tra-la-corea-del-nord-e-la-federazione-russa/",
              "snippet": "Carta delle Nazioni Unite e in conformità con la legislazione della Repubblica popolare democratica di Corea e della Federazione Russa. Articolo 5. Ciascuna ...",
              "source": "Analisi Difesa",
              "date": "18h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcSkJ6wtCiu0zlUxIJGbdInCG70RCkHAZc03flCk_JPQcDOKoRTeO46FA4GJ17MhaJuN-GdXvxgd"
            },
            {
              "title": "Vladimir Putin: \"Sarebbe un grosso errore per la Corea del Sud ...",
              "link": "https://www.trt.net.tr/italiano/mondo/2024/06/21/vladimir-putin-sarebbe-un-grosso-errore-per-la-corea-del-sud-inviare-i-suoi-aerei-in-ucraina-2154582",
              "snippet": "Secondo la dichiarazione rilasciata dal Palazzo presidenziale russo (Cremlino), Putin ha tenuto una conferenza stampa nella capitale Hanoi durante la sua ...",
              "source": "TRT",
              "date": "23h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcTKPz4tnvKMWYMmF38mhYurF-JNYLUL1GPtuVg5FNldjDpVYoJk2SJB0BSpOoWyVBljNUV5lfsC"
            },
            {
              "title": "Alcuni soldati della Corea del Nord hanno brevemente attraversato ...",
              "link": "https://www.ilpost.it/2024/06/21/soldati-corea-nord-sconfinano-sud/",
              "snippet": "Secondo la Corea del Sud gli sconfinamenti sarebbero accidentali: anche negli altri due casi, uno il 9 giugno e uno il 18 giugno, i soldati nordcoreani si erano ...",
              "source": "Il Post",
              "date": "1d ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcTVBJrB34yp5itAKaJhAuKpVdjg4klBEevPP0N_cyJjetsrK4j9jtWIHhzXUF-9_VAqNY7zszfD"
            },
            {
              "title": "La Corea del Sud risponde all'accordo di Vladimir Putin con la ...",
              "link": "https://www.ilmessaggero.it/mondo/corea_sud_russia_accordo_kim_cosa_succede-8195599.html",
              "snippet": "Insieme a Seoul, anche Usa e Giappone hanno condannato «fortemente» l'accordo firmato mercoledì a Pyongyang.",
              "source": "Il Messaggero",
              "date": "17h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcSHR4Oq72S9KdXyX84Iko-aitabVubEC5ENHQdKBS1mvib6ptFETpf77Hp0q2LiQ4TvT1wdULmE"
            },
            {
              "title": "Soffiano venti di guerra in Corea: massima allerta",
              "link": "https://www.ilfaroonline.it/2024/06/21/putin-kim-accordi-militari-pyongyang-corea-del-nord-corea-del-sud-guerra-corea-russia-ucraina/576198/",
              "snippet": "Pericolo escalation dopo il patto militare Putin-Kim. I militari sudcoreani sparano colpi d'avvertimento nella zona de-militarizzata.",
              "source": "Il Faro Online",
              "date": "20h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcQ9pRgXTy3ImGCCEt6nFpiD3JYjVasUKaNgQPspD0rHqP97oW87U-6dT_n7gOgCyJGP-tOnn7dc"
            },
            {
              "title": "Corea del Sud spara contro i soldati del Nord che avevano superato ...",
              "link": "https://notizie.virgilio.it/corea-del-sud-spara-contro-i-soldati-del-nord-che-avevano-superato-il-confine-alta-tensione-tra-i-due-paesi-1625674",
              "snippet": "Soldati sudcoreani hanno sparato dei colpi di avvertimento verso militari nordcoreani che hanno brevemente superato il confine tra i due Paesi. Un episodio che ...",
              "source": "Virgilio Notizie",
              "date": "16h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcQ3vKyiI_38uasLitoxlRvoRVcXpzB-KLYwQbO0HprJRbx8yr2C7WlTTQSBkhSt2e-Z1dIX7zFa"
            }
          ]
        },
        {
          "query": "Cocciaretto",
          "google_trends_link": "https://trends.google.com/trends/explore?q=Cocciaretto&date=now+7-d&geo=IT",
          "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Cocciaretto&tz=420",
          "traffic": 10000,
          "related_queries": [
            {
              "query": "Elisabetta Cocciaretto",
              "google_trends_link": "https://trends.google.com/trends/explore?q=Elisabetta+Cocciaretto&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Elisabetta+Cocciaretto&tz=420"
            }
          ],
          "articles": [
            {
              "title": "Cocciaretto in semifinale al WTA Birmingham: battuta Shnaider in tre ...",
              "link": "https://sport.sky.it/tennis/2024/06/21/cocciaretto-shnaider-wta-birmingham-risultato",
              "snippet": "Elisabetta Cocciaretto batte in rimonta la russa Shnaider e conquista la prima semifinale sull'erba, la quarta in carriera nel circuito maggiore.",
              "source": "Sky Sport",
              "date": "19h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcSaVI81-6qVw4sKkn1Vqkr6HzgxGdXWYGTf_NCNqPY0qIfXuYHx6OekijZKcb74joXHYYcLRlDI"
            },
            {
              "title": "Rimontona Cocciaretto: batte Shnaider ed è in semifinale",
              "link": "https://www.gazzetta.it/Tennis/WTA/21-06-2024/wta-birmingham-cocciaretto-vince-e-in-semifinale.shtml",
              "snippet": "Sull'erba inglese l'azzurra si impone 5-7 6-4 6-2 in 2 ore e 24 minuti, in un match pieno di colpi di scena.",
              "source": "La Gazzetta dello Sport",
              "date": "18h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcR4wEvqxdei6G_sLfpiEslMzypg7xUzEVoJzL_GE8j4riHTElk6GIlHhR0Jbnr7DddQzwIIeBNF"
            },
            {
              "title": "Birmingham, Cocciaretto rimonta Shnaider e conquista la semifinale ...",
              "link": "https://www.tennisitaliano.it/birmingham-cocciaretto-rimonta-shnaider-e-conquista-la-semifinale/",
              "snippet": "Elisabetta Cocciaretto è la prima semifinalista del Rothesay Classic, il WTA 250 in corso di svolgimento sull'erba di Birmingham. La tennista azzurra, che su ...",
              "source": "TennisItaliano.it",
              "date": "18h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcRUMGd-t4UtAfC1ukDWxGEjFFd75Yi3qZHVdwVV3qf_OjlzAcIGHsCqNerPDxmUS7QH0_idpNsH"
            },
            {
              "title": "Cocciaretto vince ancora e vola in semifinale a Birmingham",
              "link": "https://www.corrieredellosport.it/news/tennis/2024/06/21-129325647/torneo_di_birmingham_cocciaretto_vince_ancora_e_vola_in_semifinale",
              "snippet": "L'azzurra avanza nel Wta250 sull'erba inglese: vittoria in rimonta sulla russa Shnaider dopo quasi due ore e mezza di gioco. Sfiderà una tra Putintseva e ...",
              "source": "Corriere dello Sport.it",
              "date": "13h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcS9gTKjv862UPAlAccGC3EqZ8iX-WIpCQ0Xq1c8XtIwUAsUZ_xSJxox15CtS3yla8GPZvMW_aO9"
            },
            {
              "title": "Gli acciacchi non fermano Cocciaretto: Elisabetta è in semifinale ...",
              "link": "https://www.eurosport.it/tennis/wta-birmingham/2024/gli-acciacchi-non-fermano-cocciaretto-elisabetta-e-in-semifinale-battuta-shnaider-5-7-6-4-6-2_sto20012457/story.shtml",
              "snippet": "Elisabetta Cocciaretto non si ferma a Birmingham: bella vittoria in rimonta per l'azzurra ai quarti di finale contro Shnaider e obiettivo semifinale centrato. 5 ...",
              "source": "Eurosport.it",
              "date": "19h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcT52Oe2sqliag2zgbv3ra4HBhP65fHV_4kqMLoz4EUsRGXfTklK5f0HlA5o_4bVFS2ekJTbFX9c"
            },
            {
              "title": "Cocciaretto in semifinale a Birmingham, è rivincita contro Putintseva ...",
              "link": "https://oktennis.it/2024/06/cocciaretto-in-semifinale-a-birmingham-e-rivincita-contro-putintseva/",
              "snippet": "Tennis | WTA | Nuova splendida giornata per Elisabetta Cocciaretto, che a Birmingham ha colto la prima semifinale WTA del suo 2024.",
              "source": "oktennis",
              "date": "15h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcREz1CNTV-SkiAVdKZezqqrXRWQfoK3u-tND8UIROCoDDlHgMjKgBWF9M1-mrHGE3YHiPDUjNuN"
            },
            {
              "title": "Birmingham, “semi” Cocciaretto-Putintseva: dalle 12 aggiornamenti ...",
              "link": "https://www.supertennis.tv/News/Wta/WTA-Birmingham-semifinale-Cocciaretto-Putintseva",
              "snippet": "Dal primo all'ultimo punto la sfida sull'erba del “Rothesay International” tra Elisabetta, alla sua prima semifinale sui prati, e la kazaka.",
              "source": "SuperTennis",
              "date": "2h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcRNzmjjvdxSKrWUc53jqk2JrzGp1Cv3kwRLxIX9XBzUM0kF3FpqJwlckmdSbNjOAleDoFoE8WPD"
            }
          ]
        },
        {
          "query": "Luna piena giugno 2024",
          "google_trends_link": "https://trends.google.com/trends/explore?q=Luna+piena+giugno+2024&date=now+7-d&geo=IT",
          "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Luna+piena+giugno+2024&tz=420",
          "traffic": 10000,
          "related_queries": [
            {
              "query": "Luna Piena",
              "google_trends_link": "https://trends.google.com/trends/explore?q=Luna+Piena&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Luna+Piena&tz=420"
            }
          ],
          "articles": [
            {
              "title": "Luna delle fragole, ecco perché il prossimo plenilunio è speciale",
              "link": "https://www.gazzetta.it/attualita/21-06-2024/luna-piena-a-giugno-2024-perche-sara-unica.shtml",
              "snippet": "Tra pochi giorni, il cielo di giugno sarà testimone di un fenomeno astronomico molto particolare, la cosiddetta Luna di Fragola, un magico plenilunio ...",
              "source": "La Gazzetta dello Sport",
              "date": "18h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcSNk4OLz8HPDL5o6xHlqe1-WxZVLaiZHaRZMl9zwMlH8wEsxNTO6gsSwqJ_hhneLc83SND4Ygor"
            },
            {
              "title": "Gli effetti della luna piena di giugno (o delle fragole) sulla nostra ...",
              "link": "https://www.amica.it/2024/06/21/luna-piena-giugno-2024-significato-effetti-skincare/",
              "snippet": "Arriva la Luna piena giugno 2024: qual è il suo significato e quali sono i suoi effetti su di noi e sulla nostra skincare? Scopriamolo.",
              "source": "Amica",
              "date": "17h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcSpAX_s01rwEwobBllMEcZimKsXxHAV8Dx6_bofuJ9jblfYQ_Ts35EmGAnG3n9KW3pMC2zI2euJ"
            },
            {
              "title": "Quando vedere la Luna Piena delle Fragole sabato 22 giugno 2024 ...",
              "link": "https://www.fanpage.it/innovazione/scienze/quando-vedere-la-luna-piena-delle-fragole-sabato-22-giugno-2024-e-perche-si-chiama-cosi/",
              "snippet": "Nella notte tra venerdì 21 e sabato 22 giugno potremo ammirare nel cielo la Luna Piena delle Fragole, il plenilunio del mese.",
              "source": "Napoli Fanpage",
              "date": "14h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcT_2W-Z_sAgN69CH6XYONYK2OhWHuvzyHl1ICmnOsdM0cd_qEPeho0ciYumf0gmx9TU28acALeB"
            },
            {
              "title": "Mancano poche ore alla prima splendida Luna piena dell'estate ...",
              "link": "https://www.ilmeteo.net/notizie/attualita/mancano-poche-ore-alla-prima-splendida-luna-piena-estate-2024-ecco-quando-sorgera-in-italia.html",
              "snippet": "In arrivo il primo plenilunio dell'estate 2024: ecco quando sarà la Luna piena che segue di poche ore il solstizio, e a che ora sorgerà il satellite terrestre ...",
              "source": "Ilmeteo.net",
              "date": "19h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcTbA7Qxnej4EAQ-97fnjls9qT4WQj8Ftjk0thknqP3AS-BARMBLS-k3dajGrneUmS1GWB6Hsdox"
            },
            {
              "title": "La Luna Piena delle Fragole brillerà nel cielo (di rosso) la notte del ...",
              "link": "https://www.lofficielitalia.com/lifestyle/luna-piena-delle-fragole-rossa-perche-succede-notte-22-giugno",
              "snippet": "Scopri su L'OFFICIEL Italia la Luna Piena delle Fragole nella notte del 22 giugno 2024. Ecco perchè succede questo fenomeno e perchè si chiama così.",
              "source": "L'Officiel - Italy",
              "date": "1d ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcQgYEeOi1A9VX4ZhidgctOy3dT3IYqZ12hs5EJEEKdEIOw5G766Xa5qPxiD5aq4LpZaaDFpaJZj"
            },
            {
              "title": "Arriva la Luna Piena in Capricorno il 22 giugno 2024: è il momento ...",
              "link": "https://www.fanpage.it/attualita/arriva-la-luna-piena-in-capricorno-il-22-giugno-2024-e-il-momento-di-dare-valore-allindipendenza/",
              "snippet": "I segni zodiacali più coinvolti dalla Luna piena in Capricorno saranno i segni fissi: Ariete, Cancro, Bilancia e Capricorno. In questo plenilunio la ...",
              "source": "Donna Fanpage",
              "date": "1d ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcTHNi83NcUHQjaW-tydxc9m0bo65h8aoMKKAm9AEC6h357GQUIAxWsC544Og0Q8LrI-c55jij4_"
            },
            {
              "title": "Luna piena delle fragole 22 giugno 2024/ Il nostro satellite sarà ...",
              "link": "https://www.ilsussidiario.net/news/luna-piena-delle-fragole-22-giugno-2024-il-nostro-satellite-sara-basso-nel-cielo-e-di-colore-arancione/2721319/",
              "snippet": "Luna piena delle fragole 22 giugno 2024: il nostro satellite risulterà essere basso nel cielo e di colore arancione, ecco tutti i dettagli.",
              "source": "Il Sussidiario.net",
              "date": "18h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcQqkj5LAyby_sSs1eX8i_jlkQfUegl39UYw3U2xvh2CC8-AYMiDffmfTJn7gZrgktREW27elg5i"
            }
          ]
        },
        {
          "query": "Cuori in Atlantide",
          "google_trends_link": "https://trends.google.com/trends/explore?q=Cuori+in+Atlantide&date=now+7-d&geo=IT",
          "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Cuori+in+Atlantide&tz=420",
          "traffic": 10000,
          "articles": [
            {
              "title": "Da \"Cuori in Atlantide\" a \"Neverland\": i film da guardare stasera in tv",
              "link": "https://www.ilgiornale.it/news/cinema/cuori-atlantide-neverland-cosa-guardare-stasera-tv-2337332.html",
              "snippet": "Non amate e il calcio e non sapete cosa vedere stasera in tv? Ecco una piccola guida ai film da recuperare.",
              "source": "il Giornale",
              "date": "1d ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcSUAZT1yoV0Mxq2DENngkLgshzllGjgKaf-cUyV9M-NQkxMHVW-4yTgB4AvdpsmokbsjriccZ8F"
            },
            {
              "title": "Da Cuori in Atlantide a Neverland | i film da guardare stasera in tv",
              "link": "https://www.zazoom.it/2024-06-21/da-cuori-in-atlantide-a-neverland-i-film-da-guardare-stasera-in-tv/14949841/",
              "snippet": "Non amate e il calcio e non sapete cosa vedere stasera in tv? Ecco una piccola guida ai film da recuperare.",
              "source": "Zazoom Blog",
              "date": "23h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcRuz3wIfm5lWLggq5k8JiEeY1AUX1EiCEG0Bh9bZGe75k2PPk3bR1KXh5lJXRHXCY75fmPdDRFU"
            }
          ]
        },
        {
          "query": "Sergio Bernardini",
          "google_trends_link": "https://trends.google.com/trends/explore?q=Sergio+Bernardini&date=now+7-d&geo=IT",
          "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Sergio+Bernardini&tz=420",
          "traffic": 10000,
          "articles": [
            {
              "title": "Chi è Bruna, moglie Sergio Bernardini/ Il figlio Mario: “E' stata a La ...",
              "link": "https://www.ilsussidiario.net/news/chi-e-bruna-moglie-sergio-bernardini-il-figlio-mario-e-stata-a-la-bussola-2-volte-una-per-le-nozze/2721330/",
              "snippet": "Bruna, chi è la moglie di Sergio Bernardini: con il fondatore de La Bussola ha condiviso tutta la sua vita ed ha dato alla luce due figli: Mario e Guido.",
              "source": "Il Sussidiario.net",
              "date": "14h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcQHgdMFSZPnvWiixyNmcCpuEuObRnkNEcytqb78Qfsnz1iiEhAu2R4Aqd3ggRAohaJReI0qBPco"
            },
            {
              "title": "Chi sono Bruna, Mario e Guido, moglie e figli di Sergio Bernardini ...",
              "link": "https://amalfinotizie.it/chi-sono-bruna-mario-e-guido-moglie-e-figli-di-sergio-bernardini-storia-vera-e-cosa-fanno-oggi/",
              "snippet": "Bruna è stata la moglie di Sergio Bernardini, il fondatore di La Bussola, dal loro amore sono nati Mario e Guido. Scopriamo la loro storia.",
              "source": "Amalfi Notizie",
              "date": "14h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcRKicZtZPGreXUQnMyL9ks7c3KWfp-j6u7YP1z5jN40x_M9SENzqx9oVynYPJhqe7pUw5DwHqLPEQ"
            }
          ]
        },
        {
          "query": "Umberto Tozzi",
          "google_trends_link": "https://trends.google.com/trends/explore?q=Umberto+Tozzi&date=now+7-d&geo=IT",
          "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Umberto+Tozzi&tz=420",
          "traffic": 10000,
          "articles": [
            {
              "title": "Umberto Tozzi, sarà un lungo addio al palco. L'intervista di Sky Tg24",
              "link": "https://tg24.sky.it/spettacolo/musica/2024/06/21/umberto-tozzi-tour",
              "snippet": "Umberto Tozzi, sarà un lungo addio al palco. L'intervista di Sky Tg24. Musica. 21 giu 2024 - 17:45. Bruno Ployer. foto Cosimo Buccolieri.",
              "source": "Sky Tg24",
              "date": "16h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcS9yfVNY7Qvsl9AfLpZWM8MM-qTH6aQMe3-OLOWyUdVGJL39TsjFAJMA8CoML7y_NJKeZLfAuzx"
            },
            {
              "title": "Un emozionato Umberto Tozzi colora di rosa le Terme di Caracalla",
              "link": "https://www.iodonna.it/video-iodonna/spettacoli-video/un-emozionato-umberto-tozzi-colora-di-rosa-le-terme-di-caracalla/",
              "snippet": "Alle Terme di Caracalla di Roma \"L'Ultima notte rosa the final tour\", la tournée mondiale con cui Umberto Tozzi dà il suo addio alla scena live.",
              "source": "Io Donna",
              "date": "16h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcSSJYd_Tv2KELTfGVYIYQXg7mced2QgR5Ru06pki3S3l44cfoMx1c-JHqYYG4YVBYnKwpG-IzgW"
            },
            {
              "title": "Anche a Cirò Marina l'addio alle scene di Umberto Tozzi - CrotoneOk.it",
              "link": "https://www.crotoneok.it/anche-a-ciro-marina-laddio-alle-scene-di-umberto-tozzi/",
              "snippet": "Parte dalle splendide Terme di Caracalla di ROMA “L'ULTIMA NOTTE ROSA THE FINAL TOUR”, · L'artista celebra questo imperdibile gran finale dedicando al pubblico ...",
              "source": "CrotoneOK.it",
              "date": "12h ago"
            },
            {
              "title": "Umberto Tozzi: L'ultima notte rosa infiamma le Terme di Caracalla -",
              "link": "https://www.musicalnews.com/2024/06/21/umberto-tozzi-lultima-notte-rosa-infiamma-le-terme-di-caracalla/",
              "snippet": "Tutto esaurito giovedì 20 giugno alle Terme di Caracalla di Roma per l'atteso debutto de “L'ultima notte rosa – The Final Tour” di Umberto Tozzi.",
              "source": "Musicalnews.com",
              "date": "14h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcQ5agKJF5BBDVkElrG3Safh9vPwILrZeHrbmT8Htgv5FR4vLRmkLSGUPJtA4TdFmlmkUpGHZhlp"
            },
            {
              "title": "L'ultima notte rosa di Umberto Tozzi, poi addio ai palchi",
              "link": "https://www.paeseroma.it/intrattenimento/eventi/2024/06/21/lultima-notte-rosa-di-umberto-tozzi-poi-addio-ai-palchi/",
              "snippet": "ROMA – Un artista che non mi sarei mai aspettata di vedere dal vivo, è sicuramente Umberto Tozzi che, in una splendida cornice del Teatro dell'Opera alle ...",
              "source": "PaeseRoma.it",
              "date": "21h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcRN9eHVEi8q4D2cCn5raRciemWpR251eq-b5sGWUZ0eGTWvUOyf_15rpadMNS2P1jywBb_dvPNv"
            }
          ]
        },
        {
          "query": "Loris Karius",
          "google_trends_link": "https://trends.google.com/trends/explore?q=Loris+Karius&date=now+7-d&geo=IT",
          "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Loris+Karius&tz=420",
          "traffic": 10000,
          "related_queries": [
            {
              "query": "matrimonio diletta leotta",
              "google_trends_link": "https://trends.google.com/trends/explore?q=matrimonio+diletta+leotta&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=matrimonio+diletta+leotta&tz=420"
            },
            {
              "query": "diletta leotta matrimonio",
              "google_trends_link": "https://trends.google.com/trends/explore?q=diletta+leotta+matrimonio&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=diletta+leotta+matrimonio&tz=420"
            }
          ],
          "articles": [
            {
              "title": "Matrimonio Diletta Leotta e Loris Karius a Vulcano: gli ospiti e il ...",
              "link": "https://www.messinatoday.it/attualita/matrimonio-diletta-leotta-loris-karius-vulcano-ospiti-festa.html",
              "snippet": "Oggi Diletta Leotta e Loris Karius si sposeranno davanti a circa 160 invitati nel famoso Resort Therasia, location annunciata da tempo, che si affaccia sui ...",
              "source": "MessinaToday",
              "date": "35m ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcQFDr0ySHox-ZzYLujSUUu7IhBD_3jwOnveSRqmsesfN53B9ahjlVfMA13DYfG6qIBRe6Aa3C2b"
            },
            {
              "title": "Diletta Leotta, oggi il matrimonio con Loris Karius a Vulcano: da ...",
              "link": "https://www.ilmessaggero.it/persone/diletta_leotta_matrimonio_oggi_karius_invitati_ferragni_canalis_hunziker_ultime_notizie-8197255.html",
              "snippet": "Diversi ospiti sono giunti a destinazione già ieri e hanno preso parte a un white party pre-matrimoniale organizzato dalla coppia sull'isola.",
              "source": "Il Messaggero",
              "date": "2h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcSfnSln0N5h-u__T6IDRYq6IcBAsmAlaai_C8YjCRQ0S2P8o8-BYvPpRMnc54-UHtXHrbCU_L_3"
            },
            {
              "title": "Diletta Leotta e Loris Karius si sposano a Vulcano: due giorni di ...",
              "link": "https://www.iodonna.it/personaggi/star-italiane/2024/06/21/diletta-leotta-loris-karius-matrimonio-vulcano-quando-invitati-vip/",
              "snippet": "Al matrimonio di Diletta Leotta e del portiere saranno presenti molti invitati vip, da Chiara Ferragni a Elodie, che canterà per gli sposi.",
              "source": "Io Donna",
              "date": "17h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcS31HD2kUXS_fF2QIyRrmVfhEMlVLAhbQJz3kVvKeMt7wHBTvnKC73pAV8GOAVcAH7MAisfDEum"
            },
            {
              "title": "Diletta Leotta e Loris Karius: cosa sappiamo delle nozze a Vulcano",
              "link": "https://www.grazia.it/people-gossip/diletta-leotta-loris-karius-cosa-sappiamo-nozze-vulcano",
              "snippet": "Diletta Letto e Loris Karius pronti per il fatidico sì. La conduttrice di Dazn e il portiere del Newcastle sono a un passo dalle nozze, previste per il 22 ...",
              "source": "Grazia.it",
              "date": "18h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcQESLs9oBBK1R_A36fV6xLX6TRjUc3VdruRXp84DmmtQoayuM9nEfTgbYDEtr6Z-tVy_LoHNrVj"
            },
            {
              "title": "Matrimonio alle Isole Eolie di Diletta Leotta e Loris Karius: ecco la ...",
              "link": "https://movieplayer.it/news/matrimonio-isole-eolie-diletta-leotta-loris-karius-location-invitati_143579/",
              "snippet": "Il volto di Dazn e il portiere tedesco si sono conosciuti nell'ottobre del 2022, in precedenza la conduttrice era stata legata al pugile Daniele Scardina e ...",
              "source": "Movieplayer.it",
              "date": "15h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcTP7k1-_6u0JE2Yks2l5X9KmQYsuEza3AB36MF1BDtQ29q1H9DnGm-V73cQVwk9t_mED4hMQ76q"
            },
            {
              "title": "Diletta Leotta, tutto pronto per il matrimonio con Loris Karius",
              "link": "https://www.r101.it/news/musica-e-spettacolo/1370990/diletta-leotta-tutto-pronto-per-il-matrimonio-con-loris-karius.html",
              "snippet": "Diletta Leotta e Loris Karius, infatti, domani convoleranno a nozze giurandosi amore eterno. Lo faranno nella regione che ha visto nascere Diletta, un luogo ...",
              "source": "R101",
              "date": "20h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcRCWvHmctUXdtIkfqxR2aETHeXYWScPBdvthkK27LSYpIXi9VEgQPcHx3muiORitT0cA7M50O2N"
            },
            {
              "title": "Matrimonio Diletta Leotta e Loris Karius: dove si sposano, location ...",
              "link": "https://www.ilsussidiario.net/news/matrimonio-diletta-leotta-e-loris-karius-dove-si-sposano-location-testimoni-150-invitati-ce-la-ferragni/2721467/",
              "snippet": "Matrimonio Diletta Leotta e Loris Karius dove si sposano? La location è a Vulcano. Elenore Berlusconi tra i testimoni di nozze della sposa, ...",
              "source": "Il Sussidiario.net",
              "date": "1h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcTQgAWzOn4KbESHCTbyzLUHUN0BZtRQkfTT8Z89QJxUC6VdZDscykdFXpy5J5Y18ifvslQ68RS5"
            }
          ]
        },
        {
          "query": "Claudio Baglioni",
          "google_trends_link": "https://trends.google.com/trends/explore?q=Claudio+Baglioni&date=now+7-d&geo=IT",
          "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Claudio+Baglioni&tz=420",
          "traffic": 10000,
          "articles": [
            {
              "title": "CLAUDIO BAGLIONI: “aTUTTOCUOREplus ultra” si conclude e ...",
              "link": "https://spettacolomusicasport.com/2024/06/21/claudio-baglioni-atuttocuoreplus-ultra-si-conclude-e-completa-con-lottavo-evento-il-25-settembre-allarena-di-verona/",
              "snippet": "Il 25 settembre un nuovo appuntamento con “aTUTTOCUOREplus ultra” all'Arena di Verona si aggiunge a quelli del 19-20-21-22 e 26-27-28 settembre dopo lo ...",
              "source": "SpettacoloMusicaSport",
              "date": "13h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcTEvVWwJjmhHs-5CqMIQbOE6f5snBXX2NzspTqd1bK_Lug-f9-5ivX9tjZzEpXNWqfj_22qqmTL"
            },
            {
              "title": "Claudio Baglioni #aTUTTOCUOREplus ottava data - doremifasol.org",
              "link": "https://www.doremifasol.org/news/2024/06/21/claudio-baglioni-atuttocuoreplus-ottava-data/",
              "snippet": "CLAUDIO BAGLIONI - #aTUTTOCUOREplus ultra - ARENA DI VERONA nuova data 25 SETTEMBRE 2024 \"aTUTTOCUOREplus ultra\" SI CONCLUDE E COMPLETA CON L'OTTAVO E.",
              "source": "doremifasol.org",
              "date": "13h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcTvm4tmq5SZY260sjD52XUqHqlO2ndiYxjKdhVH2lHfNI2ph8_WV0AWw99tp4AkhdoKu6VwenTu"
            },
            {
              "title": "Claudio Baglioni in tour: nuova data per 'aTUTTOCUOREplus ultra ...",
              "link": "https://www.giornaledipuglia.com/2024/06/claudio-baglioni-in-tour-nuova-data-per.html",
              "snippet": "Questi spettacoli rappresentano il commiato di Claudio Baglioni dalle grandi arene outdoor e indoor, celebrando il legame profondo tra l'artista e l'Arena di ...",
              "source": "Giornale di Puglia",
              "date": "1h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcRN1oCVLGGrfHc1ycCM3j6PLcGcY7QQ24M3LOWnh7NEkB9xvZkCyJo_YEBxTSN37tkPu82NE19l"
            },
            {
              "title": "Claudio Baglioni: \"aTUTTOCUOREplus ultra\" si completa con l ...",
              "link": "https://endofacentury.it/2024/06/21/claudio-baglioni-atuttocuoreplus-ultra-si-completa-con-lottavo-evento-a-verona/",
              "snippet": "Il 25 settembre il nuovo appuntamento con “aTUTTOCUOREplus ultra” all'Arena di Verona si aggiunge a 19-20-21-22 e 26-27-28 settembre.",
              "source": "End of a Century",
              "date": "12h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcQzy4EwieP9YViUQ6PK9CkYJsm_Y6dxH84xYAKyMoC2QqnSsNJPRb_h_ZTlYtCOmRd_LYW9UuZ2"
            }
          ]
        },
        {
          "query": "Calhanoglu",
          "google_trends_link": "https://trends.google.com/trends/explore?q=Calhanoglu&date=now+7-d&geo=IT",
          "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Calhanoglu&tz=420",
          "traffic": 10000,
          "articles": [
            {
              "title": "Clamoroso dalla Germania: il Bayern vuole soffiare Calhanoglu all ...",
              "link": "https://www.corrieredellosport.it/news/calcio/euro-2020/turchia/2024/06/22-129356442/clamoroso_dalla_germania_il_bayern_vuole_soffiare_calhanoglu_all_inter",
              "snippet": "Secondo i rumors il centrocampista (alternativa a Palhinha) è pronto al sì, ma per i nerazzurri Hakan è incedibile.",
              "source": "Corriere dello Sport.it",
              "date": "1h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcSzYcdTc1_exUdPzMObHpGNTtVbgrRaufSpv5Mb6c8dCV99uzoxCjha05r0ao6lICBHiO8DLM_z"
            },
            {
              "title": "Turchia-Portogallo, Leao in conferenza: \"Chi temo di più? Calhanoglu\"",
              "link": "https://sport.sky.it/calcio/europei/2024/06/21/leao-turchia-portogallo-conferenza-video",
              "snippet": "L'attaccante portoghese del Milan ha parlato alla vigilia della sfida della sua nazionale contro la Turchia: \"È una partita importante contro una squadra di ...",
              "source": "Sky Sport",
              "date": "13h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcRSokEey_gheF9bWUvnrL6FyY5GhvPoXBQ0j3fa6UVgEAmw3yGft3aVMq1ppgD_feO79mLKQiou"
            },
            {
              "title": "Calhanoglu contro Bruno Fernandes: quanta Italia nella sfida chiave ...",
              "link": "https://www.gazzetta.it/Calcio/Europei/22-06-2024/turchia-portogallo-il-duello-tra-calhanoglu-e-bruno-fernandes.shtml",
              "snippet": "Il turco esaltato da Montella dopo il boom all'Inter. Il portoghese è una delle poche certezze dello United: in mezzo al campo ecco una sfida imprevedibile.",
              "source": "La Gazzetta dello Sport",
              "date": "1h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcQoSnLC0gllVSPuKk9Og4qTucqXdFbl46ZgzxwhhfM201UIH6pqLH6w5Swjok06qZC1IW1RCUGY"
            },
            {
              "title": "Di Marzio – Calhanoglu per l'Inter è incedibile: futuro ancora ...",
              "link": "https://www.fcinter1908.it/copertina/calhanoglu-incedibile-futuro-nerazzurro/",
              "snippet": "Dalla Germania si è parlato di un interesse del Bayern Monaco per Hakan Calhanoglu mentre in Turchia si sono spinti oltre, assicurando che il calciatore ...",
              "source": "fcinter1908",
              "date": "7h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcQBG-coBaoSEdzCAm_RCWDjyIup49ytQ2H0NjBP9QTIcr3CZfCixizloqYcpMK4YYhpuxU_8CPM"
            },
            {
              "title": "Calhanoglu nel mirino del Bayern Monaco: pronti 50 milioni per far ...",
              "link": "https://www.eurosport.it/calcio/calhanoglu-nel-mirino-del-bayern-monaco-pronti-50-milioni-per-far-vacillare-l-inter.-il-turco-e-nel-_sto10203515/story.shtml",
              "snippet": "CALCIOMERCATO - L'Inter potrebbe doversi difendere a stretto giro dall'assalto del Bayern Monaco ad Hakan Calhanoglu. Vincent Kompany, nuovo allenatore dei ...",
              "source": "Eurosport.it",
              "date": "10h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcTZ1j62O2GQAAoaZzhr-3zNBNbwNHPUv3GrCVMqnwQTGShHOu17nO6uBNymRDRdT5OLcSafJbxJ"
            },
            {
              "title": "Leao: \"Siamo un gruppo molto forte. Turchia? Calhanoglu fa la ...",
              "link": "https://www.fantacalcio.it/news/calcio-estero/europei/21_06_2024/leao-siamo-un-gruppo-molto-forte-turchia-calhanoglu-fa-la-differenza-462605",
              "snippet": "Intervenuto in conferenza stampa, Rafael Leao ha parlato delle potenzialità della Nazionale portoghese e di cosa si aspetta riguardo il suo Europeo.",
              "source": "Fantacalcio.it",
              "date": "14h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcQHZ-LnPcqcp9YfIL_as6o3OfABOoFGwmILsTSePx9q8T7fb1_fRioFGoW47-btg4tvmrjeu2FC"
            },
            {
              "title": "RETROSCENA CALHANOGLU: HA GIÀ RIFIUTATO UN'OFFERTA ...",
              "link": "https://www.sportmediaset.mediaset.it/mercato/retroscena-calhanoglu-ha-gia-rifiutato-un-offerta_83801791-202402k.shtml",
              "snippet": "Mentre il Bayern Monaco sembra fare sul serio per averlo, spunta un retroscena su Hakan Calhanoglu: secondo TRT Spor, portale turco, il centrocampista...",
              "source": "Sport Mediaset",
              "date": "1h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcT0WWtPr-Rsa2O9ws0bLlKf0EDB93ihDtR2M6hiup_y5tVx-GLrTN2-dD5K4z7sn7HYgU4vfC2w"
            },
            {
              "title": "Le probabili formazioni di Turchia-Portogallo: Montella cerca il ...",
              "link": "https://www.tuttomercatoweb.com/probabili-formazioni/i-le-probabili-formazioni-di-i-turchia-portogallo-montella-cerca-il-colpaccio-calhanoglu-dal-1-1981758",
              "snippet": "Turchia e Portogallo scenderanno in campo alle 18 a Dortmund in occasione della seconda giornata del Gruppo F di EURO 2024: sarà un incrocio tra.",
              "source": "TUTTO mercato WEB",
              "date": "3h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcT0Tn1jkQLvxWSRbqfg0EJIBaTmeA4Vm836hcjnc3dubY3MnrTtSDCW1ibjftseBqMyPeoOOlBa"
            },
            {
              "title": "Inter, attenta: il Bayern Monaco pensa a Calhanoglu - Radio Sportiva",
              "link": "https://www.radiosportiva.com/calcio/inter-attenta-il-bayern-monaco-pensa-a-calhanoglu/",
              "snippet": "Uno dei leader dello spogliatotio, faro del centrocampo, punto di riferimento in campo, intoccabile per Simone Inzaghi: tutto questo è stato, ed è, ...",
              "source": "Radio Sportiva",
              "date": "13h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcQL-_sF4wHf-o-O4HCx1rv5Fd-_v43ID3JbDUjM1uyVBWlmT3tXofoFonnt4jSSv_ZYSXr2SkMg"
            },
            {
              "title": "Leao avverte il Portogallo: \"Calhanoglu fa la differenza, ha ...",
              "link": "https://www.sportmediaset.mediaset.it/speciale/europei-germania-2024/leao-calhanoglu_83789437-202402k.shtml",
              "snippet": "L'attaccante rossonero dal ritiro dei lusitani: \"Attenzione anche a Guler e Yilmaz. Sogno un grande Europeo\"",
              "source": "Sport Mediaset",
              "date": "15h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcSE2UJMkH69Ig1khZFuItP50UvePXduSvsVzNMYFeh4-ou4rVVRbm7REfAj0GD04kyIBJWj41wH"
            }
          ]
        },
        {
          "query": "Bugatti Tourbillon",
          "google_trends_link": "https://trends.google.com/trends/explore?q=Bugatti+Tourbillon&date=now+7-d&geo=IT",
          "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Bugatti+Tourbillon&tz=420",
          "traffic": 10000,
          "related_queries": [
            {
              "query": "Bugatti",
              "google_trends_link": "https://trends.google.com/trends/explore?q=Bugatti&date=now+7-d&geo=IT",
              "serpapi_google_trends_link": "https://serpapi.com/search.json?data_type=TIMESERIES&date=now+7-d&engine=google_trends&geo=IT&q=Bugatti&tz=420"
            }
          ],
          "articles": [
            {
              "title": "Bugatti Tourbillon, la nuova auto ibrida da 1800 cavalli e quasi 4 ...",
              "link": "https://tg24.sky.it/tecnologia/2024/06/21/bugatti-tourbillon-nuova-auto-ibrida",
              "snippet": "Il nome della nuova hypercar Bugatti è Tourbillon, che vuol dire vortice. Sarà alimentata dal rivoluzionario motore V16 aspirato da 8,3 litri, abbinato a due ...",
              "source": "Sky Tg24",
              "date": "13h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcSfzZ-3OLgjcyL5fseiH-ABKZjtM3A61KmnkCdHh3Zj0KHdyfPRC8-b7uBK8eSfNVGPj5o0LnjO"
            },
            {
              "title": "Bugatti Tourbillon: come un orologio",
              "link": "https://www.alvolante.it/news/bugatti-tourbillon-caratteristiche-immagini-395799",
              "snippet": "L'era ibrida inizia con il botto: V16 abbinato a tre motori elettrici per 1.800 CV complessivi, 0-100 in 2 secondi e 0-300 in appena 10.",
              "source": "AlVolante",
              "date": "17h ago",
              "thumbnail": "https://t0.gstatic.com/images?q=tbn:ANd9GcRxzyFco_m6WTjlwYT4jW3x05xIeC5USdOgSMSszWQ53STx9fQeCK_JcmAKDtWdqoM2KDRLmfrh"
            },
            {
              "title": "Il quadro strumenti della Bugatti Tourbillon è un orologio svizzero",
              "link": "https://it.motor1.com/news/724063/strumentazione-bugatti-tourbillon-orologio-svizzero/",
              "snippet": "A prima vista sembra abbastanza simile a quello della Chiron, con una guglia centrale che contiene vari comandi e un volante robusto, ma c'è un nuovo elemento ...",
              "source": "Motor1 Italia",
              "date": "23h ago",
              "thumbnail": "https://t3.gstatic.com/images?q=tbn:ANd9GcRaqVqDtqFI7qdVrqTE_Nxvpy8khtLEDDG8FkjdjWQ72azUMf8pZAhPa9wP1H-dEGYKvcfXFvYP"
            },
            {
              "title": "Bugatti Tourbillon, la nuova costosissima e velocissima hypercar ...",
              "link": "https://www.autoblog.it/post/bugatti-tourbillon-la-nuova-costosissima-e-velocissima-hypercar",
              "snippet": "La Chiron ha un'erede, la Bugatti Tourbillon debutta con un possente motore W16 plug-in per raggiungere prestazioni da capogiro.",
              "source": "Autoblog.it",
              "date": "16h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcTFl97WqvqpJzFsGQOcS7u-48YID7k4ZfGL2fUo5j3y6mQ7YHPl5JQI73LQznhCTg1GW5H14c7D"
            },
            {
              "title": "La nuova Bugatti Tourbillon ibrida è potentissima, ma non scatta ...",
              "link": "https://www.tuttotech.net/news/2024/06/21/bugatti-tourbillon-confronto-su-strada-rimac-nevera.html",
              "snippet": "Bugatti ha presentato la nuova hypercar ibrida battezzata Tourbillon che monta un motore a benzina a 16 cilindri a “V” e tre unità elettriche.",
              "source": "TuttoTech.net",
              "date": "19h ago",
              "thumbnail": "https://t1.gstatic.com/images?q=tbn:ANd9GcS9nutxHuI7xtkRs0LeHXtd11FpsYah3nbaBDmgWtMnB1Ov1a2unZaGCYgSX9laAu3ojqZ8y0Yu"
            },
            {
              "title": "Bugatti Tourbillon: fulmine ibrido da 1800 cavalli",
              "link": "https://www.gazzetta.it/motori/la-mia-auto/21-06-2024/bugatti-tourbillon-motori-prestazioni-e-prezzo-della-nuova-hypercar-ibrida.shtml",
              "snippet": "Il nuovo modello della casa francese è anche la prima auto ibrida a portare il glorioso marchio di Molsheim. Monta il nuovo motore V16 aspirato da 8,3 litri ...",
              "source": "La Gazzetta dello Sport",
              "date": "23h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcSRkU93d6rosSql--EarWs30_3MjQAjfwhEjtEngSGm6J3roobkB74tJ4YmA56dhqm4qnx9a_L_"
            },
            {
              "title": "Bugatti Tourbillon, la folle hypercar da 3,8 milioni di euro, 1.800 ...",
              "link": "https://www.corriere.it/motori/news/cards/bugatti-tourbillon-tre-motori-elettrici-v16-1800-cv-38-milioni-euro-foto/erede-stirpe-hypercar.shtml",
              "snippet": "E' stata finalmente svelata l'erede della Bugatti Chiron, l'attesissima Tourbillon, il cui nome – un complicato meccanismo utilizzato sugli orologi.",
              "source": "Corriere della Sera",
              "date": "18h ago",
              "thumbnail": "https://t2.gstatic.com/images?q=tbn:ANd9GcTD_hUrsvYXGDGbLv4_S-xV9MSVpmfGbAXgoOR3d1uIrlDXHNNVu1FEx44pu1hzW66nAou57kIG"
            }
          ]
        }
      ]
    }
  ],
  "serpapi_pagination": {
    "current_date": "20240622",
    "next_date": "20240620",
    "next": "https://serpapi.com/search.json?date=20240620&engine=google_trends_trending_now&frequency=daily&geo=IT&hl=en"
  }
}

In [14]:
searches = sample_day_trends["daily_searches"][0]["searches"]
for idx, s in enumerate(searches):
    print(f"{idx}. Query: {s['query']} (Traffic: {s['traffic']})")
    for art in s["articles"]:
        print(f"\tArt title: {art['title']}")
        print(f"\tArt snippet: {art['snippet']}")
        print(f"\tArt url: {art['link']}")
        print(f"\tArt source: {art['source']}")
        print(f"\t--------------------------------")

0. Query: Platini (Traffic: 10000)
	Art title: Notti Europee, Marco Mazzocchi fa infuriare Michel Platini: “Che ca ...
	Art snippet: Il mitico "Le Roi", ospite di Notti Europee su Rai 1, stronca Mazzocchi dopo una domanda poco gradita. Attimi di imbarazzo, poi interviene Paola Ferrari.
	Art url: https://www.libero.it/magazine/news/notti-europee-mazzocchi-michel-platini-puntata-21-giugno-2024-127544
	Art source: Libero Tecnologia
	--------------------------------
	Art title: Euro2024, Platini a Notti Europee: "La Spagna è la sorpresa, ha un ...
	Art snippet: Michel Platini, ospite a Notti Europee, parla dell'Europeo e della sua Francia, ma ha anche allargato lo sguardo al futuro del calcio proponendo la sua ...
	Art url: https://sport.virgilio.it/euro2024-platini-a-notti-europee-i-giocatori-devono-governare-il-calcio-la-battuta-a-mazzocchi-867735
	Art source: Virgilio Sport
	--------------------------------
1. Query: Pele (Traffic: 5000)
	Art title: È morta Donna Celeste Arantes, la mam

In [15]:
selected_index = 1 # int(input("Please select a query by typing the number corresponding to the index: "))
chosen_trend = searches[selected_index]
print(f"\nYou selected query: {chosen_trend['query']} (Traffic: {chosen_trend['traffic']})")


You selected query: Pele (Traffic: 5000)


In [16]:
# searches = sample_day_trends["daily_searches"][0]["searches"]
# # min_traffic = min([s["traffic"] for s in searches])
# # min_traffic_searches = [s for s in searches if s["traffic"] == min_traffic]
# # chosen_query = min_traffic_searches[0]

# max_traffic = max([s["traffic"] for s in searches])
# max_traffic_searches = [s for s in searches if s["traffic"] == max_traffic]
# chosen_trend = max_traffic_searches[0]
# chosen_trend

### Get news from trend

In [17]:
# news_search = search_google(chosen_trend, search_type="nws")
# news_search
news_sample_search = {'search_metadata': {'id': '6676d3b1382427b1ac906076',
  'status': 'Success',
  'json_endpoint': 'https://serpapi.com/searches/343eac7f743b6f0b/6676d3b1382427b1ac906076.json',
  'created_at': '2024-06-22 13:37:53 UTC',
  'processed_at': '2024-06-22 13:37:53 UTC',
  'google_url': 'https://www.google.com/search?q=Platini&oq=Platini&hl=it&gl=it&tbm=nws&sourceid=chrome&ie=UTF-8',
  'raw_html_file': 'https://serpapi.com/searches/343eac7f743b6f0b/6676d3b1382427b1ac906076.html',
  'total_time_taken': 1.72},
 'search_parameters': {'engine': 'google',
  'q': 'Platini',
  'google_domain': 'google.com',
  'hl': 'it',
  'gl': 'it',
  'device': 'desktop',
  'tbm': 'nws'},
 'search_information': {'query_displayed': 'Platini',
  'total_results': 6800,
  'time_taken_displayed': 0.35,
  'news_results_state': 'Results for exact spelling'},
 'people_also_search_for': [{'name': 'Notizie su Calcio',
   'news_results': [{'position': 1,
     'link': 'https://www.gazzetta.it/Calcio/Serie-A/Juventus/20-06-2024/platini-la-superlega-si-fara-di-sicuro.shtml',
     'title': 'Platini vede nero: "Il calcio esploderà. La Superlega si farà di sicuro, è ineluttabile"',
     'source': 'La Gazzetta dello Sport',
     'date': '2 giorni fa',
     'thumbnail': 'https://serpapi.com/searches/6676d3b1382427b1ac906076/images/d74de2929e81a0de9bf99f21d81127ff67df3731d1045ff982ab222e464e79bc7bf17647d55146b25595751ba9250ba6c7a6b46a86fa3f3c.jpeg'},
    {'position': 2,
     'link': 'https://www.ilbianconero.com/a/platini-il-calcio-cambiera-la-sentenza-della-corte-europea-e-chi-27356',
     'title': "Platini: 'Il calcio cambierà, la sentenza della Corte Europea è chiara'",
     'source': 'il BiancoNero',
     'date': '16 ore fa',
     'thumbnail': 'https://serpapi.com/searches/6676d3b1382427b1ac906076/images/d74de2929e81a0de9bf99f21d81127ff67df3731d1045ff982ab222e464e79bcf4ae3ecd5d90fded703d9d64b0db9478963b93b418403dc7.jpeg'},
    {'position': 3,
     'link': 'https://www.lalaziosiamonoi.it/news/platini-tuona-il-calcio-esplodera-succederanno-tante-cose-206460',
     'title': 'Platini tuona: "Il calcio esploderà. Succederanno tante cose..."',
     'source': 'La Lazio Siamo Noi',
     'date': '1 giorno fa',
     'thumbnail': 'https://serpapi.com/searches/6676d3b1382427b1ac906076/images/d74de2929e81a0de9bf99f21d81127ff67df3731d1045ff982ab222e464e79bc67e5c2a9264c2edfd4430ad2e5ac2cc38dba72a0c4bef83d.jpeg'},
    {'position': 4,
     'link': 'https://www.tuttojuve.com/altre-notizie/platini-il-calcio-subira-grandi-cambiamenti-e-la-superlega-si-fara-691953',
     'title': 'Platini: "Il calcio subirà grandi cambiamenti e la Superlega si farà"',
     'source': 'Tutto Juve',
     'date': '2 giorni fa',
     'thumbnail': 'https://serpapi.com/searches/6676d3b1382427b1ac906076/images/d74de2929e81a0de9bf99f21d81127ff67df3731d1045ff982ab222e464e79bca3023856140982ba8a8a4fd381d064303d7fc6db9966ef9a.jpeg'}],
   'view_full_coverage_link': 'https://www.google.com/search?q=Platini&sca_esv=afd4baaf1825dfb1&gl=it&hl=it&tbm=nws&story=GiIIpQEaHQoTc3RvcnlfbGFiZWxfcGFydGlhbBIGQ2FsY2lvMi8KJfPymuzK4vHQNubVjp7V0rnTGa6VsqKdl7akmAGVut-w4tmitHYQ0NuU6QsYBXICEAI&fcs=ACgqzecuMS0CZQlMjFj7ErVBLQbMePiL3A&sa=X&ved=2ahUKEwjFoey9qu-GAxWVmokEHbnoBaQQ7IUHegQIFRAF'}],
 'news_results': [{'position': 1,
   'link': 'https://www.areanapoli.it/varie/maradona-e-la-punizione-alla-juventus-platini-a-pecci-ma-non-potevi-passargli-il-pallone-sul-destro_552770.html',
   'title': "Maradona e la punizione alla Juventus, Platini a Pecci: 'Ma non potevi passargli il pallone sul destro?'",
   'source': 'AreaNapoli.it',
   'date': '7 ore fa',
   'snippet': "Simpatico siparietto a distanza tra Platini e Pecci, l'argomento è quello \ndella magica punizione in area di Maradona contro la Juventus.",
   'thumbnail': 'https://serpapi.com/searches/6676d3b1382427b1ac906076/images/c9f62e2aaafaf9f09a8b7627802a9c720a0f58ea69f8652c4b0c942ab2d4624f.jpeg'},
  {'position': 2,
   'link': 'https://www.libero.it/magazine/news/notti-europee-mazzocchi-michel-platini-puntata-21-giugno-2024-127544',
   'title': 'Notti Europee, Marco Mazzocchi fa infuriare Michel Platini: “Che ca*ata di domanda”, imbarazzo in studio',
   'source': 'Libero',
   'date': '16 ore fa',
   'snippet': 'Il mitico "Le Roi", ospite di Notti Europee su Rai 1, stronca Mazzocchi \ndopo una domanda poco gradita. Attimi di imbarazzo, poi interviene...',
   'thumbnail': 'https://serpapi.com/searches/6676d3b1382427b1ac906076/images/c9f62e2aaafaf9f0dca1fcdc5fc358904e2258724db612b9f9940af30066f791.jpeg'},
  {'position': 3,
   'link': 'https://onefootball.com/it/notizie/platini-scatenato-in-rai-gela-il-giornalista-che-caata-di-domanda-39661201',
   'title': '💣 Platini scatenato in RAI, gela il giornalista: "Che ca**ata di domanda"',
   'source': 'OneFootball',
   'date': '4 ore fa',
   'snippet': 'Un risvolto totalmente inaspettato. Nel giorno del suo 69° compleanno, \nMichel Platini è stato ospite in collegamento di Rai 1 in occasione...',
   'thumbnail': 'https://serpapi.com/searches/6676d3b1382427b1ac906076/images/c9f62e2aaafaf9f00e47a5600564753e7971cbf3b743c60d10f305ae856b558b.jpeg'},
  {'position': 4,
   'link': 'https://m.tuttojuve.com/altre-notizie/l-eurodeputato-gozi-ha-ragione-platini-modello-uefa-finito-va-creato-un-modello-di-calcio-europeo-piu-libero-e-giusto-691977',
   'title': 'L\'eurodeputato Gozi: "Ha ragione Platini, modello Uefa finito. Va creato un modello di calcio europeo...',
   'source': 'Tutto Juve',
   'date': '1 giorno fa',
   'snippet': 'Michel Platini dice a Le Figaro dice quello che sappiamo tutti e che UEFA e \nFIGC continuano a negare: dopo la decisione della Corte Ue,.',
   'thumbnail': 'https://serpapi.com/searches/6676d3b1382427b1ac906076/images/c9f62e2aaafaf9f08a9a041ae585c25986dffb91061e7bb3a4491d6d7af681a5.jpeg'},
  {'position': 5,
   'link': 'https://sport.virgilio.it/euro2024-platini-a-notti-europee-i-giocatori-devono-governare-il-calcio-la-battuta-a-mazzocchi-867735',
   'title': 'Euro2024, Platini a Notti Europee: "La Spagna è la sorpresa, ha un bel gioco". La battuta a Mazzocchi',
   'source': 'Virgilio Sport',
   'date': '15 ore fa',
   'snippet': "Michel Platini, ospite a Notti Europee, parla dell'Europeo e della sua \nFrancia, ma ha anche allargato lo sguardo al futuro del calcio...",
   'thumbnail': 'https://serpapi.com/searches/6676d3b1382427b1ac906076/images/c9f62e2aaafaf9f09933f2bcf42a020856c9d420751d754f55545af9eb85d07e.jpeg'},
  {'position': 6,
   'link': 'https://www.juventusnews24.com/serena-platini-juve-tributo-compleanno-sky/',
   'title': 'Serena non ha dubbi: «Platini è stato il giocatore più geniale con cui ho giocato»',
   'source': 'Juventus News 24',
   'date': '7 ore fa',
   'snippet': "Serena non ha dubbi: «Platini è stato il giocatore più geniale con cui ho \ngiocato». Il tributo dell'ex Juve.",
   'thumbnail': 'https://serpapi.com/searches/6676d3b1382427b1ac906076/images/c9f62e2aaafaf9f0fbc11edaa326bda2a466301fb4c434e4ecf139bdb73196f3.jpeg'},
  {'position': 7,
   'link': 'https://www.fcinternews.it/news/platini-il-calcio-esplodera-la-superlega-si-fara-di-sicuro-957523',
   'title': 'Platini: "Il calcio esploderà, la Superlega si farà di sicuro"',
   'source': 'FC Inter News',
   'date': '1 giorno fa',
   'snippet': 'Prima o poi la Superlega si farà. Ne è sicuro Michel Platini, leggendaria \nstella del calcio francese ed ex presidente.',
   'thumbnail': 'https://serpapi.com/searches/6676d3b1382427b1ac906076/images/c9f62e2aaafaf9f065d6e1581cc925c2031cce834013cf90652cd508f1253c7b.jpeg'},
  {'position': 8,
   'link': 'https://www.bianconeranews.it/podcast/bonini-a-rbn-luiz-koopmeiners-top-juve-tieni-chiesa-platini-presidente-75484',
   'title': 'Bonini a RBN: “Luiz-Koopmeiners top. Juve tieni Chiesa. Platini presidente”',
   'source': 'Bianconera News',
   'date': '1 giorno fa',
   'snippet': "La Juve prosegue sulla strada del rafforzamento, soprattutto a centrocampo. \nDouglas Luiz sembra in dirittura d'arrivo dopo aver sostituito.",
   'thumbnail': 'https://serpapi.com/searches/6676d3b1382427b1ac906076/images/c9f62e2aaafaf9f089092ae51ee28c91abd08fcf184a87a1caad948bcc86727e.jpeg'},
  {'position': 9,
   'link': 'https://www.juvenews.eu/serie-a-news/lega-serie-a-auguri-platini/',
   'title': 'Lega Serie A, gli auguri a Michel Platini: la nota',
   'source': 'JuveNews.eu',
   'date': '22 ore fa',
   'snippet': 'Il sito ufficiale della Lega Serie A ha pubblicato una nota per fare gli \nauguri a Michel Platini, ex leggenda della Juve.',
   'thumbnail': 'https://serpapi.com/searches/6676d3b1382427b1ac906076/images/c9f62e2aaafaf9f04ca44e124c4b81aa651fa2af6c4463e0a6a33b9919f2f912.jpeg'}],
 'dmca_messages': {'title': 'Avvisi relativi ai risultati filtrati',
  'messages': [{'content': "Alcuni risultati possono essere stati rimossi nell'ambito della normativa europea sulla protezione dei dati. Ulteriori informazioni",
    'highlighted_words': [{'link': 'https://www.google.com/policies/faq',
      'text': 'Ulteriori informazioni'}]}]},
 'pagination': {'current': 1,
  'next': 'https://www.google.com/search?q=Platini&sca_esv=afd4baaf1825dfb1&gl=it&hl=it&tbm=nws&ei=stN2ZsWAEZW1ptQPudGXoAo&start=10&sa=N&ved=2ahUKEwjFoey9qu-GAxWVmokEHbnoBaQQ8NMDegQIARAY',
  'other_pages': {'2': 'https://www.google.com/search?q=Platini&sca_esv=afd4baaf1825dfb1&gl=it&hl=it&tbm=nws&ei=stN2ZsWAEZW1ptQPudGXoAo&start=10&sa=N&ved=2ahUKEwjFoey9qu-GAxWVmokEHbnoBaQQ8tMDegQIARAG',
   '3': 'https://www.google.com/search?q=Platini&sca_esv=afd4baaf1825dfb1&gl=it&hl=it&tbm=nws&ei=stN2ZsWAEZW1ptQPudGXoAo&start=20&sa=N&ved=2ahUKEwjFoey9qu-GAxWVmokEHbnoBaQQ8tMDegQIARAI',
   '4': 'https://www.google.com/search?q=Platini&sca_esv=afd4baaf1825dfb1&gl=it&hl=it&tbm=nws&ei=stN2ZsWAEZW1ptQPudGXoAo&start=30&sa=N&ved=2ahUKEwjFoey9qu-GAxWVmokEHbnoBaQQ8tMDegQIARAK',
   '5': 'https://www.google.com/search?q=Platini&sca_esv=afd4baaf1825dfb1&gl=it&hl=it&tbm=nws&ei=stN2ZsWAEZW1ptQPudGXoAo&start=40&sa=N&ved=2ahUKEwjFoey9qu-GAxWVmokEHbnoBaQQ8tMDegQIARAM',
   '6': 'https://www.google.com/search?q=Platini&sca_esv=afd4baaf1825dfb1&gl=it&hl=it&tbm=nws&ei=stN2ZsWAEZW1ptQPudGXoAo&start=50&sa=N&ved=2ahUKEwjFoey9qu-GAxWVmokEHbnoBaQQ8tMDegQIARAO',
   '7': 'https://www.google.com/search?q=Platini&sca_esv=afd4baaf1825dfb1&gl=it&hl=it&tbm=nws&ei=stN2ZsWAEZW1ptQPudGXoAo&start=60&sa=N&ved=2ahUKEwjFoey9qu-GAxWVmokEHbnoBaQQ8tMDegQIARAQ',
   '8': 'https://www.google.com/search?q=Platini&sca_esv=afd4baaf1825dfb1&gl=it&hl=it&tbm=nws&ei=stN2ZsWAEZW1ptQPudGXoAo&start=70&sa=N&ved=2ahUKEwjFoey9qu-GAxWVmokEHbnoBaQQ8tMDegQIARAS',
   '9': 'https://www.google.com/search?q=Platini&sca_esv=afd4baaf1825dfb1&gl=it&hl=it&tbm=nws&ei=stN2ZsWAEZW1ptQPudGXoAo&start=80&sa=N&ved=2ahUKEwjFoey9qu-GAxWVmokEHbnoBaQQ8tMDegQIARAU',
   '10': 'https://www.google.com/search?q=Platini&sca_esv=afd4baaf1825dfb1&gl=it&hl=it&tbm=nws&ei=stN2ZsWAEZW1ptQPudGXoAo&start=90&sa=N&ved=2ahUKEwjFoey9qu-GAxWVmokEHbnoBaQQ8tMDegQIARAW'}},
 'serpapi_pagination': {'current': 1,
  'next_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Platini&start=10&tbm=nws',
  'next': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Platini&start=10&tbm=nws',
  'other_pages': {'2': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Platini&start=10&tbm=nws',
   '3': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Platini&start=20&tbm=nws',
   '4': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Platini&start=30&tbm=nws',
   '5': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Platini&start=40&tbm=nws',
   '6': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Platini&start=50&tbm=nws',
   '7': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Platini&start=60&tbm=nws',
   '8': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Platini&start=70&tbm=nws',
   '9': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Platini&start=80&tbm=nws',
   '10': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Platini&start=90&tbm=nws'}}}

In [18]:
news_results = news_sample_search["news_results"]
news_results

[{'position': 1,
  'link': 'https://www.areanapoli.it/varie/maradona-e-la-punizione-alla-juventus-platini-a-pecci-ma-non-potevi-passargli-il-pallone-sul-destro_552770.html',
  'title': "Maradona e la punizione alla Juventus, Platini a Pecci: 'Ma non potevi passargli il pallone sul destro?'",
  'source': 'AreaNapoli.it',
  'date': '7 ore fa',
  'snippet': "Simpatico siparietto a distanza tra Platini e Pecci, l'argomento è quello \ndella magica punizione in area di Maradona contro la Juventus.",
  'thumbnail': 'https://serpapi.com/searches/6676d3b1382427b1ac906076/images/c9f62e2aaafaf9f09a8b7627802a9c720a0f58ea69f8652c4b0c942ab2d4624f.jpeg'},
 {'position': 2,
  'link': 'https://www.libero.it/magazine/news/notti-europee-mazzocchi-michel-platini-puntata-21-giugno-2024-127544',
  'title': 'Notti Europee, Marco Mazzocchi fa infuriare Michel Platini: “Che ca*ata di domanda”, imbarazzo in studio',
  'source': 'Libero',
  'date': '16 ore fa',
  'snippet': 'Il mitico "Le Roi", ospite di Notti Eu

In [19]:
from utililty import text_getter

processed_news_results = [text_getter(n["link"]) for n in news_results]
processed_news_results

[<generator object text_getter at 0x7fbda3ce13f0>,
 <generator object text_getter at 0x7fbda3ce2260>,
 <generator object text_getter at 0x7fbda3ce2340>,
 <generator object text_getter at 0x7fbda3ce27a0>,
 <generator object text_getter at 0x7fbda3ce2d50>,
 <generator object text_getter at 0x7fbda3ce30d0>,
 <generator object text_getter at 0x7fbda3ce33e0>,
 <generator object text_getter at 0x7fbda3ce3530>,
 <generator object text_getter at 0x7fbda3ce3990>]

In [20]:
processed_news_results = [list(pn)[0] for pn in processed_news_results]
processed_news_results = [pn for pn in processed_news_results if isinstance(pn, dict)]
processed_news_results

/home/simo/code/SEOautomation/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.areanapoli.it'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/simo/code/SEOautomation/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.libero.it'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/simo/code/SEOautomation/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'onefootball.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warning

[{'author': '',
  'base': 'https://www.areanapoli.it',
  'provider': 'areanapoli.it/varie/maradona-e-la-punizione-alla-juventus-platini-a-pecci-ma-non-potevi-passargli-il-pallone-sul-destro_552770',
  'published_date': None,
  'text': 'Simpatico siparietto a distanza tra Platini e Pecci, l\'argomento è quello della magica punizione in area di Maradona contro la Juventus.\n\nDiego Armando Maradona ha scritto pagine memorabili di calcio, ogni sua giocata è stata un\'opera d\'arte. Alle 15.57 del 3 novembre 1985 il Pibe de Oro tracciò nel cielo di Napoli un aquilone che smontò le leggi della fisica, sbeffeggiò quelle della dinamica e fece marameo anche al principio di gravitazione universale. Al San Paolo si giocava Napoli-Juventus, c’era molta attesa, l’incasso - record per una partita di Serie A negli anni 80 - fu di un miliardo e 552.934.700.\n\nNella serata di ieri durante il post partita tra Francia e Olanda è intervenuto Micheal Platini ai microfoni della RAI, presente in studio anc

In [21]:
processed_news_results = [{'author': '',
  'base': 'https://www.areanapoli.it',
  'provider': 'areanapoli',
  'published_date': None,
  'keywords': ['tra',
   'platini',
   'della',
   'sul',
   'maradona',
   'potevi',
   'passargli',
   'pallone',
   'di',
   'la',
   'non',
   'il',
   'ha',
   'punizione',
   'ma',
   'è',
   'pecci'],
  'summary': 'Simpatico siparietto a distanza tra Platini e Pecci, l\'argomento è quello della magica punizione in area di Maradona contro la Juventus.\nDiego Armando Maradona ha scritto pagine memorabili di calcio, ogni sua giocata è stata un\'opera d\'arte.\nNella serata di ieri durante il post partita tra Francia e Olanda è intervenuto Micheal Platini ai microfoni della RAI, presente in studio anche Eraldo Pecci che toccò il pallone a Maradona prima dell\'incredibile gol su punizione contro la Juventus.\nL\'ex campione francese ha così riferito: "Ma non potevi passargli il pallone sul destro?\nPronta la risposta di Pecci: "Diego avrebbe segnato lo stesso, era un genio!',
  'text': 'Simpatico siparietto a distanza tra Platini e Pecci, l\'argomento è quello della magica punizione in area di Maradona contro la Juventus.\n\nDiego Armando Maradona ha scritto pagine memorabili di calcio, ogni sua giocata è stata un\'opera d\'arte. Alle 15.57 del 3 novembre 1985 il Pibe de Oro tracciò nel cielo di Napoli un aquilone che smontò le leggi della fisica, sbeffeggiò quelle della dinamica e fece marameo anche al principio di gravitazione universale. Al San Paolo si giocava Napoli-Juventus, c’era molta attesa, l’incasso - record per una partita di Serie A negli anni 80 - fu di un miliardo e 552.934.700.\n\nNella serata di ieri durante il post partita tra Francia e Olanda è intervenuto Micheal Platini ai microfoni della RAI, presente in studio anche Eraldo Pecci che toccò il pallone a Maradona prima dell\'incredibile gol su punizione contro la Juventus.\n\nL\'ex campione francese ha così riferito: "Ma non potevi passargli il pallone sul destro? Così almeno avremmo pareggiato", ha detto in modo scherzoso e ironico. Pronta la risposta di Pecci: "Diego avrebbe segnato lo stesso, era un genio!".',
  'title': 'Maradona e la punizione alla Juventus, Platini a Pecci: "Ma non potevi passargli il pallone sul destro?"',
  'top_image': 'https://cdn.areanapoli.it/immagini/notizie/big/m/maradona-punizione_juve_85.jpg',
  'url': 'https://www.areanapoli.it/varie/maradona-e-la-punizione-alla-juventus-platini-a-pecci-ma-non-potevi-passargli-il-pallone-sul-destro_552770.html'},
 {'author': '',
  'base': 'https://www.libero.it',
  'provider': 'libero',
  'published_date': '2024-06-22T06:03:00+00:00',
  'keywords': ['europee',
   'platini',
   'di',
   'la',
   'è',
   'studio',
   'notti',
   'il',
   'replica',
   'ha',
   'mazzocchi',
   'una',
   'stronca',
   'caata',
   'che'],
  'summary': 'Notti Europee, Marco Mazzocchi fa infuriare Michel Platini: “Che ca*ata di domanda”, imbarazzo in studio Il mitico "Le Roi", ospite di Notti Europee su Rai 1, stronca Mazzocchi dopo una domanda poco gradita.\nStavolta è riuscito a far perdere la pazienza addirittura a uno dei mostri sacri del calcio internazionale, il mitico Michel Platini.\nQuesto può fare la differenza in un torneo che si gioca a giugno inoltrato, con giocatori stanchi dopo una lunga stagione".\nPoi è entrato in scena Marco Mazzocchi, che proprio sui titoli di coda dell’intervento di Michel Platini è riuscito a far salire il termometro dell’imbarazzo in studio con una semplice domanda.\nPoi giù risate e il provvidenziale intervento di Paola Ferrari con lo scopo di stemperare l’imbarazzo: "Platini è sempre avanti, anche nelle battute".',
  'text': 'Notti Europee, Marco Mazzocchi fa infuriare Michel Platini: “Che ca*ata di domanda”, imbarazzo in studio Il mitico "Le Roi", ospite di Notti Europee su Rai 1, stronca Mazzocchi dopo una domanda poco gradita. Attimi di imbarazzo in studio, poi interviene Paola Ferrari. Cosa è successo.\n\nCONDIVIDI\n\nIl ‘solito’ Marco Mazzocchi ne combina un’altra. Dopo l’invito a cena da ‘piacione’ rivolto a Giusy Meloni, e la battuta che qualche giorno fa ha fatto innervosire Adani, il co-conduttore di Notti Europee, l’appendice notturna Rai dedicata agli Europei 2024 di calcio, torna al centro dell’attenzione per un altro scivolone. Stavolta è riuscito a far perdere la pazienza addirittura a uno dei mostri sacri del calcio internazionale, il mitico Michel Platini. Ecco cosa è successo.\n\nEuro2024, Michel Platini ospite a Notti Europee\n\nL’ex asso della Juventus, nel giorno del suo 69esimo compleanno, si è collegato da remoto con la conduttrice Paola Ferrari e il suo studio per incassare gli auguri dei presenti (c’era anche Marco Tardelli, suo grande amico nonché ex compagno in bianconero) e dare un suo giudizio sugli Europei 2024 in corso in Germania. Tante le domande rivolte a Platini – presentato da Paola Ferrari come un "grande giocatore e un uomo speciale" – che innanzitutto ha fornito la sua personalissima lettura tecnico-tattica del torneo continentale, tirando un primo bilancio sulle favorite: "Bella atmosfera, bello spettacolo, le grandi squadre mi sembra che stiano aspettando la seconda fase per scoprirsi" ha detto "Le Roi", per poi aggiungere: "La mia Francia? Ha tanti buoni giocatori, senza giocare bene e senza segnare (l’unica rete finora siglata dei blues è stata frutto di un autogol, ndr) è già qualificata alla seconda fase. Favorite? Onestamente vedo tutte le squadre che giocano allo stesso modo, in maniera molto fisica. Solo la Spagna ha un gioco diverso, simpatico e tecnico. Questo può fare la differenza in un torneo che si gioca a giugno inoltrato, con giocatori stanchi dopo una lunga stagione".\n\nRai, Michel Platini stronca Marco Mazzocchi: gelo in studio\n\nFin qui tutto bene. Atmosfera goliardica (con Eraldo Pecci in versione mattatore) e spunti di riflessione interessanti per gli appassionati. Poi è entrato in scena Marco Mazzocchi, che proprio sui titoli di coda dell’intervento di Michel Platini è riuscito a far salire il termometro dell’imbarazzo in studio con una semplice domanda. All’atto dei saluti all’ex presidente Uefa, il giornalista Rai ha voluto strappargli un ulteriore minuto per l’ultimo quesito: "Prima di salutare Platini vorrei chiedergli ancora una cosa. Chi è secondo te il miglior calciatore francese di tutti i tempi?". La domanda, va detto piuttosto banale, non ha trovato il favore del 69enne francese. Platini infatti ha preso un sospiro e, dopo aver riflettuto qualche secondo, ha letterlamente gelato Mazzocchi con una replica secca ma velenosa: "Se permettete do una risposta politica", ha esordito Platini, piazzando quindi l’affondo stronca-Mazzocchi: "Questa è una bella ca*ata di domanda".\n\nL’uscita – inattesa e tranchant – ha inevitabilmente fatto calare il gelo in studio per qualche secondo. Poi giù risate e il provvidenziale intervento di Paola Ferrari con lo scopo di stemperare l’imbarazzo: "Platini è sempre avanti, anche nelle battute". Anche Mazzocchi l’ha presa con filosofia, riconoscendo però di aver incassato una bella ‘mazzata’: "Da uno come Platini si prende sempre gol volentieri", ha chiosato con ironia il giornalista, ormai a tutti gli effetti gaffeur principe della trasmissione.\n\nGuida TV\n\nPotrebbe interessarti anche',
  'title': 'Notti Europee, Platini stronca Mazzocchi: “Che ca*ata", la replica',
  'top_image': 'https://wips.plug.it/cips/libero.it/magazine/cms/2024/06/notti-europee-puntata-21-giugno-marco-mazzocchi-michel-platini.jpg',
  'url': 'https://www.libero.it/magazine/news/notti-europee-mazzocchi-michel-platini-puntata-21-giugno-2024-127544'},
 {'author': '',
  'base': 'https://onefootball.com',
  'provider': 'onefootball',
  'published_date': None,
  'keywords': ['qualificata',
   'onefootball',
   'è',
   'ufficiali',
   'domanda',
   'di',
   'si',
   'giornalista',
   'sorti',
   'scatenato',
   'che',
   'alle',
   'rai',
   'thuram',
   'platini',
   'q8',
   'il',
   'euro',
   'gela',
   'dal'],
  'summary': "🔴EURO '24 | UFFICIALI delle 18: Mbappé dal 1' e out Thuram, ok ArnautovicL’Italia è qualificata agli ottavi di EURO 2024, oggi si capiranno le sorti dei Gruppi C (Francia-Polonia e Olanda-Austria alle 18:00) e D (Danimarca-Serbia e Inghilterra-Slovenia alle 21:00).\nDal 1984, Q8...",
  'text': "🔴EURO '24 | UFFICIALI delle 18: Mbappé dal 1' e out Thuram, ok Arnautovic\n\nL’Italia è qualificata agli ottavi di EURO 2024, oggi si capiranno le sorti dei Gruppi C (Francia-Polonia e Olanda-Austria alle 18:00) e D (Danimarca-Serbia e Inghilterra-Slovenia alle 21:00). Dal 1984, Q8...",
  'title': '💣 Platini scatenato in RAI, gela il giornalista: "Che ca**ata di domanda" | OneFootball',
  'top_image': 'https://photobooth-api.onefootball.com/api/screenshot/%2Fv2%2Fphotobooth%2Fcms%2Fit%2F39661201%3Fdate=2024-06-25',
  'url': 'https://onefootball.com/it/notizie/platini-scatenato-in-rai-gela-il-giornalista-che-caata-di-domanda-39661201'},
 {'author': '',
  'base': 'https://m.tuttojuve.com',
  'provider': 'm.tuttojuve',
  'published_date': None,
  'keywords': ['uefa',
   'platini',
   'gozi',
   'della',
   'di',
   'la',
   'modello',
   'michel',
   'leurodeputato',
   'che',
   'il',
   'ha',
   'ragione',
   'libero',
   'più',
   'va',
   'piπ'],
  'summary': '“Michel Platini dice a Le Figaro dice quello che sappiamo tutti e che UEFA e FIGC continuano a negare: dopo la decisione della Corte Ue, il modello UEFA è finito e va creato un altro modello di calcio europeo.\nPiù libero e giusto, aggiungo io”.\nLo afferma Sandro Gozi, eurodeputato di Renew Europe e segretario generale del Partito democratico europeo, sul suo profilo "X", commentando l’intervista al quotidiano francese “Le Figaro” dell’ex stella della Juventus ed ex presidente della Uefa, Michel Platini, in cui afferma che “il calcio esploderà con la decisione della Corte di giustizia europea” e che “la FIFA e la UEFA non hanno più il monopolio delle competizioni”.\nPer Platini, “la Super League si farà”.',
  'text': '“Michel Platini dice a Le Figaro dice quello che sappiamo tutti e che UEFA e FIGC continuano a negare: dopo la decisione della Corte Ue, il modello UEFA è finito e va creato un altro modello di calcio europeo. Più libero e giusto, aggiungo io”.\n\nLo afferma Sandro Gozi, eurodeputato di Renew Europe e segretario generale del Partito democratico europeo, sul suo profilo "X", commentando l’intervista al quotidiano francese “Le Figaro” dell’ex stella della Juventus ed ex presidente della Uefa, Michel Platini, in cui afferma che “il calcio esploderà con la decisione della Corte di giustizia europea” e che “la FIFA e la UEFA non hanno più il monopolio delle competizioni”. Per Platini, “la Super League si farà”.',
  'title': 'L\'eurodeputato Gozi: "Ha ragione Platini, modello Uefa finito. Va creato un modello di calcio europeo pi√π libero e giusto"',
  'top_image': 'https://net-storage.tcccdn.com/storage/tuttojuve.com/img_notizie/thumb3/99/99c50170e5d670e72cbaa11610f94531-10712-oooz0000.jpeg',
  'url': 'https://m.tuttojuve.com/altre-notizie/l-eurodeputato-gozi-ha-ragione-platini-modello-uefa-finito-va-creato-un-modello-di-calcio-europeo-piu-libero-e-giusto-691977'},
 {'author': '',
  'base': 'https://sport.virgilio.it',
  'provider': 'sport.virgilio',
  'published_date': '2024-06-21T22:10:00+00:00',
  'keywords': ['europee',
   'spagna',
   'platini',
   'della',
   'di',
   'la',
   'si',
   'gioco',
   'notti',
   'calcio',
   'sorpresa',
   'il',
   'ha',
   'mazzocchi',
   'una',
   'è',
   'che'],
  'summary': '“La Spagna è una sorpresa, ha una bella tecnica.\nChe di rimando ha spiegato: “Il calcio di oggi è diverso da ieri, oggi c’è un rapporto più fisico col pallone, a centrocampo c’è l’abitudine di mettere tre velocisti che recuperano i palloni”.\nQuando c’è la tecnica c’è più possibilità di vincere, che giocare di fisico tutto l’anno ed arrivare poi stanchi.\nHo visto molte corse, calci d’angolo, contropiede, velocità: a parte la Spagna che ha un gioco abbastanza simpatico, le altre squadre giocano allo stesso modo.\nNon so però se sia giusto dirlo con la maglia della nazionale in un evento ufficiale dove giocano”.',
  'text': 'Dopo il primo pareggio a reti bianche di questo Europeo 2024 e che ha visto protagonista sua malgrado la Francia contro l’Olanda, Michel Platini è apparso al pubblico della Rai collegato con il post partita, nell’approfondimento di Notti Europee condotto da Paola Ferrari e Marco Mazzocchi. L’ex fantasista della Juventus e già presidente UEFA ha parlato nella serata del suo compleanno, rispondendo a qualche domanda non solo sulla sua Francia ma anche sullo stato attuale del calcio.\n\nPlatini a Notti Europee: “Euro 2024? Bella atmosfera, buon arbitraggio”\n\nInterrogato sull’idea che si è fatto sulla competizione di scena in Germania e sui match sino ad ora disputati, Platini ha rinviato il proprio giudizio: “L’atmosfera è buonissima, le partite sono aperte, ma le grandi squadre aspettano il secondo girone perché tutto si giocherà a partire dagli ottavi di finale. L’arbitraggio mi sembra buono, e da qualche anno mi sembra che anche nelle coppe europee sia così. Ma non so quale squadra arriverà in finale”.\n\nPaola Ferrari ha poi chiesto all’ex calciatore un parere sulla sua Francia, reduce come abbiamo detto dallo 0-0 con l’Olanda e che al momento ha subito un solo gol, peraltro autoinflitto, Platini ha sentenziato: “Non ho bisogno di vedere le partite di oggi per avere una impressione abbastanza buona della Francia. Ha buoni giocatori, e anche senza giocare molto bene ha già quattro punti. Come ho detto all’inizio, tutto si deciderà e si vedrà dalle partite ad eliminazione diretta”.\n\n“La Spagna è una sorpresa, ha una bella tecnica. Le altre giocano uguali”\n\nTra gli ospiti fissi di Notti Europee c’è anche Tony Damascelli, che ha notato il fatto che oggi mancano “gli artisti in mezzo al campo”, come lo è stato l’ex bianconero. Che di rimando ha spiegato: “Il calcio di oggi è diverso da ieri, oggi c’è un rapporto più fisico col pallone, a centrocampo c’è l’abitudine di mettere tre velocisti che recuperano i palloni”.\n\nLe Roi ha poi lodato la Spagna, definita come “sorpresa”: “Ha tre buoni tecnici [sic] a centrocampo e gioca bene a pallone, e quando si gioca bene a pallone a giugno si può vincere. Quando c’è la tecnica c’è più possibilità di vincere, che giocare di fisico tutto l’anno ed arrivare poi stanchi. Europeo più dei centrocampisti che degli attaccanti? I grandi goleador hanno bisogno di giocatori che danno buoni palloni e ne ho visto pochi. Ho visto molte corse, calci d’angolo, contropiede, velocità: a parte la Spagna che ha un gioco abbastanza simpatico, le altre squadre giocano allo stesso modo. Oggi tutto è fatto per vincere le gare, e si possono mettere giocatori diversi per vincerle”.\n\n“I calciatori devono governare il calcio, possono portare moralità”\n\nDopo aver svincolato sulla domanda su chi preferisca tra Guardiola o Ancelotti, visto che si parlava di bel gioco (“Sono due cari amici“), ha poi parlato del ruolo dei giocatori nel governo del calcio: “Io penso che i calciatori nuovi devono pensare al futuro, ma noi dobbiamo dargli una mano per via della nostra esperienza”.\n\nA Damascelli che fa notare come il calcio sia in mano a FIFA, UEFA e arbitri, Platini risponde: “È sempre stato così, nel futuro deve essere tutto più in mano ai calciatori che ai burocrati. Loro possono difendere la fantasia e la moralità del calcio. I Mbappé, Messi e Ronaldo devono arrivare dentro la federazione del calcio e gestire il futuro”.\n\nSulle dichiarazioni rilasciate recentemente a Le Figaro sul calcio pronto ad implodere, Platini ha spiegato riferendosi alle sentenza della Corte di Giustizia UE che l’anno scorso ha messo fine al monopolio dell’UEFA e della FIFA: “Quindi chiunque, come la Rai, può organizzare la Champions. Ci sarà una nuova gestione del calcio e spero che dentro ci siano calciatori, perché loro possono difenderlo”, ha ribadito l’ex presidente federale.\n\n“Calciatori che parlano di politica? Non con la maglia della nazionale”\n\nAlla domanda poi sulle esternazioni politiche dei calciatori come Thuram e Mbappé, schieratisi apertamente contro il Rassemblement National, Platini è stato netto: “Penso che hanno ragione a esprimere opinioni politiche perché sono liberi di dire quello che gli pare. Non so però se sia giusto dirlo con la maglia della nazionale in un evento ufficiale dove giocano”.\n\nInfine, dopo aver negato di avere rimpianti in merito alla sua avventura dirigenziale, interrogato da Mazzocchi su chi sia il più forte calciatore francese di tutti i tempi Platini non ha avuto dubbi: “Do una risposta politica: è una bella cagata di domanda”.',
  'title': 'Euro2024, Platini a Notti Europee: "La Spagna è la sorpresa, ha un bel gioco". La battuta a Mazzocchi',
  'top_image': 'https://wips.plug.it/cips/sport.virgilio.it/cms/2024/06/iolansafoto_20240622001006448_5d28f9a68418cec171eba805df258941.jpg?w=640&a=c&h=360',
  'url': 'https://sport.virgilio.it/euro2024-platini-a-notti-europee-i-giocatori-devono-governare-il-calcio-la-battuta-a-mazzocchi-867735'},
 {'author': None,
  'base': 'https://www.juventusnews24.com',
  'provider': 'juventusnews24',
  'published_date': None,
  'text': 'Serena non ha dubbi: «Platini è stato il giocatore più geniale con cui ho giocato». Il tributo dell’ex Juve Serena, ex attaccante della Juve, ospite a Sky Sport durante Calciomercato l’Originale ha reso uno speciale tributo a Platini che ieri ha festeggiato il compleanno.  SERENA OMAGGIA PLATINI – «Il 21 giugno è il compleanno di Platini, il giocatore più geniale con cui ho giocato. Un fantastico calciatore che sapeva essere il migliore in campo sotto il profilo tecnico e, fuori dal campo, anche con le parole».',
  'title': 'Serena non ha dubbi: "Platini è stato il giocatore più geniale con cui ho giocato"',
  'top_image': None,
  'url': 'https://www.juventusnews24.com/serena-platini-juve-tributo-compleanno-sky/',
  'keywords': None,
  'summary': None},
 {'author': '',
  'base': 'https://www.fcinternews.it',
  'provider': 'fcinternews',
  'published_date': None,
  'keywords': ['esploderà',
   'uefa',
   'platini',
   'di',
   'la',
   'si',
   'calcio',
   'sicuro',
   'il',
   'farà',
   'più',
   'è',
   'superlega',
   'che'],
  'summary': 'Prima o poi la Superlega si farà.\nNe è sicuro Michel Platini, leggendaria stella del calcio francese ed ex presidente UEFA: "Dopo la sentenza della Corte europea che ha stabilito che FIFA e UEFA non hanno più il monopolio delle competizioni, il calcio esploderà.\nLa Superlega si farà di sicuro - dice ai microfoni de Le Figaro -.\nLa Superlega è ineluttabile.\nSarebbe stata impossibile con me alla guida dell\'UEFA, d\'altronde uccisi il G14 che voleva uccidere il calcio.',
  'text': 'Prima o poi la Superlega si farà. Ne è sicuro Michel Platini, leggendaria stella del calcio francese ed ex presidente UEFA: "Dopo la sentenza della Corte europea che ha stabilito che FIFA e UEFA non hanno più il monopolio delle competizioni, il calcio esploderà. La Superlega si farà di sicuro - dice ai microfoni de Le Figaro -. Tanta gente che come me non ama FIFA e UEFA mi sta contattando. I tempi cambiano. La Superlega è ineluttabile. Sarebbe stata impossibile con me alla guida dell\'UEFA, d\'altronde uccisi il G14 che voleva uccidere il calcio. Oggi, tutti se ne fregano di Aleksander Ceferin. Ha fatto i conti sbagliati, cambiando la Champions solo per distribuire più partite e soldi ai club che vogliono gestirseli".',
  'title': 'Platini: "Il calcio esploderà, la Superlega si farà di sicuro"',
  'top_image': 'https://net-storage.tcccdn.com/storage/fcinternews.it/img_notizie/thumb3/ed/ed174a353eed1526d116e25b0cb2c885-50951-oooz0000.jpeg',
  'url': 'https://www.fcinternews.it/news/platini-il-calcio-esplodera-la-superlega-si-fara-di-sicuro-957523'},
 {'author': '',
  'base': 'https://www.bianconeranews.it',
  'provider': 'bianconeranews',
  'published_date': None,
  'keywords': ['luizkoopmeiners',
   'tieni',
   'platini',
   'luiz',
   'di',
   'rbn',
   'la',
   'sono',
   'se',
   'chiesa',
   'il',
   'rabiot',
   'ha',
   'juve',
   'motta',
   'presidente',
   'bonini'],
  'summary': "Il contratto del francese scade il 30 giugno, quello dell'attaccante nel 2025: le possibilità di addio sono concrete per entrambi.\nIntanto Platini tuona contro Uefa e Fifa, il sogno di molti tifosi bianconeri è di vederlo nella dirigenza.\nIn ESCLUSIVA a Fuori di Juve Massimo Bonini, ex centrocampista della Vecchia Signora.\nLa Juve ha tanti giovani su cui puntare, però se ha soldi da investire per elementi pronti va bene.\nDipenderà molto da Motta, se ripeterà quanto fatto con il Bologna, un misto di qualità e idea di gioco ben precisa.",
  'text': "Podcast Bonini a RBN: “Luiz-Koopmeiners top. Juve tieni Chiesa. Platini presidente”\n\nvedi letture\n\nLa Juve prosegue sulla strada del rafforzamento, soprattutto a centrocampo. Douglas Luiz sembra in dirittura d'arrivo dopo aver sostituito McKennie con Berrenechea come seconda contropartita tecnica insieme a Iling Junior, oltre a versare cash nelle case nelle casse dell'Aston Villa, per una cifra intorno ai 25 milioni. Giuntoli ora darà l'assalto a Koopmeiners, priorità per il nuovo tecnico Thiago Motta. L'Atalanta lo valuta 60 milioni, Huijsen potrebbe rientrare nell'operazione. Resta il doppio nodo legato a Rabiot e Chiesa. Il contratto del francese scade il 30 giugno, quello dell'attaccante nel 2025: le possibilità di addio sono concrete per entrambi. Intanto Platini tuona contro Uefa e Fifa, il sogno di molti tifosi bianconeri è di vederlo nella dirigenza. In ESCLUSIVA a Fuori di Juve Massimo Bonini, ex centrocampista della Vecchia Signora.\n\nIn primis Bonini analizza quello che potrebbe essere il futuro centrocampo della Juve, in particolare con Douglas Luiz e Koopmeiners: “Sicuramente il centrocampo è il reparto che detta i tempi a tutta la squadra, comanda il gioco. Sono due calciatori fantastici, anche se poi dipende dall'organizzazione di gioco. La Juve ha tanti giovani su cui puntare, però se ha soldi da investire per elementi pronti va bene. Dipenderà molto da Motta, se ripeterà quanto fatto con il Bologna, un misto di qualità e idea di gioco ben precisa. Douglas Luiz e Koopmeiners comunque sono tanta roba, sanno giocare con e senza palla, attaccano la profondità, sono completi, insomma due top player. Motta? Scelta giustissima. Ha dimostrato di essere un ottimo tecnico, dà un'identità alla squadra. A Bologna toglieva giocatori importanti , entravano le riserve e la musica non cambiava. Qui si vede la bravura dell'allenatore, riesce ad entrare nella testa dei suoi ragazzi e fa rendere le cose più semplici, ognuno sa ciò che deve fare”. Poi su Rabiot: “E' un nazionale francese, ha dimostrato negli ultimi anni il suo valore, di essere un grande giocatore, fisicità, segna, difficile trovare un altro centrocampista di questo livello. A me piace anche McKennie, ma credo venga ceduto. Se possibile, Rabiot bisognerebbe tenerlo. Comunque mi pare comandi la madre”.\n\nNel podcast l'intervento integrale",
  'title': 'Bonini a RBN: “Luiz-Koopmeiners top. Juve tieni Chiesa. Platini presidente”',
  'top_image': 'https://net-storage.tcccdn.com/storage/bianconeranews.it/img_notizie/thumb3/49/49733201b57c1ef34b89fcaf7463bb94-56931-oooz0000.jpeg',
  'url': 'https://www.bianconeranews.it/podcast/bonini-a-rbn-luiz-koopmeiners-top-juve-tieni-chiesa-platini-presidente-75484'},
 {'author': 'Lorenzo Focolari',
  'base': 'https://www.juvenews.eu',
  'provider': 'juvenews',
  'published_date': '2024-06-21T16:00:56+00:00',
  'keywords': ['lega',
   'coppa',
   'platini',
   'della',
   'juventus',
   'nota',
   'la',
   'serie',
   'gli',
   'michel',
   'il',
   'dei',
   'auguri',
   'ha',
   'una'],
  'summary': 'Il sito ufficiale della Lega Serie A ha pubblicato una nota per fare gli auguri a Michel Platini, ex leggenda della Juve.\nIl sito ufficiale della Lega Serie A ha pubblicato una nota per fare gli auguri all’ex calciatore della Juventus Michel Platini.\n1955: buon compleanno Michel Platini, che compie 69 anni.\nIl tre volte Pallone d’oro, considerato uno dei migliori calciatori di tutti i tempi, ha giocato 147 partite in Serie A con la Juventus segnando 68 reti.\nIn bianconero “Le Roy” ha vinto due Scudetti, una Coppa Italia, una Coppa delle Coppe, una Coppa dei Campioni, una Supercoppa UEFA e una Coppa Intercontinentale”.',
  'text': 'Il sito ufficiale della Lega Serie A ha pubblicato una nota per fare gli auguri a Michel Platini, ex leggenda della Juve.\n\nIl sito ufficiale della Lega Serie A ha pubblicato una nota per fare gli auguri all’ex calciatore della Juventus Michel Platini. Ecco il comunicato: “1931: in casa Juventus inizia l’era del “Quinquennio”. I bianconeri vincono il primo dei loro cinque Scudetti consecutivi. TANTI AUGURI! 1955: buon compleanno Michel Platini, che compie 69 anni. Il tre volte Pallone d’oro, considerato uno dei migliori calciatori di tutti i tempi, ha giocato 147 partite in Serie A con la Juventus segnando 68 reti. In bianconero “Le Roy” ha vinto due Scudetti, una Coppa Italia, una Coppa delle Coppe, una Coppa dei Campioni, una Supercoppa UEFA e una Coppa Intercontinentale”.',
  'title': 'Lega Serie A, gli auguri a Michel Platini: la nota',
  'top_image': 'https://juvenews-aws-bucket.s3.eu-south-1.amazonaws.com/202310/4a01e3e8960ffc0b8b0c5a2424ddca92-scaled.jpg',
  'url': 'https://www.juvenews.eu/serie-a-news/lega-serie-a-auguri-platini/'}]

### Cluster news articles into topics

In [22]:
from typing import Union


class Article(BaseModel):
    link: Union[str|None]
    title: Union[str|None]
    source: Union[str|None]
    date: Union[str|None]
    snippet: Union[str|None]
    summary: Union[str|None]

class Topic(BaseModel):
    # """A class representing a topic and its related news articles."""
    """A class representing a topic."""
    topic: str = Field(description="The topic name")
    articles: List[Article] = Field(description="The list of related news articles")


class TopicsGenerator(BaseModel):
    """A component responsible for generating topics based on a list of articles titles and snippets."""

    topics: List[Topic] = Field(description="The list of generated topics")

    @classmethod
    def get_system_prompt(cls) -> str:
        return """You are a news topics generator. You will generate a list of topics based on a list of articles titles and snippets. It will be used to subsequently cluster news articles into related topics. Try to generate more than one topic, even if the differences are subtle."""

    @classmethod
    def get_user_prompt(cls):
        return """List of articles:\n\n {articles}"""


class NewsClusterer(BaseModel):
    """A component responsible for clustering news articles into related topics based on their title and summary."""

    topics: List[Topic] = Field(description="The list of generated topics")

    @classmethod
    def get_system_prompt(cls) -> str:
        return """You are a news clustering generator. You will cluster news articles into related topics based on their title and snippet."""

    @classmethod
    def get_user_prompt(cls):
        return """Articles:\n\n {articles}"""
    

news_articles = [Article(
        title=n['title'],
        link=n['url'],
        source=n['provider'],
        date=n['published_date'],
        snippet=' '.join(n['summary'].split()[:50])+"..." if n['summary'] else ' '.join(n['text'].split()[:50])+"...",
        summary=n["summary"])
    for n in processed_news_results]
news_articles_shortened = [Article(
        title=n['title'],
        link="",
        source="",
        date="",
        snippet=' '.join(n['summary'].split()[:50])+"..." if n['summary'] else ' '.join(n['text'].split()[:50])+"...",
        summary="")
    for n in processed_news_results]
# news_articles = [f"Title: {n['title']}\nSnippet: {" ".join(n['summary'].split()[:50])}"
#     for n in processed_news_results]
# news_articles = "\n".join([f"Title: {n['title']}\nSnippet: {' '.join(n['summary'].split()[:50]) if n['summary'] else ' '.join(n['text'].split()[:50])}...\n----\n"
#     for n in processed_news_results])


# topics = use_component(TopicsGenerator, {"articles": news_articles_shortened})
# topics

In [23]:
### pearl jam test
news_articles_shortened = [Article(link='', title='Last-minute Pearl Jam resale tickets released for Co-op Live show tomorrow', source='', date='', snippet="The monumental rock band Pearl Jam recently started their world tour, kicking off in the USA first. And this will be the first of two chances British fans will have to see Pearl Jam hit stages this year. Here's the breakdown:How to buy Pearl Jam resale tickets nowPearl Jam tickets...", summary=''), Article(link='', title='Pearl Jam Announce New Album, Dark Matter, Out April 19, 2024 Plus World Tour', source='', date='', snippet='Produced by multi-GRAMMY® award winning producer Andrew Watt, Dark Matter, marks the band’s first release since critically-acclaimed Gigaton (2020). Writing and recording in a burst of inspiration, Dark Matter was born in just three weeks. Get more information and find stores at www.recordstoreday.comDark Matter album packaging features light painting art...', summary=''), Article(link='', title='30 years of clashes between Ticketmaster, artists and fans', source='', date='', snippet='Here’s a timeline of some of the dustups between artists, fans, and Ticketmaster over the years. (AP Photo/Jeff Chiu, File)StubHub sued Ticketmaster and the Golden State Warriors, alleging it unfairly required fans looking to resell tickets to use Ticketmaster’s resale exchange. Swifties vs Ticketmaster — 2022Taylor Swift performs at the...', summary=''), Article(link='', title='Pearl Jam Slash Ticket Prices as Poor Sales Trend Lingers', source='', date='', snippet='Pearl Jam are the latest of many acts to have to shift plans or prices amid soft demand so far in 2024. While promoters and artists have rode a wave of post-COVID demand to record-shattering profits driven by ever-spiraling ticket prices, that trend seems to have fallen apart for many...', summary=''), Article(link='', title='Australia concerts in 2024: here’s a list of shows and tours coming up', source='', date='', snippet='To celebrate, she’ll stage her debut headline shows in Australia with four stops across Brisbane, Sydney and Melbourne. In July, he’ll perform three shows in Melbourne, Sydney and Brisbane – tickets are now on sale. Tickets are now available for all shows in Sydney, Brisbane, Melbourne, Adelaide and Perth. Sofi...', summary=''), Article(link='', title='Pearl Jam asked Bill Clinton to take on Ticketmaster 30 years ago', source='', date='', snippet='Pearl Jam was at the White House, and President Bill Clinton wanted some advice from the musicians. It was April 9, 1994, and Pearl Jam was at the height of its fame after releasing “Vs.,” its second album. Days earlier, Nirvana frontman and fellow Seattle native Kurt Cobain had been...', summary=''), Article(link='', title='Pearl Jam, Neil Young and more: Several major performers announce Chicago tour stops', source='', date='', snippet='According to an announcement from Live Nation, Seattle grunge band Pearl Jam has announced a 2024 world tour with a two-night stop in Chicago at Wrigley Field. The 35-date "Dark Matter" tour, which coincides with their new album set to be released April 19, will come to Chicago Aug. 29...', summary=''), Article(link='', title='How to buy Pearl Jam tickets today for 2024 UK tour', source='', date='', snippet="Pearl Jam also announced an enormous world tour will be taking place throughout 2024. Here's what you need to know:Pearl Jam's new album cover for Dark MatterHow to buy Pearl Jam ticketsPearl Jam tickets will be released today, Friday, February 23, 2024, at 10am. Here's the link breakdown on where...", summary=''), Article(link='', title='The LA Setlist: May 21-26, 2024', source='', date='', snippet='While forever associated with the grunge genre that they helped propel out of Seattle in the early 1990s, Pearl Jam is in truth simply a fantastic band that would’ve thrived in any era of rock ‘n roll. Indeed, in some ways the quintet is a modern anomaly, not least in...', summary=''), Article(link='', title='Pearl Jam announce Spurs Stadium concert and you need to register for tickets', source='', date='', snippet="Dark Matter is being released on April 19, but you have to register for concert tickets way before that. They'll be available from Live Nation and Ticketmaster. You'll have until February 18 to register on either Live Nation and Ticketmaster, and you'll get alerted when the sale starts. You can...", summary='')]
news_art_formatted = "\n-----\n".join([f"Article Title: {a.title}\nArticle Snippet: {a.snippet}" for a in news_articles_shortened])
print(news_art_formatted)


Article Title: Last-minute Pearl Jam resale tickets released for Co-op Live show tomorrow
Article Snippet: The monumental rock band Pearl Jam recently started their world tour, kicking off in the USA first. And this will be the first of two chances British fans will have to see Pearl Jam hit stages this year. Here's the breakdown:How to buy Pearl Jam resale tickets nowPearl Jam tickets...
-----
Article Title: Pearl Jam Announce New Album, Dark Matter, Out April 19, 2024 Plus World Tour
Article Snippet: Produced by multi-GRAMMY® award winning producer Andrew Watt, Dark Matter, marks the band’s first release since critically-acclaimed Gigaton (2020). Writing and recording in a burst of inspiration, Dark Matter was born in just three weeks. Get more information and find stores at www.recordstoreday.comDark Matter album packaging features light painting art...
-----
Article Title: 30 years of clashes between Ticketmaster, artists and fans
Article Snippet: Here’s a timeline of some of the d

In [24]:
topics = use_component(TopicsGenerator, {"articles": news_art_formatted})
topics

TopicsGenerator(topics=[Topic(topic='Pearl Jam Tour and Tickets', articles=[Article(link=None, title='Last-minute Pearl Jam resale tickets released for Co-op Live show tomorrow', source=None, date=None, snippet="The monumental rock band Pearl Jam recently started their world tour, kicking off in the USA first. And this will be the first of two chances British fans will have to see Pearl Jam hit stages this year. Here's the breakdown:How to buy Pearl Jam resale tickets nowPearl Jam tickets...", summary=None), Article(link=None, title='Pearl Jam Announce New Album, Dark Matter, Out April 19, 2024 Plus World Tour', source=None, date=None, snippet='Produced by multi-GRAMMY® award winning producer Andrew Watt, Dark Matter, marks the band’s first release since critically-acclaimed Gigaton (2020). Writing and recording in a burst of inspiration, Dark Matter was born in just three weeks. Get more information and find stores at www.recordstoreday.comDark Matter album packaging features light pa

In [ ]:
topics.topics

In [ ]:
topics = TopicsGenerator(topics=[Topic(topic='Michel Platini and Maradona', articles=[Article(link=None, title='Maradona e la punizione alla Juventus, Platini a Pecci: "Ma non potevi passargli il pallone sul destro?"', source=None, date=None, snippet="Simpatico siparietto a distanza tra Platini e Pecci, l'argomento è quello della magica punizione in area di Maradona contro la Juventus. Diego Armando Maradona ha scritto pagine memorabili di calcio, ogni sua giocata è stata un'opera d'arte. Nella serata di ieri durante il post partita tra Francia e Olanda è...", summary=''), Article(link=None, title='Notti Europee, Platini stronca Mazzocchi: “Che ca*ata", la replica', source=None, date=None, snippet='Notti Europee, Marco Mazzocchi fa infuriare Michel Platini: “Che ca*ata di domanda”, imbarazzo in studio Il mitico "Le Roi", ospite di Notti Europee su Rai 1, stronca Mazzocchi dopo una domanda poco gradita. Stavolta è riuscito a far perdere la pazienza addirittura a uno dei mostri sacri del calcio internazionale...', summary=''), Article(link=None, title='💣 Platini scatenato in RAI, gela il giornalista: "Che ca**ata di domanda" | OneFootball', source=None, date=None, snippet='❌ Dimarco non si allena, ha preso un colpo: Croazia a rischioIl terzino sinistro della Nazionale Federico Dimarco si è fermato a causa di un colpo subito nel finale del match con la Spagna.Il calciatore non ha ancora recuperato e ha saltato anche...', summary=''), Article(link=None, title='Euro2024, Platini a Notti Europee: "La Spagna è la sorpresa, ha un bel gioco". La battuta a Mazzocchi', source=None, date=None, snippet='“La Spagna è una sorpresa, ha una bella tecnica. Che di rimando ha spiegato: “Il calcio di oggi è diverso da ieri, oggi c’è un rapporto più fisico col pallone, a centrocampo c’è l’abitudine di mettere tre velocisti che recuperano i palloni”. Quando c’è la tecnica c’è più possibilità di...', summary=''), Article(link=None, title='Serena non ha dubbi: "Platini è stato il giocatore più geniale con cui ho giocato"', source=None, date=None, snippet='Serena non ha dubbi: «Platini è stato il giocatore più geniale con cui ho giocato». Il tributo dell’ex Juve Serena, ex attaccante della Juve, ospite a Sky Sport durante Calciomercato l’Originale ha reso uno speciale tributo a Platini che ieri ha festeggiato il compleanno. SERENA OMAGGIA PLATINI – «Il 21...', summary=''), Article(link=None, title='Platini: "Il calcio esploderà, la Superlega si farà di sicuro"', source=None, date=None, snippet='Prima o poi la Superlega si farà. Ne è sicuro Michel Platini, leggendaria stella del calcio francese ed ex presidente UEFA: "Dopo la sentenza della Corte europea che ha stabilito che FIFA e UEFA non hanno più il monopolio delle competizioni, il calcio esploderà. La Superlega si farà di sicuro...', summary=''), Article(link=None, title='Bonini a RBN: “Luiz-Koopmeiners top. Juve tieni Chiesa. Platini presidente”', source=None, date=None, snippet="Il contratto del francese scade il 30 giugno, quello dell'attaccante nel 2025: le possibilità di addio sono concrete per entrambi. Intanto Platini tuona contro Uefa e Fifa, il sogno di molti tifosi bianconeri è di vederlo nella dirigenza. In ESCLUSIVA a Fuori di Juve Massimo Bonini, ex centrocampista della Vecchia...", summary=''), Article(link=None, title='Lega Serie A, gli auguri a Michel Platini: la nota', source=None, date=None, snippet='Il sito ufficiale della Lega Serie A ha pubblicato una nota per fare gli auguri a Michel Platini, ex leggenda della Juve. Il sito ufficiale della Lega Serie A ha pubblicato una nota per fare gli auguri all’ex calciatore della Juventus Michel Platini. 1955: buon compleanno Michel Platini, che compie...', summary='')]), Topic(topic="Platini's Views on Football Governance", articles=[Article(link=None, title='L\'eurodeputato Gozi: "Ha ragione Platini, modello Uefa finito. Va creato un modello di calcio europeo più libero e giusto"', source=None, date=None, snippet='“Michel Platini dice a Le Figaro dice quello che sappiamo tutti e che UEFA e FIGC continuano a negare: dopo la decisione della Corte Ue, il modello UEFA è finito e va creato un altro modello di calcio europeo. Più libero e giusto, aggiungo io”. Lo afferma Sandro Gozi, eurodeputato...', summary='')])])
topics

In [ ]:
for topic in topics.topics:  # readd full articles after using shortened for llm
    article_titles = [article.title for article in topic.articles]
    complete_articles = [a for a in news_articles if a.title in article_titles]
    topic.articles = complete_articles

In [ ]:
topics.topics

#### Get topic with most articles

In [ ]:
sorted_by_n_articles = sorted(topics.topics, key=lambda t: len(t.articles), reverse=True)
chosen_topic = sorted_by_n_articles[0]

In [ ]:
chosen_topic.topic

In [ ]:
chosen_topic.articles

## Initialize vectorstore

In [ ]:
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

def split_docs(docs):
    splitted_docs = []
    for d in docs:
        splits = text_splitter.split_text(d.page_content)
        splitted_d = [Document(page_content=spl, metadata={"title": d.metadata["title"], "url": d.metadata["url"], "query": d.metadata["query"]}) for spl in splits]
        for sd in splitted_d:
            if len(sd.page_content) > 50:  # scarto i titoli
                splitted_docs.append(sd)
    return splitted_docs

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
import re
import os
from hashlib import md5


vectorstore_trend_name = chosen_topic.topic.lower().replace(" ", "_")
hash_id = md5(vectorstore_trend_name.encode()).hexdigest()
vectorstore_id = re.sub(r'[^A-Za-z0-9 ]+', '', vectorstore_trend_name) + "_" + hash_id
persist_directory=f"vectorstores/{vectorstore_id}"

if not os.path.exists(persist_directory):
    print("a")
    vectorstore = Chroma.from_documents([Document(page_content="")], OpenAIEmbeddings(), persist_directory=persist_directory)
else:
    print("b")
    vectorstore = Chroma(persist_directory=persist_directory)
vectorstore = Chroma.from_documents([Document(page_content="")], OpenAIEmbeddings(), persist_directory=persist_directory)
vectorstore.get()["ids"]

In [ ]:
len(vectorstore.get()["ids"])

## Get keywords

In [ ]:
class PrimaryKeyword(BaseModel):
    """ primary keyword is the central term or phrase that best represents the main topic of a piece of content."""

    keyword: str = Field(description="The primary keyword")

class PrimaryKeywordGenerator(BaseModel):
    """A component responsible for generating a primary keyword for an article starting from a list of related articles."""

    primary_keyword: PrimaryKeyword = Field(
        description="The generated primary keyword"
    )

    @classmethod
    def get_system_prompt(cls) -> str:
        return """You are a primary keyword generator. You will generate a primary keyword to guide the writing of an SEO-efficient article starting from a list of articles"""
    
    @classmethod
    def get_user_prompt(cls):
        return """"Articles list:\n\n {articles}"""
    

class SecondaryKeyword(BaseModel):
    """A secondary keyword is a term or phrase that is related to the primary keyword and supports the main topic of your content. It's used to cover subtopics, provide additional context, and enhance the overall relevancy of the content. While they are not the main focus of the article, secondary keywords help to capture additional search traffic and improve the depth and breadth of the content."""

    keyword: str = Field(description="The secondary keyword")

class SecondaryKeywordGenerator(BaseModel):
    """A component responsible for generating a list of secondary keywords starting from a primary keyword and a list of related articles."""

    secondary_keywords: List[SecondaryKeyword] = Field(
        description="A list of generated secondary keywords."
    )

    @classmethod
    def get_system_prompt(cls) -> str:
        return """You are a secondary keyword generator. You will generate a list of secondary keywords to guide the writing of an SEO-efficient article based on the provided primary keyword and a list of related articles.\n\n Example: For an article with the primary keyword "how to bake a cake," potential secondary keywords could be:\n    "cake baking tips"\n    "best cake baking recipes"\n    "cake baking ingredients"\n    "cake baking tools"\n    "common cake baking mistakes.\n\nAVOID generic keywords such as "Obama news", be specific."""
    
    @classmethod
    def get_user_prompt(cls):
        return """Primary keyword: {kw}\n\nRelated articles:\n{articles}\n\n###Additional instructions###\n AVOID any generic keywords such as 'Obama news', be specific."""
    
    @classmethod
    def remove_news(cls, kw: list):
        for k in kw.secondary_keywords:
            if "news" in k.keyword:
                kw.secondary_keywords.remove(k)
        return kw
    

class LongtailKeyword(BaseModel):
    """A longtail keyword is a keyword that is not in the top 10 search results."""

    keyword: str = Field(description="The longtail keyword")


class LongtailKeywordsGenerator(BaseModel):
    """A component responsible for generating longtail keywords starting from a primary keyword, a list of secondary keywords and a list of related articles. """

    longtail_keywords: list[LongtailKeyword] = Field(
        description="A list of generated longtail keywords."
    )

    @classmethod
    def get_system_prompt(cls) -> str:
        return """You will generate longtail keywords starting from a primary keyword, a list of secondary keywords and a list of related articles. For example, if the primary keyword was 'bake a cake,' potential long-tail keywords could be:\n\n    'how to bake a chocolate cake from scratch'\n    'easy homemade vanilla cake recipe'\n    'gluten-free cake baking tips'\n    'best tools for cake baking beginners'\n    'how to bake a moist red velvet cake'"""
    
    @classmethod
    def get_user_prompt(cls):
        return """Primary keyword: {kw}\n\Secondary keywords: {skw}\n\nRelated articles:\n{articles}"""
    
    def remove_dups(self, secondary_kws: SecondaryKeywordGenerator) -> 'LongtailKeywordsGenerator':
        sec_keys = [k.keyword for k in secondary_kws.secondary_keywords]
        for lt in self.longtail_keywords:
            if lt.keyword in sec_keys:
                self.longtail_keywords.remove(lt)
        return self



pk_articles = chosen_topic.articles
formatted_articles = "\n----\n".join([f'Title: {a.title}\nSnippet: {a.snippet}' for a in pk_articles])
print(formatted_articles)

In [ ]:
# pk = use_component(PrimaryKeywordGenerator, {"articles": formatted_articles})
# pk

In [ ]:
pk = PrimaryKeywordGenerator(primary_keyword=PrimaryKeyword(keyword='Michel Platini'))
pk

In [ ]:
# sec_keys = use_component(
#     SecondaryKeywordGenerator, 
#     {"kw": pk.primary_keyword.keyword, "articles": formatted_articles}
# )
# sec_keys = SecondaryKeywordGenerator.remove_news(sec_keys)
# sec_keys_list = [k.keyword for k in sec_keys.secondary_keywords]
# print(sec_keys)
# print("; ".join(sec_keys_list))


In [ ]:
sec_keys = SecondaryKeywordGenerator(secondary_keywords=[SecondaryKeyword(keyword='Platini vs Maradona'), SecondaryKeyword(keyword='Platini quotes'), SecondaryKeyword(keyword='Platini career highlights'), SecondaryKeyword(keyword='Platini controversy'), SecondaryKeyword(keyword='Platini UEFA presidency'), SecondaryKeyword(keyword='Platini Superlega comments'), SecondaryKeyword(keyword='Platini Juventus legacy')])
sec_keys = SecondaryKeywordGenerator.remove_news(sec_keys)
sec_keys_list = [k.keyword for k in sec_keys.secondary_keywords]
print(sec_keys)
print("; ".join(sec_keys_list))

In [ ]:
# longtail_keys = use_component(
#     LongtailKeywordsGenerator, 
#     {"kw": pk.primary_keyword.keyword, "skw": "; ".join(sec_keys_list), "articles": formatted_articles}
# )
# longtail_keys

In [ ]:
longtail_keys = (LongtailKeywordsGenerator(longtail_keywords=[LongtailKeyword(keyword='Michel Platini vs Maradona'), LongtailKeyword(keyword='Michel Platini quotes'), LongtailKeyword(keyword='Michel Platini career highlights'), LongtailKeyword(keyword='Michel Platini controversy'), LongtailKeyword(keyword='Michel Platini UEFA presidency'), LongtailKeyword(keyword='Michel Platini Superlega comments'), LongtailKeyword(keyword='Michel Platini Juventus legacy'), LongtailKeyword(keyword='Maradona punizione Juventus Platini Pecci'), LongtailKeyword(keyword='Platini Mazzocchi Notti Europee'), LongtailKeyword(keyword='Platini gela giornalista RAI'), LongtailKeyword(keyword='Platini Euro2024 Spagna sorpresa'), LongtailKeyword(keyword='Serena Platini giocatore geniale'), LongtailKeyword(keyword='Platini Superlega esploderà'), LongtailKeyword(keyword='Bonini Luiz-Koopmeiners Juve Chiesa Platini presidente'), LongtailKeyword(keyword='Lega Serie A auguri Michel Platini')]),)[0]
longtail_keys

In [ ]:
longtail_keys2 = LongtailKeywordsGenerator.remove_dups(longtail_keys, sec_keys)  #MAKE SELF METHOD

In [ ]:
longtail_keys2

## Define outline

In [ ]:
class ParagraphWriter(BaseModel):
    """Component responsible for writing a paragraph based on provided information."""

    paragraph:str = Field(description="The written content of the H1 paragraph")

    @classmethod
    def get_system_prompt(cls) -> str:
        return """You are an expert SEO article writer. You will write the content of a paragraph based on the provided overall outline and paragraph title"""
    
    @classmethod
    def get_user_prompt(cls):
        return """Overall article outline: {outline}\n\nContext:\n{context}\n\n\nBased on the provided context, write a 50-100 words paragraph about '{p_title}'\n\n{style}"""

In [ ]:
from typing import Union


class H3Title(BaseModel):
    """A component responsible for generating the title of an H3Title"""
    
    title:str = Field(description="The title of this H2Title")
    content:Union[str|ParagraphWriter|None] = None
    web_search:Union[None|dict] = None
    sources:Union[None|list] = None


class H2Title(BaseModel):
    """A component responsible for generating the title of an H2Title"""
    
    title:str = Field(description="The title of this H2Title")
    content:Union[str|ParagraphWriter|None] = None
    h3_titles:Optional[List[H3Title]] = Field(description="The list of H3Titles that will be generated. They are related to the H2 title.")
    web_search:Union[None|dict] = None
    sources:Union[None|list] = None


In [ ]:
class OutlineGenerator(BaseModel):
    """Generate an outline to write an SEO-efficient article starting from a list of secondary keywords and a list of longtail keywords."""

    h2_titles:List[H2Title] = Field(description="The generated H2Titles")

    @classmethod
    def get_system_prompt(cls) -> str:
        return """You are an outline generator. You will generate an outline to write an SEO-efficient article starting from a list of secondary keywords and a list of longtail keywords. H1 is the title of the whole article. You will provide a minimum of 3 H2 Titles and maximum 5 H2 Titles. Leave content empty, will be filled later."""
        # return """You are an outline generator. You will generate an outline to write an SEO-efficient article starting from a primary keyword, a list of secondary keywords and a list of longtail keywords. H1 is the title of the whole article. You will provide 3-5 H2 Titles. Each H2 titles can either have 2-3 H3 Titles or none, they are optional. NEVER include an introduction nor a conclusion. Those will be added later. Add the titles that are the most interesting, current and engaging."""
    
    @classmethod
    def get_user_prompt(cls):
        return """Secondary keywords: {s_kw}\nLongtail Keywords: {l_kw}\n\n###Important instructions###\You will provide a minimum of 3 H2 Titles and maximum 5 H2 Titles. Each H2 title can have a minimum of 0 and a maximum of 2 H3 titles. NEVER include an introduction nor a conclusion. Those will be added later. Add the titles that are the most interesting, current and engaging."""
        
    def print_formatted_outline(self) -> str:
        outline_str = ""
        for h2 in self.h2_titles:
            outline_str += f"## {h2.title}\n"
            try:
                outline_str += f"{h2.content.paragraph}\n\n"
            except AttributeError:
                outline_str += f"To be filled...\n\n"
            if h2.h3_titles:
                for h3 in h2.h3_titles:
                    outline_str += f"\t### {h3.title}\n"
                    try:
                        outline_str += f"{h3.content.paragraph}\n\n"
                    except:
                        outline_str += f"To be filled...\n\n"
        return outline_str

    
# try:
#     longtail_keywords = longtail_keys.longtail_keywords
# except AttributeError:
#     longtail_keywords = longtail_keys[0].longtail_keywords
# five_longest_lt_kws = sorted([k.keyword for k in longtail_keywords], key=len, reverse=True)[:5]
# outline = use_component(
#     OutlineGenerator, 
#     {
#         "s_kw": "; ".join([k.keyword for k in sec_keys.secondary_keywords]),
#         "l_kw": "; ".join(five_longest_lt_kws)
#     }
# )
# outline

In [ ]:
outline = OutlineGenerator(h2_titles=[H2Title(title='Platini vs Maradona', content=None, h3_titles=None, web_search=None), H2Title(title='Platini Quotes', content=None, h3_titles=None, web_search=None), H2Title(title='Platini Career Highlights', content=None, h3_titles=[H3Title(title='Bonini Luiz-Koopmeiners Juve Chiesa Platini presidente', content=None, web_search=None), H3Title(title='Michel Platini career highlights', content=None, web_search=None)], web_search=None), H2Title(title='Platini Controversy', content=None, h3_titles=None, web_search=None), H2Title(title='Platini UEFA Presidency', content=None, h3_titles=None, web_search=None)])
outline

## Write paragraphs

In [ ]:
style_instructions = """### Writing style instructions ###
- Use Short Sentences: Keep sentences short and to the point.
- Where applicable, use bullet points, and numbered lists for readability.
- Active Voice: Use active voice where possible.
- Transitional Words: Use words like “therefore,” “however,” and “moreover” to improve flow.
- Make sure a 13 yeard old could understand your writing.
- The perfect paragraph should be between 50 and 150 words long.
- Banned words: vibrant, delve, unravel, journey, multifaceted, bustling, landscape, testament, realm, embark, crucial, vital, ensure.
"""

### Search info for filling content

In [ ]:
class SearchQueryGenerator(BaseModel):
    """Generate an search query to be used for finding information on the web about a topic."""

    query:str = Field(description="The generated search query")

    @classmethod
    def get_system_prompt(cls) -> str:
        return """You are SearchQueryGenerator, a specialized model designed to generate effective and relevant search queries for finding information on the web about a given topic. Your primary objective is to craft a search query that yield the most pertinent and comprehensive results, facilitating the discovery of valuable information. Below are your guidelines and responsibilities:\n\n- Identify key elements, related subtopics, and potential search keywords that encapsulate the topic comprehensively.\n- Generate a clear, concise, and targeted search query.\n- Ensure the query is specific enough to filter out irrelevant information yet broad enough to capture all relevant data.\n- """
    
    @classmethod
    def get_user_prompt(cls):
        return """Topic: {paragraph_title}"""
    
# generated_query = use_component(SearchQueryGenerator, {"paragraph_title": para_title})
# generated_query

In [ ]:
def search_web_for_outline_paragraphs(outline:OutlineGenerator, gl="us", hl="en") -> OutlineGenerator:
    total_h2 = len(outline.h2_titles)
    total_h3 = sum([len(h2.h3_titles) for h2 in outline.h2_titles if h2.h3_titles])
    print(f"[+] Preparing to make {total_h3+total_h2} searches")

    outline_copy = outline  # copy outline so i keep the empty version
    for h2 in outline_copy.h2_titles:
        generated_query = use_component(SearchQueryGenerator, {"paragraph_title": h2.title}).query
        h2_results = search_google(generated_query, hl=hl, gl=gl)
        h2.web_search = {
            "generated_query": generated_query,
            "results": h2_results
        }

        if h2.h3_titles:
            for h3 in h2.h3_titles:
                generated_query = use_component(SearchQueryGenerator, {"paragraph_title": h3.title}).query
                h3_results = search_google(generated_query, hl=hl, gl=gl)
                h3.web_search = {
                    "generated_query": generated_query,
                    "results": h3_results
        }
    return outline_copy

# outline_with_searches = search_web_for_outline_paragraphs(outline)

In [ ]:
outline_with_searches = OutlineGenerator(h2_titles=[H2Title(title='Platini vs Maradona', content=None, h3_titles=None, web_search={'generated_query': 'Comparison between Platini and Maradona', 'results': {'search_metadata': {'id': '6677fb1ef636ff91503c6289', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/f63ba76fff506626/6677fb1ef636ff91503c6289.json', 'created_at': '2024-06-23 10:38:22 UTC', 'processed_at': '2024-06-23 10:38:22 UTC', 'google_url': 'https://www.google.com/search?q=Comparison+between+Platini+and+Maradona&oq=Comparison+between+Platini+and+Maradona&hl=en&gl=us&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/f63ba76fff506626/6677fb1ef636ff91503c6289.html', 'total_time_taken': 0.55}, 'search_parameters': {'engine': 'google', 'q': 'Comparison between Platini and Maradona', 'google_domain': 'google.com', 'hl': 'en', 'gl': 'us', 'device': 'desktop'}, 'search_information': {'query_displayed': 'Comparison between Platini and Maradona', 'total_results': 650000, 'time_taken_displayed': 0.26, 'organic_results_state': 'Results for exact spelling'}, 'inline_images': [{'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&q=Comparison+between+Platini+and+Maradona&udm=2&source=univ&fir=I0A7fzNq1fr_rM%252COVKgzgd3uHgnkM%252C_%253BrdC1haEghIs8_M%252Cx1UhoveBl9aHoM%252C_%253B-E_VEiMd4nftNM%252CfRwpPvte57hJxM%252C_%253BqX7zs2rpgvCz1M%252Co5dL7lW5R977HM%252C_%253BDQqAB6TW7dzFBM%252CeocWjm9mhV6HMM%252C_%253BIooC4weXANYmKM%252CNFuIpSGU_Zth8M%252C_%253BICRbK7zgrwdp6M%252C_XaVoLAyyiYTwM%252C_%253Bjw6lzZtrn-svQM%252ClssA7xr-utU3DM%252C_%253BPpQF-CYqPU4hCM%252Cp0kA-vijztCQwM%252C_%253B3x7GC5i0a2frxM%252CeocWjm9mhV6HMM%252C_&usg=AI4_-kRmdyhDeq386oSAeE7g2PYKfRWAig&sa=X&ved=2ahUKEwiChq6cxPGGAxUW_skDHZ9MAQQQ9AF6BAgxEAA#imgrc=I0A7fzNq1fr_rM', 'source': 'https://www.youtube.com/watch?v=7QG0duSAglI', 'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR08ux0z9MCehnsx7bUvEwO9HNP8HhgzYc5Jp931Q1ok__SnS5iY4Ct0VH_fA&s', 'original': 'https://i.ytimg.com/vi/7QG0duSAglI/sddefault.jpg', 'title': 'Maradona vs Platini - The Last Dance!', 'source_name': 'YouTube'}, {'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&q=Comparison+between+Platini+and+Maradona&udm=2&source=univ&fir=I0A7fzNq1fr_rM%252COVKgzgd3uHgnkM%252C_%253BrdC1haEghIs8_M%252Cx1UhoveBl9aHoM%252C_%253B-E_VEiMd4nftNM%252CfRwpPvte57hJxM%252C_%253BqX7zs2rpgvCz1M%252Co5dL7lW5R977HM%252C_%253BDQqAB6TW7dzFBM%252CeocWjm9mhV6HMM%252C_%253BIooC4weXANYmKM%252CNFuIpSGU_Zth8M%252C_%253BICRbK7zgrwdp6M%252C_XaVoLAyyiYTwM%252C_%253Bjw6lzZtrn-svQM%252ClssA7xr-utU3DM%252C_%253BPpQF-CYqPU4hCM%252Cp0kA-vijztCQwM%252C_%253B3x7GC5i0a2frxM%252CeocWjm9mhV6HMM%252C_&usg=AI4_-kRmdyhDeq386oSAeE7g2PYKfRWAig&sa=X&ved=2ahUKEwiChq6cxPGGAxUW_skDHZ9MAQQQ9AF6BAhFEAA#imgrc=rdC1haEghIs8_M', 'source': 'https://www.seattletimes.com/sports/platini-to-ap-maradona-was-the-greatest-football-lover/', 'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSyuGveHPsNi1J5yeQMK2sCWFNqTUMI3iX7jKqxafD0WClHamoEtRGrL2KUyA&s', 'original': 'https://images.seattletimes.com/wp-content/uploads/2020/11/urn-publicid-ap-org-be8c3ad6a57cffbd405a3475ac6951afFrance_Maradona_Platini_46381.jpg?d=2040x1349', 'title': "Platini to AP: Maradona was the 'greatest football lover ...", 'source_name': 'The Seattle Times'}, {'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&q=Comparison+between+Platini+and+Maradona&udm=2&source=univ&fir=I0A7fzNq1fr_rM%252COVKgzgd3uHgnkM%252C_%253BrdC1haEghIs8_M%252Cx1UhoveBl9aHoM%252C_%253B-E_VEiMd4nftNM%252CfRwpPvte57hJxM%252C_%253BqX7zs2rpgvCz1M%252Co5dL7lW5R977HM%252C_%253BDQqAB6TW7dzFBM%252CeocWjm9mhV6HMM%252C_%253BIooC4weXANYmKM%252CNFuIpSGU_Zth8M%252C_%253BICRbK7zgrwdp6M%252C_XaVoLAyyiYTwM%252C_%253Bjw6lzZtrn-svQM%252ClssA7xr-utU3DM%252C_%253BPpQF-CYqPU4hCM%252Cp0kA-vijztCQwM%252C_%253B3x7GC5i0a2frxM%252CeocWjm9mhV6HMM%252C_&usg=AI4_-kRmdyhDeq386oSAeE7g2PYKfRWAig&sa=X&ved=2ahUKEwiChq6cxPGGAxUW_skDHZ9MAQQQ9AF6BAg1EAA#imgrc=-E_VEiMd4nftNM', 'source': 'https://www.youtube.com/watch?v=HTw21GuyJm0', 'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS00o5QARyYKwbHfxfOv1TlKth7ottgGJpvaF7KXuAgH_1QswdrIVUWUdcZQw&s', 'original': 'https://i.ytimg.com/vi/HTw21GuyJm0/maxresdefault.jpg', 'title': 'Pièce de résistance! ☆ Diego Maradona vs Michel Platini 720p', 'source_name': 'YouTube'}], 'inline_videos': [{'position': 1, 'title': 'Maradona vs Platini - The Last Dance!', 'link': 'https://www.youtube.com/watch?v=7QG0duSAglI', 'thumbnail': 'https://serpapi.com/searches/6677fb1ef636ff91503c6289/images/7d672394e9a212eb52bfd0cc462941bcbb4001aa717843a3e9ffbde634e13925.jpeg', 'channel': 'E-mot10ns HD', 'duration': '30:05', 'platform': 'YouTube', 'date': 'Mar 20, 2023'}, {'position': 2, 'title': 'MARADONA VS PLATINI', 'link': 'https://www.youtube.com/watch?v=7ikNuHH3VB8', 'thumbnail': 'https://serpapi.com/searches/6677fb1ef636ff91503c6289/images/7d672394e9a212ebacb3d4cc0876a008d115ad98f4abf74194d7b1702802be3f.jpeg', 'channel': 'Football Land', 'duration': '4:33', 'platform': 'YouTube', 'date': 'Mar 1, 2023'}, {'position': 3, 'title': 'Platini ⚽️ Maradona', 'link': 'https://www.youtube.com/watch?v=GAy-Ip2U1YY', 'thumbnail': 'https://serpapi.com/searches/6677fb1ef636ff91503c6289/images/7d672394e9a212eb885acfef957578c8305b7636cca7d172b8aa453b98e79998.jpeg', 'channel': 'CHJ', 'duration': '0:15', 'platform': 'YouTube', 'date': 'Jun 21, 2022'}, {'position': 4, 'title': 'Diego Maradona vs Michel Platini I vs Juventus(a) I Serie A 85 ...', 'link': 'https://www.youtube.com/watch?v=JI3Cgkbtf0g', 'thumbnail': 'https://serpapi.com/searches/6677fb1ef636ff91503c6289/images/7d672394e9a212eb5109ba2d70df89197992f5b068b757dcdca5157f0c396405.jpeg', 'channel': 'SB All Touch Compilations', 'duration': '14:21', 'platform': 'YouTube', 'date': 'Nov 26, 2020'}], 'related_questions': [{'question': 'Was Platini better than Maradona?', 'snippet': 'Zico and especially Maradona were on another level of technical ability than Platini. Maradona also had an explosive burst of acceleration that Zico and especially Platini lacked.', 'title': 'Who among Zico, Maradona and Platini was the True Legend ...', 'date': 'Sep 18, 2018', 'link': 'https://www.quora.com/Who-among-Zico-Maradona-and-Platini-was-the-True-Legend-of-80s-if-World-Cup-is-not-mandatory-for-Legacy#:~:text=Zico%20and%20especially%20Maradona%20were,Zico%20and%20especially%20Platini%20lacked.', 'displayed_link': 'https://www.quora.com › Who-among-Zico-Maradona-...', 'next_page_token': 'eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9vQkNreEJTa2M1U210TlJuSjFiVEp3VDBwMlNHMUtVelp3VEdOMk5UWTNSalpRYkZOSVQyRjFNa1JCUWpodE4wUTJhSFF0TmpCWVFuaG9NVkJ4UldOclIwMWllV2xpZDNGNFJUVklSMUZRRWhaSWRuUXpXbTlNWDAwMVlqaHdPRFJRYmpWdFJrbEJHaUpCUmxoeVJXTndRM1J0VTJreVdFdEZNVkJsYmkxTk16TkViVlpwY3pkdFlXMW4iLCJmY3YiOiIzIiwiZWkiOiJIdnQzWm9MX001YjhwODRQbjVtRklBIiwicWMiOiJDaWRqYjIxd1lYSnBjMjl1SUdKbGRIZGxaVzRnY0d4aGRHbHVhU0JoYm1RZ2JXRnlZV1J2Ym1FUUFIM2VGd0lfIiwicXVlc3Rpb24iOiJXYXMgUGxhdGluaSBiZXR0ZXIgdGhhbiBNYXJhZG9uYT8iLCJsayI6IkdpQjNZWE1nY0d4aGRHbHVhU0JpWlhSMFpYSWdkR2hoYmlCdFlYSmhaRzl1WVEiLCJicyI6ImMtUHk0bElNVHl4V0NNaEpMTW5NeTFSSVNpMHBTUzFTS01sSXpGUHdUU3hLVE1uUFM3U1htQnRwcENTbFVBNVVWNEJGWFM1VUhWY0dsMEY0UnI1Q1NtWUtYQzlRUldaZU5saHJTVVlxVUZ0eENjaU1TcURlMUxMVUludUpmaVlqUXluOWNxaTJYT0swY1Rsd3lZQnN5aXpHNGVCelFVYXlVdExsbUVyZ2JnM2pVdmZJTHdmeTh5b1ZuQkp6Y3ZMekZGTFVfWXVLVVYxZm5wbG5MN0U0MzBoRFNpMERwam9KcWxvaEg2bzZGMG0xQUNNQSIsImlkIjoiZmNfNSJ9', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google_related_questions&google_domain=google.com&next_page_token=eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9vQkNreEJTa2M1U210TlJuSjFiVEp3VDBwMlNHMUtVelp3VEdOMk5UWTNSalpRYkZOSVQyRjFNa1JCUWpodE4wUTJhSFF0TmpCWVFuaG9NVkJ4UldOclIwMWllV2xpZDNGNFJUVklSMUZRRWhaSWRuUXpXbTlNWDAwMVlqaHdPRFJRYmpWdFJrbEJHaUpCUmxoeVJXTndRM1J0VTJreVdFdEZNVkJsYmkxTk16TkViVlpwY3pkdFlXMW4iLCJmY3YiOiIzIiwiZWkiOiJIdnQzWm9MX001YjhwODRQbjVtRklBIiwicWMiOiJDaWRqYjIxd1lYSnBjMjl1SUdKbGRIZGxaVzRnY0d4aGRHbHVhU0JoYm1RZ2JXRnlZV1J2Ym1FUUFIM2VGd0lfIiwicXVlc3Rpb24iOiJXYXMgUGxhdGluaSBiZXR0ZXIgdGhhbiBNYXJhZG9uYT8iLCJsayI6IkdpQjNZWE1nY0d4aGRHbHVhU0JpWlhSMFpYSWdkR2hoYmlCdFlYSmhaRzl1WVEiLCJicyI6ImMtUHk0bElNVHl4V0NNaEpMTW5NeTFSSVNpMHBTUzFTS01sSXpGUHdUU3hLVE1uUFM3U1htQnRwcENTbFVBNVVWNEJGWFM1VUhWY0dsMEY0UnI1Q1NtWUtYQzlRUldaZU5saHJTVVlxVUZ0eENjaU1TcURlMUxMVUludUpmaVlqUXluOWNxaTJYT0swY1Rsd3lZQnN5aXpHNGVCelFVYXlVdExsbUVyZ2JnM2pVdmZJTHdmeTh5b1ZuQkp6Y3ZMekZGTFVfWXVLVVYxZm5wbG5MN0U0MzBoRFNpMERwam9KcWxvaEg2bzZGMG0xQUNNQSIsImlkIjoiZmNfNSJ9'}, {'question': 'Who did Maradona think was the best player ever?', 'snippet': 'Even Maradona thought Pele was the greatest soccer player of all time.', 'title': 'Pele', 'date': 'Dec 29, 2022', 'link': 'https://www.marca.com/en/football/2022/12/30/63ae262b22601d2f1d8b457a.html', 'displayed_link': 'https://www.marca.com › football › 2022/12/30', 'source_logo': 'https://serpapi.com/searches/6677fb1ef636ff91503c6289/images/e2d62413680772da9139c5b66c6697e35f749e0ad95ff37e536338cc57b14e1f.png', 'next_page_token': 'eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9vQkNreEJTa2M1U210TlJuSjFiVEp3VDBwMlNHMUtVelp3VEdOMk5UWTNSalpRYkZOSVQyRjFNa1JCUWpodE4wUTJhSFF0TmpCWVFuaG9NVkJ4UldOclIwMWllV2xpZDNGNFJUVklSMUZRRWhaSWRuUXpXbTlNWDAwMVlqaHdPRFJRYmpWdFJrbEJHaUpCUmxoeVJXTndRM1J0VTJreVdFdEZNVkJsYmkxTk16TkViVlpwY3pkdFlXMW4iLCJmY3YiOiIzIiwiZWkiOiJIdnQzWm9MX001YjhwODRQbjVtRklBIiwicWMiOiJDaWRqYjIxd1lYSnBjMjl1SUdKbGRIZGxaVzRnY0d4aGRHbHVhU0JoYm1RZ2JXRnlZV1J2Ym1FUUFIM2VGd0lfIiwicXVlc3Rpb24iOiJXaG8gZGlkIE1hcmFkb25hIHRoaW5rIHdhcyB0aGUgYmVzdCBwbGF5ZXIgZXZlcj8iLCJsayI6IkdpOTNhRzhnWkdsa0lHMWhjbUZrYjI1aElIUm9hVzVySUhkaGN5QjBhR1VnWW1WemRDQndiR0Y1WlhJZ1pYWmxjZyIsImJzIjoiYy1QeTRsSU1UeXhXQ01oSkxNbk15MVJJU2kwcFNTMVNLTWxJekZQd1RTeEtUTW5QUzdTWG1CdHBwQ1NsVUE1VVY0QkZYUzVVSFZjR2wwRjRScjVDU21ZS1hDOVFSV1plTmxoclNVWXFVRnR4Q2NpTVNxRGUxTExVSW51SmZpWWpReW45Y3FpMlhPSzBjVGx3eVlCc3lpekc0ZUJ6UVVheVV0TGxtRXJnYmczalV2ZklMd2Z5OHlvVm5CSnpjdkx6RkZMVV9ZdUtVVjFmbnBsbkw3RTQzMGhEU2kwRHBqb0pxbG9oSDZvNkYwbTFBQ01BIiwiaWQiOiJmY181In0=', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google_related_questions&google_domain=google.com&next_page_token=eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9vQkNreEJTa2M1U210TlJuSjFiVEp3VDBwMlNHMUtVelp3VEdOMk5UWTNSalpRYkZOSVQyRjFNa1JCUWpodE4wUTJhSFF0TmpCWVFuaG9NVkJ4UldOclIwMWllV2xpZDNGNFJUVklSMUZRRWhaSWRuUXpXbTlNWDAwMVlqaHdPRFJRYmpWdFJrbEJHaUpCUmxoeVJXTndRM1J0VTJreVdFdEZNVkJsYmkxTk16TkViVlpwY3pkdFlXMW4iLCJmY3YiOiIzIiwiZWkiOiJIdnQzWm9MX001YjhwODRQbjVtRklBIiwicWMiOiJDaWRqYjIxd1lYSnBjMjl1SUdKbGRIZGxaVzRnY0d4aGRHbHVhU0JoYm1RZ2JXRnlZV1J2Ym1FUUFIM2VGd0lfIiwicXVlc3Rpb24iOiJXaG8gZGlkIE1hcmFkb25hIHRoaW5rIHdhcyB0aGUgYmVzdCBwbGF5ZXIgZXZlcj8iLCJsayI6IkdpOTNhRzhnWkdsa0lHMWhjbUZrYjI1aElIUm9hVzVySUhkaGN5QjBhR1VnWW1WemRDQndiR0Y1WlhJZ1pYWmxjZyIsImJzIjoiYy1QeTRsSU1UeXhXQ01oSkxNbk15MVJJU2kwcFNTMVNLTWxJekZQd1RTeEtUTW5QUzdTWG1CdHBwQ1NsVUE1VVY0QkZYUzVVSFZjR2wwRjRScjVDU21ZS1hDOVFSV1plTmxoclNVWXFVRnR4Q2NpTVNxRGUxTExVSW51SmZpWWpReW45Y3FpMlhPSzBjVGx3eVlCc3lpekc0ZUJ6UVVheVV0TGxtRXJnYmczalV2ZklMd2Z5OHlvVm5CSnpjdkx6RkZMVV9ZdUtVVjFmbnBsbkw3RTQzMGhEU2kwRHBqb0pxbG9oSDZvNkYwbTFBQ01BIiwiaWQiOiJmY181In0%3D'}, {'question': 'Who is better than Maradona?', 'snippet': 'Pelé was a better player and had a significantly better carrer ( albein under very different context ) while both are what you could call old school number 10 or second forward Maradona was playing in a deeper role than Pelé and was more closer to a midfielder than Pelé who was more direct and goal…', 'title': 'Who was better: Pele or Maradona? Who is the greatest of all time ...', 'link': 'https://futbol.quora.com/Pele-Edges-it-for-me-never-gonna-move-on-that-https-www-quora-com-Who-was-better-Pele-or-Maradona-Who-is-the-greatest#:~:text=Pel%C3%A9%20was%20a%20better%20player,was%20more%20direct%20and%20goal%E2%80%A6', 'displayed_link': 'https://futbol.quora.com › Pele-Edges-it-for-me-never-g...', 'source_logo': 'https://serpapi.com/searches/6677fb1ef636ff91503c6289/images/e2d62413680772da9139c5b66c6697e319d98728d99c331d3f73d56601a3b9dc.png', 'next_page_token': 'eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9vQkNreEJTa2M1U210TlJuSjFiVEp3VDBwMlNHMUtVelp3VEdOMk5UWTNSalpRYkZOSVQyRjFNa1JCUWpodE4wUTJhSFF0TmpCWVFuaG9NVkJ4UldOclIwMWllV2xpZDNGNFJUVklSMUZRRWhaSWRuUXpXbTlNWDAwMVlqaHdPRFJRYmpWdFJrbEJHaUpCUmxoeVJXTndRM1J0VTJreVdFdEZNVkJsYmkxTk16TkViVlpwY3pkdFlXMW4iLCJmY3YiOiIzIiwiZWkiOiJIdnQzWm9MX001YjhwODRQbjVtRklBIiwicWMiOiJDaWRqYjIxd1lYSnBjMjl1SUdKbGRIZGxaVzRnY0d4aGRHbHVhU0JoYm1RZ2JXRnlZV1J2Ym1FUUFIM2VGd0lfIiwicXVlc3Rpb24iOiJXaG8gaXMgYmV0dGVyIHRoYW4gTWFyYWRvbmE/IiwibGsiOiJHaHQzYUc4Z2FYTWdZbVYwZEdWeUlIUm9ZVzRnYldGeVlXUnZibUUiLCJicyI6ImMtUHk0bElNVHl4V0NNaEpMTW5NeTFSSVNpMHBTUzFTS01sSXpGUHdUU3hLVE1uUFM3U1htQnRwcENTbFVBNVVWNEJGWFM1VUhWY0dsMEY0UnI1Q1NtWUtYQzlRUldaZU5saHJTVVlxVUZ0eENjaU1TcURlMUxMVUludUpmaVlqUXluOWNxaTJYT0swY1Rsd3lZQnN5aXpHNGVCelFVYXlVdExsbUVyZ2JnM2pVdmZJTHdmeTh5b1ZuQkp6Y3ZMekZGTFVfWXVLVVYxZm5wbG5MN0U0MzBoRFNpMERwam9KcWxvaEg2bzZGMG0xQUNNQSIsImlkIjoiZmNfNSJ9', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google_related_questions&google_domain=google.com&next_page_token=eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9vQkNreEJTa2M1U210TlJuSjFiVEp3VDBwMlNHMUtVelp3VEdOMk5UWTNSalpRYkZOSVQyRjFNa1JCUWpodE4wUTJhSFF0TmpCWVFuaG9NVkJ4UldOclIwMWllV2xpZDNGNFJUVklSMUZRRWhaSWRuUXpXbTlNWDAwMVlqaHdPRFJRYmpWdFJrbEJHaUpCUmxoeVJXTndRM1J0VTJreVdFdEZNVkJsYmkxTk16TkViVlpwY3pkdFlXMW4iLCJmY3YiOiIzIiwiZWkiOiJIdnQzWm9MX001YjhwODRQbjVtRklBIiwicWMiOiJDaWRqYjIxd1lYSnBjMjl1SUdKbGRIZGxaVzRnY0d4aGRHbHVhU0JoYm1RZ2JXRnlZV1J2Ym1FUUFIM2VGd0lfIiwicXVlc3Rpb24iOiJXaG8gaXMgYmV0dGVyIHRoYW4gTWFyYWRvbmE%2FIiwibGsiOiJHaHQzYUc4Z2FYTWdZbVYwZEdWeUlIUm9ZVzRnYldGeVlXUnZibUUiLCJicyI6ImMtUHk0bElNVHl4V0NNaEpMTW5NeTFSSVNpMHBTUzFTS01sSXpGUHdUU3hLVE1uUFM3U1htQnRwcENTbFVBNVVWNEJGWFM1VUhWY0dsMEY0UnI1Q1NtWUtYQzlRUldaZU5saHJTVVlxVUZ0eENjaU1TcURlMUxMVUludUpmaVlqUXluOWNxaTJYT0swY1Rsd3lZQnN5aXpHNGVCelFVYXlVdExsbUVyZ2JnM2pVdmZJTHdmeTh5b1ZuQkp6Y3ZMekZGTFVfWXVLVVYxZm5wbG5MN0U0MzBoRFNpMERwam9KcWxvaEg2bzZGMG0xQUNNQSIsImlkIjoiZmNfNSJ9'}, {'question': "How many Ballon d'Ors did Maradona win?", 'snippet': "Diego Maradona never won the Ballon d'Or as only players of European nationality were eligible to win the award during the large majority of his career. However, he did receive an honourary award in 1995 that was named the Golden Ballon d'Or. Not too bad and pretty well deserved!", 'title': "Maradona's honourary Ballon d'Or - The Athletic - The New York Times", 'date': 'Oct 30, 2023', 'link': 'https://www.nytimes.com/athletic/live-blogs/ballon-dor-2023-live-updates-winners-time-messi-haaland/V946EQBmSmh8/bRdVixLEYmXw/#:~:text=Diego%20Maradona%20never%20won%20the,bad%20and%20pretty%20well%20deserved!', 'displayed_link': 'https://www.nytimes.com › live-blogs › bRdVixLEYmXw', 'source_logo': 'https://serpapi.com/searches/6677fb1ef636ff91503c6289/images/e2d62413680772da9139c5b66c6697e315daafddbc27e933cc531f404256050a.png', 'next_page_token': 'eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9vQkNreEJTa2M1U210TlJuSjFiVEp3VDBwMlNHMUtVelp3VEdOMk5UWTNSalpRYkZOSVQyRjFNa1JCUWpodE4wUTJhSFF0TmpCWVFuaG9NVkJ4UldOclIwMWllV2xpZDNGNFJUVklSMUZRRWhaSWRuUXpXbTlNWDAwMVlqaHdPRFJRYmpWdFJrbEJHaUpCUmxoeVJXTndRM1J0VTJreVdFdEZNVkJsYmkxTk16TkViVlpwY3pkdFlXMW4iLCJmY3YiOiIzIiwiZWkiOiJIdnQzWm9MX001YjhwODRQbjVtRklBIiwicWMiOiJDaWRqYjIxd1lYSnBjMjl1SUdKbGRIZGxaVzRnY0d4aGRHbHVhU0JoYm1RZ2JXRnlZV1J2Ym1FUUFIM2VGd0lfIiwicXVlc3Rpb24iOiJIb3cgbWFueSBCYWxsb24gZCdPcnMgZGlkIE1hcmFkb25hIHdpbj8iLCJsayI6IkdpWm9iM2NnYldGdWVTQmlZV3hzYjI0Z1pDQnZjbk1nWkdsa0lHMWhjbUZrYjI1aElIZHBiZyIsImJzIjoiYy1QeTRsSU1UeXhXQ01oSkxNbk15MVJJU2kwcFNTMVNLTWxJekZQd1RTeEtUTW5QUzdTWG1CdHBwQ1NsVUE1VVY0QkZYUzVVSFZjR2wwRjRScjVDU21ZS1hDOVFSV1plTmxoclNVWXFVRnR4Q2NpTVNxRGUxTExVSW51SmZpWWpReW45Y3FpMlhPSzBjVGx3eVlCc3lpekc0ZUJ6UVVheVV0TGxtRXJnYmczalV2ZklMd2Z5OHlvVm5CSnpjdkx6RkZMVV9ZdUtVVjFmbnBsbkw3RTQzMGhEU2kwRHBqb0pxbG9oSDZvNkYwbTFBQ01BIiwiaWQiOiJmY181In0=', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google_related_questions&google_domain=google.com&next_page_token=eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9vQkNreEJTa2M1U210TlJuSjFiVEp3VDBwMlNHMUtVelp3VEdOMk5UWTNSalpRYkZOSVQyRjFNa1JCUWpodE4wUTJhSFF0TmpCWVFuaG9NVkJ4UldOclIwMWllV2xpZDNGNFJUVklSMUZRRWhaSWRuUXpXbTlNWDAwMVlqaHdPRFJRYmpWdFJrbEJHaUpCUmxoeVJXTndRM1J0VTJreVdFdEZNVkJsYmkxTk16TkViVlpwY3pkdFlXMW4iLCJmY3YiOiIzIiwiZWkiOiJIdnQzWm9MX001YjhwODRQbjVtRklBIiwicWMiOiJDaWRqYjIxd1lYSnBjMjl1SUdKbGRIZGxaVzRnY0d4aGRHbHVhU0JoYm1RZ2JXRnlZV1J2Ym1FUUFIM2VGd0lfIiwicXVlc3Rpb24iOiJIb3cgbWFueSBCYWxsb24gZCdPcnMgZGlkIE1hcmFkb25hIHdpbj8iLCJsayI6IkdpWm9iM2NnYldGdWVTQmlZV3hzYjI0Z1pDQnZjbk1nWkdsa0lHMWhjbUZrYjI1aElIZHBiZyIsImJzIjoiYy1QeTRsSU1UeXhXQ01oSkxNbk15MVJJU2kwcFNTMVNLTWxJekZQd1RTeEtUTW5QUzdTWG1CdHBwQ1NsVUE1VVY0QkZYUzVVSFZjR2wwRjRScjVDU21ZS1hDOVFSV1plTmxoclNVWXFVRnR4Q2NpTVNxRGUxTExVSW51SmZpWWpReW45Y3FpMlhPSzBjVGx3eVlCc3lpekc0ZUJ6UVVheVV0TGxtRXJnYmczalV2ZklMd2Z5OHlvVm5CSnpjdkx6RkZMVV9ZdUtVVjFmbnBsbkw3RTQzMGhEU2kwRHBqb0pxbG9oSDZvNkYwbTFBQ01BIiwiaWQiOiJmY181In0%3D'}], 'organic_results': [{'position': 1, 'title': 'A discussion of Maradona v Platini', 'link': 'https://www.bigsoccer.com/threads/a-discussion-of-maradona-v-platini.1989797/', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.bigsoccer.com/threads/a-discussion-of-maradona-v-platini.1989797/&ved=2ahUKEwiChq6cxPGGAxUW_skDHZ9MAQQQFnoECCIQAQ', 'displayed_link': 'https://www.bigsoccer.com › ... › Players & Legends', 'favicon': 'https://serpapi.com/searches/6677fb1ef636ff91503c6289/images/fdcb8d4581cd37268a409fcee9c66bf8600379fa5e2df465c698eadc5a75bdaa.png', 'date': 'Jun 27, 2013', 'snippet': 'Maradona was twice South American player of the year and finished 2nd once. Platini played 358 games in France, scored 209 goals. Maradona ...', 'snippet_highlighted_words': ['the year and finished 2nd once. Platini played 358 games in France, scored 209 goals'], 'sitelinks': {'list': [{'title': 'Review: - Platini vs Maradona - the king vs the robber', 'link': 'https://www.bigsoccer.com/threads/platini-vs-maradona-the-king-vs-the-robber.2022392/', 'date': 'Feb 12, 2020'}, {'title': 'Platini va Maradons vs Zico | BigSoccer Forum', 'link': 'https://www.bigsoccer.com/threads/platini-va-maradons-vs-zico.1702664/', 'date': 'Feb 12, 2020'}, {'title': 'Head-to-head matchups between the legends', 'link': 'https://www.bigsoccer.com/threads/head-to-head-matchups-between-the-legends.1697269/', 'date': 'Feb 12, 2020'}, {'title': 'Is Zico, not Maradona, the second best player?ever?', 'link': 'https://www.bigsoccer.com/threads/is-zico-not-maradona-the-second-best-player-ever.1784415/', 'date': 'Feb 12, 2020'}]}, 'source': 'BigSoccer Forum'}, {'position': 2, 'title': 'Who among Zico, Maradona and Platini was the True ...', 'link': 'https://www.quora.com/Who-among-Zico-Maradona-and-Platini-was-the-True-Legend-of-80s-if-World-Cup-is-not-mandatory-for-Legacy', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.quora.com/Who-among-Zico-Maradona-and-Platini-was-the-True-Legend-of-80s-if-World-Cup-is-not-mandatory-for-Legacy&ved=2ahUKEwiChq6cxPGGAxUW_skDHZ9MAQQQFnoECD8QAQ', 'displayed_link': 'https://www.quora.com › Who-among-Zico-Maradona-...', 'favicon': 'https://serpapi.com/searches/6677fb1ef636ff91503c6289/images/fdcb8d4581cd37268a409fcee9c66bf895d805e2fca8ca92484f3c721bae5d35.png', 'snippet': 'Messi and Maradona are the faces of Argentina for the last 40 years or so and are evidently the best players the country has ever produced.', 'snippet_highlighted_words': ['Messi'], 'sitelinks': {'list': [{'title': 'Zidane, Maradona, Platini, Cruyff or Messi? - Quora', 'link': 'https://www.quora.com/Who-is-the-best-number-10-Zidane-Maradona-Platini-Cruyff-or-Messi', 'answer_count': 1, 'date': 'Nov 20, 2023'}, {'title': 'Was Maradona better than Zico? - Quora', 'link': 'https://www.quora.com/Was-Maradona-better-than-Zico', 'answer_count': 3, 'date': 'Jan 28, 2021'}, {'title': 'How do Messi and Ronaldo stack up against legends ...', 'link': 'https://www.quora.com/How-do-Messi-and-Ronaldo-stack-up-against-legends-from-the-past-like-Maradona-Cruyff-Platini-and-Batistuta-Do-they-only-stand-out-as-theres-less-great-players-nowadays', 'answer_count': 8, 'date': 'Oct 9, 2018'}]}, 'source': 'Quora'}, {'position': 3, 'title': "Platini to AP: Maradona was the 'greatest football lover'", 'link': 'https://www.sandiegouniontribune.com/sports/national/story/2020-11-26/platini-to-ap-maradona-was-the-greatest-football-lover/', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.sandiegouniontribune.com/sports/national/story/2020-11-26/platini-to-ap-maradona-was-the-greatest-football-lover/&ved=2ahUKEwiChq6cxPGGAxUW_skDHZ9MAQQQFnoECCkQAQ', 'displayed_link': 'https://www.sandiegouniontribune.com › national › story', 'favicon': 'https://serpapi.com/searches/6677fb1ef636ff91503c6289/images/fdcb8d4581cd37268a409fcee9c66bf88ea917df4d15422f4c2f2dff3fbfdcbf.png', 'date': 'Nov 26, 2020', 'snippet': 'Platini, who is 65, and Maradona played against each other at club level at a time when the Italian league was arguably the best in the world.', 'snippet_highlighted_words': ['Maradona played against each other at club level'], 'source': 'San Diego Union-Tribune'}, {'position': 4, 'title': 'Did Maradona have a "rival", like Messi/CR7 or Mbappe/ ...', 'link': 'https://www.reddit.com/r/football/comments/11d1liw/did_maradona_have_a_rival_like_messicr7_or/', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.reddit.com/r/football/comments/11d1liw/did_maradona_have_a_rival_like_messicr7_or/&ved=2ahUKEwiChq6cxPGGAxUW_skDHZ9MAQQQFnoECC8QAQ', 'displayed_link': '50+ comments · 1 year ago', 'favicon': 'https://serpapi.com/searches/6677fb1ef636ff91503c6289/images/fdcb8d4581cd37268a409fcee9c66bf8a496600760408fc82a89d6508d9c61ef.png', 'snippet': "I know people compare him to Pelé but it doesn't count since they're different eras and can't really be compared.", 'snippet_highlighted_words': ['him to Pelé'], 'source': 'Reddit · r/football'}, {'position': 5, 'title': 'Juventus vs. Napoli: The Fixture Where Diego Maradona ...', 'link': 'https://bleacherreport.com/articles/2671994-juventus-vs-napoli-the-fixture-where-diego-maradona-vanquished-michel-platini', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://bleacherreport.com/articles/2671994-juventus-vs-napoli-the-fixture-where-diego-maradona-vanquished-michel-platini&ved=2ahUKEwiChq6cxPGGAxUW_skDHZ9MAQQQFnoECCoQAQ', 'displayed_link': 'https://bleacherreport.com › World Football › Serie A', 'favicon': 'https://serpapi.com/searches/6677fb1ef636ff91503c6289/images/fdcb8d4581cd37268a409fcee9c66bf8b825875d37d4b7bcec56b9cd54bc8786.png', 'date': 'Oct 28, 2016', 'snippet': 'Juventus vs. Napoli: The Fixture Where Diego Maradona Vanquished Michel Platini | News, Scores, Highlights, Stats, and Rumors | Bleacher Report.', 'snippet_highlighted_words': ['Juventus vs. Napoli'], 'source': 'Bleacher Report'}, {'position': 6, 'title': 'Diego Maradona vs. Michel Platini: Titles Won Comparison', 'link': 'https://www.aworldofsoccer.com/players_comparison/diego_maradona_vs_michel_platini.htm', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.aworldofsoccer.com/players_comparison/diego_maradona_vs_michel_platini.htm&ved=2ahUKEwiChq6cxPGGAxUW_skDHZ9MAQQQFnoECC4QAQ', 'displayed_link': 'https://www.aworldofsoccer.com › players_comparison', 'favicon': 'https://serpapi.com/searches/6677fb1ef636ff91503c6289/images/fdcb8d4581cd37268a409fcee9c66bf86f91b3d9219f93fa768d50cb180569b4.png', 'snippet': 'With his national team, Diego Maradona won 1 title more than Michel Platini, 3 against 2. Also in FIFA World Cups won Diego Maradona has the advantage over ...', 'snippet_highlighted_words': ['Diego Maradona won 1 title more than Michel Platini, 3 against 2'], 'source': 'A World of Soccer.com'}, {'position': 7, 'title': "Platini to AP: Maradona was the 'greatest football lover'", 'link': 'https://apnews.com/platini-to-ap-maradona-was-the-greatest-football-lover-be8c3ad6a57cffbd405a3475ac6951af', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://apnews.com/platini-to-ap-maradona-was-the-greatest-football-lover-be8c3ad6a57cffbd405a3475ac6951af&ved=2ahUKEwiChq6cxPGGAxUW_skDHZ9MAQQQFnoECCsQAQ', 'displayed_link': 'https://apnews.com › platini-to-ap-maradona-was-the-gr...', 'favicon': 'https://serpapi.com/searches/6677fb1ef636ff91503c6289/images/fdcb8d4581cd37268a409fcee9c66bf8e98d8ba1c6da55c2bd9a54133008c35c.png', 'date': 'Nov 26, 2020', 'snippet': 'Platini, who is 65, and Maradona played against each other at club level at a time when the Italian league was arguably the best in the world.', 'snippet_highlighted_words': ['Maradona played against each other at club level'], 'source': 'AP News'}, {'position': 8, 'title': 'Why is the same criteria not applied to Maradona as ...', 'link': 'https://www.reddit.com/r/football/comments/108esej/why_is_the_same_criteria_not_applied_to_maradona/', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.reddit.com/r/football/comments/108esej/why_is_the_same_criteria_not_applied_to_maradona/&ved=2ahUKEwiChq6cxPGGAxUW_skDHZ9MAQQQFnoECDAQAQ', 'displayed_link': '200+ comments · 1 year ago', 'favicon': 'https://serpapi.com/searches/6677fb1ef636ff91503c6289/images/fdcb8d4581cd37268a409fcee9c66bf801ac0cd2e3fbe1768ceaa113d5a44873.png', 'snippet': "So they just say Maradona is the GOAT because it'll just make them look old and wise. I love you for this.", 'snippet_highlighted_words': ["Maradona is the GOAT because it'll just make them look old and"], 'source': 'Reddit · r/football'}], 'pagination': {'current': 1, 'next': 'https://www.google.com/search?q=Comparison+between+Platini+and+Maradona&oq=Comparison+between+Platini+and+Maradona&hl=en&gl=us&start=10&sourceid=chrome&ie=UTF-8', 'other_pages': {'2': 'https://www.google.com/search?q=Comparison+between+Platini+and+Maradona&oq=Comparison+between+Platini+and+Maradona&hl=en&gl=us&start=10&sourceid=chrome&ie=UTF-8', '3': 'https://www.google.com/search?q=Comparison+between+Platini+and+Maradona&oq=Comparison+between+Platini+and+Maradona&hl=en&gl=us&start=20&sourceid=chrome&ie=UTF-8', '4': 'https://www.google.com/search?q=Comparison+between+Platini+and+Maradona&oq=Comparison+between+Platini+and+Maradona&hl=en&gl=us&start=30&sourceid=chrome&ie=UTF-8', '5': 'https://www.google.com/search?q=Comparison+between+Platini+and+Maradona&oq=Comparison+between+Platini+and+Maradona&hl=en&gl=us&start=40&sourceid=chrome&ie=UTF-8'}}, 'serpapi_pagination': {'current': 1, 'next_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Comparison+between+Platini+and+Maradona&start=10', 'next': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Comparison+between+Platini+and+Maradona&start=10', 'other_pages': {'2': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Comparison+between+Platini+and+Maradona&start=10', '3': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Comparison+between+Platini+and+Maradona&start=20', '4': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Comparison+between+Platini+and+Maradona&start=30', '5': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Comparison+between+Platini+and+Maradona&start=40'}}}}), H2Title(title='Platini Quotes', content=None, h3_titles=None, web_search={'generated_query': 'Platini Quotes', 'results': {'search_metadata': {'id': '6677fd329c449d91fbb435ed', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/4f329ebceb94b720/6677fd329c449d91fbb435ed.json', 'created_at': '2024-06-23 10:47:14 UTC', 'processed_at': '2024-06-23 10:47:14 UTC', 'google_url': 'https://www.google.com/search?q=Platini+Quotes&oq=Platini+Quotes&hl=en&gl=us&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/4f329ebceb94b720/6677fd329c449d91fbb435ed.html', 'total_time_taken': 0.47}, 'search_parameters': {'engine': 'google', 'q': 'Platini Quotes', 'google_domain': 'google.com', 'hl': 'en', 'gl': 'us', 'device': 'desktop'}, 'search_information': {'query_displayed': 'Platini Quotes', 'total_results': 115000, 'time_taken_displayed': 0.2, 'organic_results_state': 'Results for exact spelling'}, 'inline_images': [{'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Platini+Quotes&tbm=isch&source=univ&fir=7OoPYFWjryIYCM%252CymsUBnVyC_ih5M%252C_%253BXrxFdLvD4XlAcM%252C3FLX3uCou-1RyM%252C_%253BJB_41FCfAFaNAM%252C2tBLWecU3NirRM%252C_%253Bul4SABLh5cBefM%252CVnaQhuyXDJsn_M%252C_%253BCEt-D4Cu95kAcM%252CXmFQtCvFqdiNyM%252C_%253BTCPxnGLKakS0MM%252CIwRcWsJqwi5-GM%252C_%253BfRRc-cAt0HMwsM%252CdBwy-ZE-9zoh4M%252C_%253BIg6eh6sps6SlBM%252CN2aE81uNislj0M%252C_%253BiI5Pogo1yQQFHM%252CGxt8NARrh8rljM%252C_%253B5foPx5hycy0AYM%252CXmFQtCvFqdiNyM%252C_&usg=AI4_-kSfneUcGcRzohtOu4CjNHYRC7PmJA&sa=X&ved=2ahUKEwjPl9eZxvGGAxW368kDHVKwBUgQ9AF6BAgUEAA#imgrc=7OoPYFWjryIYCM', 'source': 'https://www.azquotes.com/quotes/topics/platini.html', 'thumbnail': 'https://serpapi.com/searches/6677fd329c449d91fbb435ed/images/4cae3a2513a4472f42f082f33b8f66e0913684fb0fbbc4b95b58461e37e7daee.jpeg', 'original': 'https://www.azquotes.com/picture-quotes/quote-what-differences-were-there-between-maradona-and-platini-michel-patini-100-59-22.jpg', 'title': 'TOP 6 PLATINI QUOTES | A-Z Quotes', 'source_name': 'A-Z Quotes'}, {'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Platini+Quotes&tbm=isch&source=univ&fir=7OoPYFWjryIYCM%252CymsUBnVyC_ih5M%252C_%253BXrxFdLvD4XlAcM%252C3FLX3uCou-1RyM%252C_%253BJB_41FCfAFaNAM%252C2tBLWecU3NirRM%252C_%253Bul4SABLh5cBefM%252CVnaQhuyXDJsn_M%252C_%253BCEt-D4Cu95kAcM%252CXmFQtCvFqdiNyM%252C_%253BTCPxnGLKakS0MM%252CIwRcWsJqwi5-GM%252C_%253BfRRc-cAt0HMwsM%252CdBwy-ZE-9zoh4M%252C_%253BIg6eh6sps6SlBM%252CN2aE81uNislj0M%252C_%253BiI5Pogo1yQQFHM%252CGxt8NARrh8rljM%252C_%253B5foPx5hycy0AYM%252CXmFQtCvFqdiNyM%252C_&usg=AI4_-kSfneUcGcRzohtOu4CjNHYRC7PmJA&sa=X&ved=2ahUKEwjPl9eZxvGGAxW368kDHVKwBUgQ9AF6BAgSEAA#imgrc=XrxFdLvD4XlAcM', 'source': 'https://www.azquotes.com/quote/1112387', 'thumbnail': 'https://serpapi.com/searches/6677fd329c449d91fbb435ed/images/4cae3a2513a4472fdf4372e76e3a5d6b06d8227b2bd4ffc700f78a43669e0b84.jpeg', 'original': 'https://www.azquotes.com/picture-quotes/quote-michel-platini-is-a-good-player-not-a-great-player-eamon-111-23-87.jpg', 'title': 'Eamon quote: Michel Platini is a good player, not a great player', 'source_name': 'A-Z Quotes'}, {'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Platini+Quotes&tbm=isch&source=univ&fir=7OoPYFWjryIYCM%252CymsUBnVyC_ih5M%252C_%253BXrxFdLvD4XlAcM%252C3FLX3uCou-1RyM%252C_%253BJB_41FCfAFaNAM%252C2tBLWecU3NirRM%252C_%253Bul4SABLh5cBefM%252CVnaQhuyXDJsn_M%252C_%253BCEt-D4Cu95kAcM%252CXmFQtCvFqdiNyM%252C_%253BTCPxnGLKakS0MM%252CIwRcWsJqwi5-GM%252C_%253BfRRc-cAt0HMwsM%252CdBwy-ZE-9zoh4M%252C_%253BIg6eh6sps6SlBM%252CN2aE81uNislj0M%252C_%253BiI5Pogo1yQQFHM%252CGxt8NARrh8rljM%252C_%253B5foPx5hycy0AYM%252CXmFQtCvFqdiNyM%252C_&usg=AI4_-kSfneUcGcRzohtOu4CjNHYRC7PmJA&sa=X&ved=2ahUKEwjPl9eZxvGGAxW368kDHVKwBUgQ9AF6BAgTEAA#imgrc=JB_41FCfAFaNAM', 'source': 'https://quotefancy.com/quote/1659153/Eamon-Dunphy-Michel-Platini-has-no-bottle-He-is-not-a-great-player', 'thumbnail': 'https://serpapi.com/searches/6677fd329c449d91fbb435ed/images/4cae3a2513a4472f1c38261d3f646ddafefb32fca0f88f3a827ed8b1a2469b93.jpeg', 'original': 'https://quotefancy.com/media/wallpaper/3840x2160/1414065-Eamon-Dunphy-Quote-Michel-Platini-has-no-bottle-He-is-not-a-great.jpg', 'title': 'Eamon Dunphy Quote: “Michel Platini has no bottle. He is not ...', 'source_name': 'QuoteFancy'}], 'related_questions': [{'question': 'What is Michel Platini famous for?', 'snippet': 'Michel Platini (born June 21, 1955, Joeuf, France) is a French professional football (soccer) player and administrator who was named the European Footballer of the Year three times (1983–85) and served as president of the Union of European Football Associations (UEFA; 2007–16).', 'title': 'Michel Platini | Biography, FIFA, & Facts - Britannica', 'date': '6 days ago', 'link': 'https://www.britannica.com/biography/Michel-Platini#:~:text=Michel%20Platini%20(born%20June%2021,UEFA%3B%202007%E2%80%9316).', 'displayed_link': 'https://www.britannica.com › biography › Michel-Platini', 'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTwT7h-5f4paCrJ8dGAAGz-CoYR3HTkumhV0fDpGI5Ytw&s', 'source_logo': 'https://serpapi.com/searches/6677fd329c449d91fbb435ed/images/1555d3c6550428a57cf278f0c73444573fed82794c875062d09b20afa6fea673.png', 'next_page_token': 'eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210TldVaHVNM0l5WmtwM1JGSkJSMlJvUWt4YWJYUkpTVzVUUlVkVGVucHlXR2xFWkU1VE1VSXlNVEpYT1hWclV6QTJORE5WUkRVeU1YRlRhekIxVTJ4dVpXRXRRUzB4RWhkTmRqRXpXbk10TWtKeVpsaHdPRFJRTUhWRFYzZEJVUm9pUVVaWWNrVmpjV00zUkVkbVJEaDZYeTFMVG1OSWJIaE5jRTlRUmt4MGFYb3hVUSIsImZjdiI6IjMiLCJlaSI6Ik12MTNacy0yQnJmWHA4NFAwdUNXd0FRIiwicWMiOiJDZzV3YkdGMGFXNXBJSEYxYjNSbGN4QUFmY3h4d1Q0IiwicXVlc3Rpb24iOiJXaGF0IGlzIE1pY2hlbCBQbGF0aW5pIGZhbW91cyBmb3I/IiwibGsiOiJHaUYzYUdGMElHbHpJRzFwWTJobGJDQndiR0YwYVc1cElHWmhiVzkxY3lCbWIzSSIsImJzIjoiYy1QeTRWSUt6MGdzVWNnc1Z2RE5UTTVJelZFSXlFa3N5Y3pMVkVoTHpNMHZMVlpJeXkteWw3aGVZcVFzcFZnT1ZaZ0xVVmlBb1pETGtVc0diRnBpVWFwQ2VuNS1pa0p4Zm5KeWFwRkNZV2wtU1dxeHZjVEdGQ001S1JtWU9ZbFkxSEFoT1FncUQ1WUFHUV9FLVNWSmlUazU5aEpUa3BFZGhGTWhsd21YQk15MGtveFVCZF9VNHVKTWlEcDdpUjEtUnFKU3duQlBJYVFFR0FFIiwiaWQiOiJmY18zIn0=', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google_related_questions&google_domain=google.com&next_page_token=eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210TldVaHVNM0l5WmtwM1JGSkJSMlJvUWt4YWJYUkpTVzVUUlVkVGVucHlXR2xFWkU1VE1VSXlNVEpYT1hWclV6QTJORE5WUkRVeU1YRlRhekIxVTJ4dVpXRXRRUzB4RWhkTmRqRXpXbk10TWtKeVpsaHdPRFJRTUhWRFYzZEJVUm9pUVVaWWNrVmpjV00zUkVkbVJEaDZYeTFMVG1OSWJIaE5jRTlRUmt4MGFYb3hVUSIsImZjdiI6IjMiLCJlaSI6Ik12MTNacy0yQnJmWHA4NFAwdUNXd0FRIiwicWMiOiJDZzV3YkdGMGFXNXBJSEYxYjNSbGN4QUFmY3h4d1Q0IiwicXVlc3Rpb24iOiJXaGF0IGlzIE1pY2hlbCBQbGF0aW5pIGZhbW91cyBmb3I%2FIiwibGsiOiJHaUYzYUdGMElHbHpJRzFwWTJobGJDQndiR0YwYVc1cElHWmhiVzkxY3lCbWIzSSIsImJzIjoiYy1QeTRWSUt6MGdzVWNnc1Z2RE5UTTVJelZFSXlFa3N5Y3pMVkVoTHpNMHZMVlpJeXkteWw3aGVZcVFzcFZnT1ZaZ0xVVmlBb1pETGtVc0diRnBpVWFwQ2VuNS1pa0p4Zm5KeWFwRkNZV2wtU1dxeHZjVEdGQ001S1JtWU9ZbFkxSEFoT1FncUQ1WUFHUV9FLVNWSmlUazU5aEpUa3BFZGhGTWhsd21YQk15MGtveFVCZF9VNHVKTWlEcDdpUjEtUnFKU3duQlBJYVFFR0FFIiwiaWQiOiJmY18zIn0%3D'}, {'question': 'What are good soccer quotes?', 'title': 'Top 10 Best Soccer Quotes', 'date': 'Feb 27, 2024', 'link': 'https://www.olliesports.com/post/top-10-best-soccer-quotes', 'list': ['“Success is no accident. ... ', "“I don't believe skill was, or ever will be, the result of coaches. ... ", '“The only victory that counts is the one over yourself.” – ... ', '“You have to fight to reach your dream. ... ', '“You have to fight to reach your dream. ... ', "“I don't believe in luck, I only believe in hard work.” –"], 'displayed_link': 'https://www.olliesports.com › post › top-10-best-soccer-...', 'source_logo': 'https://serpapi.com/searches/6677fd329c449d91fbb435ed/images/1555d3c6550428a57cf278f0c73444573f8c5c046b26d784fada23fef04f3504.png', 'next_page_token': 'eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210TldVaHVNM0l5WmtwM1JGSkJSMlJvUWt4YWJYUkpTVzVUUlVkVGVucHlXR2xFWkU1VE1VSXlNVEpYT1hWclV6QTJORE5WUkRVeU1YRlRhekIxVTJ4dVpXRXRRUzB4RWhkTmRqRXpXbk10TWtKeVpsaHdPRFJRTUhWRFYzZEJVUm9pUVVaWWNrVmpjV00zUkVkbVJEaDZYeTFMVG1OSWJIaE5jRTlRUmt4MGFYb3hVUSIsImZjdiI6IjMiLCJlaSI6Ik12MTNacy0yQnJmWHA4NFAwdUNXd0FRIiwicWMiOiJDZzV3YkdGMGFXNXBJSEYxYjNSbGN4QUFmY3h4d1Q0IiwicXVlc3Rpb24iOiJXaGF0IGFyZSBnb29kIHNvY2NlciBxdW90ZXM/IiwibGsiOiJHaHgzYUdGMElHbHpJR0VnWjI5dlpDQnpiMk5qWlhJZ2NYVnZkR1Z6IiwiYnMiOiJjLVB5NFZJS3owZ3NVY2dzVnZETlRNNUl6VkVJeUVrc3ljekxWRWhMek0wdkxWWkl5eS15bDdoZVlxUXNwVmdPVlpnTFVWaUFvWkRMa1VzR2JGcGlVYXBDZW41LWlrSnhmbkp5YXBGQ1lXbC1TV3F4dmNUR0ZDTTVLUm1ZT1lsWTFIQWhPUWdxRDVZQUdRX0UtU1ZKaVRrNTloSlRrcEVkaEZNaGx3bVhCTXkwa294VUJkX1U0dUpNaURwN2lSMS1ScUpTd25CUElhUUVHQUUiLCJpZCI6ImZjXzMifQ==', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google_related_questions&google_domain=google.com&next_page_token=eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210TldVaHVNM0l5WmtwM1JGSkJSMlJvUWt4YWJYUkpTVzVUUlVkVGVucHlXR2xFWkU1VE1VSXlNVEpYT1hWclV6QTJORE5WUkRVeU1YRlRhekIxVTJ4dVpXRXRRUzB4RWhkTmRqRXpXbk10TWtKeVpsaHdPRFJRTUhWRFYzZEJVUm9pUVVaWWNrVmpjV00zUkVkbVJEaDZYeTFMVG1OSWJIaE5jRTlRUmt4MGFYb3hVUSIsImZjdiI6IjMiLCJlaSI6Ik12MTNacy0yQnJmWHA4NFAwdUNXd0FRIiwicWMiOiJDZzV3YkdGMGFXNXBJSEYxYjNSbGN4QUFmY3h4d1Q0IiwicXVlc3Rpb24iOiJXaGF0IGFyZSBnb29kIHNvY2NlciBxdW90ZXM%2FIiwibGsiOiJHaHgzYUdGMElHbHpJR0VnWjI5dlpDQnpiMk5qWlhJZ2NYVnZkR1Z6IiwiYnMiOiJjLVB5NFZJS3owZ3NVY2dzVnZETlRNNUl6VkVJeUVrc3ljekxWRWhMek0wdkxWWkl5eS15bDdoZVlxUXNwVmdPVlpnTFVWaUFvWkRMa1VzR2JGcGlVYXBDZW41LWlrSnhmbkp5YXBGQ1lXbC1TV3F4dmNUR0ZDTTVLUm1ZT1lsWTFIQWhPUWdxRDVZQUdRX0UtU1ZKaVRrNTloSlRrcEVkaEZNaGx3bVhCTXkwa294VUJkX1U0dUpNaURwN2lSMS1ScUpTd25CUElhUUVHQUUiLCJpZCI6ImZjXzMifQ%3D%3D'}, {'question': 'What is a good quote for football?', 'title': 'Browse this list to find the best football sayings for your team:', 'date': 'Aug 4, 2021', 'link': 'https://www.totallypromotional.com/blog/25-game-day-quotes-for-football-lovers/', 'list': ["“It's game day!”", '“A win is a win”', '“Sweat + Spirit = Success”', '“Some wish for it. Champions work for it.”', '“Fall means football”', '“Blessed, stressed and football-obsessed”', '“Win. Lose. But never defeated.”', '“Victory requires a down payment”'], 'displayed_link': 'https://www.totallypromotional.com › blog › 25-game-d...', 'source_logo': 'https://serpapi.com/searches/6677fd329c449d91fbb435ed/images/1555d3c6550428a57cf278f0c73444577b7512b4831bd44cd5c588b9c5765a9f.png', 'next_page_token': 'eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210TldVaHVNM0l5WmtwM1JGSkJSMlJvUWt4YWJYUkpTVzVUUlVkVGVucHlXR2xFWkU1VE1VSXlNVEpYT1hWclV6QTJORE5WUkRVeU1YRlRhekIxVTJ4dVpXRXRRUzB4RWhkTmRqRXpXbk10TWtKeVpsaHdPRFJRTUhWRFYzZEJVUm9pUVVaWWNrVmpjV00zUkVkbVJEaDZYeTFMVG1OSWJIaE5jRTlRUmt4MGFYb3hVUSIsImZjdiI6IjMiLCJlaSI6Ik12MTNacy0yQnJmWHA4NFAwdUNXd0FRIiwicWMiOiJDZzV3YkdGMGFXNXBJSEYxYjNSbGN4QUFmY3h4d1Q0IiwicXVlc3Rpb24iOiJXaGF0IGlzIGEgZ29vZCBxdW90ZSBmb3IgZm9vdGJhbGw/IiwibGsiOiJHaUYzYUdGMElHbHpJR0VnWjI5dlpDQnhkVzkwWlNCbWIzSWdabTl2ZEdKaGJHdyIsImJzIjoiYy1QeTRWSUt6MGdzVWNnc1Z2RE5UTTVJelZFSXlFa3N5Y3pMVkVoTHpNMHZMVlpJeXkteWw3aGVZcVFzcFZnT1ZaZ0xVVmlBb1pETGtVc0diRnBpVWFwQ2VuNS1pa0p4Zm5KeWFwRkNZV2wtU1dxeHZjVEdGQ001S1JtWU9ZbFkxSEFoT1FncUQ1WUFHUV9FLVNWSmlUazU5aEpUa3BFZGhGTWhsd21YQk15MGtveFVCZF9VNHVKTWlEcDdpUjEtUnFKU3duQlBJYVFFR0FFIiwiaWQiOiJmY18zIn0=', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google_related_questions&google_domain=google.com&next_page_token=eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210TldVaHVNM0l5WmtwM1JGSkJSMlJvUWt4YWJYUkpTVzVUUlVkVGVucHlXR2xFWkU1VE1VSXlNVEpYT1hWclV6QTJORE5WUkRVeU1YRlRhekIxVTJ4dVpXRXRRUzB4RWhkTmRqRXpXbk10TWtKeVpsaHdPRFJRTUhWRFYzZEJVUm9pUVVaWWNrVmpjV00zUkVkbVJEaDZYeTFMVG1OSWJIaE5jRTlRUmt4MGFYb3hVUSIsImZjdiI6IjMiLCJlaSI6Ik12MTNacy0yQnJmWHA4NFAwdUNXd0FRIiwicWMiOiJDZzV3YkdGMGFXNXBJSEYxYjNSbGN4QUFmY3h4d1Q0IiwicXVlc3Rpb24iOiJXaGF0IGlzIGEgZ29vZCBxdW90ZSBmb3IgZm9vdGJhbGw%2FIiwibGsiOiJHaUYzYUdGMElHbHpJR0VnWjI5dlpDQnhkVzkwWlNCbWIzSWdabTl2ZEdKaGJHdyIsImJzIjoiYy1QeTRWSUt6MGdzVWNnc1Z2RE5UTTVJelZFSXlFa3N5Y3pMVkVoTHpNMHZMVlpJeXkteWw3aGVZcVFzcFZnT1ZaZ0xVVmlBb1pETGtVc0diRnBpVWFwQ2VuNS1pa0p4Zm5KeWFwRkNZV2wtU1dxeHZjVEdGQ001S1JtWU9ZbFkxSEFoT1FncUQ1WUFHUV9FLVNWSmlUazU5aEpUa3BFZGhGTWhsd21YQk15MGtveFVCZF9VNHVKTWlEcDdpUjEtUnFKU3duQlBJYVFFR0FFIiwiaWQiOiJmY18zIn0%3D'}, {'question': 'What is the Messi quote?', 'snippet': 'You have to fight to reach your dream. You have to sacrifice and work hard for it.', 'title': 'Lionel Messi - Quotes - IMDb', 'link': 'https://www.imdb.com/name/nm2177779/quotes/#:~:text=You%20have%20to%20fight%20to,and%20work%20hard%20for%20it.', 'displayed_link': 'https://www.imdb.com › name › quotes', 'source_logo': 'https://serpapi.com/searches/6677fd329c449d91fbb435ed/images/1555d3c6550428a57cf278f0c73444577c76810efb6351de77390bcf4f437867.png', 'next_page_token': 'eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210TldVaHVNM0l5WmtwM1JGSkJSMlJvUWt4YWJYUkpTVzVUUlVkVGVucHlXR2xFWkU1VE1VSXlNVEpYT1hWclV6QTJORE5WUkRVeU1YRlRhekIxVTJ4dVpXRXRRUzB4RWhkTmRqRXpXbk10TWtKeVpsaHdPRFJRTUhWRFYzZEJVUm9pUVVaWWNrVmpjV00zUkVkbVJEaDZYeTFMVG1OSWJIaE5jRTlRUmt4MGFYb3hVUSIsImZjdiI6IjMiLCJlaSI6Ik12MTNacy0yQnJmWHA4NFAwdUNXd0FRIiwicWMiOiJDZzV3YkdGMGFXNXBJSEYxYjNSbGN4QUFmY3h4d1Q0IiwicXVlc3Rpb24iOiJXaGF0IGlzIHRoZSBNZXNzaSBxdW90ZT8iLCJsayI6IkdoTjNhR0YwSUdseklHMWxjM05wSUhGMWIzUmwiLCJicyI6ImMtUHk0VklLejBnc1VjZ3NWdkROVE01SXpWRUl5RWtzeWN6TFZFaEx6TTB2TFZaSXl5LXlsN2hlWXFRc3BWZ09WWmdMVVZpQW9aRExrVXNHYkZwaVVhcENlbjUtaWtKeGZuSnlhcEZDWVdsLVNXcXh2Y1RHRkNNNUtSbVlPWWxZMUhBaE9RZ3FENVlBR1FfRS1TVkppVGs1OWhKVGtwRWRoRk1obHdtWEJNeTBrb3hVQmRfVTR1Sk1pRHA3aVIxLVJxSlN3bkJQSWFRRUdBRSIsImlkIjoiZmNfMyJ9', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google_related_questions&google_domain=google.com&next_page_token=eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210TldVaHVNM0l5WmtwM1JGSkJSMlJvUWt4YWJYUkpTVzVUUlVkVGVucHlXR2xFWkU1VE1VSXlNVEpYT1hWclV6QTJORE5WUkRVeU1YRlRhekIxVTJ4dVpXRXRRUzB4RWhkTmRqRXpXbk10TWtKeVpsaHdPRFJRTUhWRFYzZEJVUm9pUVVaWWNrVmpjV00zUkVkbVJEaDZYeTFMVG1OSWJIaE5jRTlRUmt4MGFYb3hVUSIsImZjdiI6IjMiLCJlaSI6Ik12MTNacy0yQnJmWHA4NFAwdUNXd0FRIiwicWMiOiJDZzV3YkdGMGFXNXBJSEYxYjNSbGN4QUFmY3h4d1Q0IiwicXVlc3Rpb24iOiJXaGF0IGlzIHRoZSBNZXNzaSBxdW90ZT8iLCJsayI6IkdoTjNhR0YwSUdseklHMWxjM05wSUhGMWIzUmwiLCJicyI6ImMtUHk0VklLejBnc1VjZ3NWdkROVE01SXpWRUl5RWtzeWN6TFZFaEx6TTB2TFZaSXl5LXlsN2hlWXFRc3BWZ09WWmdMVVZpQW9aRExrVXNHYkZwaVVhcENlbjUtaWtKeGZuSnlhcEZDWVdsLVNXcXh2Y1RHRkNNNUtSbVlPWWxZMUhBaE9RZ3FENVlBR1FfRS1TVkppVGs1OWhKVGtwRWRoRk1obHdtWEJNeTBrb3hVQmRfVTR1Sk1pRHA3aVIxLVJxSlN3bkJQSWFRRUdBRSIsImlkIjoiZmNfMyJ9'}], 'answer_box': {'type': 'organic_result', 'title': 'Top 25 Michel Patini Quotes (2024 Update) - QuoteFancy', 'link': 'https://quotefancy.com/michel-patini-quotes', 'displayed_link': 'https://quotefancy.com › michel-patini-quotes', 'snippet': 'Top 25 Michel Patini Quotes (2024 Update)', 'snippet_highlighted_words': ['Top 25 Michel Patini Quotes (2024 Update)'], 'list': ["“In any case, you can't turn back the clock.” ... ", '“For me, football is just a game, not a drama.” ... ', '“What Zidane can do with a football, Maradona could do with an orange.” ... ', '“Some games you win, some you lose, and some you draw.” ... ', '“A football team represents a way of being, a culture.”'], 'favicon': 'https://serpapi.com/searches/6677fd329c449d91fbb435ed/images/726d20126a65c0f4998bd982ed3d7cceebfb558c3e97ee9c.png', 'source': 'QuoteFancy'}, 'organic_results': [{'position': 1, 'title': 'TOP 25 QUOTES BY MICHEL PATINI', 'link': 'https://www.azquotes.com/author/11380-Michel_Patini', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.azquotes.com/author/11380-Michel_Patini&ved=2ahUKEwjPl9eZxvGGAxW368kDHVKwBUgQFnoECBkQAQ', 'displayed_link': 'https://www.azquotes.com › author › 11380-Michel_Patini', 'thumbnail': 'https://serpapi.com/searches/6677fd329c449d91fbb435ed/images/b02c5b5912906a98de2e27f3943b5bebdc9697ba265c9a90b7b39ce5ed3bfdc9.jpeg', 'favicon': 'https://serpapi.com/searches/6677fd329c449d91fbb435ed/images/b02c5b5912906a98de2e27f3943b5beb8144318a4cdb85d79bf81d1491285a03.png', 'snippet': 'Michel Patini Quotes · It is a game before a product, a sport before a market, a show before a business. · A football team represents a way of being, a culture.', 'snippet_highlighted_words': ['Quotes'], 'source': 'A-Z Quotes'}, {'position': 2, 'title': 'Michel Patini Quotes', 'link': 'https://www.brainyquote.com/authors/michel-patini-quotes', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.brainyquote.com/authors/michel-patini-quotes&ved=2ahUKEwjPl9eZxvGGAxW368kDHVKwBUgQFnoECB4QAQ', 'displayed_link': 'https://www.brainyquote.com › authors › michel-patini-...', 'favicon': 'https://serpapi.com/searches/6677fd329c449d91fbb435ed/images/b02c5b5912906a98de2e27f3943b5beb33fb76a85027fa1b86a78732c85923da.png', 'snippet': 'Enjoy the best Michel Patini Quotes at BrainyQuote. Quotations by Michel ... What differences were there between Maradona and Platini? Share this Quote.', 'snippet_highlighted_words': ['Quotes', 'Quotations', 'Platini', 'Quote'], 'source': 'BrainyQuote'}, {'position': 3, 'title': 'Michel Platini - Quotes', 'link': 'https://www.imdb.com/name/nm1177672/quotes/', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.imdb.com/name/nm1177672/quotes/&ved=2ahUKEwjPl9eZxvGGAxW368kDHVKwBUgQFnoECCEQAQ', 'displayed_link': 'https://www.imdb.com › name › quotes', 'favicon': 'https://serpapi.com/searches/6677fd329c449d91fbb435ed/images/b02c5b5912906a98de2e27f3943b5bebefc5c9cc4434494505991bc717bb7cb2.png', 'snippet': '"The English are so stupid." (On the frequent exclusion of the gifted Glenn Hoddle from England teams). What Zidane does with a ball, Maradona could do with ...', 'source': 'IMDb'}, {'position': 4, 'title': 'Quotes - MichelPlatini.org', 'link': 'http://www.michelplatini.org/Quotes.html', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=http://www.michelplatini.org/Quotes.html&ved=2ahUKEwjPl9eZxvGGAxW368kDHVKwBUgQFnoECB8QAQ', 'displayed_link': 'http://www.michelplatini.org › Quotes', 'favicon': 'https://serpapi.com/searches/6677fd329c449d91fbb435ed/images/b02c5b5912906a98de2e27f3943b5bebf744cf970b58a82541468ae6182d854c.png', 'snippet': '"This is a very sad day. Another gift to us from above has come and gone. Platini will remain in our memories as one of Juventus\' greatest ever players."', 'snippet_highlighted_words': ['Platini'], 'source': 'michelplatini.org'}, {'position': 5, 'title': 'Michel Platini Quotes', 'link': 'https://www.goodreads.com/author/quotes/233046.Michel_Platini', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.goodreads.com/author/quotes/233046.Michel_Platini&ved=2ahUKEwjPl9eZxvGGAxW368kDHVKwBUgQFnoECCAQAQ', 'displayed_link': 'https://www.goodreads.com › 233...', 'favicon': 'https://serpapi.com/searches/6677fd329c449d91fbb435ed/images/b02c5b5912906a98de2e27f3943b5bebaf17d5c88a81960f72586e4b5fcd70b5.png', 'snippet': "2 quotes from Michel Platini: 'Está el Pelé hombre y el Pelé jugador. Para jugar como Pelé hay que jugar como Dios.' and 'A great amongst the greats.'", 'snippet_highlighted_words': ['quotes', 'Platini'], 'source': 'Goodreads'}, {'position': 6, 'title': 'Quote by Michel Platini: “A great amongst the greats.”', 'link': 'https://www.goodreads.com/quotes/1280164-a-great-amongst-the-greats', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.goodreads.com/quotes/1280164-a-great-amongst-the-greats&ved=2ahUKEwjPl9eZxvGGAxW368kDHVKwBUgQFnoECCIQAQ', 'displayed_link': 'https://www.goodreads.com › quotes › 1280164-a-grea...', 'favicon': 'https://serpapi.com/searches/6677fd329c449d91fbb435ed/images/b02c5b5912906a98de2e27f3943b5beb81a8ba5ce36bc0a539155bd64fdafb10.png', 'snippet': "Michel Platini — 'A great amongst the greats.'", 'snippet_highlighted_words': ['Platini'], 'source': 'Goodreads'}, {'position': 7, 'title': 'Best Michel Platini Quotes #quotes', 'link': 'https://www.youtube.com/watch?v=9b1IBqetAcs', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.youtube.com/watch%3Fv%3D9b1IBqetAcs&ved=2ahUKEwjPl9eZxvGGAxW368kDHVKwBUgQFnoECCUQAQ', 'displayed_link': '90+ views · 1 year ago', 'favicon': 'https://serpapi.com/searches/6677fd329c449d91fbb435ed/images/b02c5b5912906a98de2e27f3943b5beba1f5e86a1347d457e42d0929125018dc.png', 'snippet': 'Best Michel Platini Quotes Michel François Platini is a French football administrator and former player and manager.', 'snippet_highlighted_words': ['Platini Quotes'], 'source': 'YouTube · The Magic Pearl'}, {'position': 8, 'title': 'Michel Platini - Football Quotes', 'link': 'https://twitter.com/FootballQuote_/status/369900494829129728', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://twitter.com/FootballQuote_/status/369900494829129728&ved=2ahUKEwjPl9eZxvGGAxW368kDHVKwBUgQFnoECCMQAQ', 'displayed_link': '20+ likes · 10 years ago', 'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTvQ15vycYUI7sdgtJ8C7NqHceXosxN8StEqQQXdfNoUOb7LVHQzeYe&usqp=CAE&s', 'favicon': 'https://serpapi.com/searches/6677fd329c449d91fbb435ed/images/b02c5b5912906a98de2e27f3943b5beb5d1c71d09df2ca0a500872db5732f37b.png', 'snippet': '"The street is the best way to become a good footballer." - Michel Platini.', 'snippet_highlighted_words': ['Platini'], 'source': 'X · FootballQuote_'}], 'pagination': {'current': 1, 'next': 'https://www.google.com/search?q=Platini+Quotes&oq=Platini+Quotes&hl=en&gl=us&start=10&sourceid=chrome&ie=UTF-8', 'other_pages': {'2': 'https://www.google.com/search?q=Platini+Quotes&oq=Platini+Quotes&hl=en&gl=us&start=10&sourceid=chrome&ie=UTF-8', '3': 'https://www.google.com/search?q=Platini+Quotes&oq=Platini+Quotes&hl=en&gl=us&start=20&sourceid=chrome&ie=UTF-8', '4': 'https://www.google.com/search?q=Platini+Quotes&oq=Platini+Quotes&hl=en&gl=us&start=30&sourceid=chrome&ie=UTF-8', '5': 'https://www.google.com/search?q=Platini+Quotes&oq=Platini+Quotes&hl=en&gl=us&start=40&sourceid=chrome&ie=UTF-8'}}, 'serpapi_pagination': {'current': 1, 'next_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+Quotes&start=10', 'next': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+Quotes&start=10', 'other_pages': {'2': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+Quotes&start=10', '3': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+Quotes&start=20', '4': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+Quotes&start=30', '5': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+Quotes&start=40'}}}}), H2Title(title='Platini Career Highlights', content=None, h3_titles=[H3Title(title='Bonini Luiz-Koopmeiners Juve Chiesa Platini presidente', content=None, web_search={'generated_query': 'Bonini Luiz-Koopmeiners Juve Chiesa Platini presidente', 'results': {'search_metadata': {'id': '6677fd762348848db8e3cf1d', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/09da1436d2b0d774/6677fd762348848db8e3cf1d.json', 'created_at': '2024-06-23 10:48:22 UTC', 'processed_at': '2024-06-23 10:48:22 UTC', 'google_url': 'https://www.google.com/search?q=Bonini+Luiz-Koopmeiners+Juve+Chiesa+Platini+presidente&oq=Bonini+Luiz-Koopmeiners+Juve+Chiesa+Platini+presidente&hl=en&gl=us&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/09da1436d2b0d774/6677fd762348848db8e3cf1d.html', 'total_time_taken': 0.87}, 'search_parameters': {'engine': 'google', 'q': 'Bonini Luiz-Koopmeiners Juve Chiesa Platini presidente', 'google_domain': 'google.com', 'hl': 'en', 'gl': 'us', 'device': 'desktop'}, 'search_information': {'query_displayed': 'Bonini Luiz-Koopmeiners Juve Chiesa Platini presidente', 'total_results': 9, 'time_taken_displayed': 0.34, 'organic_results_state': 'Results for exact spelling'}, 'inline_images': [{'link': 'https://www.bianconeranews.it/podcast/bonini-a-rbn-luiz-koopmeiners-top-juve-tieni-chiesa-platini-presidente-75484', 'source': 'https://www.bianconeranews.it/podcast/bonini-a-rbn-luiz-koopmeiners-top-juve-tieni-chiesa-platini-presidente-75484', 'thumbnail': 'https://serpapi.com/searches/6677fd762348848db8e3cf1d/images/f4ab6e891a53b31750d0aba6b59411ec82b9128430512dfcd1061f0a457b0437.jpeg', 'original': 'https://net-storage.tcccdn.com/storage/bianconeranews.it/img_notizie/thumb3/49/49733201b57c1ef34b89fcaf7463bb94-56931-oooz0000.jpeg', 'title': 'Bonini a RBN: “Luiz-Koopmeiners top. Juve tieni Chiesa ...', 'source_name': 'Bianconera News'}, {'link': 'https://sports.yahoo.com/koopmeiners-juventus-ready-submit-offer-092800255.html', 'source': 'https://sports.yahoo.com/koopmeiners-juventus-ready-submit-offer-092800255.html', 'thumbnail': 'https://serpapi.com/searches/6677fd762348848db8e3cf1d/images/f4ab6e891a53b31762edc18cb55da1a5ce3461009d391a27a19b18ddf5a303e7.jpeg', 'original': 'https://media.zenfs.com/en/football_italia_articles_132/f2ca9099c96751c0af8ee1d66111a135', 'title': 'Koopmeiners: Juventus ready to submit new offer to Atalanta ...', 'source_name': 'Yahoo Sports'}, {'link': 'https://onefootball.com/en/news/koopmeiners-is-waiting-for-juventus-as-the-bianconeri-struggle-with-douglas-luiz-deal-39646699', 'source': 'https://onefootball.com/en/news/koopmeiners-is-waiting-for-juventus-as-the-bianconeri-struggle-with-douglas-luiz-deal-39646699', 'thumbnail': 'https://serpapi.com/searches/6677fd762348848db8e3cf1d/images/f4ab6e891a53b3171a75a679f1b80489737ec6f2a13b95dc47fe182be5c2b73e.jpeg', 'original': 'https://image-service.onefootball.com/transform?w=280&h=210&dpr=2&image=https%3A%2F%2Ficdn.juvefc.com%2Fwp-content%2Fuploads%2F2024%2F03%2F3568869-72686548-2560-1440.jpg', 'title': 'Koopmeiners is waiting for Juventus as the Bianconeri ...', 'source_name': 'OneFootball'}], 'organic_results': [{'position': 1, 'title': 'Luiz-Koopmeiners top. Juve tieni Chiesa. Platini presidente', 'link': 'https://www.bianconeranews.it/podcast/bonini-a-rbn-luiz-koopmeiners-top-juve-tieni-chiesa-platini-presidente-75484', 'redirect_link': 'https://www.google.com/url?sa=t&url=https://www.bianconeranews.it/podcast/bonini-a-rbn-luiz-koopmeiners-top-juve-tieni-chiesa-platini-presidente-75484&usg=AOvVaw2_b8JsosgwAZqIvhBBKjoc', 'displayed_link': 'https://www.bianconeranews.it › bo...', 'favicon': 'https://serpapi.com/searches/6677fd762348848db8e3cf1d/images/e7263791ccb8417ec03ace71fc299ab58cfffce7f326ccff3ee7a384af9166a4.webp', 'snippet': 'Bonini a RBN: “Luiz-Koopmeiners top. Juve tieni Chiesa. Platini presidente”. Podcast. 21.06.2024 10:30 di Quintiliano Giampietro vedi letture.', 'snippet_highlighted_words': ['21.06.2024 10:30'], 'source': 'Bianconera News'}, {'position': 2, 'title': 'Bonini a RBN: “Luiz e Koopmeiners tanta roba. Motta ...', 'link': 'https://www.tuttojuve.com/podcast-tj/bonini-a-rbn-luiz-e-koopmeiners-tanta-roba-motta-scelta-giustissima-platini-presidente-della-juve-692047', 'redirect_link': 'https://www.google.com/url?sa=t&url=https://www.tuttojuve.com/podcast-tj/bonini-a-rbn-luiz-e-koopmeiners-tanta-roba-motta-scelta-giustissima-platini-presidente-della-juve-692047&usg=AOvVaw1B-NheijDleo30dWGrqjJ3', 'displayed_link': 'https://www.tuttojuve.com › bonini...', 'favicon': 'https://serpapi.com/searches/6677fd762348848db8e3cf1d/images/e7263791ccb8417ec03ace71fc299ab5da1af642a09e18fb10c589d72fd21c70.webp', 'snippet': 'Bonini a RBN: “Luiz e Koopmeiners tanta roba. Motta scelta giustissima. Platini? Presidente della Juve ”. 21.06.2024 11:00 di Quintiliano Giampietro', 'snippet_highlighted_words': ['21.06.2024 11:00'], 'source': 'Tutto Juve'}, {'position': 3, 'title': 'Chiesa, tre formazioni dI Premier su di lui', 'link': 'https://www.tuttojuve.com/calciomercato/chiesa-tre-formazioni-di-premier-su-di-lui-690345', 'redirect_link': 'https://www.google.com/url?sa=t&url=https://www.tuttojuve.com/calciomercato/chiesa-tre-formazioni-di-premier-su-di-lui-690345&usg=AOvVaw2YFa0YIxPE2w4iIIGcGKmw', 'displayed_link': 'https://www.tuttojuve.com › chiesa-...', 'favicon': 'https://serpapi.com/searches/6677fd762348848db8e3cf1d/images/e7263791ccb8417ec03ace71fc299ab599eb8caa51896d5554e533e66e74a31b.webp', 'date': 'Jun 7, 2024', 'snippet': 'Chiesa, tre formazioni dI Premier su di lui ... Bonini a RBN: “Luiz e Koopmeiners tanta roba. Motta scelta giustissima. Platini? Presidente della ...', 'snippet_highlighted_words': ['Chiesa', 'Bonini', 'Luiz', 'Koopmeiners', 'Platini', 'Presidente'], 'source': 'Tutto Juve'}, {'position': 4, 'title': 'Platini: “John Elkann ci tiene e la Juve tornerà vincente. Io ...', 'link': 'https://www.bianconeranews.it/primo-piano/platini-john-elkann-ci-tiene-e-la-juve-tornera-vincente-io-presidente-74178', 'redirect_link': 'https://www.google.com/url?sa=t&url=https://www.bianconeranews.it/primo-piano/platini-john-elkann-ci-tiene-e-la-juve-tornera-vincente-io-presidente-74178&usg=AOvVaw02vVdxziGHkWVFQ4USBrB-', 'displayed_link': 'https://www.bianconeranews.it › pla...', 'favicon': 'https://serpapi.com/searches/6677fd762348848db8e3cf1d/images/e7263791ccb8417ec03ace71fc299ab5e529488c1d8fef33eb8a95e6820f776c.webp', 'date': 'Jun 3, 2024', 'snippet': 'Platini parla della Juve e, convinto che ... Bonini a RBN: “Luiz-Koopmeiners top. Juve tieni Chiesa. Platini presidente” 10:15 Primo piano Juve ...', 'snippet_highlighted_words': ['10:15 Primo'], 'source': 'Bianconera News'}, {'position': 5, 'title': 'Massimo Bonini', 'link': 'https://en.wikipedia.org/wiki/Massimo_Bonini', 'redirect_link': 'https://www.google.com/url?sa=t&url=https://en.wikipedia.org/wiki/Massimo_Bonini&usg=AOvVaw1NdCKVu0nhUBMaOlAvJjIL', 'displayed_link': 'https://en.wikipedia.org › wiki › Massimo_Bonini', 'favicon': 'https://serpapi.com/searches/6677fd762348848db8e3cf1d/images/e7263791ccb8417ec03ace71fc299ab5734adb3fe130e0d45dd4e7cbf92bd63e.webp', 'snippet': 'Massimo Bonini (born 13 October 1959) is a Sammarinese former professional football player and coach, who played as a midfielder for Italian sides Bellaria ...', 'snippet_highlighted_words': ['Massimo Bonini'], 'missing': ['Koopmeiners', 'presidente'], 'source': 'Wikipedia'}, {'position': 6, 'title': "Bonini: «Alla Juve manca il gioco. Risultati negativi? C'è ...", 'link': 'https://www.juventusnews24.com/bonini-juve-crisi-gioco-risultati-negativi-scudetto-parole-dichiarazioni-intervista-esclusiva/', 'redirect_link': 'https://www.google.com/url?sa=t&url=https://www.juventusnews24.com/bonini-juve-crisi-gioco-risultati-negativi-scudetto-parole-dichiarazioni-intervista-esclusiva/&usg=AOvVaw1ffRl-aajCdL0VnSaIWvOd', 'displayed_link': 'https://www.juventusnews24.com › ...', 'favicon': 'https://serpapi.com/searches/6677fd762348848db8e3cf1d/images/e7263791ccb8417ec03ace71fc299ab541924f6b778907ac50c805b89691c02f.webp', 'date': 'Feb 22, 2024', 'snippet': "Massimo Bonini, in esclusiva per Juventusnews24, ha così parlato dell'attuale momento che sta vivendo la formazione di Allegri. Una squadra che, ...", 'snippet_highlighted_words': ['Bonini'], 'source': 'Juventus News 24'}, {'position': 7, 'title': "Chiesa Juve, l'ex bianconero ha dubbi: «Non convince in ...", 'link': 'https://www.juventusnews24.com/bonini-juve-chiesa-modulo-allegri-parole-intervista-ex-centrocampista/', 'redirect_link': 'https://www.google.com/url?sa=t&url=https://www.juventusnews24.com/bonini-juve-chiesa-modulo-allegri-parole-intervista-ex-centrocampista/&usg=AOvVaw06EKE-VO__T6-KWdqG7i9t', 'displayed_link': 'https://www.juventusnews24.com › ...', 'favicon': 'https://serpapi.com/searches/6677fd762348848db8e3cf1d/images/e7263791ccb8417ec03ace71fc299ab5c6d7b05ca0611b6fce0c10b01e55cb57.webp', 'date': 'Mar 24, 2024', 'snippet': "... Koopmeiners con Douglas Luiz, cala l'ottimismo sul rinnovo di Rabiot ... Massimo Bonini, ex centrocampista della Juve, analizza così alcuni ...", 'snippet_highlighted_words': ['Koopmeiners', 'Luiz', 'Bonini', 'Juve'], 'source': 'Juventus News 24'}, {'position': 8, 'title': 'La grande storia del calcio | ✅️ Nel 1988 ha allenato il Cagliari ...', 'link': 'https://m.facebook.com/groups/408670337078409/posts/1161298688482233/', 'redirect_link': 'https://www.google.com/url?sa=t&url=https://m.facebook.com/groups/408670337078409/posts/1161298688482233/&usg=AOvVaw1n1QsU3yPECTKr4KDPDzM-', 'displayed_link': 'https://m.facebook.com › groups › posts', 'favicon': 'https://serpapi.com/searches/6677fd762348848db8e3cf1d/images/e7263791ccb8417ec03ace71fc299ab52d9bac05979815897d7d5343f162f42e.webp', 'snippet': 'Bonini, F. Emoli, R. Bentancur, S. Khedira, G ... Chiesa, Douglas Costa, G. Menichelli, G. Stacchini, F ... #juve #juventus #ajax #amsterdam #calcio', 'snippet_highlighted_words': ['Bonini', 'Chiesa', 'juve', 'juventus'], 'source': 'Facebook'}, {'position': 9, 'title': 'La grande storia del calcio | https://youtu. be/mI7OZZGoglU', 'link': 'https://m.facebook.com/groups/408670337078409/posts/1094861781792591/', 'redirect_link': 'https://www.google.com/url?sa=t&url=https://m.facebook.com/groups/408670337078409/posts/1094861781792591/&usg=AOvVaw3BtYoY0XGuWCfZiHYBQf-A', 'displayed_link': 'https://m.facebook.com › groups › posts', 'favicon': 'https://serpapi.com/searches/6677fd762348848db8e3cf1d/images/e7263791ccb8417ec03ace71fc299ab57b4c27cb7c300aeb22b784515c40bf9f.webp', 'snippet': "Bonini, F. Emoli, R. Bentancur, S. Khedira, G ... Chiesa, Douglas Costa, G. Menichelli, G ... Juve del Quinquennio d'Oro, di Combi, Rosetta e", 'snippet_highlighted_words': ['Bonini', 'Chiesa', 'Juve'], 'source': 'Facebook'}], 'dmca_messages': {'title': 'Notices about Filtered Results', 'messages': [{'content': 'In order to show you the most relevant results, we have omitted some entries very similar to the 9 already displayed.If you like, you can repeat the search with the omitted results included.', 'highlighted_words': [{'link': 'https://www.google.com/search?q=Bonini+Luiz-Koopmeiners+Juve+Chiesa+Platini+presidente&sca_esv=8f0324ad6f2b8a65&sca_upv=1&gl=us&hl=en&filter=0', 'text': 'repeat the search with the omitted results included'}]}]}, 'pagination': {'current': 1, 'next': 'https://www.google.com/search?q=Bonini+Luiz-Koopmeiners+Juve+Chiesa+Platini+presidente&oq=Bonini+Luiz-Koopmeiners+Juve+Chiesa+Platini+presidente&hl=en&gl=us&start=10&sourceid=chrome&ie=UTF-8', 'other_pages': {'2': 'https://www.google.com/search?q=Bonini+Luiz-Koopmeiners+Juve+Chiesa+Platini+presidente&oq=Bonini+Luiz-Koopmeiners+Juve+Chiesa+Platini+presidente&hl=en&gl=us&start=10&sourceid=chrome&ie=UTF-8', '3': 'https://www.google.com/search?q=Bonini+Luiz-Koopmeiners+Juve+Chiesa+Platini+presidente&oq=Bonini+Luiz-Koopmeiners+Juve+Chiesa+Platini+presidente&hl=en&gl=us&start=20&sourceid=chrome&ie=UTF-8', '4': 'https://www.google.com/search?q=Bonini+Luiz-Koopmeiners+Juve+Chiesa+Platini+presidente&oq=Bonini+Luiz-Koopmeiners+Juve+Chiesa+Platini+presidente&hl=en&gl=us&start=30&sourceid=chrome&ie=UTF-8', '5': 'https://www.google.com/search?q=Bonini+Luiz-Koopmeiners+Juve+Chiesa+Platini+presidente&oq=Bonini+Luiz-Koopmeiners+Juve+Chiesa+Platini+presidente&hl=en&gl=us&start=40&sourceid=chrome&ie=UTF-8'}}, 'serpapi_pagination': {'current': 1, 'next_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Bonini+Luiz-Koopmeiners+Juve+Chiesa+Platini+presidente&start=10', 'next': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Bonini+Luiz-Koopmeiners+Juve+Chiesa+Platini+presidente&start=10', 'other_pages': {'2': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Bonini+Luiz-Koopmeiners+Juve+Chiesa+Platini+presidente&start=10', '3': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Bonini+Luiz-Koopmeiners+Juve+Chiesa+Platini+presidente&start=20', '4': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Bonini+Luiz-Koopmeiners+Juve+Chiesa+Platini+presidente&start=30', '5': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Bonini+Luiz-Koopmeiners+Juve+Chiesa+Platini+presidente&start=40'}}}}), H3Title(title='Michel Platini career highlights', content=None, web_search={'generated_query': 'Michel Platini career highlights', 'results': {'search_metadata': {'id': '6677fdd38ccee0eba895a9fe', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/3da1829b265e7cd4/6677fdd38ccee0eba895a9fe.json', 'created_at': '2024-06-23 10:49:55 UTC', 'processed_at': '2024-06-23 10:49:55 UTC', 'google_url': 'https://www.google.com/search?q=Michel+Platini+career+highlights&oq=Michel+Platini+career+highlights&hl=en&gl=us&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/3da1829b265e7cd4/6677fdd38ccee0eba895a9fe.html', 'total_time_taken': 1.22}, 'search_parameters': {'engine': 'google', 'q': 'Michel Platini career highlights', 'google_domain': 'google.com', 'hl': 'en', 'gl': 'us', 'device': 'desktop'}, 'search_information': {'query_displayed': 'Michel Platini career highlights', 'total_results': 101000, 'time_taken_displayed': 0.3, 'organic_results_state': 'Results for exact spelling'}, 'sports_results': {'title': 'Michel Platini', 'team': 'Former president of the UEFA', 'tables': [{'title': 'France · World Cup', 'games': [{'title': '19861986', 'matches_games_played': '66', 'goals_goals': '22', 'assists_assists': '--'}, {'title': '19821982', 'matches_games_played': '55', 'goals_goals': '22', 'assists_assists': '--'}, {'title': '19781978', 'matches_games_played': '33', 'goals_goals': '11', 'assists_assists': '--'}]}]}, 'knowledge_graph': {'title': 'Michel Platini', 'type': 'Former president of the UEFA', 'entity_type': 'people, athlete, soccer_player, people', 'main_tab_text': 'Overview', 'kgmid': '/m/01fdsm', 'knowledge_graph_search_link': 'https://www.google.com/search?kgmid=/m/01fdsm&hl=en-US&q=Michel+Platini&kgs=b5b2ad910c0470f4&shndl=17&shem=ssim&source=sh/x/kp/osrp/m5/1', 'serpapi_knowledge_graph_search_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en-US&kgmid=%2Fm%2F01fdsm&q=Michel+Platini', 'tabs': [{'text': 'Stats', 'si': 'ACC90ny9-jaHPXqh7KxEKXGGccqzWWM698HRflpADe-WgJzacnRWT5cGDCKpwjgXe1-Mo7yKjPKSWOAR9JHS4jgbVPCRhSHRU8BECSQW1igCjNP9_s0OHXde5y9El_1oQzD0odhjJCRD', 'link': 'https://www.google.com/search?si=ACC90ny9-jaHPXqh7KxEKXGGccqzWWM698HRflpADe-WgJzacnRWT5cGDCKpwjgXe1-Mo7yKjPKSWOAR9JHS4jgbVPCRhSHRU8BECSQW1igCjNP9_s0OHXde5y9El_1oQzD0odhjJCRD', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Michel+Platini+career+highlights&si=ACC90ny9-jaHPXqh7KxEKXGGccqzWWM698HRflpADe-WgJzacnRWT5cGDCKpwjgXe1-Mo7yKjPKSWOAR9JHS4jgbVPCRhSHRU8BECSQW1igCjNP9_s0OHXde5y9El_1oQzD0odhjJCRD'}, {'text': 'Career', 'si': 'ACC90nz4lwyS23sm8hcph5fuM7ckJRDUKweKVcOYR8DyxN4kUsudyH6ZoJlxyWz0RbIAI53QWfZZa2SOzX0duqk8Zytx-LBJrKwlVAshPQgBGtFwfk7ifdhqrVd5DDH-JIAfQ-Vwmttk8ghTSEJ6cjvUVxSwAS-7lnbbKgZ2SWYOUEW-iD7Q9Wk=', 'link': 'https://www.google.com/search?si=ACC90nz4lwyS23sm8hcph5fuM7ckJRDUKweKVcOYR8DyxN4kUsudyH6ZoJlxyWz0RbIAI53QWfZZa2SOzX0duqk8Zytx-LBJrKwlVAshPQgBGtFwfk7ifdhqrVd5DDH-JIAfQ-Vwmttk8ghTSEJ6cjvUVxSwAS-7lnbbKgZ2SWYOUEW-iD7Q9Wk=', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Michel+Platini+career+highlights&si=ACC90nz4lwyS23sm8hcph5fuM7ckJRDUKweKVcOYR8DyxN4kUsudyH6ZoJlxyWz0RbIAI53QWfZZa2SOzX0duqk8Zytx-LBJrKwlVAshPQgBGtFwfk7ifdhqrVd5DDH-JIAfQ-Vwmttk8ghTSEJ6cjvUVxSwAS-7lnbbKgZ2SWYOUEW-iD7Q9Wk%3D'}, {'text': 'Songs', 'si': 'ACC90nyR35EXIVqsrAONouAC2If7im8eG9AWdkeY7WEHZ67Ix1jLqk_EMnamLYhwJRXmtD9Ue2jE_uJyDpQ56VIzIFKRON52uCnDA7Y-JTWphL0p_Q6mkvtIw1lbsmn_sNPow8zcnO1J', 'link': 'https://www.google.com/search?si=ACC90nyR35EXIVqsrAONouAC2If7im8eG9AWdkeY7WEHZ67Ix1jLqk_EMnamLYhwJRXmtD9Ue2jE_uJyDpQ56VIzIFKRON52uCnDA7Y-JTWphL0p_Q6mkvtIw1lbsmn_sNPow8zcnO1J', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Michel+Platini+career+highlights&si=ACC90nyR35EXIVqsrAONouAC2If7im8eG9AWdkeY7WEHZ67Ix1jLqk_EMnamLYhwJRXmtD9Ue2jE_uJyDpQ56VIzIFKRON52uCnDA7Y-JTWphL0p_Q6mkvtIw1lbsmn_sNPow8zcnO1J'}, {'text': 'Videos', 'si': 'ACC90nxAHrInVEcjasiFOJd3T0BSx1l8-3GnIvMMlxgboSkRXQHdYeVd1MFAuxgjiTvJGCGzAZuajXgGVfKG_lp3F5NPN28FC_yyZJ7zYBQeujgkftzHyYsz1dLuVjG1LVtTzeghdhcn', 'link': 'https://www.google.com/search?si=ACC90nxAHrInVEcjasiFOJd3T0BSx1l8-3GnIvMMlxgboSkRXQHdYeVd1MFAuxgjiTvJGCGzAZuajXgGVfKG_lp3F5NPN28FC_yyZJ7zYBQeujgkftzHyYsz1dLuVjG1LVtTzeghdhcn', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Michel+Platini+career+highlights&si=ACC90nxAHrInVEcjasiFOJd3T0BSx1l8-3GnIvMMlxgboSkRXQHdYeVd1MFAuxgjiTvJGCGzAZuajXgGVfKG_lp3F5NPN28FC_yyZJ7zYBQeujgkftzHyYsz1dLuVjG1LVtTzeghdhcn'}, {'text': 'Listen', 'si': 'ACC90nylNfUkJaxznrzSwClq9nw0u5DVmerFU1ZS4WsDMouPEMnzG8ReT5iL8vphLl8-P45M045uFTLorHkPq6YVBMkw13fn8QY1MIa-RQE6bNNXcMli4N0Do9Rd6S3DdTvXfVtyCIRr', 'link': 'https://www.google.com/search?si=ACC90nylNfUkJaxznrzSwClq9nw0u5DVmerFU1ZS4WsDMouPEMnzG8ReT5iL8vphLl8-P45M045uFTLorHkPq6YVBMkw13fn8QY1MIa-RQE6bNNXcMli4N0Do9Rd6S3DdTvXfVtyCIRr', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Michel+Platini+career+highlights&si=ACC90nylNfUkJaxznrzSwClq9nw0u5DVmerFU1ZS4WsDMouPEMnzG8ReT5iL8vphLl8-P45M045uFTLorHkPq6YVBMkw13fn8QY1MIa-RQE6bNNXcMli4N0Do9Rd6S3DdTvXfVtyCIRr'}], 'header_images': [{'image': 'https://serpapi.com/searches/6677fdd38ccee0eba895a9fe/images/dea6603174d7b3616af815240197535b3acc769f1d0e379c3a841555b933b5e1c3846e6659369eb7.webp'}, {'image': 'https://serpapi.com/searches/6677fdd38ccee0eba895a9fe/images/dea6603174d7b3616af815240197535b3acc769f1d0e379c47953ab2917a107f1899121a86bb8299.webp'}, {'image': 'https://serpapi.com/searches/6677fdd38ccee0eba895a9fe/images/dea6603174d7b3616af815240197535b3acc769f1d0e379c40ee00b10cac6255e14a873ec0ba34e9.webp'}], 'description': "Michel François Platini is a French football administrator and former player and manager. Regarded as one of the greatest footballers of all time, Platini won the Ballon d'Or three times in a row, in 1983, 1984 and 1985, and came seventh in the FIFA Player of the Century vote.", 'source': {'name': 'Wikipedia', 'link': 'https://en.wikipedia.org/wiki/Michel_Platini'}, 'born': '1955 (age 69 years), Jœuf, France', 'born_links': [{'text': 'Jœuf, France', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=J%C5%93uf&si=ACC90nzx_D3_zUKRnpAjmO0UBLNxnt7EyN4YYdru6U3bxLI-LxvQdBRRSnQQlq7vIxw88DJUHTO63N009AlYiTewIE3c91x7FGA2Thx5A77bgpEf9jH7RPeki1QZqxUgbMyuyGDwnrWZ8nu4-0rU7qpI4rMa6zL1jbEuTwDT7ho6LX74GjxqOR8DDv3Up7JBsX9EZqMugaW9&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQmxMoAHoECBgQAg'}], 'dates_joined': '1982 (Juventus F.C.), 1976 (France national football team), 1972 (AS Nancy Lorraine)', 'dates_joined_links': [{'text': 'Juventus F.C.', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Juventus&si=ACC90nyvvWro6QmnyY1IfSdgk5wwjB1r8BGd_IWRjXqmKPQqm076E0SqGvBZ--FsN5xL5jFyoBDfUEBT56M47ItDximwbf8ypERic8qGdsjcDRn1eVbqqHdlRsRmooT3CRMqHXq6CnP_cSIxwQMiGQHu3tt69_5ThLko2SL60vVDGpyshggnUs_e19D7H48bfq0U6ukVovIsAWfN6R8KPqET8lgKOIMGCuM9XDPfUSO9NvnBcSS_czaJZJYj8ImeyUl-bV1iJt8PbWcSBPhMWnfTX8PKpUEZPw%3D%3D&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQmxMoAHoECBsQAg'}, {'text': 'France national football team', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=France+national+football+team&si=ACC90nzx_D3_zUKRnpAjmO0UBLNxnt7EyN4YYdru6U3bxLI-L3o9p6uSFMcOhGy_sKj525KYh9KnhjJ5s6XHxwLhy3nmVuca0YEahqRfhjgXM7fsIXMH7rHWtAkgijlvVKjk-rB3hdqeut2_9ppemer7QJc58NIXgTSrT9JwEIEl_ExXMVGdFcdu8M7mU3fgFoIZzWmjfye9CPGa9hcz7amSz9Xba4cFAzomUA2C_ArTzGSiof5vAUSwGpzM0JvXtGVF0WU5ZwfGzN8U-3qZbjfZVNeyGI0TPX3M60pp7CJyoc-VHLnJjT4%3D&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQmxMoAXoECBsQAw'}, {'text': 'AS Nancy Lorraine', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=AS+Nancy+Lorraine&si=ACC90nzx_D3_zUKRnpAjmO0UBLNxnt7EyN4YYdru6U3bxLI-L4igLxRnCsxOQ8dTh_h4Ajbfat76fjM1MuA4ZNJiT8X2Axq8X10yOQay4c0ZINEiob0DsHi3gEbdPFk9MvGN80HgOAJbBxO_ltCh_aEEbn63ofjHV1DZxWudhqvskYHVlcnJIzPKUXxNt4Gbj3V-oLMMH0wEnJwlgHAAfi9yTm-g0o63H2KRbZX9Un5nO9baeCvZvoyuyaB8B3kHlNvoleMuWmNjZvVpjdg3tQN5ZFitYLpAIg%3D%3D&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQmxMoAnoECBsQBA'}], 'position': 'Midfielder', 'position_links': [{'text': 'Midfielder', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=&si=ACC90nzx_D3_zUKRnpAjmO0UBLNxnt7EyN4YYdru6U3bxLI-L7WAdTL-YwWnQpdY4j2K-lFe6PkhNVQynek3Z4mN3IUjnI2V27wjtzeMfLdwkyyququ-YjNz2xISAI0Yj4t6JJM6AxKEK2iW0D-aPoEgApturWjG4Z1k25NQ8uD4sHY4cr3Wh78%3D&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQmxMoAHoECBkQAg'}], 'parents': 'Aldo Platini, Anna Platini', 'parents_links': [{'text': 'Aldo Platini', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Aldo+Platini&si=ACC90nxYhNno81_TzuVO0e1EieRzQwXK2DrwuMdAl3IyVcpNQ4FyHQ0IG7ls3z9f0-A7XqNVR7rMHHkXB-pKywLzoMGuQOnD6AWi2_pIZRb_7EOcdsUJAHeYX2LDhemq0xafPhoe0-m3nsZ0JMI0Av0hPL3UUapoVIYZPCE9nP4dvr_Whdg9MLNIqFrTGk4GHbYgS5c1iV2vGykdtyBIn_GQC4238BQUlNh2WSrlGs-_1Jrsad6k4le_-QkLeWiQHks1K5WwRwaRntgaHrnG3l2olAhpy8gDIw%3D%3D&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQmxMoAHoECCEQAg'}, {'text': 'Anna Platini', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Anna+Platini&si=ACC90nxYhNno81_TzuVO0e1EieRzQwXK2DrwuMdAl3IyVcpNQz0RKrNvs_mMGtn16CmvjDPU4aCMjQ8ohDqWxxHbYJPVrH4gwg6KtvQy3Ta9tAPXL3x0LjUVMOrJJ7ag7uSJGN72ELiUu621PHg0vWt8DccsPgs9awpabXQ6W1ce0V60HSJstKxp03-W9r_w3WLLDYk55hEQxFYgPftFj5iMpRItS4Rx2VRZRAHe9yynIeWpOBKR-hH9rtJ4QFlAOUp6tny8RCZRpD_CRikThaqQ__WY9cNsiA%3D%3D&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQmxMoAXoECCEQAw'}], 'spouse': 'Christelle Platini (m. 1977)', 'spouse_links': [{'text': 'Christelle Platini', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Christelle+Platini&si=ACC90ny8E30vD16OoPAAI4cStfcliGy35W8UAhb0TsHNc_ISQewA087VSDWbyOz5-3RS_0CVXF1JcD4_90KwGkF2IdpGNSFOHwwJ07IDvuoM3TOBhN2eeGEtKfYi2eb6380eRscsYLAfC6Ut1nBsLN_bE5kWGQ_wrcHp7bIysQ4E-GqhViHZ73lS0WiZCzFD0aJxlyuMMyrVYGdND7usWm_HLc-Q3ScHqg%3D%3D&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQmxMoAHoECBoQAg'}], 'children': 'Laurent Platini, Marine Platini', 'children_links': [{'text': 'Laurent Platini', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Laurent+Platini&si=ACC90nwLLwns5sISZcdzuISy7t-NHozt8Cbt6G3WNQfC9ekAgAuCKhUGcOl2a35eG0h_haGSCG09Ngl684dSvpax23l8hQAyxldnMV_jZpCTRNMQ6DCJO71-hb5RVHycB2ikVJ4XDRq8jAMljrbsf1zy4OtY4EPIcR-Pk2rMpa5jSbIzCMKYE3XaK2N04vlQhCEDBafUDRgiR0kaXcqi5WEnPw4w5yhoauSw3W6K_jAyyz_m6D08_k-IX_zp_W3X_9y626eep-UNmCUnzxqichfQhUL6nkax4A%3D%3D&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQmxMoAHoECBcQAg'}, {'text': 'Marine Platini', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Marine+Platini&si=ACC90nwLLwns5sISZcdzuISy7t-NHozt8Cbt6G3WNQfC9ekAgLL5wXOoGx8qgyyuHBItiDzKPXD_rdY16qGcpU59HclAfgdz0CYflEv6_IBPhrmODpCwRiODvtm3R_SuzKm4cWbKdedHMN6T1TAuRYJEf5UkNLOEUe_-Mk3gHF7KJHXmcm3Bkxh1Rw5z7RXq0fdCMU8Myoh4Qf9VVgnEOsTD-Ia_6f8PBWg-NlnjE46jTCrR2G5FLcm0fB9-qs9bUC1K8L_Y4LZ9N6eXJxppxRIo26mzhUeJ9A%3D%3D&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQmxMoAXoECBcQAw'}], 'web_results': [{'title': 'Age', 'snippet': '69 yearsJun 21, 1955', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=michel+platini+age&stick=H4sIAAAAAAAAAOPgE-LUz9U3MExLKc7VEs1OttIvSM0vyEkFUkXF-XlWiempi1iFcjOTM1JzFApyEksy8zIVgIIAe1MZZzkAAAA&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQ18AJegQIHxAB'}, {'title': 'Dates joined', 'snippet': '1982 (Juventus F.C.), 1976 (France national football team), 1972 (AS Nancy Lorraine)', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=michel+platini+dates+joined&stick=H4sIAAAAAAAAAOPgE-LUz9U3MExLKc7VUs4ot9JPzs_JSU0uyczP008sychJLUkttirITM5OTVFISSxJXcQqnZuZnJGao1CQk1iSmZcJFi1WyMrPzEtNAQDkaj3pUAAAAA&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQ18AJegQIIxAB'}, {'thumbnail': 'https://serpapi.com/searches/6677fdd38ccee0eba895a9fe/images/dea6603174d7b3616af815240197535ba775736d623e0f5c5997123e0cf700271324830e2f97b04c.jpeg', 'title': "The Art of Scoring: Michel Platini's Unforgettable Goals with ...", 'source': 'YouTube • Juventus', 'source_image': 'https://serpapi.com/searches/6677fdd38ccee0eba895a9fe/images/dea6603174d7b3616af815240197535ba775736d623e0f5c9e28cf711b8eb1c72a4eb454e77809a5f15c34a65b1d312a.png', 'link': 'https://www.youtube.com/watch?v=lSnnmFI9cRM', 'duration': '3:13'}], 'listen': [{'name': 'Spotify', 'link': 'https://open.spotify.com/artist/6xma1aDFf3bz3c5HGRIlVr?autoplay=true', 'image': 'https://serpapi.com/searches/6677fdd38ccee0eba895a9fe/images/dea6603174d7b3616af815240197535b65c7ddb81e87ba9e901adaa8bd64cba8.png'}, {'name': 'Apple Music', 'link': 'https://music.apple.com/us/artist/platini/1718461656', 'image': 'https://serpapi.com/searches/6677fdd38ccee0eba895a9fe/images/dea6603174d7b3616af815240197535b79eeeefba8728104258046a056cb407a.png'}, {'name': 'Pandora', 'link': 'http://www.pandora.com/?sc=miR10880634', 'image': 'https://serpapi.com/searches/6677fdd38ccee0eba895a9fe/images/dea6603174d7b3616af815240197535b2c2ca9677c49ce627b66d7873f84ed05.png'}], 'profiles': [{'name': 'YouTube', 'link': 'https://www.youtube.com/channel/UCsSClQ7XXRWuN0VQElQhvgQ', 'image': 'https://serpapi.com/searches/6677fdd38ccee0eba895a9fe/images/dea6603174d7b3616af815240197535b744538fc1c6a8271b46534490259053954a1a23c2780b1a1.png'}, {'name': 'Instagram', 'link': 'https://www.instagram.com/michelplatiniofficiel/', 'image': 'https://serpapi.com/searches/6677fdd38ccee0eba895a9fe/images/dea6603174d7b3616af815240197535b744538fc1c6a82715be3b464608578fd1eb4b0c3c5f18f09.png'}], 'songs': [{'name': 'Hier encore', 'extensions': 'Les Enfoirés au Grand Rex · 1994', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=les+enfoir%C3%A9s+hier+encore&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tFP1zcsNM1KKilLNtcS9C0tzkx2LCrJLC4JyQ_Oz0tfxCqZk1qskJqXlp9ZdHhlsUJGZmoRkJucX5QKADafiSlKAAAA&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQri56BAh_EAM', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=les+enfoir%C3%A9s+hier+encore&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tFP1zcsNM1KKilLNtcS9C0tzkx2LCrJLC4JyQ_Oz0tfxCqZk1qskJqXlp9ZdHhlsUJGZmoRkJucX5QKADafiSlKAAAA'}, {'name': "Jusqu'au dernier", 'extensions': 'Les Enfoirés 2024, On a 35 ans ! · 2024', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=les+enfoir%C3%A9s+jusqu%27au+dernier&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tVP1zc0rDTKyMrIMkjSEvQtLc5MdiwqySwuCckPzs9LX8Qql5NarJCal5afWXR4ZbFCVmlxYal6YqlCSmpRXmZqEQAFTZYRUAAAAA&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQri56BAh_EAU', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=les+enfoir%C3%A9s+jusqu%27au+dernier&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tVP1zc0rDTKyMrIMkjSEvQtLc5MdiwqySwuCckPzs9LX8Qql5NarJCal5afWXR4ZbFCVmlxYal6YqlCSmpRXmZqEQAFTZYRUAAAAA'}, {'name': 'La chanson des restos', 'extensions': "Tournée d'Enfoirés · 1989", 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=les+enfoir%C3%A9s+la+chanson+des+restos&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tFP1zcsNjAtMbQ0qdQS9C0tzkx2LCrJLC4JyQ_Oz0tfxKqck1qskJqXlp9ZdHhlsUJOokJyRmJecX6eQgpQoii1uCS_GAC9YiMSVAAAAA&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQri56BAh_EAc', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=les+enfoir%C3%A9s+la+chanson+des+restos&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tFP1zcsNjAtMbQ0qdQS9C0tzkx2LCrJLC4JyQ_Oz0tfxKqck1qskJqXlp9ZdHhlsUJOokJyRmJecX6eQgpQoii1uCS_GAC9YiMSVAAAAA', 'image': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTwT5M_653obvZxnU9qzAjBeA4Dqc4sdte-L3aa&s=0'}, {'name': 'Rêvons', 'extensions': '2023 Enfoirés un jour, toujours · 2023', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=les+enfoir%C3%A9s+r%C3%AAvons&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tVP1zc0LE4pT89NL0nTEvQtLc5MdiwqySwuCckPzs9LX8QqmpNarJCal5afWXR4ZbFC0eFVZfl5xQClALwHRwAAAA&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQri56BAh_EAo', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=les+enfoir%C3%A9s+r%C3%AAvons&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tVP1zc0LE4pT89NL0nTEvQtLc5MdiwqySwuCckPzs9LX8QqmpNarJCal5afWXR4ZbFC0eFVZfl5xQClALwHRwAAAA'}, {'name': 'On trace', 'extensions': 'Le Monde des Enfoirés · 2019', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=les+enfoir%C3%A9s+on+trace&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tVP1zc0TMs2jrcoqErTEvQtLc5MdiwqySwuCckPzs9LX8QqlpNarJCal5afWXR4ZbFCfp5CSVFicioA_CA3sEgAAAA&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQri56BAh_EAw', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=les+enfoir%C3%A9s+on+trace&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tVP1zc0TMs2jrcoqErTEvQtLc5MdiwqySwuCckPzs9LX8QqlpNarJCal5afWXR4ZbFCfp5CSVFicioA_CA3sEgAAAA', 'image': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT8SedPpjcZYaV5ugkIABl45sb5k2doQ-tG8x06&s=0'}, {'name': 'Attention au départ', 'extensions': 'La boîte à musique des Enfoirés (Live) · 2013', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=les+enfoir%C3%A9s+attention+au+d%C3%A9part&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tVP1zc0TDLPqzIyMbHUEvQtLc5MdiwqySwuCckPzs9LX8SqlJNarJCal5afWXR4ZbFCYklJal5JZn6eQmKpQsrhlQWJRSUA2dckZlQAAAA&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQri56BAh_EA8', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=les+enfoir%C3%A9s+attention+au+d%C3%A9part&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tVP1zc0TDLPqzIyMbHUEvQtLc5MdiwqySwuCckPzs9LX8SqlJNarJCal5afWXR4ZbFCYklJal5JZn6eQmKpQsrhlQWJRSUA2dckZlQAAAA', 'image': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRGPBd3Xh6WW60zSKgnJmhTxQa5ZrEUlBAQ1PP6&s=0'}, {'name': "Ella, elle l'a", 'extensions': 'Les Enfoirés au Grand Rex · 1994', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=les+enfoir%C3%A9s+ella,+elle+l%27a&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tFP1zcsNM1KSTIvr9IS9C0tzkx2LCrJLC4JyQ_Oz0tfxCqTk1qskJqXlp9ZdHglkJWTk6gDIlMVctQTAeERV4ZNAAAA&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQri56BAh_EBI', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=les+enfoir%C3%A9s+ella%2C+elle+l%27a&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tFP1zcsNM1KSTIvr9IS9C0tzkx2LCrJLC4JyQ_Oz0tfxCqTk1qskJqXlp9ZdHglkJWTk6gDIlMVctQTAeERV4ZNAAAA', 'image': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR6UfegyONC_nUo01dXgu74I1qAusHCF7ORDVqp&s=0'}, {'name': 'Popcorn salé', 'extensions': 'Les Enfoirés 2024, On a 35 ans ! · 2024', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=les+enfoir%C3%A9s+popcorn+sal%C3%A9&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tVP1zc0zEkpSi5IqajQEvQtLc5MdiwqySwuCckPzs9LX8QqnZNarJCal5afWXR4ZbFCQX5Bcn5RnkJxYs7hlQDiKVveTQAAAA&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQri56BAh_EBU', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=les+enfoir%C3%A9s+popcorn+sal%C3%A9&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tVP1zc0zEkpSi5IqajQEvQtLc5MdiwqySwuCckPzs9LX8QqnZNarJCal5afWXR4ZbFCQX5Bcn5RnkJxYs7hlQDiKVveTQAAAA'}]}, 'inline_videos': [{'position': 1, 'title': 'Michel Platini, Le Roi [Goals & Skills]', 'link': 'https://www.youtube.com/watch?v=Ws6QQC7fmBE', 'thumbnail': 'https://serpapi.com/searches/6677fdd38ccee0eba895a9fe/images/448c3ccb01d81ad1c7df5e5a9e5ead8d6771c185ecfe62e6892274a985338e13.jpeg', 'channel': 'Barbosa Fútbol Videos', 'duration': '8:42', 'platform': 'YouTube', 'date': 'Oct 14, 2017'}, {'position': 2, 'title': 'Michel Platini [Best Skills & Goals]', 'link': 'https://www.youtube.com/watch?v=Eq0wUKgETek', 'thumbnail': 'https://serpapi.com/searches/6677fdd38ccee0eba895a9fe/images/448c3ccb01d81ad1e17ca1ec4f88c6d920205c3004064f98b7712105bd822359.jpeg', 'channel': 'VM Soccer Legends', 'duration': '9:08', 'platform': 'YouTube', 'date': 'Jun 20, 2022'}, {'position': 3, 'title': "The Art of Scoring: Michel Platini's Unforgettable Goals with ...", 'link': 'https://www.youtube.com/watch?v=lSnnmFI9cRM', 'thumbnail': 'https://serpapi.com/searches/6677fdd38ccee0eba895a9fe/images/448c3ccb01d81ad1fb61751f3fdc06bc29da0b6188e854ae6ec659a668adf957.jpeg', 'channel': 'Juventus', 'duration': '3:13', 'platform': 'YouTube', 'date': 'Jun 21, 2023'}, {'position': 4, 'title': 'Michel Platini | Top 10 goals', 'link': 'https://www.youtube.com/watch?v=E84oxh3gmLs', 'thumbnail': 'https://serpapi.com/searches/6677fdd38ccee0eba895a9fe/images/448c3ccb01d81ad11eb2eb76b06879dabc64b15bab8602ef963261bba455b602.jpeg', 'channel': 'Legendary Football', 'duration': '2:01', 'platform': 'YouTube', 'date': 'Feb 15, 2023'}], 'related_questions': [{'question': 'What is Michel Platini famous for?', 'snippet': 'Michel Platini (born June 21, 1955, Joeuf, France) is a French professional football (soccer) player and administrator who was named the European Footballer of the Year three times (1983–85) and served as president of the Union of European Football Associations (UEFA; 2007–16).', 'title': 'Michel Platini | Biography, FIFA, & Facts - Britannica', 'date': '6 days ago', 'link': 'https://www.britannica.com/biography/Michel-Platini', 'displayed_link': 'https://www.britannica.com › ... › Other Sports', 'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTwT7h-5f4paCrJ8dGAAGz-CoYR3HTkumhV0fDpGI5Ytw&s', 'source_logo': 'https://serpapi.com/searches/6677fdd38ccee0eba895a9fe/images/7bc24e74d18188dd260881528d38efcc5215878b5f3e8c2b97c7d35b6eb87863.webp', 'next_page_token': 'eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210UFUwNXNjVWRSVUV0TWVERjVkelZxZWpWR01WazVaRVZYYVZCd2NYbGpPRlF4Y2xFME5HbHFXR1ZoTFRBM2MzRlRWM05XT0dsamNFUmxVMm93V1hKR1NGQnNVWE42RWhjeFVERXpXbkpFTFVkT2FVcHJVRWxRZUY5TExUUkJNQm9pUVVaWWNrVmpjVlZpYVZvNE1UWm5URFpZWW05MWNUWk5ZV2x4UlZKZmEwVXRkdyIsImZjdiI6IjMiLCJlaSI6IjFQMTNackQtR05pSmtQSVB4X0stNEEwIiwicWMiOiJDaUJ0YVdOb1pXd2djR3hoZEdsdWFTQmpZWEpsWlhJZ2FHbG5hR3hwWjJoMGN4QUFmVnlrOUQ0IiwicXVlc3Rpb24iOiJXaGF0IGlzIE1pY2hlbCBQbGF0aW5pIGZhbW91cyBmb3I/IiwibGsiOiJHaUYzYUdGMElHbHpJRzFwWTJobGJDQndiR0YwYVc1cElHWmhiVzkxY3lCbWIzSSIsImJzIjoiYzNYUU1RckNNQlFHWUZ3ek9aWk9yM1VWbFY2Z3E2Q0NtLUNneERZMUQ5b2tKSkdnZF9GV2dwZndFS1lxa1ZMZHZfOV9QNDhzU2JyaDFBSWFXR0hCV1Ezcm1sb1VDQlZ0NU1sQUpYVWUzVzAyaWhQM2djMGJxaDRrRlpuT3BZT0dpak1jSmEwTmxGaUdSbE5JelFBRmNGOVNVTTJZcjc0bDJTeWU4SDVLX1UtUlJidGFncU1HRHN4YXBzZmhpTlN3eFpJS2xrZlhmWmJHNERvdzFIcDNlVG5pWHhCR1d5MFZSOWJkN1ZEazBXUFh2b0RfaE9vTGg0TW4iLCJpZCI6ImZjXzUifQ==', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google_related_questions&google_domain=google.com&next_page_token=eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210UFUwNXNjVWRSVUV0TWVERjVkelZxZWpWR01WazVaRVZYYVZCd2NYbGpPRlF4Y2xFME5HbHFXR1ZoTFRBM2MzRlRWM05XT0dsamNFUmxVMm93V1hKR1NGQnNVWE42RWhjeFVERXpXbkpFTFVkT2FVcHJVRWxRZUY5TExUUkJNQm9pUVVaWWNrVmpjVlZpYVZvNE1UWm5URFpZWW05MWNUWk5ZV2x4UlZKZmEwVXRkdyIsImZjdiI6IjMiLCJlaSI6IjFQMTNackQtR05pSmtQSVB4X0stNEEwIiwicWMiOiJDaUJ0YVdOb1pXd2djR3hoZEdsdWFTQmpZWEpsWlhJZ2FHbG5hR3hwWjJoMGN4QUFmVnlrOUQ0IiwicXVlc3Rpb24iOiJXaGF0IGlzIE1pY2hlbCBQbGF0aW5pIGZhbW91cyBmb3I%2FIiwibGsiOiJHaUYzYUdGMElHbHpJRzFwWTJobGJDQndiR0YwYVc1cElHWmhiVzkxY3lCbWIzSSIsImJzIjoiYzNYUU1RckNNQlFHWUZ3ek9aWk9yM1VWbFY2Z3E2Q0NtLUNneERZMUQ5b2tKSkdnZF9GV2dwZndFS1lxa1ZMZHZfOV9QNDhzU2JyaDFBSWFXR0hCV1Ezcm1sb1VDQlZ0NU1sQUpYVWUzVzAyaWhQM2djMGJxaDRrRlpuT3BZT0dpak1jSmEwTmxGaUdSbE5JelFBRmNGOVNVTTJZcjc0bDJTeWU4SDVLX1UtUlJidGFncU1HRHN4YXBzZmhpTlN3eFpJS2xrZlhmWmJHNERvdzFIcDNlVG5pWHhCR1d5MFZSOWJkN1ZEazBXUFh2b0RfaE9vTGg0TW4iLCJpZCI6ImZjXzUifQ%3D%3D'}, {'question': 'How many goals did Platini score in his career?', 'title': 'Career stats', 'link': 'https://www.transfermarkt.co.uk/michel-platini/leistungsdaten/spieler/88994', 'displayed_link': 'https://www.transfermarkt.co.uk › leistungsdaten › spieler', 'source_logo': 'https://serpapi.com/searches/6677fdd38ccee0eba895a9fe/images/7bc24e74d18188dd260881528d38efcc7159d314eaef949a367eb36f0275e72f.png', 'next_page_token': 'eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210UFUwNXNjVWRSVUV0TWVERjVkelZxZWpWR01WazVaRVZYYVZCd2NYbGpPRlF4Y2xFME5HbHFXR1ZoTFRBM2MzRlRWM05XT0dsamNFUmxVMm93V1hKR1NGQnNVWE42RWhjeFVERXpXbkpFTFVkT2FVcHJVRWxRZUY5TExUUkJNQm9pUVVaWWNrVmpjVlZpYVZvNE1UWm5URFpZWW05MWNUWk5ZV2x4UlZKZmEwVXRkdyIsImZjdiI6IjMiLCJlaSI6IjFQMTNackQtR05pSmtQSVB4X0stNEEwIiwicWMiOiJDaUJ0YVdOb1pXd2djR3hoZEdsdWFTQmpZWEpsWlhJZ2FHbG5hR3hwWjJoMGN4QUFmVnlrOUQ0IiwicXVlc3Rpb24iOiJIb3cgbWFueSBnb2FscyBkaWQgUGxhdGluaSBzY29yZSBpbiBoaXMgY2FyZWVyPyIsImxrIjoiR2k1b2IzY2diV0Z1ZVNCbmIyRnNjeUJrYVdRZ2NHeGhkR2x1YVNCelkyOXlaU0JwYmlCb2FYTWdZMkZ5WldWeSIsImJzIjoiYzNYUU1RckNNQlFHWUZ3ek9aWk9yM1VWbFY2Z3E2Q0NtLUNneERZMUQ5b2tKSkdnZF9GV2dwZndFS1lxa1ZMZHZfOV9QNDhzU2JyaDFBSWFXR0hCV1Ezcm1sb1VDQlZ0NU1sQUpYVWUzVzAyaWhQM2djMGJxaDRrRlpuT3BZT0dpak1jSmEwTmxGaUdSbE5JelFBRmNGOVNVTTJZcjc0bDJTeWU4SDVLX1UtUlJidGFncU1HRHN4YXBzZmhpTlN3eFpJS2xrZlhmWmJHNERvdzFIcDNlVG5pWHhCR1d5MFZSOWJkN1ZEazBXUFh2b0RfaE9vTGg0TW4iLCJpZCI6ImZjXzUifQ==', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google_related_questions&google_domain=google.com&next_page_token=eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210UFUwNXNjVWRSVUV0TWVERjVkelZxZWpWR01WazVaRVZYYVZCd2NYbGpPRlF4Y2xFME5HbHFXR1ZoTFRBM2MzRlRWM05XT0dsamNFUmxVMm93V1hKR1NGQnNVWE42RWhjeFVERXpXbkpFTFVkT2FVcHJVRWxRZUY5TExUUkJNQm9pUVVaWWNrVmpjVlZpYVZvNE1UWm5URFpZWW05MWNUWk5ZV2x4UlZKZmEwVXRkdyIsImZjdiI6IjMiLCJlaSI6IjFQMTNackQtR05pSmtQSVB4X0stNEEwIiwicWMiOiJDaUJ0YVdOb1pXd2djR3hoZEdsdWFTQmpZWEpsWlhJZ2FHbG5hR3hwWjJoMGN4QUFmVnlrOUQ0IiwicXVlc3Rpb24iOiJIb3cgbWFueSBnb2FscyBkaWQgUGxhdGluaSBzY29yZSBpbiBoaXMgY2FyZWVyPyIsImxrIjoiR2k1b2IzY2diV0Z1ZVNCbmIyRnNjeUJrYVdRZ2NHeGhkR2x1YVNCelkyOXlaU0JwYmlCb2FYTWdZMkZ5WldWeSIsImJzIjoiYzNYUU1RckNNQlFHWUZ3ek9aWk9yM1VWbFY2Z3E2Q0NtLUNneERZMUQ5b2tKSkdnZF9GV2dwZndFS1lxa1ZMZHZfOV9QNDhzU2JyaDFBSWFXR0hCV1Ezcm1sb1VDQlZ0NU1sQUpYVWUzVzAyaWhQM2djMGJxaDRrRlpuT3BZT0dpak1jSmEwTmxGaUdSbE5JelFBRmNGOVNVTTJZcjc0bDJTeWU4SDVLX1UtUlJidGFncU1HRHN4YXBzZmhpTlN3eFpJS2xrZlhmWmJHNERvdzFIcDNlVG5pWHhCR1d5MFZSOWJkN1ZEazBXUFh2b0RfaE9vTGg0TW4iLCJpZCI6ImZjXzUifQ%3D%3D'}, {'question': 'Who was better, Platini or Zidane?', 'snippet': 'Platini was a better player relative to his era but on paper, Zidane had a better and more complete career. Same amount of Champions League/European Cups but Zidane had tougher opponents. Platini scored a penalty in the final and Zidane scored one of the best CL goals ever in the final. Both made the final twice.', 'title': "Hot take: Platini's career was better and more complete then Zidane's", 'date': 'Jul 26, 2023', 'link': 'https://www.reddit.com/r/football/comments/159ywh9/hot_take_platinis_career_was_better_and_more/', 'displayed_link': 'https://www.reddit.com › football › comments › hot_tak...', 'source_logo': 'https://serpapi.com/searches/6677fdd38ccee0eba895a9fe/images/7bc24e74d18188dd260881528d38efcc10b310ef1c1214d9db3a2ba1df20947b.png', 'next_page_token': 'eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210UFUwNXNjVWRSVUV0TWVERjVkelZxZWpWR01WazVaRVZYYVZCd2NYbGpPRlF4Y2xFME5HbHFXR1ZoTFRBM2MzRlRWM05XT0dsamNFUmxVMm93V1hKR1NGQnNVWE42RWhjeFVERXpXbkpFTFVkT2FVcHJVRWxRZUY5TExUUkJNQm9pUVVaWWNrVmpjVlZpYVZvNE1UWm5URFpZWW05MWNUWk5ZV2x4UlZKZmEwVXRkdyIsImZjdiI6IjMiLCJlaSI6IjFQMTNackQtR05pSmtQSVB4X0stNEEwIiwicWMiOiJDaUJ0YVdOb1pXd2djR3hoZEdsdWFTQmpZWEpsWlhJZ2FHbG5hR3hwWjJoMGN4QUFmVnlrOUQ0IiwicXVlc3Rpb24iOiJXaG8gd2FzIGJldHRlciwgUGxhdGluaSBvciBaaWRhbmU/IiwibGsiOiJHaUIzYUc4Z2QyRnpJR0psZEhSbGNpQndiR0YwYVc1cElHOXlJSHBwWkdGdVpRIiwiYnMiOiJjM1hRTVFyQ01CUUdZRnd6T1paT3IzVVZsVjZncTZDQ20tQ2d4RFkxRDlva0pKR2dkX0ZXZ3Bmd0VLWXFrVkxkdl85X1A0OHNTYnJoMUFJYVdHSEJXUTNybWxvVUNCVnQ1TWxBSlhVZTNXMDJpaFAzZ2MwYnFoNGtGWm5PcFlPR2lqTWNKYTBObEZpR1JsTkl6UUFGY0Y5U1VNMllyNzRsMlN5ZThINUtfVS1SUmJ0YWdxTUdEc3hhcHNmaGlOU3d4WklLbGtmWGZaYkc0RG93MUhwM2VUbmlYeEJHV3kwVlI5YmQ3VkRrMFdQWHZvRF9oT29MaDRNbiIsImlkIjoiZmNfNSJ9', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google_related_questions&google_domain=google.com&next_page_token=eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210UFUwNXNjVWRSVUV0TWVERjVkelZxZWpWR01WazVaRVZYYVZCd2NYbGpPRlF4Y2xFME5HbHFXR1ZoTFRBM2MzRlRWM05XT0dsamNFUmxVMm93V1hKR1NGQnNVWE42RWhjeFVERXpXbkpFTFVkT2FVcHJVRWxRZUY5TExUUkJNQm9pUVVaWWNrVmpjVlZpYVZvNE1UWm5URFpZWW05MWNUWk5ZV2x4UlZKZmEwVXRkdyIsImZjdiI6IjMiLCJlaSI6IjFQMTNackQtR05pSmtQSVB4X0stNEEwIiwicWMiOiJDaUJ0YVdOb1pXd2djR3hoZEdsdWFTQmpZWEpsWlhJZ2FHbG5hR3hwWjJoMGN4QUFmVnlrOUQ0IiwicXVlc3Rpb24iOiJXaG8gd2FzIGJldHRlciwgUGxhdGluaSBvciBaaWRhbmU%2FIiwibGsiOiJHaUIzYUc4Z2QyRnpJR0psZEhSbGNpQndiR0YwYVc1cElHOXlJSHBwWkdGdVpRIiwiYnMiOiJjM1hRTVFyQ01CUUdZRnd6T1paT3IzVVZsVjZncTZDQ20tQ2d4RFkxRDlva0pKR2dkX0ZXZ3Bmd0VLWXFrVkxkdl85X1A0OHNTYnJoMUFJYVdHSEJXUTNybWxvVUNCVnQ1TWxBSlhVZTNXMDJpaFAzZ2MwYnFoNGtGWm5PcFlPR2lqTWNKYTBObEZpR1JsTkl6UUFGY0Y5U1VNMllyNzRsMlN5ZThINUtfVS1SUmJ0YWdxTUdEc3hhcHNmaGlOU3d4WklLbGtmWGZaYkc0RG93MUhwM2VUbmlYeEJHV3kwVlI5YmQ3VkRrMFdQWHZvRF9oT29MaDRNbiIsImlkIjoiZmNfNSJ9'}, {'question': 'How many trophies did Platini win?', 'snippet': "He won the Serie A title with Juventus in 1984 and 1986, the European Cup Winners' Cup in 1984 (setting up Vignola's opening goal in the Turin club's 2–1 victory over Porto in the final of the competition), the 1984 European Super Cup, the European Cup in 1985 and the 1985 Intercontinental Cup.", 'title': 'Michel Platini: The Maestro of French Football. - LinkedIn', 'date': 'Jun 4, 2023', 'link': 'https://www.linkedin.com/pulse/michel-platini-maestro-french-football-javad-ghorbani', 'displayed_link': 'https://www.linkedin.com › pulse › michel-platini-maestr...', 'source_logo': 'https://serpapi.com/searches/6677fdd38ccee0eba895a9fe/images/7bc24e74d18188dd260881528d38efcc649556577d536c9900afd1e4a9320d78.png', 'next_page_token': 'eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210UFUwNXNjVWRSVUV0TWVERjVkelZxZWpWR01WazVaRVZYYVZCd2NYbGpPRlF4Y2xFME5HbHFXR1ZoTFRBM2MzRlRWM05XT0dsamNFUmxVMm93V1hKR1NGQnNVWE42RWhjeFVERXpXbkpFTFVkT2FVcHJVRWxRZUY5TExUUkJNQm9pUVVaWWNrVmpjVlZpYVZvNE1UWm5URFpZWW05MWNUWk5ZV2x4UlZKZmEwVXRkdyIsImZjdiI6IjMiLCJlaSI6IjFQMTNackQtR05pSmtQSVB4X0stNEEwIiwicWMiOiJDaUJ0YVdOb1pXd2djR3hoZEdsdWFTQmpZWEpsWlhJZ2FHbG5hR3hwWjJoMGN4QUFmVnlrOUQ0IiwicXVlc3Rpb24iOiJIb3cgbWFueSB0cm9waGllcyBkaWQgUGxhdGluaSB3aW4/IiwibGsiOiJHaUZvYjNjZ2JXRnVlU0IwY205d2FHbGxjeUJrYVdRZ2NHeGhkR2x1YVNCM2FXNCIsImJzIjoiYzNYUU1RckNNQlFHWUZ3ek9aWk9yM1VWbFY2Z3E2Q0NtLUNneERZMUQ5b2tKSkdnZF9GV2dwZndFS1lxa1ZMZHZfOV9QNDhzU2JyaDFBSWFXR0hCV1Ezcm1sb1VDQlZ0NU1sQUpYVWUzVzAyaWhQM2djMGJxaDRrRlpuT3BZT0dpak1jSmEwTmxGaUdSbE5JelFBRmNGOVNVTTJZcjc0bDJTeWU4SDVLX1UtUlJidGFncU1HRHN4YXBzZmhpTlN3eFpJS2xrZlhmWmJHNERvdzFIcDNlVG5pWHhCR1d5MFZSOWJkN1ZEazBXUFh2b0RfaE9vTGg0TW4iLCJpZCI6ImZjXzUifQ==', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google_related_questions&google_domain=google.com&next_page_token=eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210UFUwNXNjVWRSVUV0TWVERjVkelZxZWpWR01WazVaRVZYYVZCd2NYbGpPRlF4Y2xFME5HbHFXR1ZoTFRBM2MzRlRWM05XT0dsamNFUmxVMm93V1hKR1NGQnNVWE42RWhjeFVERXpXbkpFTFVkT2FVcHJVRWxRZUY5TExUUkJNQm9pUVVaWWNrVmpjVlZpYVZvNE1UWm5URFpZWW05MWNUWk5ZV2x4UlZKZmEwVXRkdyIsImZjdiI6IjMiLCJlaSI6IjFQMTNackQtR05pSmtQSVB4X0stNEEwIiwicWMiOiJDaUJ0YVdOb1pXd2djR3hoZEdsdWFTQmpZWEpsWlhJZ2FHbG5hR3hwWjJoMGN4QUFmVnlrOUQ0IiwicXVlc3Rpb24iOiJIb3cgbWFueSB0cm9waGllcyBkaWQgUGxhdGluaSB3aW4%2FIiwibGsiOiJHaUZvYjNjZ2JXRnVlU0IwY205d2FHbGxjeUJrYVdRZ2NHeGhkR2x1YVNCM2FXNCIsImJzIjoiYzNYUU1RckNNQlFHWUZ3ek9aWk9yM1VWbFY2Z3E2Q0NtLUNneERZMUQ5b2tKSkdnZF9GV2dwZndFS1lxa1ZMZHZfOV9QNDhzU2JyaDFBSWFXR0hCV1Ezcm1sb1VDQlZ0NU1sQUpYVWUzVzAyaWhQM2djMGJxaDRrRlpuT3BZT0dpak1jSmEwTmxGaUdSbE5JelFBRmNGOVNVTTJZcjc0bDJTeWU4SDVLX1UtUlJidGFncU1HRHN4YXBzZmhpTlN3eFpJS2xrZlhmWmJHNERvdzFIcDNlVG5pWHhCR1d5MFZSOWJkN1ZEazBXUFh2b0RfaE9vTGg0TW4iLCJpZCI6ImZjXzUifQ%3D%3D'}], 'organic_results': [{'position': 1, 'title': 'Michel Platini', 'link': 'https://en.wikipedia.org/wiki/Michel_Platini', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://en.wikipedia.org/wiki/Michel_Platini&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQFnoECEsQAQ&usg=AOvVaw0TVBgm3Q_6A-CURhLIQWhw', 'displayed_link': 'https://en.wikipedia.org › wiki › Michel_Platini', 'thumbnail': 'https://serpapi.com/searches/6677fdd38ccee0eba895a9fe/images/ead709b343488184720a55c8a0843d70eff3db658d6907d75f08950ba7dacc1b.jpeg', 'favicon': 'https://serpapi.com/searches/6677fdd38ccee0eba895a9fe/images/ead709b343488184720a55c8a0843d70333d6ccd6eaf96c7bde9e8cc2ea6a293.webp', 'snippet': "Regarded as one of the greatest footballers of all time, Platini won the Ballon d'Or three times in a row, in 1983, 1984 and 1985, and came seventh in the FIFA ...", 'snippet_highlighted_words': ['Platini'], 'source': 'Wikipedia'}, {'position': 2, 'title': 'Michel Platini | Biography, FIFA, & Facts', 'link': 'https://www.britannica.com/biography/Michel-Platini', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.britannica.com/biography/Michel-Platini&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQFnoECEkQAQ&usg=AOvVaw0mEX4HX-10Jr3FSggZv78L', 'displayed_link': 'https://www.britannica.com › ... › Other Sports', 'thumbnail': 'https://serpapi.com/searches/6677fdd38ccee0eba895a9fe/images/ead709b343488184720a55c8a0843d70a446851e3404fe2d8d0e6e85cfd1f9d8.jpeg', 'favicon': 'https://serpapi.com/searches/6677fdd38ccee0eba895a9fe/images/ead709b343488184720a55c8a0843d70d657781ad85c9c95b77d03e454984115.webp', 'date': '6 days ago', 'snippet': 'Michel Platini, French professional football (soccer) player who was a three-time European Footballer of the Year (1983–85) and president of ...', 'snippet_highlighted_words': ['Michel Platini', 'professional'], 'source': 'Britannica'}, {'position': 3, 'title': 'Michel Platini biography and career accomplishments', 'link': 'https://www.footballhistory.org/player/michel-platini.html', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.footballhistory.org/player/michel-platini.html&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQFnoECG4QAQ&usg=AOvVaw0q5yyHp8dLU8bRX8JL_4UB', 'displayed_link': 'https://www.footballhistory.org › player › michel-platini', 'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSP8_rq-Mvf4hyRQNzcazgxxmNiYTZDlFS1HRr3jhKYqZNJvTn6QhO8&usqp=CAE&s', 'favicon': 'https://serpapi.com/searches/6677fdd38ccee0eba895a9fe/images/ead709b343488184720a55c8a0843d70058dbc761ed6044cefb14dc8e8b26e0c.webp', 'snippet': 'The footbal icon Michel Platini has served as a successful offensive midfielder, captain as well as manager of the French team. After his career he did become ...', 'snippet_highlighted_words': ['Michel Platini', 'career'], 'source': 'The history of football (soccer)'}, {'position': 4, 'title': 'MICHEAL PLATINI Football Career Achievements and ...', 'link': 'https://medium.com/@mondebaron/michel-platini-a-french-football-legend-stands-among-the-greatest-players-in-history-8b2f89a4abe6', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://medium.com/%40mondebaron/michel-platini-a-french-football-legend-stands-among-the-greatest-players-in-history-8b2f89a4abe6&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQFnoECEwQAQ&usg=AOvVaw0mePWYdDP0djXHlMChjotA', 'displayed_link': '5 months ago', 'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTc4xsVXyuLuuAt6zQgCbvR20Tow-16XuVxFyLJGHkNUd4SL4qYB1kg&usqp=CAE&s', 'favicon': 'https://serpapi.com/searches/6677fdd38ccee0eba895a9fe/images/ead709b343488184720a55c8a0843d70f5639cbf2b4655ec74596cdb9b13ba88.webp', 'snippet': "His remarkable career includes three consecutive Ballon d'Or wins and leading France to victory in the 1984 European Championship. Beyond ...", 'snippet_highlighted_words': ['career'], 'source': 'Medium · Mondebaron'}, {'position': 5, 'title': 'Michel Platini - Titles & achievements - Transfermarkt', 'link': 'https://www.transfermarkt.us/michel-platini/erfolge/spieler/88994', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.transfermarkt.us/michel-platini/erfolge/spieler/88994&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQFnoECEoQAQ&usg=AOvVaw32sJRO5FIcmYPnw1Wlzk3H', 'displayed_link': 'https://www.transfermarkt.us › erfolge › spieler', 'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSVsQ_zdqfxE4zIElOyq_Vxxlrvn0wDFpbB_u9fEST4ZHqYCv64Rqu3&usqp=CAE&s', 'favicon': 'https://serpapi.com/searches/6677fdd38ccee0eba895a9fe/images/ead709b343488184720a55c8a0843d70be8c5d1c7b470ff10a786bf9de32a2d2.webp', 'snippet': "Michel Platini ; 3x Winner Ballon d'Or. Winner Ballon d'Or. 1985 · 2x Footballer of the Year ; 4x Top goal scorer. Top goal scorer. 84/85, Serie A - 18 Goals · 1x ...", 'snippet_highlighted_words': ['Michel Platini', 'Goals'], 'source': 'transfermarkt.us'}, {'position': 6, 'title': "Legends Revisited: Michel Platini - France's Greatest Player?", 'link': 'https://bleacherreport.com/articles/262769-legends-revisited-the-life-and-times-of-michel-platini', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://bleacherreport.com/articles/262769-legends-revisited-the-life-and-times-of-michel-platini&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQFnoECE8QAQ&usg=AOvVaw0KDvj0SP75rmpsq9vaOaG5', 'displayed_link': 'https://bleacherreport.com › World Football › Juventus', 'favicon': 'https://serpapi.com/searches/6677fdd38ccee0eba895a9fe/images/ead709b343488184720a55c8a0843d7013fc8b31d5ce22c5844e5d983e2e0735.webp', 'date': 'Sep 28, 2009', 'snippet': 'Michel Platini came to national repute playing for AS Nancy in the late 70s by scoring 98 goals from 181 appearances in midfield, during a ...', 'snippet_highlighted_words': ['Michel Platini', 'goals'], 'source': 'Bleacher Report'}, {'position': 7, 'title': 'A few career goals from Michel Platini', 'link': 'https://www.youtube.com/watch?v=PWzTttpBBxE', 'displayed_link': '760+ views · 2 months ago', 'thumbnail': 'https://i.ytimg.com/vi/PWzTttpBBxE/mqdefault.jpg?sqp=-oaymwEFCJQBEFM&rs=AMzJL3k4dhhlJTow3XJ-eMIFHhjlQywk-A', 'favicon': 'https://serpapi.com/searches/6677fdd38ccee0eba895a9fe/images/ead709b343488184720a55c8a0843d70fd12eac5730598ea70d6f8e47e56d319.png', 'snippet': 'Michel Platini A French midfielder who played for Nancy, Saint-Étienne and Juventus. He scored over 250 goals during his career so here are ...', 'missing': ['highlights'], 'must_include': {'word': 'highlights', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Michel+Platini+career+%22highlights%22&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQ5t4CegQIVxAT'}, 'video_link': 'https://encrypted-vtbn0.gstatic.com/video?q=tbn:ANd9GcSFBtGDjR5If5Iz_eJenlfle-RhHFfC72XqnA', 'source': "YouTube · Now That's What I Call a Goal"}], 'related_searches': [{'block_position': 1, 'query': 'michel platini clubs', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Michel+Platini+clubs&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQ1QJ6BAhGEAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Michel+Platini+clubs'}, {'block_position': 1, 'query': 'michel platini trophies', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Michel+Platini+trophies&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQ1QJ6BAhFEAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Michel+Platini+trophies'}, {'block_position': 1, 'query': 'michel platini age', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Michel+platini+age&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQ1QJ6BAhEEAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Michel+platini+age'}, {'block_position': 1, 'query': 'michel platini net worth', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Michel+Platini+Net+worth&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQ1QJ6BAhDEAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Michel+Platini+Net+worth'}, {'block_position': 1, 'query': 'sports career of michel platini', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Sports+career+of+Michel+Platini&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQ1QJ6BAhCEAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Sports+career+of+Michel+Platini'}, {'block_position': 1, 'query': 'michel platini fifa card', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Michel+Platini+FIFA+card&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQ1QJ6BAhBEAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Michel+Platini+FIFA+card'}, {'block_position': 1, 'query': 'michel platini number', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Michel+platini+number&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQ1QJ6BAhAEAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Michel+platini+number'}, {'block_position': 1, 'query': 'michel platini retirement', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Michel+Platini+retirement&sa=X&ved=2ahUKEwiwuInnxvGGAxXYBEQIHUe5D9wQ1QJ6BAg-EAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Michel+Platini+retirement'}], 'pagination': {'current': 1, 'next': 'https://www.google.com/search?q=Michel+Platini+career+highlights&oq=Michel+Platini+career+highlights&hl=en&gl=us&start=10&sourceid=chrome&ie=UTF-8', 'other_pages': {'2': 'https://www.google.com/search?q=Michel+Platini+career+highlights&oq=Michel+Platini+career+highlights&hl=en&gl=us&start=10&sourceid=chrome&ie=UTF-8', '3': 'https://www.google.com/search?q=Michel+Platini+career+highlights&oq=Michel+Platini+career+highlights&hl=en&gl=us&start=20&sourceid=chrome&ie=UTF-8', '4': 'https://www.google.com/search?q=Michel+Platini+career+highlights&oq=Michel+Platini+career+highlights&hl=en&gl=us&start=30&sourceid=chrome&ie=UTF-8', '5': 'https://www.google.com/search?q=Michel+Platini+career+highlights&oq=Michel+Platini+career+highlights&hl=en&gl=us&start=40&sourceid=chrome&ie=UTF-8'}}, 'serpapi_pagination': {'current': 1, 'next_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Michel+Platini+career+highlights&start=10', 'next': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Michel+Platini+career+highlights&start=10', 'other_pages': {'2': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Michel+Platini+career+highlights&start=10', '3': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Michel+Platini+career+highlights&start=20', '4': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Michel+Platini+career+highlights&start=30', '5': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Michel+Platini+career+highlights&start=40'}}}})], web_search={'generated_query': 'Platini career highlights', 'results': {'search_metadata': {'id': '6677fd5f5f22034182fc0927', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/33387a73a5e0dec3/6677fd5f5f22034182fc0927.json', 'created_at': '2024-06-23 10:47:59 UTC', 'processed_at': '2024-06-23 10:47:59 UTC', 'google_url': 'https://www.google.com/search?q=Platini+career+highlights&oq=Platini+career+highlights&hl=en&gl=us&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/33387a73a5e0dec3/6677fd5f5f22034182fc0927.html', 'total_time_taken': 1.1}, 'search_parameters': {'engine': 'google', 'q': 'Platini career highlights', 'google_domain': 'google.com', 'hl': 'en', 'gl': 'us', 'device': 'desktop'}, 'search_information': {'query_displayed': 'Platini career highlights', 'total_results': 243000, 'time_taken_displayed': 0.36, 'organic_results_state': 'Results for exact spelling'}, 'sports_results': {'title': 'Michel Platini', 'team': 'Former president of the UEFA', 'tables': [{'title': 'France · World Cup', 'games': [{'title': '19861986', 'matches_games_played': '66', 'goals_goals': '22', 'assists_assists': '--'}, {'title': '19821982', 'matches_games_played': '55', 'goals_goals': '22', 'assists_assists': '--'}, {'title': '19781978', 'matches_games_played': '33', 'goals_goals': '11', 'assists_assists': '--'}]}]}, 'knowledge_graph': {'title': 'Michel Platini', 'type': 'Former president of the UEFA', 'entity_type': 'people, athlete, soccer_player, people', 'main_tab_text': 'Overview', 'kgmid': '/m/01fdsm', 'knowledge_graph_search_link': 'https://www.google.com/search?kgmid=/m/01fdsm&hl=en-US&q=Michel+Platini&kgs=4e61ede91f0d880e&shndl=17&shem=ssim&source=sh/x/kp/osrp/m5/1', 'serpapi_knowledge_graph_search_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en-US&kgmid=%2Fm%2F01fdsm&q=Michel+Platini', 'tabs': [{'text': 'Stats', 'si': 'ACC90ny9-jaHPXqh7KxEKXGGccqzWWM698HRflpADe-WgJzacnRWT5cGDCKpwjgXe1-Mo7yKjPKSWOAR9JHS4jgbVPCRhSHRU8BECSQW1igCjNP9_s0OHXde5y9El_1oQzD0odhjJCRD', 'link': 'https://www.google.com/search?si=ACC90ny9-jaHPXqh7KxEKXGGccqzWWM698HRflpADe-WgJzacnRWT5cGDCKpwjgXe1-Mo7yKjPKSWOAR9JHS4jgbVPCRhSHRU8BECSQW1igCjNP9_s0OHXde5y9El_1oQzD0odhjJCRD', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+career+highlights&si=ACC90ny9-jaHPXqh7KxEKXGGccqzWWM698HRflpADe-WgJzacnRWT5cGDCKpwjgXe1-Mo7yKjPKSWOAR9JHS4jgbVPCRhSHRU8BECSQW1igCjNP9_s0OHXde5y9El_1oQzD0odhjJCRD'}, {'text': 'Career', 'si': 'ACC90nz4lwyS23sm8hcph5fuM7ckJRDUKweKVcOYR8DyxN4kUsudyH6ZoJlxyWz0RbIAI53QWfZZa2SOzX0duqk8Zytx-LBJrKwlVAshPQgBGtFwfk7ifdhqrVd5DDH-JIAfQ-Vwmttk8ghTSEJ6cjvUVxSwAS-7lnbbKgZ2SWYOUEW-iD7Q9Wk=', 'link': 'https://www.google.com/search?si=ACC90nz4lwyS23sm8hcph5fuM7ckJRDUKweKVcOYR8DyxN4kUsudyH6ZoJlxyWz0RbIAI53QWfZZa2SOzX0duqk8Zytx-LBJrKwlVAshPQgBGtFwfk7ifdhqrVd5DDH-JIAfQ-Vwmttk8ghTSEJ6cjvUVxSwAS-7lnbbKgZ2SWYOUEW-iD7Q9Wk=', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+career+highlights&si=ACC90nz4lwyS23sm8hcph5fuM7ckJRDUKweKVcOYR8DyxN4kUsudyH6ZoJlxyWz0RbIAI53QWfZZa2SOzX0duqk8Zytx-LBJrKwlVAshPQgBGtFwfk7ifdhqrVd5DDH-JIAfQ-Vwmttk8ghTSEJ6cjvUVxSwAS-7lnbbKgZ2SWYOUEW-iD7Q9Wk%3D'}, {'text': 'Songs', 'si': 'ACC90nyR35EXIVqsrAONouAC2If7im8eG9AWdkeY7WEHZ67Ix1jLqk_EMnamLYhwJRXmtD9Ue2jE_uJyDpQ56VIzIFKRON52uCnDA7Y-JTWphL0p_Q6mkvtIw1lbsmn_sNPow8zcnO1J', 'link': 'https://www.google.com/search?si=ACC90nyR35EXIVqsrAONouAC2If7im8eG9AWdkeY7WEHZ67Ix1jLqk_EMnamLYhwJRXmtD9Ue2jE_uJyDpQ56VIzIFKRON52uCnDA7Y-JTWphL0p_Q6mkvtIw1lbsmn_sNPow8zcnO1J', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+career+highlights&si=ACC90nyR35EXIVqsrAONouAC2If7im8eG9AWdkeY7WEHZ67Ix1jLqk_EMnamLYhwJRXmtD9Ue2jE_uJyDpQ56VIzIFKRON52uCnDA7Y-JTWphL0p_Q6mkvtIw1lbsmn_sNPow8zcnO1J'}, {'text': 'Listen', 'si': 'ACC90nylNfUkJaxznrzSwClq9nw0u5DVmerFU1ZS4WsDMouPEMnzG8ReT5iL8vphLl8-P45M045uFTLorHkPq6YVBMkw13fn8QY1MIa-RQE6bNNXcMli4N0Do9Rd6S3DdTvXfVtyCIRr', 'link': 'https://www.google.com/search?si=ACC90nylNfUkJaxznrzSwClq9nw0u5DVmerFU1ZS4WsDMouPEMnzG8ReT5iL8vphLl8-P45M045uFTLorHkPq6YVBMkw13fn8QY1MIa-RQE6bNNXcMli4N0Do9Rd6S3DdTvXfVtyCIRr', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+career+highlights&si=ACC90nylNfUkJaxznrzSwClq9nw0u5DVmerFU1ZS4WsDMouPEMnzG8ReT5iL8vphLl8-P45M045uFTLorHkPq6YVBMkw13fn8QY1MIa-RQE6bNNXcMli4N0Do9Rd6S3DdTvXfVtyCIRr'}], 'header_images': [{'image': 'https://serpapi.com/searches/6677fd5f5f22034182fc0927/images/dd01939741925414b3a8c6394d25e008ae649b11150b8e5c44bf30eceb354f667cdbaddee20ffaaa.webp'}, {'image': 'https://serpapi.com/searches/6677fd5f5f22034182fc0927/images/dd01939741925414b3a8c6394d25e008ae649b11150b8e5cfef05094fedf45f4524a7cf715bdc7d5.webp'}, {'image': 'https://serpapi.com/searches/6677fd5f5f22034182fc0927/images/dd01939741925414b3a8c6394d25e008ae649b11150b8e5cfa1f05fbd05a01f9a49e799dfe2822ef.webp'}], 'description': "Michel François Platini is a French football administrator and former player and manager. Regarded as one of the greatest footballers of all time, Platini won the Ballon d'Or three times in a row, in 1983, 1984 and 1985, and came seventh in the FIFA Player of the Century vote.", 'source': {'name': 'Wikipedia', 'link': 'https://en.wikipedia.org/wiki/Michel_Platini'}, 'born': '1955 (age 69 years), Jœuf, France', 'born_links': [{'text': 'Jœuf, France', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=J%C5%93uf&si=ACC90nzx_D3_zUKRnpAjmO0UBLNxnt7EyN4YYdru6U3bxLI-LxvQdBRRSnQQlq7vIxw88DJUHTO63N009AlYiTewIE3c91x7FGA2Thx5A77bgpEf9jH7RPeki1QZqxUgbMyuyGDwnrWZ8nu4-0rU7qpI4rMa6zL1jbEuTwDT7ho6LX74GjxqOR8DDv3Up7JBsX9EZqMugaW9&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQmxMoAHoECBcQAg'}], 'dates_joined': '1982 (Juventus F.C.), 1976 (France national football team), 1972 (AS Nancy Lorraine)', 'dates_joined_links': [{'text': 'Juventus F.C.', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Juventus&si=ACC90nyvvWro6QmnyY1IfSdgk5wwjB1r8BGd_IWRjXqmKPQqm076E0SqGvBZ--FsN5xL5jFyoBDfUEBT56M47ItDximwbf8ypERic8qGdsjcDRn1eVbqqHdlRsRmooT3CRMqHXq6CnP_cSIxwQMiGQHu3tt69_5ThLko2SL60vVDGpyshggnUs_e19D7H48bfq0U6ukVovIsAWfN6R8KPqET8lgKOIMGCuM9XDPfUSO9NvnBcSS_czaJZJYj8ImeyUl-bV1iJt8PbWcSBPhMWnfTX8PKpUEZPw%3D%3D&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQmxMoAHoECBUQAg'}, {'text': 'France national football team', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=France+national+football+team&si=ACC90nzx_D3_zUKRnpAjmO0UBLNxnt7EyN4YYdru6U3bxLI-L3o9p6uSFMcOhGy_sKj525KYh9KnhjJ5s6XHxwLhy3nmVuca0YEahqRfhjgXM7fsIXMH7rHWtAkgijlvVKjk-rB3hdqeut2_9ppemer7QJc58NIXgTSrT9JwEIEl_ExXMVGdFcdu8M7mU3fgFoIZzWmjfye9CPGa9hcz7amSz9Xba4cFAzomUA2C_ArTzGSiof5vAUSwGpzM0JvXtGVF0WU5ZwfGzN8U-3qZbjfZVNeyGI0TPX3M60pp7CJyoc-VHLnJjT4%3D&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQmxMoAXoECBUQAw'}, {'text': 'AS Nancy Lorraine', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=AS+Nancy+Lorraine&si=ACC90nzx_D3_zUKRnpAjmO0UBLNxnt7EyN4YYdru6U3bxLI-L4igLxRnCsxOQ8dTh_h4Ajbfat76fjM1MuA4ZNJiT8X2Axq8X10yOQay4c0ZINEiob0DsHi3gEbdPFk9MvGN80HgOAJbBxO_ltCh_aEEbn63ofjHV1DZxWudhqvskYHVlcnJIzPKUXxNt4Gbj3V-oLMMH0wEnJwlgHAAfi9yTm-g0o63H2KRbZX9Un5nO9baeCvZvoyuyaB8B3kHlNvoleMuWmNjZvVpjdg3tQN5ZFitYLpAIg%3D%3D&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQmxMoAnoECBUQBA'}], 'position': 'Midfielder', 'position_links': [{'text': 'Midfielder', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=&si=ACC90nzx_D3_zUKRnpAjmO0UBLNxnt7EyN4YYdru6U3bxLI-L7WAdTL-YwWnQpdY4j2K-lFe6PkhNVQynek3Z4mN3IUjnI2V27wjtzeMfLdwkyyququ-YjNz2xISAI0Yj4t6JJM6AxKEK2iW0D-aPoEgApturWjG4Z1k25NQ8uD4sHY4cr3Wh78%3D&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQmxMoAHoECBQQAg'}], 'parents': 'Aldo Platini, Anna Platini', 'parents_links': [{'text': 'Aldo Platini', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Aldo+Platini&si=ACC90nxYhNno81_TzuVO0e1EieRzQwXK2DrwuMdAl3IyVcpNQ4FyHQ0IG7ls3z9f0-A7XqNVR7rMHHkXB-pKywLzoMGuQOnD6AWi2_pIZRb_7EOcdsUJAHeYX2LDhemq0xafPhoe0-m3nsZ0JMI0Av0hPL3UUapoVIYZPCE9nP4dvr_Whdg9MLNIqFrTGk4GHbYgS5c1iV2vGykdtyBIn_GQC4238BQUlNh2WSrlGs-_1Jrsad6k4le_-QkLeWiQHks1K5WwRwaRntgaHrnG3l2olAhpy8gDIw%3D%3D&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQmxMoAHoECB8QAg'}, {'text': 'Anna Platini', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Anna+Platini&si=ACC90nxYhNno81_TzuVO0e1EieRzQwXK2DrwuMdAl3IyVcpNQz0RKrNvs_mMGtn16CmvjDPU4aCMjQ8ohDqWxxHbYJPVrH4gwg6KtvQy3Ta9tAPXL3x0LjUVMOrJJ7ag7uSJGN72ELiUu621PHg0vWt8DccsPgs9awpabXQ6W1ce0V60HSJstKxp03-W9r_w3WLLDYk55hEQxFYgPftFj5iMpRItS4Rx2VRZRAHe9yynIeWpOBKR-hH9rtJ4QFlAOUp6tny8RCZRpD_CRikThaqQ__WY9cNsiA%3D%3D&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQmxMoAXoECB8QAw'}], 'spouse': 'Christelle Platini (m. 1977)', 'spouse_links': [{'text': 'Christelle Platini', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Christelle+Platini&si=ACC90ny8E30vD16OoPAAI4cStfcliGy35W8UAhb0TsHNc_ISQewA087VSDWbyOz5-3RS_0CVXF1JcD4_90KwGkF2IdpGNSFOHwwJ07IDvuoM3TOBhN2eeGEtKfYi2eb6380eRscsYLAfC6Ut1nBsLN_bE5kWGQ_wrcHp7bIysQ4E-GqhViHZ73lS0WiZCzFD0aJxlyuMMyrVYGdND7usWm_HLc-Q3ScHqg%3D%3D&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQmxMoAHoECB0QAg'}], 'children': 'Laurent Platini, Marine Platini', 'children_links': [{'text': 'Laurent Platini', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Laurent+Platini&si=ACC90nwLLwns5sISZcdzuISy7t-NHozt8Cbt6G3WNQfC9ekAgAuCKhUGcOl2a35eG0h_haGSCG09Ngl684dSvpax23l8hQAyxldnMV_jZpCTRNMQ6DCJO71-hb5RVHycB2ikVJ4XDRq8jAMljrbsf1zy4OtY4EPIcR-Pk2rMpa5jSbIzCMKYE3XaK2N04vlQhCEDBafUDRgiR0kaXcqi5WEnPw4w5yhoauSw3W6K_jAyyz_m6D08_k-IX_zp_W3X_9y626eep-UNmCUnzxqichfQhUL6nkax4A%3D%3D&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQmxMoAHoECBwQAg'}, {'text': 'Marine Platini', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Marine+Platini&si=ACC90nwLLwns5sISZcdzuISy7t-NHozt8Cbt6G3WNQfC9ekAgLL5wXOoGx8qgyyuHBItiDzKPXD_rdY16qGcpU59HclAfgdz0CYflEv6_IBPhrmODpCwRiODvtm3R_SuzKm4cWbKdedHMN6T1TAuRYJEf5UkNLOEUe_-Mk3gHF7KJHXmcm3Bkxh1Rw5z7RXq0fdCMU8Myoh4Qf9VVgnEOsTD-Ia_6f8PBWg-NlnjE46jTCrR2G5FLcm0fB9-qs9bUC1K8L_Y4LZ9N6eXJxppxRIo26mzhUeJ9A%3D%3D&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQmxMoAXoECBwQAw'}], 'web_results': [{'title': 'Age', 'snippet': '69 yearsJun 21, 1955', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=michel+platini+age&stick=H4sIAAAAAAAAAOPgE-LUz9U3MExLKc7VEs1OttIvSM0vyEkFUkXF-XlWiempi1iFcjOTM1JzFApyEksy8zIVgIIAe1MZZzkAAAA&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQ18AJegQIIxAB'}, {'title': 'Dates joined', 'snippet': '1982 (Juventus F.C.), 1976 (France national football team), 1972 (AS Nancy Lorraine)', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=michel+platini+dates+joined&stick=H4sIAAAAAAAAAOPgE-LUz9U3MExLKc7VUs4ot9JPzs_JSU0uyczP008sychJLUkttirITM5OTVFISSxJXcQqnZuZnJGao1CQk1iSmZcJFi1WyMrPzEtNAQDkaj3pUAAAAA&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQ18AJegQIJBAB'}, {'thumbnail': 'https://serpapi.com/searches/6677fd5f5f22034182fc0927/images/dd01939741925414b3a8c6394d25e00894204e5b287a89cc01522b5dff0ceb609b08b3210d00f3aa.jpeg', 'title': "Highlights: Michel Platini's playing career", 'source': 'YouTube • UEFA', 'source_image': 'https://serpapi.com/searches/6677fd5f5f22034182fc0927/images/dd01939741925414b3a8c6394d25e00894204e5b287a89ccd3c29dbdf3b29b8d0f4c73259b8f5df0733a5842c3a61b51.png', 'link': 'https://www.youtube.com/watch?v=kmA0jzqcaXI', 'duration': '1:13'}], 'listen': [{'name': 'Spotify', 'link': 'https://open.spotify.com/artist/6xma1aDFf3bz3c5HGRIlVr?autoplay=true', 'image': 'https://serpapi.com/searches/6677fd5f5f22034182fc0927/images/dd01939741925414b3a8c6394d25e008050841bdd867abdf858f354703c4d4b4.png'}, {'name': 'Apple Music', 'link': 'https://music.apple.com/us/artist/platini/1718461656', 'image': 'https://serpapi.com/searches/6677fd5f5f22034182fc0927/images/dd01939741925414b3a8c6394d25e008063dbd60db1bd8b28a8fd334d923087b.png'}, {'name': 'Pandora', 'link': 'http://www.pandora.com/?sc=miR10880634', 'image': 'https://serpapi.com/searches/6677fd5f5f22034182fc0927/images/dd01939741925414b3a8c6394d25e008b99bff65d6369e8d3dc61fbc76b2ba14.png'}], 'profiles': [{'name': 'YouTube', 'link': 'https://www.youtube.com/channel/UCsSClQ7XXRWuN0VQElQhvgQ', 'image': 'https://serpapi.com/searches/6677fd5f5f22034182fc0927/images/dd01939741925414b3a8c6394d25e0080e4d11842ea2ea662c441ac1017b4af2b496dd6d8a1b4f1c.png'}, {'name': 'Instagram', 'link': 'https://www.instagram.com/michelplatiniofficiel/', 'image': 'https://serpapi.com/searches/6677fd5f5f22034182fc0927/images/dd01939741925414b3a8c6394d25e0080e4d11842ea2ea669c72d4cc0e9c77899d3dfcc219f0e18c.png'}], 'songs': [{'name': 'Hier encore', 'extensions': 'Les Enfoirés au Grand Rex · 1994', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=les+enfoir%C3%A9s+hier+encore&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tFP1zcsNM1KKilLNtcS9C0tzkx2LCrJLC4JyQ_Oz0tfxCqZk1qskJqXlp9ZdHhlsUJGZmoRkJucX5QKADafiSlKAAAA&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQri56BAhAEAM', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=les+enfoir%C3%A9s+hier+encore&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tFP1zcsNM1KKilLNtcS9C0tzkx2LCrJLC4JyQ_Oz0tfxCqZk1qskJqXlp9ZdHhlsUJGZmoRkJucX5QKADafiSlKAAAA'}, {'name': "Jusqu'au dernier", 'extensions': 'Les Enfoirés 2024, On a 35 ans ! · 2024', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=les+enfoir%C3%A9s+jusqu%27au+dernier&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tVP1zc0rDTKyMrIMkjSEvQtLc5MdiwqySwuCckPzs9LX8Qql5NarJCal5afWXR4ZbFCVmlxYal6YqlCSmpRXmZqEQAFTZYRUAAAAA&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQri56BAhAEAU', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=les+enfoir%C3%A9s+jusqu%27au+dernier&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tVP1zc0rDTKyMrIMkjSEvQtLc5MdiwqySwuCckPzs9LX8Qql5NarJCal5afWXR4ZbFCVmlxYal6YqlCSmpRXmZqEQAFTZYRUAAAAA'}, {'name': 'La chanson des restos', 'extensions': "Tournée d'Enfoirés · 1989", 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=les+enfoir%C3%A9s+la+chanson+des+restos&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tFP1zcsNjAtMbQ0qdQS9C0tzkx2LCrJLC4JyQ_Oz0tfxKqck1qskJqXlp9ZdHhlsUJOokJyRmJecX6eQgpQoii1uCS_GAC9YiMSVAAAAA&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQri56BAhAEAc', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=les+enfoir%C3%A9s+la+chanson+des+restos&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tFP1zcsNjAtMbQ0qdQS9C0tzkx2LCrJLC4JyQ_Oz0tfxKqck1qskJqXlp9ZdHhlsUJOokJyRmJecX6eQgpQoii1uCS_GAC9YiMSVAAAAA', 'image': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTwT5M_653obvZxnU9qzAjBeA4Dqc4sdte-L3aa&s=0'}, {'name': 'Rêvons', 'extensions': '2023 Enfoirés un jour, toujours · 2023', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=les+enfoir%C3%A9s+r%C3%AAvons&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tVP1zc0LE4pT89NL0nTEvQtLc5MdiwqySwuCckPzs9LX8QqmpNarJCal5afWXR4ZbFC0eFVZfl5xQClALwHRwAAAA&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQri56BAhAEAo', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=les+enfoir%C3%A9s+r%C3%AAvons&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tVP1zc0LE4pT89NL0nTEvQtLc5MdiwqySwuCckPzs9LX8QqmpNarJCal5afWXR4ZbFC0eFVZfl5xQClALwHRwAAAA'}, {'name': 'On trace', 'extensions': 'Le Monde des Enfoirés · 2019', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=les+enfoir%C3%A9s+on+trace&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tVP1zc0TMs2jrcoqErTEvQtLc5MdiwqySwuCckPzs9LX8QqlpNarJCal5afWXR4ZbFCfp5CSVFicioA_CA3sEgAAAA&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQri56BAhAEAw', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=les+enfoir%C3%A9s+on+trace&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tVP1zc0TMs2jrcoqErTEvQtLc5MdiwqySwuCckPzs9LX8QqlpNarJCal5afWXR4ZbFCfp5CSVFicioA_CA3sEgAAAA', 'image': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT8SedPpjcZYaV5ugkIABl45sb5k2doQ-tG8x06&s=0'}, {'name': 'Attention au départ', 'extensions': 'La boîte à musique des Enfoirés (Live) · 2013', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=les+enfoir%C3%A9s+attention+au+d%C3%A9part&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tVP1zc0TDLPqzIyMbHUEvQtLc5MdiwqySwuCckPzs9LX8SqlJNarJCal5afWXR4ZbFCYklJal5JZn6eQmKpQsrhlQWJRSUA2dckZlQAAAA&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQri56BAhAEA8', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=les+enfoir%C3%A9s+attention+au+d%C3%A9part&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tVP1zc0TDLPqzIyMbHUEvQtLc5MdiwqySwuCckPzs9LX8SqlJNarJCal5afWXR4ZbFCYklJal5JZn6eQmKpQsrhlQWJRSUA2dckZlQAAAA', 'image': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRGPBd3Xh6WW60zSKgnJmhTxQa5ZrEUlBAQ1PP6&s=0'}, {'name': "Ella, elle l'a", 'extensions': 'Les Enfoirés au Grand Rex · 1994', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=les+enfoir%C3%A9s+ella,+elle+l%27a&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tFP1zcsNM1KSTIvr9IS9C0tzkx2LCrJLC4JyQ_Oz0tfxCqTk1qskJqXlp9ZdHglkJWTk6gDIlMVctQTAeERV4ZNAAAA&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQri56BAhAEBI', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=les+enfoir%C3%A9s+ella%2C+elle+l%27a&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tFP1zcsNM1KSTIvr9IS9C0tzkx2LCrJLC4JyQ_Oz0tfxCqTk1qskJqXlp9ZdHglkJWTk6gDIlMVctQTAeERV4ZNAAAA', 'image': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR6UfegyONC_nUo01dXgu74I1qAusHCF7ORDVqp&s=0'}, {'name': 'Popcorn salé', 'extensions': 'Les Enfoirés 2024, On a 35 ans ! · 2024', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=les+enfoir%C3%A9s+popcorn+sal%C3%A9&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tVP1zc0zEkpSi5IqajQEvQtLc5MdiwqySwuCckPzs9LX8QqnZNarJCal5afWXR4ZbFCQX5Bcn5RnkJxYs7hlQDiKVveTQAAAA&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQri56BAhAEBU', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=les+enfoir%C3%A9s+popcorn+sal%C3%A9&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tVP1zc0zEkpSi5IqajQEvQtLc5MdiwqySwuCckPzs9LX8QqnZNarJCal5afWXR4ZbFCQX5Bcn5RnkJxYs7hlQDiKVveTQAAAA'}]}, 'related_questions': [{'question': 'How many goals did Platini score in his career?', 'title': 'Career stats', 'link': 'https://www.transfermarkt.co.uk/michel-platini/leistungsdaten/spieler/88994', 'displayed_link': 'https://www.transfermarkt.co.uk › leistungsdaten › spieler', 'source_logo': 'https://serpapi.com/searches/6677fd5f5f22034182fc0927/images/c9878775d8b355753df2c72cbcea58a0419a173887f8b4ae2f23e8241a6f792f.png', 'next_page_token': 'eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210TlYwRk9VRmgxY2xWUVdFcHRiM2h1WlhCRWEwOXBkMUV0WkdJNFJEUm5RMGN4YW5sWWVUY3pTRVp1T1hkWFowbHNWVWhsYkdoTlNteGZUVFpQUlVGeFZrazBaUzFvRWhkWlVERXpXbkYyVTBGalJHRnJVRWxRTFRjdFZEUkJjeG9pUVVaWWNrVmpjRk5XVDJoNGJHczVUazVJU0daNVpDMW5iVFZ6UzB0eVJrMDVRUSIsImZjdiI6IjMiLCJlaSI6IllQMTNacXZTQWNEYWtQSVAtNy1UNEFzIiwicWMiOiJDaGx3YkdGMGFXNXBJR05oY21WbGNpQm9hV2RvYkdsbmFIUnpFQUI5MmtEdVBnIiwicXVlc3Rpb24iOiJIb3cgbWFueSBnb2FscyBkaWQgUGxhdGluaSBzY29yZSBpbiBoaXMgY2FyZWVyPyIsImxrIjoiR2k1b2IzY2diV0Z1ZVNCbmIyRnNjeUJrYVdRZ2NHeGhkR2x1YVNCelkyOXlaU0JwYmlCb2FYTWdZMkZ5WldWeSIsImJzIjoiYzNXUVFRckNNQkJGY1p1Vnk5TFZ0RzVGcFJmb1ZsREJuZUJDR2R2VURMUkpTU0pCNy1LdEJDX2hJV3hWSXFXNkhkNV84X21zWU5PNWNsQ2hQTU5SWVdrZ3B4eldKVnFTQkNaVG1nTkpFR1FnUTgyNVRvTmJsTXpDaWVpbjZ2OHB0bVR4UnFDRjVyS2lUUERTX3lpd1VpY0RoV3JVZDV1TXdzaDl3T29OMWoyUUxWcWJBb2NHRHR4YXJzZGVwelJzS1VmSjAtQzZULUlRWEFmMHRvYTd2TGkybXBfQWFsVUw0dDBWSE1rMGVPemFhdUluV0hfQjRlQUoiLCJpZCI6ImZjXzUifQ==', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google_related_questions&google_domain=google.com&next_page_token=eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210TlYwRk9VRmgxY2xWUVdFcHRiM2h1WlhCRWEwOXBkMUV0WkdJNFJEUm5RMGN4YW5sWWVUY3pTRVp1T1hkWFowbHNWVWhsYkdoTlNteGZUVFpQUlVGeFZrazBaUzFvRWhkWlVERXpXbkYyVTBGalJHRnJVRWxRTFRjdFZEUkJjeG9pUVVaWWNrVmpjRk5XVDJoNGJHczVUazVJU0daNVpDMW5iVFZ6UzB0eVJrMDVRUSIsImZjdiI6IjMiLCJlaSI6IllQMTNacXZTQWNEYWtQSVAtNy1UNEFzIiwicWMiOiJDaGx3YkdGMGFXNXBJR05oY21WbGNpQm9hV2RvYkdsbmFIUnpFQUI5MmtEdVBnIiwicXVlc3Rpb24iOiJIb3cgbWFueSBnb2FscyBkaWQgUGxhdGluaSBzY29yZSBpbiBoaXMgY2FyZWVyPyIsImxrIjoiR2k1b2IzY2diV0Z1ZVNCbmIyRnNjeUJrYVdRZ2NHeGhkR2x1YVNCelkyOXlaU0JwYmlCb2FYTWdZMkZ5WldWeSIsImJzIjoiYzNXUVFRckNNQkJGY1p1Vnk5TFZ0RzVGcFJmb1ZsREJuZUJDR2R2VURMUkpTU0pCNy1LdEJDX2hJV3hWSXFXNkhkNV84X21zWU5PNWNsQ2hQTU5SWVdrZ3B4eldKVnFTQkNaVG1nTkpFR1FnUTgyNVRvTmJsTXpDaWVpbjZ2OHB0bVR4UnFDRjVyS2lUUERTX3lpd1VpY0RoV3JVZDV1TXdzaDl3T29OMWoyUUxWcWJBb2NHRHR4YXJzZGVwelJzS1VmSjAtQzZULUlRWEFmMHRvYTd2TGkybXBfQWFsVUw0dDBWSE1rMGVPemFhdUluV0hfQjRlQUoiLCJpZCI6ImZjXzUifQ%3D%3D'}, {'question': 'What is Michel Platini famous for?', 'snippet': 'Michel Platini (born June 21, 1955, Joeuf, France) is a French professional football (soccer) player and administrator who was named the European Footballer of the Year three times (1983–85) and served as president of the Union of European Football Associations (UEFA; 2007–16).', 'title': 'Michel Platini | Biography, FIFA, & Facts - Britannica', 'date': '6 days ago', 'link': 'https://www.britannica.com/biography/Michel-Platini#:~:text=Michel%20Platini%20(born%20June%2021,UEFA%3B%202007%E2%80%9316).', 'displayed_link': 'https://www.britannica.com › ... › Other Sports', 'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTwT7h-5f4paCrJ8dGAAGz-CoYR3HTkumhV0fDpGI5Ytw&s', 'source_logo': 'https://serpapi.com/searches/6677fd5f5f22034182fc0927/images/c9878775d8b355753df2c72cbcea58a00edd11ca4fa29650d46b1f754c7050c4.png', 'next_page_token': 'eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210TlYwRk9VRmgxY2xWUVdFcHRiM2h1WlhCRWEwOXBkMUV0WkdJNFJEUm5RMGN4YW5sWWVUY3pTRVp1T1hkWFowbHNWVWhsYkdoTlNteGZUVFpQUlVGeFZrazBaUzFvRWhkWlVERXpXbkYyVTBGalJHRnJVRWxRTFRjdFZEUkJjeG9pUVVaWWNrVmpjRk5XVDJoNGJHczVUazVJU0daNVpDMW5iVFZ6UzB0eVJrMDVRUSIsImZjdiI6IjMiLCJlaSI6IllQMTNacXZTQWNEYWtQSVAtNy1UNEFzIiwicWMiOiJDaGx3YkdGMGFXNXBJR05oY21WbGNpQm9hV2RvYkdsbmFIUnpFQUI5MmtEdVBnIiwicXVlc3Rpb24iOiJXaGF0IGlzIE1pY2hlbCBQbGF0aW5pIGZhbW91cyBmb3I/IiwibGsiOiJHaUYzYUdGMElHbHpJRzFwWTJobGJDQndiR0YwYVc1cElHWmhiVzkxY3lCbWIzSSIsImJzIjoiYzNXUVFRckNNQkJGY1p1Vnk5TFZ0RzVGcFJmb1ZsREJuZUJDR2R2VURMUkpTU0pCNy1LdEJDX2hJV3hWSXFXNkhkNV84X21zWU5PNWNsQ2hQTU5SWVdrZ3B4eldKVnFTQkNaVG1nTkpFR1FnUTgyNVRvTmJsTXpDaWVpbjZ2OHB0bVR4UnFDRjVyS2lUUERTX3lpd1VpY0RoV3JVZDV1TXdzaDl3T29OMWoyUUxWcWJBb2NHRHR4YXJzZGVwelJzS1VmSjAtQzZULUlRWEFmMHRvYTd2TGkybXBfQWFsVUw0dDBWSE1rMGVPemFhdUluV0hfQjRlQUoiLCJpZCI6ImZjXzUifQ==', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google_related_questions&google_domain=google.com&next_page_token=eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210TlYwRk9VRmgxY2xWUVdFcHRiM2h1WlhCRWEwOXBkMUV0WkdJNFJEUm5RMGN4YW5sWWVUY3pTRVp1T1hkWFowbHNWVWhsYkdoTlNteGZUVFpQUlVGeFZrazBaUzFvRWhkWlVERXpXbkYyVTBGalJHRnJVRWxRTFRjdFZEUkJjeG9pUVVaWWNrVmpjRk5XVDJoNGJHczVUazVJU0daNVpDMW5iVFZ6UzB0eVJrMDVRUSIsImZjdiI6IjMiLCJlaSI6IllQMTNacXZTQWNEYWtQSVAtNy1UNEFzIiwicWMiOiJDaGx3YkdGMGFXNXBJR05oY21WbGNpQm9hV2RvYkdsbmFIUnpFQUI5MmtEdVBnIiwicXVlc3Rpb24iOiJXaGF0IGlzIE1pY2hlbCBQbGF0aW5pIGZhbW91cyBmb3I%2FIiwibGsiOiJHaUYzYUdGMElHbHpJRzFwWTJobGJDQndiR0YwYVc1cElHWmhiVzkxY3lCbWIzSSIsImJzIjoiYzNXUVFRckNNQkJGY1p1Vnk5TFZ0RzVGcFJmb1ZsREJuZUJDR2R2VURMUkpTU0pCNy1LdEJDX2hJV3hWSXFXNkhkNV84X21zWU5PNWNsQ2hQTU5SWVdrZ3B4eldKVnFTQkNaVG1nTkpFR1FnUTgyNVRvTmJsTXpDaWVpbjZ2OHB0bVR4UnFDRjVyS2lUUERTX3lpd1VpY0RoV3JVZDV1TXdzaDl3T29OMWoyUUxWcWJBb2NHRHR4YXJzZGVwelJzS1VmSjAtQzZULUlRWEFmMHRvYTd2TGkybXBfQWFsVUw0dDBWSE1rMGVPemFhdUluV0hfQjRlQUoiLCJpZCI6ImZjXzUifQ%3D%3D'}, {'question': 'Who was better, Platini or Zidane?', 'snippet': 'Platini was a better player relative to his era but on paper, Zidane had a better and more complete career. Same amount of Champions League/European Cups but Zidane had tougher opponents. Platini scored a penalty in the final and Zidane scored one of the best CL goals ever in the final. Both made the final twice.', 'title': "Hot take: Platini's career was better and more complete then Zidane's", 'date': 'Jul 26, 2023', 'link': 'https://www.reddit.com/r/football/comments/159ywh9/hot_take_platinis_career_was_better_and_more/#:~:text=Platini%20was%20a%20better%20player,Both%20made%20the%20final%20twice.', 'displayed_link': 'https://www.reddit.com › football › comments › hot_tak...', 'source_logo': 'https://serpapi.com/searches/6677fd5f5f22034182fc0927/images/c9878775d8b355753df2c72cbcea58a0b14f3e600b1dbc4094c0f28ea88cd0f9.png', 'next_page_token': 'eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210TlYwRk9VRmgxY2xWUVdFcHRiM2h1WlhCRWEwOXBkMUV0WkdJNFJEUm5RMGN4YW5sWWVUY3pTRVp1T1hkWFowbHNWVWhsYkdoTlNteGZUVFpQUlVGeFZrazBaUzFvRWhkWlVERXpXbkYyVTBGalJHRnJVRWxRTFRjdFZEUkJjeG9pUVVaWWNrVmpjRk5XVDJoNGJHczVUazVJU0daNVpDMW5iVFZ6UzB0eVJrMDVRUSIsImZjdiI6IjMiLCJlaSI6IllQMTNacXZTQWNEYWtQSVAtNy1UNEFzIiwicWMiOiJDaGx3YkdGMGFXNXBJR05oY21WbGNpQm9hV2RvYkdsbmFIUnpFQUI5MmtEdVBnIiwicXVlc3Rpb24iOiJXaG8gd2FzIGJldHRlciwgUGxhdGluaSBvciBaaWRhbmU/IiwibGsiOiJHaUIzYUc4Z2QyRnpJR0psZEhSbGNpQndiR0YwYVc1cElHOXlJSHBwWkdGdVpRIiwiYnMiOiJjM1dRUVFyQ01CQkZjWnVWeTlMVnRHNUZwUmZvVmxEQm5lQkNHZHZVRExSSlNTSkI3LUt0QkNfaElXeFZJcVc2SGQ1XzhfbXNZTk81Y2xDaFBNTlJZV2tncHh6V0pWcVNCQ1pUbWdOSkVHUWdRODI1VG9OYmxNekNpZWluNnY4cHRtVHhScUNGNXJLaVRQRFNfeWl3VWljRGhXclVkNXVNd3NoOXdPb04xajJRTFZxYkFvY0dEdHhhcnNkZXB6UnNLVWZKMC1DNlQtSVFYQWYwdG9hN3ZMaTJtcF9BYWxVTDR0MFZITWswZU96YWF1SW5XSF9CNGVBSiIsImlkIjoiZmNfNSJ9', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google_related_questions&google_domain=google.com&next_page_token=eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210TlYwRk9VRmgxY2xWUVdFcHRiM2h1WlhCRWEwOXBkMUV0WkdJNFJEUm5RMGN4YW5sWWVUY3pTRVp1T1hkWFowbHNWVWhsYkdoTlNteGZUVFpQUlVGeFZrazBaUzFvRWhkWlVERXpXbkYyVTBGalJHRnJVRWxRTFRjdFZEUkJjeG9pUVVaWWNrVmpjRk5XVDJoNGJHczVUazVJU0daNVpDMW5iVFZ6UzB0eVJrMDVRUSIsImZjdiI6IjMiLCJlaSI6IllQMTNacXZTQWNEYWtQSVAtNy1UNEFzIiwicWMiOiJDaGx3YkdGMGFXNXBJR05oY21WbGNpQm9hV2RvYkdsbmFIUnpFQUI5MmtEdVBnIiwicXVlc3Rpb24iOiJXaG8gd2FzIGJldHRlciwgUGxhdGluaSBvciBaaWRhbmU%2FIiwibGsiOiJHaUIzYUc4Z2QyRnpJR0psZEhSbGNpQndiR0YwYVc1cElHOXlJSHBwWkdGdVpRIiwiYnMiOiJjM1dRUVFyQ01CQkZjWnVWeTlMVnRHNUZwUmZvVmxEQm5lQkNHZHZVRExSSlNTSkI3LUt0QkNfaElXeFZJcVc2SGQ1XzhfbXNZTk81Y2xDaFBNTlJZV2tncHh6V0pWcVNCQ1pUbWdOSkVHUWdRODI1VG9OYmxNekNpZWluNnY4cHRtVHhScUNGNXJLaVRQRFNfeWl3VWljRGhXclVkNXVNd3NoOXdPb04xajJRTFZxYkFvY0dEdHhhcnNkZXB6UnNLVWZKMC1DNlQtSVFYQWYwdG9hN3ZMaTJtcF9BYWxVTDR0MFZITWswZU96YWF1SW5XSF9CNGVBSiIsImlkIjoiZmNfNSJ9'}, {'question': 'How many trophies did Platini win?', 'snippet': "He won the Serie A title with Juventus in 1984 and 1986, the European Cup Winners' Cup in 1984 (setting up Vignola's opening goal in the Turin club's 2–1 victory over Porto in the final of the competition), the 1984 European Super Cup, the European Cup in 1985 and the 1985 Intercontinental Cup.", 'title': 'Michel Platini: The Maestro of French Football. - LinkedIn', 'date': 'Jun 4, 2023', 'link': 'https://www.linkedin.com/pulse/michel-platini-maestro-french-football-javad-ghorbani#:~:text=He%20won%20the%20Serie%20A,and%20the%201985%20Intercontinental%20Cup.', 'displayed_link': 'https://www.linkedin.com › pulse › michel-platini-maestr...', 'source_logo': 'https://serpapi.com/searches/6677fd5f5f22034182fc0927/images/c9878775d8b355753df2c72cbcea58a02de109947556d3968cade5c1e95a476b.png', 'next_page_token': 'eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210TlYwRk9VRmgxY2xWUVdFcHRiM2h1WlhCRWEwOXBkMUV0WkdJNFJEUm5RMGN4YW5sWWVUY3pTRVp1T1hkWFowbHNWVWhsYkdoTlNteGZUVFpQUlVGeFZrazBaUzFvRWhkWlVERXpXbkYyVTBGalJHRnJVRWxRTFRjdFZEUkJjeG9pUVVaWWNrVmpjRk5XVDJoNGJHczVUazVJU0daNVpDMW5iVFZ6UzB0eVJrMDVRUSIsImZjdiI6IjMiLCJlaSI6IllQMTNacXZTQWNEYWtQSVAtNy1UNEFzIiwicWMiOiJDaGx3YkdGMGFXNXBJR05oY21WbGNpQm9hV2RvYkdsbmFIUnpFQUI5MmtEdVBnIiwicXVlc3Rpb24iOiJIb3cgbWFueSB0cm9waGllcyBkaWQgUGxhdGluaSB3aW4/IiwibGsiOiJHaUZvYjNjZ2JXRnVlU0IwY205d2FHbGxjeUJrYVdRZ2NHeGhkR2x1YVNCM2FXNCIsImJzIjoiYzNXUVFRckNNQkJGY1p1Vnk5TFZ0RzVGcFJmb1ZsREJuZUJDR2R2VURMUkpTU0pCNy1LdEJDX2hJV3hWSXFXNkhkNV84X21zWU5PNWNsQ2hQTU5SWVdrZ3B4eldKVnFTQkNaVG1nTkpFR1FnUTgyNVRvTmJsTXpDaWVpbjZ2OHB0bVR4UnFDRjVyS2lUUERTX3lpd1VpY0RoV3JVZDV1TXdzaDl3T29OMWoyUUxWcWJBb2NHRHR4YXJzZGVwelJzS1VmSjAtQzZULUlRWEFmMHRvYTd2TGkybXBfQWFsVUw0dDBWSE1rMGVPemFhdUluV0hfQjRlQUoiLCJpZCI6ImZjXzUifQ==', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google_related_questions&google_domain=google.com&next_page_token=eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210TlYwRk9VRmgxY2xWUVdFcHRiM2h1WlhCRWEwOXBkMUV0WkdJNFJEUm5RMGN4YW5sWWVUY3pTRVp1T1hkWFowbHNWVWhsYkdoTlNteGZUVFpQUlVGeFZrazBaUzFvRWhkWlVERXpXbkYyVTBGalJHRnJVRWxRTFRjdFZEUkJjeG9pUVVaWWNrVmpjRk5XVDJoNGJHczVUazVJU0daNVpDMW5iVFZ6UzB0eVJrMDVRUSIsImZjdiI6IjMiLCJlaSI6IllQMTNacXZTQWNEYWtQSVAtNy1UNEFzIiwicWMiOiJDaGx3YkdGMGFXNXBJR05oY21WbGNpQm9hV2RvYkdsbmFIUnpFQUI5MmtEdVBnIiwicXVlc3Rpb24iOiJIb3cgbWFueSB0cm9waGllcyBkaWQgUGxhdGluaSB3aW4%2FIiwibGsiOiJHaUZvYjNjZ2JXRnVlU0IwY205d2FHbGxjeUJrYVdRZ2NHeGhkR2x1YVNCM2FXNCIsImJzIjoiYzNXUVFRckNNQkJGY1p1Vnk5TFZ0RzVGcFJmb1ZsREJuZUJDR2R2VURMUkpTU0pCNy1LdEJDX2hJV3hWSXFXNkhkNV84X21zWU5PNWNsQ2hQTU5SWVdrZ3B4eldKVnFTQkNaVG1nTkpFR1FnUTgyNVRvTmJsTXpDaWVpbjZ2OHB0bVR4UnFDRjVyS2lUUERTX3lpd1VpY0RoV3JVZDV1TXdzaDl3T29OMWoyUUxWcWJBb2NHRHR4YXJzZGVwelJzS1VmSjAtQzZULUlRWEFmMHRvYTd2TGkybXBfQWFsVUw0dDBWSE1rMGVPemFhdUluV0hfQjRlQUoiLCJpZCI6ImZjXzUifQ%3D%3D'}], 'organic_results': [{'position': 1, 'title': 'Michel Platini | Biography, FIFA, & Facts', 'link': 'https://www.britannica.com/biography/Michel-Platini', 'redirect_link': 'https://www.google.com/url?sa=t&url=https://www.britannica.com/biography/Michel-Platini&usg=AOvVaw0mEX4HX-10Jr3FSggZv78L', 'displayed_link': 'https://www.britannica.com › ... › Other Sports', 'thumbnail': 'https://serpapi.com/searches/6677fd5f5f22034182fc0927/images/8ad815eb923ed82eb340f537b707250d533eb667768bac45c683c1cd882a46b0.jpeg', 'favicon': 'https://serpapi.com/searches/6677fd5f5f22034182fc0927/images/8ad815eb923ed82eb340f537b707250df34a4f76856bc3442159ec0d4d363e4e.png', 'date': '6 days ago', 'snippet': 'Michel Platini, French professional football (soccer) player who was a three-time European Footballer of the Year (1983–85) and president of ...', 'snippet_highlighted_words': ['Platini', 'professional'], 'source': 'Britannica'}, {'position': 2, 'title': 'Michel Platini', 'link': 'https://en.wikipedia.org/wiki/Michel_Platini', 'redirect_link': 'https://www.google.com/url?sa=t&url=https://en.wikipedia.org/wiki/Michel_Platini&usg=AOvVaw0TVBgm3Q_6A-CURhLIQWhw', 'displayed_link': 'https://en.wikipedia.org › wiki › Michel_Platini', 'thumbnail': 'https://serpapi.com/searches/6677fd5f5f22034182fc0927/images/8ad815eb923ed82eb340f537b707250de81db61f16c50fc6dadd9598391b92e8.jpeg', 'favicon': 'https://serpapi.com/searches/6677fd5f5f22034182fc0927/images/8ad815eb923ed82eb340f537b707250d2f4ec8f9ffa2acc97f35b350f659513d.png', 'snippet': 'Platini was a key player of the France national team that won the 1984 European Championship, a tournament in which he was the top scorer and best player, and ...', 'snippet_highlighted_words': ['Platini'], 'source': 'Wikipedia'}, {'position': 3, 'title': 'Michel Platini, Le Roi [Goals & Skills]', 'link': 'https://www.youtube.com/watch?v=Ws6QQC7fmBE', 'displayed_link': '1M+ views · 6 years ago', 'thumbnail': 'https://serpapi.com/searches/6677fd5f5f22034182fc0927/images/8ad815eb923ed82eb340f537b707250d504ccbda55b69ae61c07b3878c484e86.jpeg', 'favicon': 'https://serpapi.com/searches/6677fd5f5f22034182fc0927/images/8ad815eb923ed82eb340f537b707250de245f8433d4d23d0070bd298c230655d.png', 'snippet': "9 goals in 5 matchs during Euro 1984. 3 times Ballon d'Or. Designed best player of Juventus history. Best French player of all time and one ...", 'missing': ['career'], 'must_include': {'word': 'career', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Platini+%22career%22+highlights&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQ5t4CegQIYhAT'}, 'video_link': 'https://encrypted-vtbn0.gstatic.com/video?q=tbn:ANd9GcRAM3SU4Cu-VS9USTJo0y4S33u70ETk7QhIhQ', 'source': 'YouTube · Barbosa Fútbol Videos'}, {'position': 4, 'title': 'Michel Platini | Top 10 goals', 'link': 'https://www.youtube.com/watch?v=E84oxh3gmLs', 'displayed_link': '26.4K+ views · 1 year ago', 'thumbnail': 'https://serpapi.com/searches/6677fd5f5f22034182fc0927/images/8ad815eb923ed82eb340f537b707250d4dbde8519426c5d9543c66289da5d3fb.jpeg', 'favicon': 'https://serpapi.com/searches/6677fd5f5f22034182fc0927/images/8ad815eb923ed82eb340f537b707250d6146f0e1967e24993a6ae84c30fae3a2.png', 'snippet': 'Michel François Platini is a French footballer considered as one of the greatest footballers of all time During his career, Platini played ...', 'missing': ['highlights'], 'must_include': {'word': 'highlights', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Platini+career+%22highlights%22&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQ5t4CegQIYBAT'}, 'video_link': 'https://encrypted-vtbn0.gstatic.com/video?q=tbn:ANd9GcRmxK4lKeos1AomkEfgoQpI8Y5Y9IRjNYctuA', 'source': 'YouTube · Legendary Football'}, {'position': 5, 'title': 'Michel Platini biography and career accomplishments', 'link': 'https://www.footballhistory.org/player/michel-platini.html', 'redirect_link': 'https://www.google.com/url?sa=t&url=https://www.footballhistory.org/player/michel-platini.html&usg=AOvVaw0q5yyHp8dLU8bRX8JL_4UB', 'displayed_link': 'https://www.footballhistory.org › player › michel-platini', 'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSP8_rq-Mvf4hyRQNzcazgxxmNiYTZDlFS1HRr3jhKYqZNJvTn6QhO8&usqp=CAE&s', 'favicon': 'https://serpapi.com/searches/6677fd5f5f22034182fc0927/images/8ad815eb923ed82eb340f537b707250db3c6f4308dbeb067470743c3b0260a7b.png', 'snippet': 'The footbal icon Michel Platini has served as a successful offensive midfielder, captain as well as manager of the French team. After his career he did become ...', 'snippet_highlighted_words': ['Platini', 'career'], 'source': 'The history of football (soccer)'}, {'position': 6, 'title': 'MICHEAL PLATINI Football Career Achievements and ...', 'link': 'https://medium.com/@mondebaron/michel-platini-a-french-football-legend-stands-among-the-greatest-players-in-history-8b2f89a4abe6', 'redirect_link': 'https://www.google.com/url?sa=t&url=https://medium.com/@mondebaron/michel-platini-a-french-football-legend-stands-among-the-greatest-players-in-history-8b2f89a4abe6&usg=AOvVaw0mePWYdDP0djXHlMChjotA', 'displayed_link': '5 months ago', 'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTc4xsVXyuLuuAt6zQgCbvR20Tow-16XuVxFyLJGHkNUd4SL4qYB1kg&usqp=CAE&s', 'favicon': 'https://serpapi.com/searches/6677fd5f5f22034182fc0927/images/8ad815eb923ed82eb340f537b707250de8b7b3b45b46938314d6eed974b9baad.png', 'snippet': "Platini's illustrious career boasts 18 trophies, encompassing French and Italian league titles, European Cup, European Championship, and more.", 'snippet_highlighted_words': ["Platini's", 'career'], 'source': 'Medium · Mondebaron'}, {'position': 7, 'title': 'Michel Platini [Best Skills & Goals]', 'link': 'https://www.youtube.com/watch?v=Eq0wUKgETek', 'displayed_link': '14.5K+ views · 2 years ago', 'thumbnail': 'https://i.ytimg.com/vi/Eq0wUKgETek/mqdefault.jpg?sqp=-oaymwEFCJQBEFM&rs=AMzJL3mY02XCZ5_HsqsDnDZL-aAaD1B-7w', 'favicon': 'https://serpapi.com/searches/6677fd5f5f22034182fc0927/images/8ad815eb923ed82eb340f537b707250d39202e1188580813637634b8a093d060.png', 'snippet': '... Career Award: 2012 Juventus Greatest XI of All Time: 2017 Ballon d ... / vmsoccerlegends michel platini top 10 goals, michel platini skills, ...', 'video_link': 'https://encrypted-vtbn0.gstatic.com/video?q=tbn:ANd9GcTO4KbnTLfkBoYP8m7ommco1zASnxh2xy0paw', 'source': 'YouTube · VM Soccer Legends'}, {'position': 8, 'title': "Legends Revisited: Michel Platini - France's Greatest Player?", 'link': 'https://bleacherreport.com/articles/262769-legends-revisited-the-life-and-times-of-michel-platini', 'redirect_link': 'https://www.google.com/url?sa=t&url=https://bleacherreport.com/articles/262769-legends-revisited-the-life-and-times-of-michel-platini&usg=AOvVaw0KDvj0SP75rmpsq9vaOaG5', 'displayed_link': 'https://bleacherreport.com › World Football › Juventus', 'favicon': 'https://serpapi.com/searches/6677fd5f5f22034182fc0927/images/8ad815eb923ed82eb340f537b707250d9115bc10136588608e899112a3bce064.png', 'date': 'Sep 28, 2009', 'snippet': 'Michel Platini came to national repute playing for AS Nancy in the late 70s by scoring 98 goals from 181 appearances in midfield, during a ...', 'snippet_highlighted_words': ['Platini', 'goals'], 'source': 'Bleacher Report'}, {'position': 9, 'title': 'Michel Platini - Titles & achievements - Transfermarkt', 'link': 'https://www.transfermarkt.us/michel-platini/erfolge/spieler/88994', 'redirect_link': 'https://www.google.com/url?sa=t&url=https://www.transfermarkt.us/michel-platini/erfolge/spieler/88994&usg=AOvVaw32sJRO5FIcmYPnw1Wlzk3H', 'displayed_link': 'https://www.transfermarkt.us › erfolge › spieler', 'favicon': 'https://serpapi.com/searches/6677fd5f5f22034182fc0927/images/8ad815eb923ed82eb340f537b707250d0e394ddade0a4a9a0d613df0e626cbc3.jpeg', 'snippet': "Michel Platini · 3x Winner Ballon d'Or · 2x Footballer of the Year · 4x Top goal scorer · 1x European champion · 1x European Champion Clubs' Cup winner · 2x Italian ...", 'snippet_highlighted_words': ['Platini'], 'source': 'transfermarkt.us'}, {'position': 10, 'title': 'A few career goals from Michel Platini', 'link': 'https://www.youtube.com/watch?v=PWzTttpBBxE', 'displayed_link': '760+ views · 2 months ago', 'thumbnail': 'https://i.ytimg.com/vi/PWzTttpBBxE/mqdefault.jpg?sqp=-oaymwEFCJQBEFM&rs=AMzJL3k4dhhlJTow3XJ-eMIFHhjlQywk-A', 'favicon': 'https://serpapi.com/searches/6677fd5f5f22034182fc0927/images/8ad815eb923ed82eb340f537b707250d833d793430e65a0c217f0e6e3191a8e6.png', 'snippet': 'Michel Platini A French midfielder who played for Nancy, Saint-Étienne and Juventus. He scored over 250 goals during his career so here are ...', 'missing': ['highlights'], 'must_include': {'word': 'highlights', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Platini+career+%22highlights%22&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQ5t4CegQIPBAT'}, 'video_link': 'https://encrypted-vtbn0.gstatic.com/video?q=tbn:ANd9GcSFBtGDjR5If5Iz_eJenlfle-RhHFfC72XqnA', 'source': "YouTube · Now That's What I Call a Goal"}], 'related_searches': [{'block_position': 1, 'query': "platini ballon d'or", 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=platini+ballon+d%27or&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQ1QJ6BAhHEAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=platini+ballon+d%27or'}, {'block_position': 1, 'query': 'Platini goals', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Platini+goals&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQ1QJ6BAhGEAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+goals'}, {'block_position': 1, 'query': 'Platini clubs', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Platini+clubs&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQ1QJ6BAhDEAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+clubs'}, {'block_position': 1, 'query': 'Sports career of Michel Platini', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Sports+career+of+Michel+Platini&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQ1QJ6BAg-EAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Sports+career+of+Michel+Platini'}, {'block_position': 1, 'query': 'Platini trophies', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Platini+trophies&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQ1QJ6BAg9EAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+trophies'}, {'block_position': 1, 'query': 'Michel platini age', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Michel+platini+age&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQ1QJ6BAg7EAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Michel+platini+age'}, {'block_position': 1, 'query': 'Platini FIFA card', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Platini+FIFA+card&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQ1QJ6BAg6EAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+FIFA+card'}, {'block_position': 1, 'query': 'Platini nationality', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Platini+nationality&sa=X&ved=2ahUKEwirgsqvxvGGAxVALUQIHfvfBLwQ1QJ6BAg5EAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+nationality'}], 'pagination': {'current': 1, 'next': 'https://www.google.com/search?q=Platini+career+highlights&oq=Platini+career+highlights&hl=en&gl=us&start=10&sourceid=chrome&ie=UTF-8', 'other_pages': {'2': 'https://www.google.com/search?q=Platini+career+highlights&oq=Platini+career+highlights&hl=en&gl=us&start=10&sourceid=chrome&ie=UTF-8', '3': 'https://www.google.com/search?q=Platini+career+highlights&oq=Platini+career+highlights&hl=en&gl=us&start=20&sourceid=chrome&ie=UTF-8', '4': 'https://www.google.com/search?q=Platini+career+highlights&oq=Platini+career+highlights&hl=en&gl=us&start=30&sourceid=chrome&ie=UTF-8', '5': 'https://www.google.com/search?q=Platini+career+highlights&oq=Platini+career+highlights&hl=en&gl=us&start=40&sourceid=chrome&ie=UTF-8'}}, 'serpapi_pagination': {'current': 1, 'next_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+career+highlights&start=10', 'next': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+career+highlights&start=10', 'other_pages': {'2': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+career+highlights&start=10', '3': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+career+highlights&start=20', '4': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+career+highlights&start=30', '5': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+career+highlights&start=40'}}}}), H2Title(title='Platini Controversy', content=None, h3_titles=None, web_search={'generated_query': 'Platini Controversy', 'results': {'search_metadata': {'id': '6677fde192564113b8d0dd46', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/e1c49879d20b5a87/6677fde192564113b8d0dd46.json', 'created_at': '2024-06-23 10:50:09 UTC', 'processed_at': '2024-06-23 10:50:09 UTC', 'google_url': 'https://www.google.com/search?q=Platini+Controversy&oq=Platini+Controversy&hl=en&gl=us&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/e1c49879d20b5a87/6677fde192564113b8d0dd46.html', 'total_time_taken': 2.53}, 'search_parameters': {'engine': 'google', 'q': 'Platini Controversy', 'google_domain': 'google.com', 'hl': 'en', 'gl': 'us', 'device': 'desktop'}, 'search_information': {'query_displayed': 'Platini Controversy', 'total_results': 66500, 'time_taken_displayed': 0.31, 'organic_results_state': 'Results for exact spelling'}, 'related_questions': [{'question': 'What is Platini accused of?', 'snippet': 'Once news of the invoice and its recipient came to light, both Blatter and Platini were accused of fraud and forgery by the Swiss Office of the Attorney General in September 2015. Earlier in May 2015, seven FIFA officials were arrested on charges of wire fraud, racketeering, and money laundering.', 'title': 'Explained: How tainted football administrators Blatter and Platini ...', 'date': 'Jul 8, 2022', 'link': 'https://indianexpress.com/article/explained/explained-tainted-football-administrators-blatter-platini-escaped-criminal-charges-8017965/#:~:text=Once%20news%20of%20the%20invoice,%2C%20racketeering%2C%20and%20money%20laundering.', 'displayed_link': 'https://indianexpress.com › article › explained › explaine...', 'source_logo': 'https://serpapi.com/searches/6677fde192564113b8d0dd46/images/e83b2b9b11ec503b801ff02b236ac3840f1d6502d1bdff0d4e7c6dec81588611.png', 'next_page_token': 'eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210UVkwbHVRMEpvTm1ScGRHRndNMnhHTmsxUVUwcFdRbTlFU1ZaaFIzQlBPVWxaWXpKMGVHazRkbVp3VTNWTVdreHdXa1JHUnpoU05Ga3dZbE13YkRkcVJFRjBTRkpuRWhjMFh6RXpXbTkxYTAxMFpYQTFUbTlRTFRkcFQzZEJjeG9pUVVaWWNrVmpjVjlWZUdkcU5VdzBhMGhLTVdSc1pEVnVXbkI2WlhsTmNHeFdadyIsImZjdiI6IjMiLCJlaSI6IjRfMTNab3VrTXRlcDVOb1AtN2lPd0FzIiwicWMiOiJDaE53YkdGMGFXNXBJR052Ym5SeWIzWmxjbk41RUFCOU1nYzFQdyIsInF1ZXN0aW9uIjoiV2hhdCBpcyBQbGF0aW5pIGFjY3VzZWQgb2Y/IiwibGsiOiJHaHAzYUdGMElHbHpJSEJzWVhScGJta2dZV05qZFhObFpDQnZaZyIsImJzIjoiYy1PeTQ1SU96MGdzVWNnc1ZnaklTU3pKek10VVNFeE9MaTFPVFZISVQ3T1hlS3BvSkNNbFZRNVZVWUNoZ3N1RVN3eXNQeU1SWVVCS2ZsNnF2Y1QxRWlOeEtkRnltR1FCa2lSWEFwY09XRmRLWm9wQ2NHcEJnWUlUVUxZa3RVZ2hNUzlGd1Rjek9TTTFCOGswZTRsT0RpTmRLZTF5bUpaaWtKWWtKQzI1RUMwSU93UVlBUSIsImlkIjoiZmNfMSJ9', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google_related_questions&google_domain=google.com&next_page_token=eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210UVkwbHVRMEpvTm1ScGRHRndNMnhHTmsxUVUwcFdRbTlFU1ZaaFIzQlBPVWxaWXpKMGVHazRkbVp3VTNWTVdreHdXa1JHUnpoU05Ga3dZbE13YkRkcVJFRjBTRkpuRWhjMFh6RXpXbTkxYTAxMFpYQTFUbTlRTFRkcFQzZEJjeG9pUVVaWWNrVmpjVjlWZUdkcU5VdzBhMGhLTVdSc1pEVnVXbkI2WlhsTmNHeFdadyIsImZjdiI6IjMiLCJlaSI6IjRfMTNab3VrTXRlcDVOb1AtN2lPd0FzIiwicWMiOiJDaE53YkdGMGFXNXBJR052Ym5SeWIzWmxjbk41RUFCOU1nYzFQdyIsInF1ZXN0aW9uIjoiV2hhdCBpcyBQbGF0aW5pIGFjY3VzZWQgb2Y%2FIiwibGsiOiJHaHAzYUdGMElHbHpJSEJzWVhScGJta2dZV05qZFhObFpDQnZaZyIsImJzIjoiYy1PeTQ1SU96MGdzVWNnc1ZnaklTU3pKek10VVNFeE9MaTFPVFZISVQ3T1hlS3BvSkNNbFZRNVZVWUNoZ3N1RVN3eXNQeU1SWVVCS2ZsNnF2Y1QxRWlOeEtkRnltR1FCa2lSWEFwY09XRmRLWm9wQ2NHcEJnWUlUVUxZa3RVZ2hNUzlGd1Rjek9TTTFCOGswZTRsT0RpTmRLZTF5bUpaaWtKWWtKQzI1RUMwSU93UVlBUSIsImlkIjoiZmNfMSJ9'}, {'question': 'What has Platini done?', 'snippet': 'Michel Platini (born June 21, 1955, Joeuf, France) is a French professional football (soccer) player and administrator who was named the European Footballer of the Year three times (1983–85) and served as president of the Union of European Football Associations (UEFA; 2007–16).', 'title': 'Michel Platini | Biography, FIFA, & Facts - Britannica', 'date': '6 days ago', 'link': 'https://www.britannica.com/biography/Michel-Platini#:~:text=Michel%20Platini%20(born%20June%2021,UEFA%3B%202007%E2%80%9316).', 'displayed_link': 'https://www.britannica.com › biography › Michel-Platini', 'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTwT7h-5f4paCrJ8dGAAGz-CoYR3HTkumhV0fDpGI5Ytw&s', 'source_logo': 'https://serpapi.com/searches/6677fde192564113b8d0dd46/images/e83b2b9b11ec503b801ff02b236ac3843fdf84a3d6a8a3c3906b9a1b5b1d3bbb.png', 'next_page_token': 'eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210UVkwbHVRMEpvTm1ScGRHRndNMnhHTmsxUVUwcFdRbTlFU1ZaaFIzQlBPVWxaWXpKMGVHazRkbVp3VTNWTVdreHdXa1JHUnpoU05Ga3dZbE13YkRkcVJFRjBTRkpuRWhjMFh6RXpXbTkxYTAxMFpYQTFUbTlRTFRkcFQzZEJjeG9pUVVaWWNrVmpjVjlWZUdkcU5VdzBhMGhLTVdSc1pEVnVXbkI2WlhsTmNHeFdadyIsImZjdiI6IjMiLCJlaSI6IjRfMTNab3VrTXRlcDVOb1AtN2lPd0FzIiwicWMiOiJDaE53YkdGMGFXNXBJR052Ym5SeWIzWmxjbk41RUFCOU1nYzFQdyIsInF1ZXN0aW9uIjoiV2hhdCBoYXMgUGxhdGluaSBkb25lPyIsImxrIjoiR2hWM2FHRjBJR2hoY3lCd2JHRjBhVzVwSUdSdmJtVSIsImJzIjoiYy1PeTQ1SU96MGdzVWNnc1ZnaklTU3pKek10VVNFeE9MaTFPVFZISVQ3T1hlS3BvSkNNbFZRNVZVWUNoZ3N1RVN3eXNQeU1SWVVCS2ZsNnF2Y1QxRWlOeEtkRnltR1FCa2lSWEFwY09XRmRLWm9wQ2NHcEJnWUlUVUxZa3RVZ2hNUzlGd1Rjek9TTTFCOGswZTRsT0RpTmRLZTF5bUpaaWtKWWtKQzI1RUMwSU93UVlBUSIsImlkIjoiZmNfMSJ9', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google_related_questions&google_domain=google.com&next_page_token=eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210UVkwbHVRMEpvTm1ScGRHRndNMnhHTmsxUVUwcFdRbTlFU1ZaaFIzQlBPVWxaWXpKMGVHazRkbVp3VTNWTVdreHdXa1JHUnpoU05Ga3dZbE13YkRkcVJFRjBTRkpuRWhjMFh6RXpXbTkxYTAxMFpYQTFUbTlRTFRkcFQzZEJjeG9pUVVaWWNrVmpjVjlWZUdkcU5VdzBhMGhLTVdSc1pEVnVXbkI2WlhsTmNHeFdadyIsImZjdiI6IjMiLCJlaSI6IjRfMTNab3VrTXRlcDVOb1AtN2lPd0FzIiwicWMiOiJDaE53YkdGMGFXNXBJR052Ym5SeWIzWmxjbk41RUFCOU1nYzFQdyIsInF1ZXN0aW9uIjoiV2hhdCBoYXMgUGxhdGluaSBkb25lPyIsImxrIjoiR2hWM2FHRjBJR2hoY3lCd2JHRjBhVzVwSUdSdmJtVSIsImJzIjoiYy1PeTQ1SU96MGdzVWNnc1ZnaklTU3pKek10VVNFeE9MaTFPVFZISVQ3T1hlS3BvSkNNbFZRNVZVWUNoZ3N1RVN3eXNQeU1SWVVCS2ZsNnF2Y1QxRWlOeEtkRnltR1FCa2lSWEFwY09XRmRLWm9wQ2NHcEJnWUlUVUxZa3RVZ2hNUzlGd1Rjek9TTTFCOGswZTRsT0RpTmRLZTF5bUpaaWtKWWtKQzI1RUMwSU93UVlBUSIsImlkIjoiZmNfMSJ9'}, {'question': 'What did Sepp Blatter and Michel Platini do?', 'snippet': 'The two, once among the most powerful figures in world football, had denied the charges against them. Prosecutors had accused Blatter, a Swiss who led FIFA for 17 years, and Platini of unlawfully arranging for FIFA to pay the Frenchman two million Swiss francs (£1.7m) in 2011.', 'title': 'Sepp Blatter and Michel Platini: Former heads of FIFA and ...', 'date': 'Jul 8, 2022', 'link': 'https://www.skysports.com/football/news/11095/12648027/sepp-blatter-and-michel-platini-former-heads-of-fifa-and-uefa-cleared-of-corruption-charges-by-swiss-court#:~:text=The%20two%2C%20once%20among%20the,%C2%A31.7m)%20in%202011.', 'displayed_link': 'https://www.skysports.com › football › news › sepp-bla...', 'next_page_token': 'eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210UVkwbHVRMEpvTm1ScGRHRndNMnhHTmsxUVUwcFdRbTlFU1ZaaFIzQlBPVWxaWXpKMGVHazRkbVp3VTNWTVdreHdXa1JHUnpoU05Ga3dZbE13YkRkcVJFRjBTRkpuRWhjMFh6RXpXbTkxYTAxMFpYQTFUbTlRTFRkcFQzZEJjeG9pUVVaWWNrVmpjVjlWZUdkcU5VdzBhMGhLTVdSc1pEVnVXbkI2WlhsTmNHeFdadyIsImZjdiI6IjMiLCJlaSI6IjRfMTNab3VrTXRlcDVOb1AtN2lPd0FzIiwicWMiOiJDaE53YkdGMGFXNXBJR052Ym5SeWIzWmxjbk41RUFCOU1nYzFQdyIsInF1ZXN0aW9uIjoiV2hhdCBkaWQgU2VwcCBCbGF0dGVyIGFuZCBNaWNoZWwgUGxhdGluaSBkbz8iLCJsayI6IkdpdDNhR0YwSUdScFpDQnpaWEJ3SUdKc1lYUjBaWElnWVc1a0lHMXBZMmhsYkNCd2JHRjBhVzVwSUdSdiIsImJzIjoiYy1PeTQ1SU96MGdzVWNnc1ZnaklTU3pKek10VVNFeE9MaTFPVFZISVQ3T1hlS3BvSkNNbFZRNVZVWUNoZ3N1RVN3eXNQeU1SWVVCS2ZsNnF2Y1QxRWlOeEtkRnltR1FCa2lSWEFwY09XRmRLWm9wQ2NHcEJnWUlUVUxZa3RVZ2hNUzlGd1Rjek9TTTFCOGswZTRsT0RpTmRLZTF5bUpaaWtKWWtKQzI1RUMwSU93UVlBUSIsImlkIjoiZmNfMSJ9', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google_related_questions&google_domain=google.com&next_page_token=eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210UVkwbHVRMEpvTm1ScGRHRndNMnhHTmsxUVUwcFdRbTlFU1ZaaFIzQlBPVWxaWXpKMGVHazRkbVp3VTNWTVdreHdXa1JHUnpoU05Ga3dZbE13YkRkcVJFRjBTRkpuRWhjMFh6RXpXbTkxYTAxMFpYQTFUbTlRTFRkcFQzZEJjeG9pUVVaWWNrVmpjVjlWZUdkcU5VdzBhMGhLTVdSc1pEVnVXbkI2WlhsTmNHeFdadyIsImZjdiI6IjMiLCJlaSI6IjRfMTNab3VrTXRlcDVOb1AtN2lPd0FzIiwicWMiOiJDaE53YkdGMGFXNXBJR052Ym5SeWIzWmxjbk41RUFCOU1nYzFQdyIsInF1ZXN0aW9uIjoiV2hhdCBkaWQgU2VwcCBCbGF0dGVyIGFuZCBNaWNoZWwgUGxhdGluaSBkbz8iLCJsayI6IkdpdDNhR0YwSUdScFpDQnpaWEJ3SUdKc1lYUjBaWElnWVc1a0lHMXBZMmhsYkNCd2JHRjBhVzVwSUdSdiIsImJzIjoiYy1PeTQ1SU96MGdzVWNnc1ZnaklTU3pKek10VVNFeE9MaTFPVFZISVQ3T1hlS3BvSkNNbFZRNVZVWUNoZ3N1RVN3eXNQeU1SWVVCS2ZsNnF2Y1QxRWlOeEtkRnltR1FCa2lSWEFwY09XRmRLWm9wQ2NHcEJnWUlUVUxZa3RVZ2hNUzlGd1Rjek9TTTFCOGswZTRsT0RpTmRLZTF5bUpaaWtKWWtKQzI1RUMwSU93UVlBUSIsImlkIjoiZmNfMSJ9'}], 'answer_box': {'type': 'organic_result', 'title': 'Blatter, Platini cleared of corruption charges at FIFA trial - ESPN', 'link': 'https://www.espn.com/soccer/story/_/id/37629890/blatter-platini-cleared-corruption-charges-fifa-trial#:~:text=Prosecutors%20accused%20Blatter%20and%20Platini,FIFA%202.23%20million%20Swiss%20francs.', 'displayed_link': 'https://www.espn.com › soccer › story › blatter-platini-...', 'date': 'Jul 8, 2022', 'hover_info': [{'text': 'Blatter', 'info': 'This is the latest accepted revision, reviewed on 11 June 2024. Joseph Sepp Blatter (born Josef Blatter; 10 March 1936) is a Swiss former football administrator who served as the eighth President of FIFA from 1998 to 2015.', 'title': 'Sepp Blatter - Wikipedia', 'link': 'https://en.wikipedia.org/wiki/Sepp_Blatter', 'displayed_link': 'https://en.wikipedia.org › wiki › Sepp_Blatter'}], 'snippet': 'Prosecutors accused Blatter and Platini of unlawfully arranging for FIFA to pay the Frenchman 2 million Swiss francs ($2.06 million) in 2011. Prosecutors had asked the court to impose suspended 20-month prison sentences on both men, and to compel them to repay Zurich-based FIFA 2.23 million Swiss francs.', 'snippet_highlighted_words': ['unlawfully arranging for FIFA to pay the Frenchman 2 million Swiss francs ($2.06 million) in 2011'], 'favicon': 'https://serpapi.com/searches/6677fde192564113b8d0dd46/images/c2442f6442bbef90f0a0802b10d89bb0e538af88c37bb8c8.png', 'source': 'ESPN'}, 'organic_results': [{'position': 1, 'title': 'Sepp Blatter and Michel Platini cleared of corruption charges', 'link': 'https://www.cnbc.com/2022/07/08/sepp-blatter-and-michel-platini-cleared-of-corruption-charges.html', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.cnbc.com/2022/07/08/sepp-blatter-and-michel-platini-cleared-of-corruption-charges.html&ved=2ahUKEwjLobbuxvGGAxXXFFkFHXucA7gQFnoECB4QAQ', 'displayed_link': 'https://www.cnbc.com › 2022/07/08 › sepp-blatter-and-...', 'favicon': 'https://serpapi.com/searches/6677fde192564113b8d0dd46/images/ce23f6b57ab09b0b78474986ebec66d7929d1d5c2eee65aa2990396613b35263.png', 'date': 'Jul 8, 2022', 'snippet': 'Former FIFA president Sepp Blatter and former UEFA president Michel Platini were both cleared of corruption charges by a Swiss court on Friday.', 'snippet_highlighted_words': ['cleared of corruption charges'], 'source': 'CNBC'}, {'position': 2, 'title': 'Sepp Blatter and Michel Platini: Former heads of FIFA and ...', 'link': 'https://www.skysports.com/football/news/11095/12648027/sepp-blatter-and-michel-platini-former-heads-of-fifa-and-uefa-cleared-of-corruption-charges-by-swiss-court', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.skysports.com/football/news/11095/12648027/sepp-blatter-and-michel-platini-former-heads-of-fifa-and-uefa-cleared-of-corruption-charges-by-swiss-court&ved=2ahUKEwjLobbuxvGGAxXXFFkFHXucA7gQFnoECBwQAQ', 'displayed_link': 'https://www.skysports.com › football › news › sepp-bla...', 'favicon': 'https://serpapi.com/searches/6677fde192564113b8d0dd46/images/ce23f6b57ab09b0b78474986ebec66d7260edd010f5fd5546d144deb5e81d215.png', 'date': 'Jul 8, 2022', 'snippet': 'Former FIFA president Sepp Blatter and former UEFA president Michel Platini were both cleared of corruption charges by a Swiss court on Friday.', 'snippet_highlighted_words': ['cleared of corruption charges by a Swiss court'], 'source': 'Sky Sports'}, {'position': 3, 'title': 'Criminal Charges: Why Swiss Court Found Blatter & Platini ...', 'link': 'https://www.lawinsport.com/topics/item/criminal-charges-why-swiss-court-found-blatter-platini-not-guilty-and-what-happens-next', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.lawinsport.com/topics/item/criminal-charges-why-swiss-court-found-blatter-platini-not-guilty-and-what-happens-next&ved=2ahUKEwjLobbuxvGGAxXXFFkFHXucA7gQFnoECDUQAQ', 'displayed_link': 'https://www.lawinsport.com › topics › item › criminal-ch...', 'favicon': 'https://serpapi.com/searches/6677fde192564113b8d0dd46/images/ce23f6b57ab09b0b78474986ebec66d7b713a05c86cf892fcf161803c48b9ba8.png', 'date': 'Aug 19, 2022', 'snippet': 'This article examines the case, looking at. The Background – a hotbed of corruption allegations. The investigation. The retrospective payment.', 'snippet_highlighted_words': ['corruption allegations'], 'source': 'LawInSport'}, {'position': 4, 'title': 'Sepp Blatter and Michel Platini cleared of corruption', 'link': 'https://www.dw.com/en/fifa-trial-sepp-blatter-and-michel-platini-cleared-of-corruption/a-62404516', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.dw.com/en/fifa-trial-sepp-blatter-and-michel-platini-cleared-of-corruption/a-62404516&ved=2ahUKEwjLobbuxvGGAxXXFFkFHXucA7gQFnoECDkQAQ', 'displayed_link': 'https://www.dw.com › fifa-trial-sepp-blatter-and-michel...', 'favicon': 'https://serpapi.com/searches/6677fde192564113b8d0dd46/images/ce23f6b57ab09b0b78474986ebec66d7a8ce83c9e3609a95720611e301a5b2cb.jpeg', 'date': 'Jul 8, 2022', 'snippet': 'Former FIFA President Sepp Blatter and France footballing legend Michel Platini were acquitted on charges of illegally siphoning FIFA funds.', 'snippet_highlighted_words': ['were acquitted on charges of illegally siphoning FIFA funds'], 'source': 'DW'}, {'position': 5, 'title': 'Sepp Blatter and Michel Platini Acquitted of Fraud', 'link': 'https://www.nytimes.com/2022/07/08/sports/soccer/sepp-blatter-michel-platini-trial.html', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.nytimes.com/2022/07/08/sports/soccer/sepp-blatter-michel-platini-trial.html&ved=2ahUKEwjLobbuxvGGAxXXFFkFHXucA7gQFnoECDgQAQ', 'displayed_link': 'https://www.nytimes.com › Sports › Soccer', 'favicon': 'https://serpapi.com/searches/6677fde192564113b8d0dd46/images/ce23f6b57ab09b0b78474986ebec66d7dab6d99d59b55e7f9c2e34356e4824d8.png', 'date': 'Jul 8, 2022', 'snippet': 'Sepp Blatter, the former president of FIFA, and his onetime ally Michel Platini were acquitted of fraud on Friday in the latest attempt by ...', 'snippet_highlighted_words': ['were acquitted of fraud on'], 'source': 'The New York Times'}, {'position': 6, 'title': 'Former top football execs Blatter and Platini cleared on ...', 'link': 'https://www.politico.eu/article/football-boss-sepp-blatter-michel-platini-corruption-fifa-uefa/', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.politico.eu/article/football-boss-sepp-blatter-michel-platini-corruption-fifa-uefa/&ved=2ahUKEwjLobbuxvGGAxXXFFkFHXucA7gQFnoECDcQAQ', 'displayed_link': 'https://www.politico.eu › article › football-boss-sepp-bl...', 'favicon': 'https://serpapi.com/searches/6677fde192564113b8d0dd46/images/ce23f6b57ab09b0b78474986ebec66d76948f7d56aa1ee14392714567ba309ee.png', 'date': 'Jul 8, 2022', 'snippet': "Former FIFA boss Sepp Blatter and ex-UEFA president Michel Platini have been cleared of corruption charges, Switzerland's Federal Criminal ...", 'snippet_highlighted_words': ['been cleared of corruption charges'], 'source': 'politico.eu'}, {'position': 7, 'title': 'Ex-Fifa president Blatter and ex-Uefa boss Platini charged ...', 'link': 'https://www.bbc.com/news/world-europe-59133079', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.bbc.com/news/world-europe-59133079&ved=2ahUKEwjLobbuxvGGAxXXFFkFHXucA7gQFnoECDsQAQ', 'displayed_link': 'https://www.bbc.com › news › world-europe-59133079', 'favicon': 'https://serpapi.com/searches/6677fde192564113b8d0dd46/images/ce23f6b57ab09b0b78474986ebec66d70081d25443d2c3efa333ca28a77056b0.png', 'date': 'Nov 2, 2021', 'snippet': 'Former Fifa officials Sepp Blatter and Michel Platini have been charged with fraud and other offences in Switzerland.', 'snippet_highlighted_words': ['charged with fraud and other offences in Switzerland'], 'source': 'BBC'}, {'position': 8, 'title': 'Sepp Blatter and Michel Platini acquitted of fraud by Swiss ...', 'link': 'https://www.theguardian.com/football/2022/jul/08/sepp-blatter-and-michel-platini-acquitted-of-fraud-swiss-court', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.theguardian.com/football/2022/jul/08/sepp-blatter-and-michel-platini-acquitted-of-fraud-swiss-court&ved=2ahUKEwjLobbuxvGGAxXXFFkFHXucA7gQFnoECDYQAQ', 'displayed_link': 'https://www.theguardian.com › football › jul › sepp-bla...', 'favicon': 'https://serpapi.com/searches/6677fde192564113b8d0dd46/images/ce23f6b57ab09b0b78474986ebec66d77d7692489421f17021aa7cb773783df5.png', 'date': 'Jul 8, 2022', 'snippet': 'Sepp Blatter and Michel Platini have been acquitted of fraud in a Swiss court over a payment of 2.2m Swiss francs made in 2011.', 'snippet_highlighted_words': ['Sepp Blatter and Michel Platini have been acquitted of fraud in a Swiss'], 'source': 'The Guardian'}], 'top_stories_link': 'https://www.google.com/search?q=Platini+Controversy&sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&udm=18&sa=X&ved=2ahUKEwjLobbuxvGGAxXXFFkFHXucA7gQ8poLegQISxAF', 'top_stories_serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+Controversy', 'related_searches': [{'block_position': 1, 'query': 'platini fifa 23', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Platini+FIFA+23&sa=X&ved=2ahUKEwjLobbuxvGGAxXXFFkFHXucA7gQ1QJ6BAg6EAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+FIFA+23'}, {'block_position': 1, 'query': 'what happened to sepp blatter', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=What+happened+to+Sepp+Blatter&sa=X&ved=2ahUKEwjLobbuxvGGAxXXFFkFHXucA7gQ1QJ6BAg0EAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=What+happened+to+Sepp+Blatter'}, {'block_position': 1, 'query': 'platini fifa card', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Platini+FIFA+card&sa=X&ved=2ahUKEwjLobbuxvGGAxXXFFkFHXucA7gQ1QJ6BAgzEAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+FIFA+card'}, {'block_position': 1, 'query': 'sepp blatter net worth', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Sepp+Blatter+net+worth&sa=X&ved=2ahUKEwjLobbuxvGGAxXXFFkFHXucA7gQ1QJ6BAgyEAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Sepp+Blatter+net+worth'}, {'block_position': 1, 'query': 'fifa corruption scandal', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=FIFA+corruption+scandal&sa=X&ved=2ahUKEwjLobbuxvGGAxXXFFkFHXucA7gQ1QJ6BAgxEAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=FIFA+corruption+scandal'}, {'block_position': 1, 'query': 'fifa corruption scandal 2015', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=FIFA+corruption+scandal+2015&sa=X&ved=2ahUKEwjLobbuxvGGAxXXFFkFHXucA7gQ1QJ6BAgwEAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=FIFA+corruption+scandal+2015'}, {'block_position': 1, 'query': 'fifa president', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=FIFA+president&sa=X&ved=2ahUKEwjLobbuxvGGAxXXFFkFHXucA7gQ1QJ6BAgvEAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=FIFA+president'}, {'block_position': 1, 'query': 'fifa controversy', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=FIFA+controversy&sa=X&ved=2ahUKEwjLobbuxvGGAxXXFFkFHXucA7gQ1QJ6BAguEAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=FIFA+controversy'}], 'discussions_and_forums': [{'title': "If Sepp Blatter and Platini are fraudulent crooks, it's saying ...", 'link': 'https://www.quora.com/If-Sepp-Blatter-and-Platini-are-fraudulent-crooks-its-saying-they-have-suspect-judgment-so-why-cant-everything-they-put-in-as-FIFA-president-not-be-thrown-out', 'date': '8mo ago', 'extensions': ['2 answers'], 'source': 'Quora'}, {'title': 'Sepp Blatter and Michel Platini acquitted of corruption ... - Reddit', 'link': 'https://www.reddit.com/r/soccer/comments/vu5njd/sepp_blatter_and_michel_platini_acquitted_of/', 'date': '1y ago', 'extensions': ['r/soccer'], 'source': 'Reddit'}, {'title': 'Former FIFA president Sepp Blatter and former UEFA ... - Reddit', 'link': 'https://www.reddit.com/r/sports/comments/vu62hs/former_fifa_president_sepp_blatter_and_former/', 'date': '1y ago', 'extensions': ['r/sports'], 'source': 'Reddit'}], 'pagination': {'current': 1, 'next': 'https://www.google.com/search?q=Platini+Controversy&oq=Platini+Controversy&hl=en&gl=us&start=10&sourceid=chrome&ie=UTF-8', 'other_pages': {'2': 'https://www.google.com/search?q=Platini+Controversy&oq=Platini+Controversy&hl=en&gl=us&start=10&sourceid=chrome&ie=UTF-8', '3': 'https://www.google.com/search?q=Platini+Controversy&oq=Platini+Controversy&hl=en&gl=us&start=20&sourceid=chrome&ie=UTF-8', '4': 'https://www.google.com/search?q=Platini+Controversy&oq=Platini+Controversy&hl=en&gl=us&start=30&sourceid=chrome&ie=UTF-8', '5': 'https://www.google.com/search?q=Platini+Controversy&oq=Platini+Controversy&hl=en&gl=us&start=40&sourceid=chrome&ie=UTF-8'}}, 'serpapi_pagination': {'current': 1, 'next_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+Controversy&start=10', 'next': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+Controversy&start=10', 'other_pages': {'2': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+Controversy&start=10', '3': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+Controversy&start=20', '4': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+Controversy&start=30', '5': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+Controversy&start=40'}}}}), H2Title(title='Platini UEFA Presidency', content=None, h3_titles=None, web_search={'generated_query': 'Platini UEFA Presidency', 'results': {'search_metadata': {'id': '6677fdfb224db13c280ce5f5', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/62909b9283fe6522/6677fdfb224db13c280ce5f5.json', 'created_at': '2024-06-23 10:50:35 UTC', 'processed_at': '2024-06-23 10:50:35 UTC', 'google_url': 'https://www.google.com/search?q=Platini+UEFA+Presidency&oq=Platini+UEFA+Presidency&hl=en&gl=us&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/62909b9283fe6522/6677fdfb224db13c280ce5f5.html', 'total_time_taken': 1.85}, 'search_parameters': {'engine': 'google', 'q': 'Platini UEFA Presidency', 'google_domain': 'google.com', 'hl': 'en', 'gl': 'us', 'device': 'desktop'}, 'search_information': {'query_displayed': 'Platini UEFA Presidency', 'total_results': 400000, 'time_taken_displayed': 0.45, 'organic_results_state': 'Some results for exact spelling but showing fixed spelling', 'spelling_fix': 'Platini UEFA President', 'showing_results_for': 'Platini UEFA President'}, 'sports_results': {'title': 'Michel Platini', 'team': 'Former president of the UEFA', 'tables': [{'title': 'France · World Cup', 'games': [{'title': '19861986', 'matches_games_played': '66', 'goals_goals': '22', 'assists_assists': '--'}, {'title': '19821982', 'matches_games_played': '55', 'goals_goals': '22', 'assists_assists': '--'}, {'title': '19781978', 'matches_games_played': '33', 'goals_goals': '11', 'assists_assists': '--'}]}]}, 'knowledge_graph': {'title': 'Michel Platini', 'type': 'Former president of the UEFA', 'entity_type': 'people, athlete, soccer_player, people', 'kgmid': '/m/01fdsm', 'knowledge_graph_search_link': 'https://www.google.com/search?kgmid=/m/01fdsm&hl=en-US&q=Michel Platini', 'serpapi_knowledge_graph_search_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en-US&kgmid=%2Fm%2F01fdsm&q=Michel+Platini', 'description': "Michel François Platini is a French football administrator and former player and manager. Regarded as one of the greatest footballers of all time, Platini won the Ballon d'Or three times in a row, in 1983, 1984 and 1985, and came seventh in the FIFA Player of the Century vote.", 'source': {'name': 'Wikipedia', 'link': 'https://en.wikipedia.org/wiki/Michel_Platini'}, 'born': '1955 (age 69 years), Jœuf, France', 'born_links': [{'text': 'Jœuf, France', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=J%C5%93uf&si=ACC90nzx_D3_zUKRnpAjmO0UBLNxnt7EyN4YYdru6U3bxLI-LxvQdBRRSnQQlq7vIxw88DJUHTO63N009AlYiTewIE3c91x7FGA2Thx5A77bgpEf9jH7RPeki1QZqxUgbMyuyGDwnrWZ8nu4-0rU7qpI4rMa6zL1jbEuTwDT7ho6LX74GjxqOR8DDv3Up7JBsX9EZqMugaW9&sa=X&ved=2ahUKEwip6Kr6xvGGAxWxr4QIHbQeATcQmxMoAHoECD8QAg'}], 'dates_joined': '1982 (Juventus F.C.), 1976 (France national football team), 1972 (AS Nancy Lorraine)', 'dates_joined_links': [{'text': 'Juventus F.C.', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Juventus&si=ACC90nyvvWro6QmnyY1IfSdgk5wwjB1r8BGd_IWRjXqmKPQqm076E0SqGvBZ--FsN5xL5jFyoBDfUEBT56M47ItDximwbf8ypERic8qGdsjcDRn1eVbqqHdlRsRmooT3CRMqHXq6CnP_cSIxwQMiGQHu3tt69_5ThLko2SL60vVDGpyshggnUs_e19D7H48bfq0U6ukVovIsAWfN6R8KPqET8lgKOIMGCuM9XDPfUSO9NvnBcSS_czaJZJYj8ImeyUl-bV1iJt8PbWcSBPhMWnfTX8PKpUEZPw%3D%3D&sa=X&ved=2ahUKEwip6Kr6xvGGAxWxr4QIHbQeATcQmxMoAHoECDwQAg'}, {'text': 'France national football team', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=France+national+football+team&si=ACC90nzx_D3_zUKRnpAjmO0UBLNxnt7EyN4YYdru6U3bxLI-L3o9p6uSFMcOhGy_sKj525KYh9KnhjJ5s6XHxwLhy3nmVuca0YEahqRfhjgXM7fsIXMH7rHWtAkgijlvVKjk-rB3hdqeut2_9ppemer7QJc58NIXgTSrT9JwEIEl_ExXMVGdFcdu8M7mU3fgFoIZzWmjfye9CPGa9hcz7amSz9Xba4cFAzomUA2C_ArTzGSiof5vAUSwGpzM0JvXtGVF0WU5ZwfGzN8U-3qZbjfZVNeyGI0TPX3M60pp7CJyoc-VHLnJjT4%3D&sa=X&ved=2ahUKEwip6Kr6xvGGAxWxr4QIHbQeATcQmxMoAXoECDwQAw'}, {'text': 'AS Nancy Lorraine', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=AS+Nancy+Lorraine&si=ACC90nzx_D3_zUKRnpAjmO0UBLNxnt7EyN4YYdru6U3bxLI-L4igLxRnCsxOQ8dTh_h4Ajbfat76fjM1MuA4ZNJiT8X2Axq8X10yOQay4c0ZINEiob0DsHi3gEbdPFk9MvGN80HgOAJbBxO_ltCh_aEEbn63ofjHV1DZxWudhqvskYHVlcnJIzPKUXxNt4Gbj3V-oLMMH0wEnJwlgHAAfi9yTm-g0o63H2KRbZX9Un5nO9baeCvZvoyuyaB8B3kHlNvoleMuWmNjZvVpjdg3tQN5ZFitYLpAIg%3D%3D&sa=X&ved=2ahUKEwip6Kr6xvGGAxWxr4QIHbQeATcQmxMoAnoECDwQBA'}], 'position': 'Midfielder', 'position_links': [{'text': 'Midfielder', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=&si=ACC90nzx_D3_zUKRnpAjmO0UBLNxnt7EyN4YYdru6U3bxLI-L7WAdTL-YwWnQpdY4j2K-lFe6PkhNVQynek3Z4mN3IUjnI2V27wjtzeMfLdwkyyququ-YjNz2xISAI0Yj4t6JJM6AxKEK2iW0D-aPoEgApturWjG4Z1k25NQ8uD4sHY4cr3Wh78%3D&sa=X&ved=2ahUKEwip6Kr6xvGGAxWxr4QIHbQeATcQmxMoAHoECD0QAg'}], 'parents': 'Aldo Platini, Anna Platini', 'parents_links': [{'text': 'Aldo Platini', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Aldo+Platini&si=ACC90nxYhNno81_TzuVO0e1EieRzQwXK2DrwuMdAl3IyVcpNQ4FyHQ0IG7ls3z9f0-A7XqNVR7rMHHkXB-pKywLzoMGuQOnD6AWi2_pIZRb_7EOcdsUJAHeYX2LDhemq0xafPhoe0-m3nsZ0JMI0Av0hPL3UUapoVIYZPCE9nP4dvr_Whdg9MLNIqFrTGk4GHbYgS5c1iV2vGykdtyBIn_GQC4238BQUlNh2WSrlGs-_1Jrsad6k4le_-QkLeWiQHks1K5WwRwaRntgaHrnG3l2olAhpy8gDIw%3D%3D&sa=X&ved=2ahUKEwip6Kr6xvGGAxWxr4QIHbQeATcQmxMoAHoECEAQAg'}, {'text': 'Anna Platini', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Anna+Platini&si=ACC90nxYhNno81_TzuVO0e1EieRzQwXK2DrwuMdAl3IyVcpNQz0RKrNvs_mMGtn16CmvjDPU4aCMjQ8ohDqWxxHbYJPVrH4gwg6KtvQy3Ta9tAPXL3x0LjUVMOrJJ7ag7uSJGN72ELiUu621PHg0vWt8DccsPgs9awpabXQ6W1ce0V60HSJstKxp03-W9r_w3WLLDYk55hEQxFYgPftFj5iMpRItS4Rx2VRZRAHe9yynIeWpOBKR-hH9rtJ4QFlAOUp6tny8RCZRpD_CRikThaqQ__WY9cNsiA%3D%3D&sa=X&ved=2ahUKEwip6Kr6xvGGAxWxr4QIHbQeATcQmxMoAXoECEAQAw'}], 'spouse': 'Christelle Platini (m. 1977)', 'spouse_links': [{'text': 'Christelle Platini', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Christelle+Platini&si=ACC90ny8E30vD16OoPAAI4cStfcliGy35W8UAhb0TsHNc_ISQewA087VSDWbyOz5-3RS_0CVXF1JcD4_90KwGkF2IdpGNSFOHwwJ07IDvuoM3TOBhN2eeGEtKfYi2eb6380eRscsYLAfC6Ut1nBsLN_bE5kWGQ_wrcHp7bIysQ4E-GqhViHZ73lS0WiZCzFD0aJxlyuMMyrVYGdND7usWm_HLc-Q3ScHqg%3D%3D&sa=X&ved=2ahUKEwip6Kr6xvGGAxWxr4QIHbQeATcQmxMoAHoECD4QAg'}], 'children': 'Laurent Platini, Marine Platini', 'children_links': [{'text': 'Laurent Platini', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Laurent+Platini&si=ACC90nwLLwns5sISZcdzuISy7t-NHozt8Cbt6G3WNQfC9ekAgAuCKhUGcOl2a35eG0h_haGSCG09Ngl684dSvpax23l8hQAyxldnMV_jZpCTRNMQ6DCJO71-hb5RVHycB2ikVJ4XDRq8jAMljrbsf1zy4OtY4EPIcR-Pk2rMpa5jSbIzCMKYE3XaK2N04vlQhCEDBafUDRgiR0kaXcqi5WEnPw4w5yhoauSw3W6K_jAyyz_m6D08_k-IX_zp_W3X_9y626eep-UNmCUnzxqichfQhUL6nkax4A%3D%3D&sa=X&ved=2ahUKEwip6Kr6xvGGAxWxr4QIHbQeATcQmxMoAHoECDsQAg'}, {'text': 'Marine Platini', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Marine+Platini&si=ACC90nwLLwns5sISZcdzuISy7t-NHozt8Cbt6G3WNQfC9ekAgLL5wXOoGx8qgyyuHBItiDzKPXD_rdY16qGcpU59HclAfgdz0CYflEv6_IBPhrmODpCwRiODvtm3R_SuzKm4cWbKdedHMN6T1TAuRYJEf5UkNLOEUe_-Mk3gHF7KJHXmcm3Bkxh1Rw5z7RXq0fdCMU8Myoh4Qf9VVgnEOsTD-Ia_6f8PBWg-NlnjE46jTCrR2G5FLcm0fB9-qs9bUC1K8L_Y4LZ9N6eXJxppxRIo26mzhUeJ9A%3D%3D&sa=X&ved=2ahUKEwip6Kr6xvGGAxWxr4QIHbQeATcQmxMoAXoECDsQAw'}], 'listen': [{'name': 'Spotify', 'link': 'https://open.spotify.com/artist/6xma1aDFf3bz3c5HGRIlVr?autoplay=true', 'image': 'https://serpapi.com/searches/6677fdfb224db13c280ce5f5/images/cbbf69363db0557abe206027bb921d33a7fc1b860943dc0c24ad227668d10741.png'}, {'name': 'Apple Music', 'link': 'https://music.apple.com/us/artist/platini/1718461656', 'image': 'https://serpapi.com/searches/6677fdfb224db13c280ce5f5/images/cbbf69363db0557abe206027bb921d330b4e79a2a14b7644a3028e753a08df7b.png'}, {'name': 'Pandora', 'link': 'http://www.pandora.com/?sc=miR10880634', 'image': 'https://serpapi.com/searches/6677fdfb224db13c280ce5f5/images/cbbf69363db0557abe206027bb921d33dee972ef707c7da2775cbfbc5add3dd2.png'}], 'profiles': [{'name': 'YouTube', 'link': 'https://www.youtube.com/channel/UCsSClQ7XXRWuN0VQElQhvgQ', 'image': 'https://serpapi.com/searches/6677fdfb224db13c280ce5f5/images/cbbf69363db0557abe206027bb921d332046e5b5de7f58f951cee7d9131bbbf3d3c7733a02ad1fe1.png'}, {'name': 'Instagram', 'link': 'https://www.instagram.com/michelplatiniofficiel/', 'image': 'https://serpapi.com/searches/6677fdfb224db13c280ce5f5/images/cbbf69363db0557abe206027bb921d332046e5b5de7f58f944a906e2bfe3c5064f612a66f4c51655.png'}], 'songs': [{'name': 'Hier encore', 'extensions': 'Les Enfoirés au Grand Rex · 1994', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=les+enfoir%C3%A9s+hier+encore&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tFP1zcsNM1KKilLNtcS9C0tzkx2LCrJLC4JyQ_Oz0tfxCqZk1qskJqXlp9ZdHhlsUJGZmoRkJucX5QKADafiSlKAAAA&sa=X&ved=2ahUKEwip6Kr6xvGGAxWxr4QIHbQeATcQri56BAhCEAM', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=les+enfoir%C3%A9s+hier+encore&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tFP1zcsNM1KKilLNtcS9C0tzkx2LCrJLC4JyQ_Oz0tfxCqZk1qskJqXlp9ZdHhlsUJGZmoRkJucX5QKADafiSlKAAAA'}, {'name': "Jusqu'au dernier", 'extensions': 'Les Enfoirés 2024, On a 35 ans ! · 2024', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=les+enfoir%C3%A9s+jusqu%27au+dernier&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tVP1zc0rDTKyMrIMkjSEvQtLc5MdiwqySwuCckPzs9LX8Qql5NarJCal5afWXR4ZbFCVmlxYal6YqlCSmpRXmZqEQAFTZYRUAAAAA&sa=X&ved=2ahUKEwip6Kr6xvGGAxWxr4QIHbQeATcQri56BAhCEAU', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=les+enfoir%C3%A9s+jusqu%27au+dernier&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tVP1zc0rDTKyMrIMkjSEvQtLc5MdiwqySwuCckPzs9LX8Qql5NarJCal5afWXR4ZbFCVmlxYal6YqlCSmpRXmZqEQAFTZYRUAAAAA'}, {'name': 'La chanson des restos', 'extensions': "Tournée d'Enfoirés · 1989", 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=les+enfoir%C3%A9s+la+chanson+des+restos&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tFP1zcsNjAtMbQ0qdQS9C0tzkx2LCrJLC4JyQ_Oz0tfxKqck1qskJqXlp9ZdHhlsUJOokJyRmJecX6eQgpQoii1uCS_GAC9YiMSVAAAAA&sa=X&ved=2ahUKEwip6Kr6xvGGAxWxr4QIHbQeATcQri56BAhCEAc', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=les+enfoir%C3%A9s+la+chanson+des+restos&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tFP1zcsNjAtMbQ0qdQS9C0tzkx2LCrJLC4JyQ_Oz0tfxKqck1qskJqXlp9ZdHhlsUJOokJyRmJecX6eQgpQoii1uCS_GAC9YiMSVAAAAA', 'image': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTwT5M_653obvZxnU9qzAjBeA4Dqc4sdte-L3aa&s=0'}, {'name': 'Rêvons', 'extensions': '2023 Enfoirés un jour, toujours · 2023', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=les+enfoir%C3%A9s+r%C3%AAvons&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tVP1zc0LE4pT89NL0nTEvQtLc5MdiwqySwuCckPzs9LX8QqmpNarJCal5afWXR4ZbFC0eFVZfl5xQClALwHRwAAAA&sa=X&ved=2ahUKEwip6Kr6xvGGAxWxr4QIHbQeATcQri56BAhCEAo', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=les+enfoir%C3%A9s+r%C3%AAvons&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tVP1zc0LE4pT89NL0nTEvQtLc5MdiwqySwuCckPzs9LX8QqmpNarJCal5afWXR4ZbFC0eFVZfl5xQClALwHRwAAAA'}], 'people_also_search_for': [{'name': 'Sepp Blatter', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&si=ACC90nyvvWro6QmnyY1IfSdgk5wwjB1r8BGd_IWRjXqmKPQqm1xJY05G-_pJvUE639mUNh3pMH_KteVul5RRkXiITWKV9Oc7h43ju57hNB2q4Sl7Hj7lJ8dDWhIBfDvaJCvR4V1VUibDnvVXdmR9fMymdy_WRwRpkrAKjMd7b7UXJmSPYh3dYnskA1Rx5BRmJY_8RlL3zbY8kPrj34ip_aNaBXwQmol0h-3SyA2N96gDf3fB1Jms45izPCejqyeqTo6MnH-TCQZ0Q2g9IlIM7nWuntFnQ5u-aTxCympE9MTGj9vShOdJMM8%3D&q=Sepp+Blatter&sa=X&ved=2ahUKEwip6Kr6xvGGAxWxr4QIHbQeATcQs9oBKAB6BAg6EAI', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Sepp+Blatter&si=ACC90nyvvWro6QmnyY1IfSdgk5wwjB1r8BGd_IWRjXqmKPQqm1xJY05G-_pJvUE639mUNh3pMH_KteVul5RRkXiITWKV9Oc7h43ju57hNB2q4Sl7Hj7lJ8dDWhIBfDvaJCvR4V1VUibDnvVXdmR9fMymdy_WRwRpkrAKjMd7b7UXJmSPYh3dYnskA1Rx5BRmJY_8RlL3zbY8kPrj34ip_aNaBXwQmol0h-3SyA2N96gDf3fB1Jms45izPCejqyeqTo6MnH-TCQZ0Q2g9IlIM7nWuntFnQ5u-aTxCympE9MTGj9vShOdJMM8%3D', 'image': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRKFje-UFWkADcPBPmNS8Gb_o-xTHHjyqDiD8zlsF-srtmnqwcA3Ksv&s=0'}, {'name': 'Zinedine Zidane', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&si=ACC90nyvvWro6QmnyY1IfSdgk5wwjB1r8BGd_IWRjXqmKPQqm89rwvErki7_Dqx8AVQv16zWAHbzf1kVg7SQWh5X8WFoWEMN8qBpuCpyTgqjQTkxPoQygOWhKMoMNo0fY1KC8l3gyinUd9iLUuyozUKqJThTvJVKESHzvA454zLuATnWcrvBjQJeQIAMmicpQUWjZz51mVydU6wC-QnEgqUOkRQt0KfOBWZMt2p92P11JSNgIeEcFapZvEBzmz0WtUwfhGQ7KVSjtEzfqf7RiF8zfFGph7mB8IhUYGndtaNux3iywm6kPzw%3D&q=Zinedine+Zidane&sa=X&ved=2ahUKEwip6Kr6xvGGAxWxr4QIHbQeATcQs9oBKAF6BAg6EAM', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Zinedine+Zidane&si=ACC90nyvvWro6QmnyY1IfSdgk5wwjB1r8BGd_IWRjXqmKPQqm89rwvErki7_Dqx8AVQv16zWAHbzf1kVg7SQWh5X8WFoWEMN8qBpuCpyTgqjQTkxPoQygOWhKMoMNo0fY1KC8l3gyinUd9iLUuyozUKqJThTvJVKESHzvA454zLuATnWcrvBjQJeQIAMmicpQUWjZz51mVydU6wC-QnEgqUOkRQt0KfOBWZMt2p92P11JSNgIeEcFapZvEBzmz0WtUwfhGQ7KVSjtEzfqf7RiF8zfFGph7mB8IhUYGndtaNux3iywm6kPzw%3D', 'image': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRfht4htDMNDpJArk64A-EIUGULYMULvF3k-FiSYdEy_phKl1BbD7Rx&s=0'}, {'name': 'Christelle Platini', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&si=ACC90ny8E30vD16OoPAAI4cStfcliGy35W8UAhb0TsHNc_ISQewA087VSDWbyOz5-3RS_0Di_8Fy3vYgJX59tFrxZvviNw2cBVfD40ouh8zK_4dflrbWV5F8vrUEo11-1FRiMP-rNaWbFB1LrWPvGYm4zRmtTzMOd__-1Ey--Wf9cMMTbxfcNK1tgG1YbZWnQ6ggttYTjeAWh6V8LMFMsoIo02tKm86Smav3oP_0Pnb-TvFoYMTp89I2fCSfgbXdEUjIQ2LI-xxAg9UPer-zgRy99_OWBa1Y8Th4XtDcqLP8G7ctt1XGBQDD0j-9eHptHrga-V9ypIEm&q=Christelle+Platini&sa=X&ved=2ahUKEwip6Kr6xvGGAxWxr4QIHbQeATcQs9oBKAJ6BAg6EAQ', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Christelle+Platini&si=ACC90ny8E30vD16OoPAAI4cStfcliGy35W8UAhb0TsHNc_ISQewA087VSDWbyOz5-3RS_0Di_8Fy3vYgJX59tFrxZvviNw2cBVfD40ouh8zK_4dflrbWV5F8vrUEo11-1FRiMP-rNaWbFB1LrWPvGYm4zRmtTzMOd__-1Ey--Wf9cMMTbxfcNK1tgG1YbZWnQ6ggttYTjeAWh6V8LMFMsoIo02tKm86Smav3oP_0Pnb-TvFoYMTp89I2fCSfgbXdEUjIQ2LI-xxAg9UPer-zgRy99_OWBa1Y8Th4XtDcqLP8G7ctt1XGBQDD0j-9eHptHrga-V9ypIEm', 'image': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTjibowx_CxBLZ62XB26rPT7Okjoy6XpFwVsIaON-aZZex5o2kY_uP2&s=0'}, {'name': 'Franz Beckenbauer', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&si=ACC90nzx_D3_zUKRnpAjmO0UBLNxnt7EyN4YYdru6U3bxLI-L1ilM8ZSnnVl4jHUUZbE_kN7S_n-CQftdTGtK2CJDv1P5XOvo3bvFbggswoHYX4cMOEFC5BNtzxGLvePzuV2NfCnGTtH65QbCgIp9ZweXadkKM6s4D89_7F1omfI9a70MbHET_fP7yjzq5jj_gbhpWZ2YYrhP4Y5dNC4OEAfkbY9imy0SOxP4PkMrkogBvov1C7mu_omgELSgIpL5utJxAMoRjGFR_7KOI3jHZ-YGmkp766FUP-5_58RVvQnStymwoMjH-E%3D&q=Franz+Beckenbauer&sa=X&ved=2ahUKEwip6Kr6xvGGAxWxr4QIHbQeATcQs9oBKAN6BAg6EAU', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Franz+Beckenbauer&si=ACC90nzx_D3_zUKRnpAjmO0UBLNxnt7EyN4YYdru6U3bxLI-L1ilM8ZSnnVl4jHUUZbE_kN7S_n-CQftdTGtK2CJDv1P5XOvo3bvFbggswoHYX4cMOEFC5BNtzxGLvePzuV2NfCnGTtH65QbCgIp9ZweXadkKM6s4D89_7F1omfI9a70MbHET_fP7yjzq5jj_gbhpWZ2YYrhP4Y5dNC4OEAfkbY9imy0SOxP4PkMrkogBvov1C7mu_omgELSgIpL5utJxAMoRjGFR_7KOI3jHZ-YGmkp766FUP-5_58RVvQnStymwoMjH-E%3D', 'image': 'https://encrypted-tbn0.gstatic.com/licensed-image?q=tbn:ANd9GcR7tnE0tNaEVhGhMFYGCBr7BVDp3FiYzjhwYXt6byfVTfNSW3HlfBOg_TEFEk08hpgTkpIle78&s=19'}]}, 'related_questions': [{'question': 'What was Michel Platini accused of?', 'snippet': 'Blatter, Platini cleared of corruption at FIFA trial Platini, a former France national team captain and manager, was also acquitted of fraud. Prosecutors accused Blatter and Platini of unlawfully arranging for FIFA to pay the Frenchman 2 million Swiss francs ($2.06 million) in 2011.', 'title': 'unlawfully arranging for FIFA', 'date': 'Jul 8, 2022', 'link': 'https://www.espn.com/soccer/story/_/id/37629890/blatter-platini-cleared-corruption-charges-fifa-trial#:~:text=Blatter%2C%20Platini%20cleared%20of%20corruption%20at%20FIFA%20trial&text=Platini%2C%20a%20former%20France%20national,(%242.06%20million)%20in%202011.', 'displayed_link': 'https://www.espn.com › soccer › story › blatter-platini-cl...', 'source_logo': 'https://serpapi.com/searches/6677fdfb224db13c280ce5f5/images/9139fc04705a8cc5397390671da5f0a843663b300954a4a36f61e43006295af5.png', 'next_page_token': 'eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210TlIwWXRjVzFIVFZOdE1sRjFhemhDWXpKaVRHbE1TMmhuVVhCTFVUUTJVRlZhTkZGT2NqWnpaa1F5U0RsellXNXdUSFJuVTFwaE5XaFlTRk5TYm5WbllVZHRjVFZ5RWhkZlVERXpXbkZ1TmsxTVNHWnJkbEZRZEV3eVJYVkJUUm9pUVVaWWNrVmpiMnRYWVdoQ1F6TnhNbGhZZFVWeE4yODFNMmxtUkZwcVRFVkJVUSIsImZjdiI6IjMiLCJlaSI6Il9QMTNacW42TUxIZmt2UVB0TDJFdUFNIiwicWMiOiJDaFp3YkdGMGFXNXBJSFZsWm1FZ2NISmxjMmxrWlc1MEVBQjlzbXcyUHciLCJxdWVzdGlvbiI6IldoYXQgd2FzIE1pY2hlbCBQbGF0aW5pIGFjY3VzZWQgb2Y/IiwibGsiOiJHaUozYUdGMElIZGhjeUJ0YVdOb1pXd2djR3hoZEdsdWFTQmhZMk4xYzJWa0lHOW0iLCJicyI6ImM0M09NUXJDTUJUR2NWd3pPWmFJOEtnS0trNjlRQkhVUlJSQnBPZ2lzWGtsZ2RxV05EWGdHUnk4Z0FmeVZwcEtpOFhGN2NINzhmRW5hOUlMQk5OZ1dBNHJHUXFNWVJNekxSTUpMQXlMSERta2tlX2M5bDZmdXFhUzU0X01maVE1a0ZFZ1VwQTVhSUVRRmtwaG9pRlRtRXR1cnpRcUg3djVZdW83ejRFM3BrUHpoeTh3WW1SSlhMdHRBMDZvTmFwSjNab3FPRWpPRXZTZHg5RnpLWmdHckV2ZjdsbzZNaU1RQ0V5QVMxNnZ4TWd1LU5WMzMzcGQyakVWeXhyTUpyVmJMdyIsImlkIjoiZmNfMSJ9', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google_related_questions&google_domain=google.com&next_page_token=eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210TlIwWXRjVzFIVFZOdE1sRjFhemhDWXpKaVRHbE1TMmhuVVhCTFVUUTJVRlZhTkZGT2NqWnpaa1F5U0RsellXNXdUSFJuVTFwaE5XaFlTRk5TYm5WbllVZHRjVFZ5RWhkZlVERXpXbkZ1TmsxTVNHWnJkbEZRZEV3eVJYVkJUUm9pUVVaWWNrVmpiMnRYWVdoQ1F6TnhNbGhZZFVWeE4yODFNMmxtUkZwcVRFVkJVUSIsImZjdiI6IjMiLCJlaSI6Il9QMTNacW42TUxIZmt2UVB0TDJFdUFNIiwicWMiOiJDaFp3YkdGMGFXNXBJSFZsWm1FZ2NISmxjMmxrWlc1MEVBQjlzbXcyUHciLCJxdWVzdGlvbiI6IldoYXQgd2FzIE1pY2hlbCBQbGF0aW5pIGFjY3VzZWQgb2Y%2FIiwibGsiOiJHaUozYUdGMElIZGhjeUJ0YVdOb1pXd2djR3hoZEdsdWFTQmhZMk4xYzJWa0lHOW0iLCJicyI6ImM0M09NUXJDTUJUR2NWd3pPWmFJOEtnS0trNjlRQkhVUlJSQnBPZ2lzWGtsZ2RxV05EWGdHUnk4Z0FmeVZwcEtpOFhGN2NINzhmRW5hOUlMQk5OZ1dBNHJHUXFNWVJNekxSTUpMQXlMSERta2tlX2M5bDZmdXFhUzU0X01maVE1a0ZFZ1VwQTVhSUVRRmtwaG9pRlRtRXR1cnpRcUg3djVZdW83ejRFM3BrUHpoeTh3WW1SSlhMdHRBMDZvTmFwSjNab3FPRWpPRXZTZHg5RnpLWmdHckV2ZjdsbzZNaU1RQ0V5QVMxNnZ4TWd1LU5WMzMzcGQyakVWeXhyTUpyVmJMdyIsImlkIjoiZmNfMSJ9'}, {'question': 'Who is the current president of the UEFA?', 'snippet': "UEFA president Aleksander Ceferin will not stand for a fourth term in 2027, despite winning a contentious vote on the matter at the European governing body's annual congress in Paris on Thursday.", 'title': 'Aleksander Čeferin', 'date': 'Feb 8, 2024', 'info': 'Union of European Football Associations / President', 'link': 'https://www.nytimes.com/athletic/5259860/2024/02/08/aleksander-ceferin-uefa-president-2027/#:~:text=UEFA%20president%20Aleksander%20Ceferin%20will,congress%20in%20Paris%20on%20Thursday.', 'displayed_link': 'https://www.nytimes.com › athletic › 2024/02/08 › aleks...', 'source_logo': 'https://serpapi.com/searches/6677fdfb224db13c280ce5f5/images/9139fc04705a8cc5397390671da5f0a87fc6d54918f3e87337f501da7bfcbc78.png', 'next_page_token': 'eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210TlIwWXRjVzFIVFZOdE1sRjFhemhDWXpKaVRHbE1TMmhuVVhCTFVUUTJVRlZhTkZGT2NqWnpaa1F5U0RsellXNXdUSFJuVTFwaE5XaFlTRk5TYm5WbllVZHRjVFZ5RWhkZlVERXpXbkZ1TmsxTVNHWnJkbEZRZEV3eVJYVkJUUm9pUVVaWWNrVmpiMnRYWVdoQ1F6TnhNbGhZZFVWeE4yODFNMmxtUkZwcVRFVkJVUSIsImZjdiI6IjMiLCJlaSI6Il9QMTNacW42TUxIZmt2UVB0TDJFdUFNIiwicWMiOiJDaFp3YkdGMGFXNXBJSFZsWm1FZ2NISmxjMmxrWlc1MEVBQjlzbXcyUHciLCJxdWVzdGlvbiI6IldobyBpcyB0aGUgY3VycmVudCBwcmVzaWRlbnQgb2YgdGhlIFVFRkE/IiwibGsiOiJjNVBTS01fSVY4Z3NWaWpKU0ZWSUxpMHFTczByVVNnb1NpM09UQUd4OHRQQUVxV3BhWWtBIiwiYnMiOiJjNDNPTVFyQ01CVEdjVnd6T1phSThLZ0tLazY5UUJIVVJSUkJwT2dpc1hrbGdkcVdORFhnR1J5OGdBZnlWcHBLaThYRjdjSDc4ZkVuYTlJTEJOTmdXQTRyR1FxTVlSTXpMUk1KTEF5TEhEbWtrZV9jOWw2ZnVxYVM1NF9NZmlRNWtGRWdVcEE1YUlFUUZrcGhvaUZUbUV0dXJ6UXFIN3Y1WXVvN3o0RTNwa1B6aHk4d1ltUkpYTHR0QTA2b05hcEozWm9xT0VqT0V2U2R4OUZ6S1pnR3JFdmY3bG82TWlNUUNFeUFTMTZ2eE1ndS1OVjMzM3BkMmpFVnl4ck1KclZiTHciLCJpZCI6ImZjXzEifQ==', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google_related_questions&google_domain=google.com&next_page_token=eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210TlIwWXRjVzFIVFZOdE1sRjFhemhDWXpKaVRHbE1TMmhuVVhCTFVUUTJVRlZhTkZGT2NqWnpaa1F5U0RsellXNXdUSFJuVTFwaE5XaFlTRk5TYm5WbllVZHRjVFZ5RWhkZlVERXpXbkZ1TmsxTVNHWnJkbEZRZEV3eVJYVkJUUm9pUVVaWWNrVmpiMnRYWVdoQ1F6TnhNbGhZZFVWeE4yODFNMmxtUkZwcVRFVkJVUSIsImZjdiI6IjMiLCJlaSI6Il9QMTNacW42TUxIZmt2UVB0TDJFdUFNIiwicWMiOiJDaFp3YkdGMGFXNXBJSFZsWm1FZ2NISmxjMmxrWlc1MEVBQjlzbXcyUHciLCJxdWVzdGlvbiI6IldobyBpcyB0aGUgY3VycmVudCBwcmVzaWRlbnQgb2YgdGhlIFVFRkE%2FIiwibGsiOiJjNVBTS01fSVY4Z3NWaWpKU0ZWSUxpMHFTczByVVNnb1NpM09UQUd4OHRQQUVxV3BhWWtBIiwiYnMiOiJjNDNPTVFyQ01CVEdjVnd6T1phSThLZ0tLazY5UUJIVVJSUkJwT2dpc1hrbGdkcVdORFhnR1J5OGdBZnlWcHBLaThYRjdjSDc4ZkVuYTlJTEJOTmdXQTRyR1FxTVlSTXpMUk1KTEF5TEhEbWtrZV9jOWw2ZnVxYVM1NF9NZmlRNWtGRWdVcEE1YUlFUUZrcGhvaUZUbUV0dXJ6UXFIN3Y1WXVvN3o0RTNwa1B6aHk4d1ltUkpYTHR0QTA2b05hcEozWm9xT0VqT0V2U2R4OUZ6S1pnR3JFdmY3bG82TWlNUUNFeUFTMTZ2eE1ndS1OVjMzM3BkMmpFVnl4ck1KclZiTHciLCJpZCI6ImZjXzEifQ%3D%3D'}, {'question': 'Who was better, Platini or Zidane?', 'snippet': 'Platini was a better player relative to his era but on paper, Zidane had a better and more complete career. Same amount of Champions League/European Cups but Zidane had tougher opponents. Platini scored a penalty in the final and Zidane scored one of the best CL goals ever in the final. Both made the final twice.', 'title': "Hot take: Platini's career was better and more complete then Zidane's", 'date': 'Jul 26, 2023', 'link': 'https://www.reddit.com/r/football/comments/159ywh9/hot_take_platinis_career_was_better_and_more/#:~:text=Platini%20was%20a%20better%20player,Both%20made%20the%20final%20twice.', 'displayed_link': 'https://www.reddit.com › football › comments › hot_tak...', 'source_logo': 'https://serpapi.com/searches/6677fdfb224db13c280ce5f5/images/9139fc04705a8cc5397390671da5f0a819e9ae1b47b25c0169b7e4d2c817b7e0.png', 'next_page_token': 'eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210TlIwWXRjVzFIVFZOdE1sRjFhemhDWXpKaVRHbE1TMmhuVVhCTFVUUTJVRlZhTkZGT2NqWnpaa1F5U0RsellXNXdUSFJuVTFwaE5XaFlTRk5TYm5WbllVZHRjVFZ5RWhkZlVERXpXbkZ1TmsxTVNHWnJkbEZRZEV3eVJYVkJUUm9pUVVaWWNrVmpiMnRYWVdoQ1F6TnhNbGhZZFVWeE4yODFNMmxtUkZwcVRFVkJVUSIsImZjdiI6IjMiLCJlaSI6Il9QMTNacW42TUxIZmt2UVB0TDJFdUFNIiwicWMiOiJDaFp3YkdGMGFXNXBJSFZsWm1FZ2NISmxjMmxrWlc1MEVBQjlzbXcyUHciLCJxdWVzdGlvbiI6IldobyB3YXMgYmV0dGVyLCBQbGF0aW5pIG9yIFppZGFuZT8iLCJsayI6IkdpQjNhRzhnZDJGeklHSmxkSFJsY2lCd2JHRjBhVzVwSUc5eUlIcHBaR0Z1WlEiLCJicyI6ImM0M09NUXJDTUJUR2NWd3pPWmFJOEtnS0trNjlRQkhVUlJSQnBPZ2lzWGtsZ2RxV05EWGdHUnk4Z0FmeVZwcEtpOFhGN2NINzhmRW5hOUlMQk5OZ1dBNHJHUXFNWVJNekxSTUpMQXlMSERta2tlX2M5bDZmdXFhUzU0X01maVE1a0ZFZ1VwQTVhSUVRRmtwaG9pRlRtRXR1cnpRcUg3djVZdW83ejRFM3BrUHpoeTh3WW1SSlhMdHRBMDZvTmFwSjNab3FPRWpPRXZTZHg5RnpLWmdHckV2ZjdsbzZNaU1RQ0V5QVMxNnZ4TWd1LU5WMzMzcGQyakVWeXhyTUpyVmJMdyIsImlkIjoiZmNfMSJ9', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google_related_questions&google_domain=google.com&next_page_token=eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210TlIwWXRjVzFIVFZOdE1sRjFhemhDWXpKaVRHbE1TMmhuVVhCTFVUUTJVRlZhTkZGT2NqWnpaa1F5U0RsellXNXdUSFJuVTFwaE5XaFlTRk5TYm5WbllVZHRjVFZ5RWhkZlVERXpXbkZ1TmsxTVNHWnJkbEZRZEV3eVJYVkJUUm9pUVVaWWNrVmpiMnRYWVdoQ1F6TnhNbGhZZFVWeE4yODFNMmxtUkZwcVRFVkJVUSIsImZjdiI6IjMiLCJlaSI6Il9QMTNacW42TUxIZmt2UVB0TDJFdUFNIiwicWMiOiJDaFp3YkdGMGFXNXBJSFZsWm1FZ2NISmxjMmxrWlc1MEVBQjlzbXcyUHciLCJxdWVzdGlvbiI6IldobyB3YXMgYmV0dGVyLCBQbGF0aW5pIG9yIFppZGFuZT8iLCJsayI6IkdpQjNhRzhnZDJGeklHSmxkSFJsY2lCd2JHRjBhVzVwSUc5eUlIcHBaR0Z1WlEiLCJicyI6ImM0M09NUXJDTUJUR2NWd3pPWmFJOEtnS0trNjlRQkhVUlJSQnBPZ2lzWGtsZ2RxV05EWGdHUnk4Z0FmeVZwcEtpOFhGN2NINzhmRW5hOUlMQk5OZ1dBNHJHUXFNWVJNekxSTUpMQXlMSERta2tlX2M5bDZmdXFhUzU0X01maVE1a0ZFZ1VwQTVhSUVRRmtwaG9pRlRtRXR1cnpRcUg3djVZdW83ejRFM3BrUHpoeTh3WW1SSlhMdHRBMDZvTmFwSjNab3FPRWpPRXZTZHg5RnpLWmdHckV2ZjdsbzZNaU1RQ0V5QVMxNnZ4TWd1LU5WMzMzcGQyakVWeXhyTUpyVmJMdyIsImlkIjoiZmNfMSJ9'}, {'question': 'When did Platini leave the UEFA?', 'snippet': 'Term of office: 26 January 2007 – 14 September 2016 Michel Platini embarked on a career as a football administrator, in France as well as at FIFA and UEFA, after an outstanding career as both a player and a coach.', 'title': 'Previous UEFA presidents', 'date': 'Apr 5, 2023', 'link': 'https://www.uefa.com/news-media/news/0280-17b4199e535e-64d8a4443b2c-1000--previous-uefa-presidents/#:~:text=Term%20of%20office%3A%2026%20January,a%20player%20and%20a%20coach.', 'displayed_link': 'https://www.uefa.com › news-media › news › 0280-17...', 'next_page_token': 'eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210TlIwWXRjVzFIVFZOdE1sRjFhemhDWXpKaVRHbE1TMmhuVVhCTFVUUTJVRlZhTkZGT2NqWnpaa1F5U0RsellXNXdUSFJuVTFwaE5XaFlTRk5TYm5WbllVZHRjVFZ5RWhkZlVERXpXbkZ1TmsxTVNHWnJkbEZRZEV3eVJYVkJUUm9pUVVaWWNrVmpiMnRYWVdoQ1F6TnhNbGhZZFVWeE4yODFNMmxtUkZwcVRFVkJVUSIsImZjdiI6IjMiLCJlaSI6Il9QMTNacW42TUxIZmt2UVB0TDJFdUFNIiwicWMiOiJDaFp3YkdGMGFXNXBJSFZsWm1FZ2NISmxjMmxrWlc1MEVBQjlzbXcyUHciLCJxdWVzdGlvbiI6IldoZW4gZGlkIFBsYXRpbmkgbGVhdmUgdGhlIFVFRkE/IiwibGsiOiJHaHQzYUdWdUlHUnBaQ0J3YkdGMGFXNXBJR3hsWVhabElIVmxabUUiLCJicyI6ImM0M09NUXJDTUJUR2NWd3pPWmFJOEtnS0trNjlRQkhVUlJSQnBPZ2lzWGtsZ2RxV05EWGdHUnk4Z0FmeVZwcEtpOFhGN2NINzhmRW5hOUlMQk5OZ1dBNHJHUXFNWVJNekxSTUpMQXlMSERta2tlX2M5bDZmdXFhUzU0X01maVE1a0ZFZ1VwQTVhSUVRRmtwaG9pRlRtRXR1cnpRcUg3djVZdW83ejRFM3BrUHpoeTh3WW1SSlhMdHRBMDZvTmFwSjNab3FPRWpPRXZTZHg5RnpLWmdHckV2ZjdsbzZNaU1RQ0V5QVMxNnZ4TWd1LU5WMzMzcGQyakVWeXhyTUpyVmJMdyIsImlkIjoiZmNfMSJ9', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google_related_questions&google_domain=google.com&next_page_token=eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210TlIwWXRjVzFIVFZOdE1sRjFhemhDWXpKaVRHbE1TMmhuVVhCTFVUUTJVRlZhTkZGT2NqWnpaa1F5U0RsellXNXdUSFJuVTFwaE5XaFlTRk5TYm5WbllVZHRjVFZ5RWhkZlVERXpXbkZ1TmsxTVNHWnJkbEZRZEV3eVJYVkJUUm9pUVVaWWNrVmpiMnRYWVdoQ1F6TnhNbGhZZFVWeE4yODFNMmxtUkZwcVRFVkJVUSIsImZjdiI6IjMiLCJlaSI6Il9QMTNacW42TUxIZmt2UVB0TDJFdUFNIiwicWMiOiJDaFp3YkdGMGFXNXBJSFZsWm1FZ2NISmxjMmxrWlc1MEVBQjlzbXcyUHciLCJxdWVzdGlvbiI6IldoZW4gZGlkIFBsYXRpbmkgbGVhdmUgdGhlIFVFRkE%2FIiwibGsiOiJHaHQzYUdWdUlHUnBaQ0J3YkdGMGFXNXBJR3hsWVhabElIVmxabUUiLCJicyI6ImM0M09NUXJDTUJUR2NWd3pPWmFJOEtnS0trNjlRQkhVUlJSQnBPZ2lzWGtsZ2RxV05EWGdHUnk4Z0FmeVZwcEtpOFhGN2NINzhmRW5hOUlMQk5OZ1dBNHJHUXFNWVJNekxSTUpMQXlMSERta2tlX2M5bDZmdXFhUzU0X01maVE1a0ZFZ1VwQTVhSUVRRmtwaG9pRlRtRXR1cnpRcUg3djVZdW83ejRFM3BrUHpoeTh3WW1SSlhMdHRBMDZvTmFwSjNab3FPRWpPRXZTZHg5RnpLWmdHckV2ZjdsbzZNaU1RQ0V5QVMxNnZ4TWd1LU5WMzMzcGQyakVWeXhyTUpyVmJMdyIsImlkIjoiZmNfMSJ9'}], 'answer_box': {'type': 'organic_result', 'title': 'Michel Platini | Biography, FIFA, & Facts - Britannica', 'link': 'https://www.britannica.com/biography/Michel-Platini#:~:text=Michel%20Platini%20(born%20June%2021,UEFA%3B%202007%E2%80%9316).', 'displayed_link': 'https://www.britannica.com › ... › Other Sports', 'date': '6 days ago', 'snippet': 'Michel Platini (born June 21, 1955, Joeuf, France) is a French professional football (soccer) player and administrator who was named the European Footballer of the Year three times (1983–85) and served as president of the Union of European Football Associations (UEFA; 2007–16).', 'snippet_highlighted_words': ['2007–16'], 'favicon': 'https://serpapi.com/searches/6677fdfb224db13c280ce5f5/images/095fa1670525e3664840003c4d99c017749ff6acc36a4d10.png', 'source': 'Britannica', 'images': ['https://serpapi.com/searches/6677fdfb224db13c280ce5f5/images/095fa1670525e366f0be090dac6416890aa5660d396e4b38.jpeg', 'https://serpapi.com/searches/6677fdfb224db13c280ce5f5/images/095fa1670525e366f0be090dac641689f145d3b7ff11100d.jpeg']}, 'organic_results': [{'position': 1, 'title': 'Michel Platini', 'link': 'https://en.wikipedia.org/wiki/Michel_Platini', 'redirect_link': 'https://www.google.com/url?sa=t&url=https://en.wikipedia.org/wiki/Michel_Platini&usg=AOvVaw0TVBgm3Q_6A-CURhLIQWhw', 'displayed_link': 'https://en.wikipedia.org › wiki › Michel_Platini', 'favicon': 'https://serpapi.com/searches/6677fdfb224db13c280ce5f5/images/95be797ace94db9fe821411da73726490366d1b9f50dc52fc2960ae4a3fd5728.png', 'snippet': 'In 2007, he became the first former player to be elected as the president of the Union of European Football Associations (UEFA). ... FIFA president, Platini was ...', 'snippet_highlighted_words': ['FIFA president', 'Platini'], 'source': 'Wikipedia'}, {'position': 2, 'title': 'Previous UEFA presidents', 'link': 'https://www.uefa.com/news-media/news/0280-17b4199e535e-64d8a4443b2c-1000--previous-uefa-presidents/', 'redirect_link': 'https://www.google.com/url?sa=t&url=https://www.uefa.com/news-media/news/0280-17b4199e535e-64d8a4443b2c-1000--previous-uefa-presidents/&usg=AOvVaw1BhKt5_kQWml-DM0BFP9zn', 'displayed_link': 'https://www.uefa.com › news-media › news › 0280-17...', 'favicon': 'https://serpapi.com/searches/6677fdfb224db13c280ce5f5/images/95be797ace94db9fe821411da7372649a75d8a7bd2201b62f0ba3c682dec78d7.png', 'date': 'Apr 5, 2023', 'snippet': 'Michel Platini resigned as UEFA president on 14 September 2016. Lennart Johansson (Sweden). Term of office: 19 April 1990 – 26 January 2007', 'snippet_highlighted_words': ['Michel Platini resigned as UEFA president on 14 September 2016'], 'source': 'UEFA.com'}, {'position': 3, 'title': 'Ex-Fifa president Blatter and ex-Uefa boss Platini charged ...', 'link': 'https://www.bbc.com/news/world-europe-59133079', 'redirect_link': 'https://www.google.com/url?sa=t&url=https://www.bbc.com/news/world-europe-59133079&usg=AOvVaw1LY2i9u32BCJqh8JkrX9vn', 'displayed_link': 'https://www.bbc.com › news › world-europe-59133079', 'favicon': 'https://serpapi.com/searches/6677fdfb224db13c280ce5f5/images/95be797ace94db9fe821411da7372649055e6a1f0169399ebb778dd1b7ca235b.png', 'date': 'Nov 2, 2021', 'snippet': 'Sepp Blatter and Michel Platini, former world and European football heads, are charged with fraud.', 'snippet_highlighted_words': ['charged with fraud'], 'source': 'BBC'}, {'position': 4, 'title': 'Michel Platini to stand for the FIFA presidency', 'link': 'https://www.uefa.com/news-media/news/0223-0f8aa042d199-765034b57c20-1000--michel-platini-to-stand-for-the-fifa-presidency/', 'redirect_link': 'https://www.google.com/url?sa=t&url=https://www.uefa.com/news-media/news/0223-0f8aa042d199-765034b57c20-1000--michel-platini-to-stand-for-the-fifa-presidency/&usg=AOvVaw30S6um7HbFF9qhyUQ5r9ZD', 'displayed_link': 'https://www.uefa.com › news-media › news › 0223-0f8...', 'favicon': 'https://serpapi.com/searches/6677fdfb224db13c280ce5f5/images/95be797ace94db9fe821411da7372649a433734ecb67e78056e2ab1800e7ccff.png', 'date': 'Jul 29, 2015', 'snippet': 'The President of UEFA, Michel Platini, has today announced his intention to stand as a candidate for the FIFA presidency.', 'snippet_highlighted_words': ['Michel Platini'], 'source': 'UEFA.com'}, {'position': 5, 'title': "Michel Platini: The Former UEFA President's Controversial ...", 'link': 'https://www.si.com/soccer/2019/06/18/michel-platini-former-uefa-presidents-controversial-career-football-politics', 'redirect_link': 'https://www.google.com/url?sa=t&url=https://www.si.com/soccer/2019/06/18/michel-platini-former-uefa-presidents-controversial-career-football-politics&usg=AOvVaw3pyhrxDkY5Bjby0UBFPVpw', 'displayed_link': 'https://www.si.com › soccer › 2019/06/18 › michel-plat...', 'favicon': 'https://serpapi.com/searches/6677fdfb224db13c280ce5f5/images/95be797ace94db9fe821411da7372649f15547f883e343c3f85566bfcfafda7e.png', 'date': 'Jun 18, 2019', 'snippet': 'The former UEFA president was recently arrested over allegations of corruption in the awarding of the 2022 World Cup to Qatar, but this is ...', 'snippet_highlighted_words': ['recently arrested over allegations of corruption in the awarding of the 2022 World'], 'source': 'Sports Illustrated'}, {'position': 6, 'title': 'Michel Platini resigns as Uefa president after ban reduced ...', 'link': 'https://www.theguardian.com/football/2016/may/09/michel-platini-cas-appeal-ban-football', 'redirect_link': 'https://www.google.com/url?sa=t&url=https://www.theguardian.com/football/2016/may/09/michel-platini-cas-appeal-ban-football&usg=AOvVaw3avNTCJuR9fS-83v1ct_kq', 'displayed_link': 'https://www.theguardian.com › football › may › michel...', 'favicon': 'https://serpapi.com/searches/6677fdfb224db13c280ce5f5/images/95be797ace94db9fe821411da7372649280c655f9d00f6e5b064ad2caacee5ae.png', 'date': 'May 9, 2016', 'snippet': 'The Uefa president, Michel Platini, will finally resign from the post after losing his battle to appeal against his ban from all football over a ...', 'snippet_highlighted_words': ['Michel Platini, will finally resign from the post'], 'source': 'The Guardian'}, {'position': 7, 'title': 'Sepp Blatter and Michel Platini: Former heads of FIFA and ...', 'link': 'https://www.skysports.com/football/news/11095/12648027/sepp-blatter-and-michel-platini-former-heads-of-fifa-and-uefa-cleared-of-corruption-charges-by-swiss-court', 'redirect_link': 'https://www.google.com/url?sa=t&url=https://www.skysports.com/football/news/11095/12648027/sepp-blatter-and-michel-platini-former-heads-of-fifa-and-uefa-cleared-of-corruption-charges-by-swiss-court&usg=AOvVaw3ocsgl5HQwzediXuwpOy9H', 'displayed_link': 'https://www.skysports.com › football › news › sepp-bla...', 'favicon': 'https://serpapi.com/searches/6677fdfb224db13c280ce5f5/images/95be797ace94db9fe821411da737264972835769a4015f1b2652f6c8bad127dd.png', 'date': 'Jul 8, 2022', 'snippet': 'Sepp Blatter was head of FIFA for 17 years before being forced out in December 2015; Michel Platini ran UEFA for eight years before leaving ...', 'snippet_highlighted_words': ['Sepp Blatter'], 'source': 'Sky Sports'}, {'position': 8, 'title': 'Former UEFA president Michel Platini files criminal ...', 'link': 'https://www.espn.com/soccer/story/_/id/37627101/former-uefa-president-michel-platini-files-criminal-complaint-fifa-gianni-infantino', 'redirect_link': 'https://www.google.com/url?sa=t&url=https://www.espn.com/soccer/story/_/id/37627101/former-uefa-president-michel-platini-files-criminal-complaint-fifa-gianni-infantino&usg=AOvVaw2Vw3EoWqIm9cnIMrIuQGfI', 'displayed_link': 'https://www.espn.com › soccer › story › former-uefa-pr...', 'favicon': 'https://serpapi.com/searches/6677fdfb224db13c280ce5f5/images/95be797ace94db9fe821411da7372649d2a44bcdce470d456ece5872cee907f3.png', 'date': 'Apr 5, 2022', 'snippet': 'Former France great Michel Platini has filed a criminal complaint against FIFA president Gianni Infantino.', 'snippet_highlighted_words': ['a criminal complaint against FIFA president Gianni Infantino'], 'source': 'ESPN'}, {'position': 9, 'title': 'Michel Platini - latest news, breaking stories and comment', 'link': 'https://www.independent.co.uk/topic/michel-platini', 'redirect_link': 'https://www.google.com/url?sa=t&url=https://www.independent.co.uk/topic/michel-platini&usg=AOvVaw09CTlBTFOPt90xjDsUZpHp', 'displayed_link': 'https://www.independent.co.uk › topic › michel-platini', 'favicon': 'https://serpapi.com/searches/6677fdfb224db13c280ce5f5/images/95be797ace94db9fe821411da7372649aba08c101f19afeb143d56c9ed0f34de.png', 'snippet': "European · Aleksander Ceferin elected to succeed Michel Platini as Uefa president · Uefa president Michel Platini has been banned from this weekend's Euro 2016 ...", 'snippet_highlighted_words': ['Aleksander Ceferin'], 'source': 'The Independent'}], 'related_searches': [{'block_position': 1, 'query': "platini ballon d'or", 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=platini+ballon+d%27or&sa=X&ved=2ahUKEwip6Kr6xvGGAxWxr4QIHbQeATcQ1QJ6BAhfEAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=platini+ballon+d%27or'}, {'block_position': 1, 'query': 'Michel Platini clubs', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Michel+Platini+clubs&sa=X&ved=2ahUKEwip6Kr6xvGGAxWxr4QIHbQeATcQ1QJ6BAhcEAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Michel+Platini+clubs'}, {'block_position': 1, 'query': 'When did Platini retire', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=When+did+Platini+retire&sa=X&ved=2ahUKEwip6Kr6xvGGAxWxr4QIHbQeATcQ1QJ6BAhYEAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=When+did+Platini+retire'}, {'block_position': 1, 'query': 'Michel platini age', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Michel+platini+age&sa=X&ved=2ahUKEwip6Kr6xvGGAxWxr4QIHbQeATcQ1QJ6BAhXEAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Michel+platini+age'}, {'block_position': 1, 'query': 'Michel Platini Net worth', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Michel+Platini+Net+worth&sa=X&ved=2ahUKEwip6Kr6xvGGAxWxr4QIHbQeATcQ1QJ6BAhWEAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Michel+Platini+Net+worth'}, {'block_position': 1, 'query': 'Michel platini juventus number', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Michel+platini+juventus+number&sa=X&ved=2ahUKEwip6Kr6xvGGAxWxr4QIHbQeATcQ1QJ6BAhVEAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Michel+platini+juventus+number'}, {'block_position': 1, 'query': 'Platini nationality', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Platini+nationality&sa=X&ved=2ahUKEwip6Kr6xvGGAxWxr4QIHbQeATcQ1QJ6BAhUEAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+nationality'}, {'block_position': 1, 'query': 'Did Platini win the World Cup', 'link': 'https://www.google.com/search?sca_esv=8f0324ad6f2b8a65&gl=us&hl=en&q=Did+Platini+win+the+World+Cup&sa=X&ved=2ahUKEwip6Kr6xvGGAxWxr4QIHbQeATcQ1QJ6BAhTEAE', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Did+Platini+win+the+World+Cup'}], 'pagination': {'current': 1, 'next': 'https://www.google.com/search?q=Platini+UEFA+Presidency&oq=Platini+UEFA+Presidency&hl=en&gl=us&start=10&sourceid=chrome&ie=UTF-8', 'other_pages': {'2': 'https://www.google.com/search?q=Platini+UEFA+Presidency&oq=Platini+UEFA+Presidency&hl=en&gl=us&start=10&sourceid=chrome&ie=UTF-8', '3': 'https://www.google.com/search?q=Platini+UEFA+Presidency&oq=Platini+UEFA+Presidency&hl=en&gl=us&start=20&sourceid=chrome&ie=UTF-8', '4': 'https://www.google.com/search?q=Platini+UEFA+Presidency&oq=Platini+UEFA+Presidency&hl=en&gl=us&start=30&sourceid=chrome&ie=UTF-8', '5': 'https://www.google.com/search?q=Platini+UEFA+Presidency&oq=Platini+UEFA+Presidency&hl=en&gl=us&start=40&sourceid=chrome&ie=UTF-8'}}, 'serpapi_pagination': {'current': 1, 'next_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+UEFA+Presidency&start=10', 'next': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+UEFA+Presidency&start=10', 'other_pages': {'2': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+UEFA+Presidency&start=10', '3': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+UEFA+Presidency&start=20', '4': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+UEFA+Presidency&start=30', '5': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=us&google_domain=google.com&hl=en&q=Platini+UEFA+Presidency&start=40'}}}})])

In [ ]:
# generated_query = SearchQueryGenerator(query='Platini UEFA Presidency')

In [ ]:
# sample_search = {'search_metadata': {'id': '667739eaf218019f4cf6fa1a',
#   'status': 'Success',
#   'json_endpoint': 'https://serpapi.com/searches/89a407cbdf11b9a5/667739eaf218019f4cf6fa1a.json',
#   'created_at': '2024-06-22 20:54:02 UTC',
#   'processed_at': '2024-06-22 20:54:02 UTC',
#   'google_url': 'https://www.google.com/search?q=Platini+UEFA+Presidency&oq=Platini+UEFA+Presidency&hl=it&gl=it&sourceid=chrome&ie=UTF-8',
#   'raw_html_file': 'https://serpapi.com/searches/89a407cbdf11b9a5/667739eaf218019f4cf6fa1a.html',
#   'total_time_taken': 0.72},
#  'search_parameters': {'engine': 'google',
#   'q': 'Platini UEFA Presidency',
#   'google_domain': 'google.com',
#   'hl': 'it',
#   'gl': 'it',
#   'device': 'desktop'},
#  'search_information': {'query_displayed': 'Platini UEFA Presidency',
#   'total_results': 384000,
#   'time_taken_displayed': 0.41,
#   'organic_results_state': 'Some results for exact spelling but showing fixed spelling',
#   'spelling_fix': 'Platini UEFA President',
#   'showing_results_for': 'Platini UEFA President'},
#  'sports_results': {'title': 'Michel Platini',
#   'team': 'Dirigente sportivo e allenatore',
#   'tables': [{'title': 'Francia · Mondiali',
#     'games': [{'title': '19861986',
#       'partite_games_played': '66',
#       'gol_gol': '22',
#       'assist_assist': '--'},
#      {'title': '19821982',
#       'partite_games_played': '55',
#       'gol_gol': '22',
#       'assist_assist': '--'},
#      {'title': '19781978',
#       'partite_games_played': '33',
#       'gol_gol': '11',
#       'assist_assist': '--'}]}]},
#  'knowledge_graph': {'title': 'Michel Platini',
#   'type': 'Dirigente sportivo e allenatore',
#   'entity_type': 'people, athlete, soccer_player, people',
#   'kgmid': '/m/01fdsm',
#   'knowledge_graph_search_link': 'https://www.google.com/search?kgmid=/m/01fdsm&hl=it-IT&q=Michel Platini',
#   'serpapi_knowledge_graph_search_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it-IT&kgmid=%2Fm%2F01fdsm&q=Michel+Platini',
#   'header_images': [{'image': 'https://serpapi.com/searches/667739eaf218019f4cf6fa1a/images/0ac927d909d3ea54d689d1dce752cc10f40ceb46f4732ef900b2cd2112bf7261290c63b484043c3a.jpeg',
#     'source': 'http://t2.gstatic.com/licensed-image?q=tbn:ANd9GcSVXrwWc-BoMdHE4E5rfWvmo1re6XhDNobZE1mBlh7oYYwPA6bKKIb0u2scI-s1lXpQ'},
#    {'image': 'https://serpapi.com/searches/667739eaf218019f4cf6fa1a/images/0ac927d909d3ea54d689d1dce752cc10f40ceb46f4732ef97ffd05c383f63062d8080bc070e692b0.jpeg',
#     'source': 'https://it.uefa.com/news-media/news/021f-0f8ec44048fa-51ead69ffd7b-1000--michel-platini-rieletto-presidente-uefa/'},
#    {'image': 'https://serpapi.com/searches/667739eaf218019f4cf6fa1a/images/0ac927d909d3ea54d689d1dce752cc10f40ceb46f4732ef9c9a1572674016e6767705031d9e547ef.jpeg',
#     'source': 'https://www.dailymail.co.uk/sport/football/article-3009342/We-love-FIFA-roars-Michel-Platini-UEFA-president-elected-unopposed.html'},
#    {'image': 'https://serpapi.com/searches/667739eaf218019f4cf6fa1a/images/0ac927d909d3ea54d689d1dce752cc10f40ceb46f4732ef9645dab5a8d4f8a0f98f97e3d4ab903ac.jpeg',
#     'source': 'https://it.uefa.com/news-media/news/0226-0f8aa4c5a18e-0edbc146e133-1000--dichiarazione-del-presidente-uefa-michel-platini/'},
#    {'image': 'https://serpapi.com/searches/667739eaf218019f4cf6fa1a/images/0ac927d909d3ea54d689d1dce752cc10f40ceb46f4732ef92e21da445adf8f8f262b55971266c2b6.jpeg',
#     'source': 'https://www.skysports.com/football/news/30964/9551138/champions-league-uefa-president-michel-platini-happy-with-2022-dates-switch'},
#    {'image': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRNEQxl3eDkwuH-2IARQBxUbdJmMzWGe0cd3c_5GedrRg&s',
#     'source': 'https://www.britannica.com/biography/Michel-Platini'},
#    {'image': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS6ypOrmIAGNzF5wnCSOBCwVRzCErONJIf9BRiPk03XkA&s',
#     'source': 'https://www.skysports.com/football/news/11095/12648027/sepp-blatter-and-michel-platini-former-heads-of-fifa-and-uefa-cleared-of-corruption-charges-by-swiss-court'}],
#   'description': 'Michel François Platini è un dirigente sportivo, allenatore di calcio ed ex calciatore francese, di ruolo centrocampista.',
#   'source': {'name': 'Wikipedia',
#    'link': 'https://it.wikipedia.org/wiki/Michel_Platini'},
#   'nascita': '21 giugno 1955 (età 69 anni), Jœuf, Francia',
#   'nascita_links': [{'text': 'Jœuf, Francia',
#     'link': 'https://www.google.com/search?sca_esv=dc08701f44d53b0e&hl=it&gl=it&q=J%C5%93uf&si=ACC90nzx_D3_zUKRnpAjmO0UBLNxnt7EyN4YYdru6U3bxLI-LxvQdBRRSnQQlq7vIxw88DJUHTO63N009AlYiTewIE3c91x7FGA2Thx5A77bgpEf9jH7RPeki1QZqxUgbMyuyGDwnrWZ8nu4-0rU7qpI4rMa6zL1jbEuTwDT7ho6LX74GjxqOR8DDv3Up7JBsX9EZqMugaW9&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQmxMoAHoECDoQAg'}],
#   'posizione': 'Centrocampista',
#   'posizione_links': [{'text': 'Centrocampista',
#     'link': 'https://www.google.com/search?sca_esv=dc08701f44d53b0e&hl=it&gl=it&q=&si=ACC90nzx_D3_zUKRnpAjmO0UBLNxnt7EyN4YYdru6U3bxLI-L7WAdTL-YwWnQpdY4j2K-lFe6PkhNVQynek3Z4mN3IUjnI2V27wjtzeMfLdwkyyququ-YjNz2xISAI0Yj4t6JJM6AxKEK2iW0D-aPoEgApturWjG4Z1k25NQ8uD4sHY4cr3Wh78%3D&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQmxMoAHoECDcQAg'}],
#   'coniuge': 'Christelle Platini (s. 1977)',
#   'coniuge_links': [{'text': 'Christelle Platini',
#     'link': 'https://www.google.com/search?sca_esv=dc08701f44d53b0e&hl=it&gl=it&q=christelle+platini&si=ACC90ny8E30vD16OoPAAI4cStfcliGy35W8UAhb0TsHNc_ISQRfdSrgO5UQY6YbcEQx1Bxqy-F7ZtzGm0NtvS7mvUtqESlwt3nL2TLZt--Q2dxV5UwbcZ9sDfbsMmuO0xGyDzcDs_wogX9CNlFpjf29BhlrrdPyvzZdQ2tDVErwOi4GvA-FPwRqoHmYqKj44vJjppVi8a9KD0sdZtd6MErXZxiTAdw5qtg%3D%3D&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQmxMoAHoECDYQAg'}],
#   'figli': 'Laurent Platini, Marine Platini',
#   'figli_links': [{'text': 'Laurent Platini',
#     'link': 'https://www.google.com/search?sca_esv=dc08701f44d53b0e&hl=it&gl=it&q=laurent+platini&si=ACC90nwLLwns5sISZcdzuISy7t-NHozt8Cbt6G3WNQfC9ekAgGE9RUZGaskwbE-HjiUK49gDR8uLWrON7EnvCUrZLGTBrM46LCcPZFxuZaq13I9tW4yvOxb4QXZk4BtJHDymen2XVMHEuM3DvYlMbAYjaRBOTsGytOeF5FbvWz3MOM7RjQX-8GXYLC0Vy5Wc4qj49QEKcrkd-_2pVpXcftTUMQjWOc3NwhJI7exyBtr31PvV6nKj6SHnOmkLS_OHVO30bZFpXJm91o4zluYZHyir247x5fwWWg%3D%3D&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQmxMoAHoECDUQAg'},
#    {'text': 'Marine Platini',
#     'link': 'https://www.google.com/search?sca_esv=dc08701f44d53b0e&hl=it&gl=it&q=Marine+Platini&si=ACC90nwLLwns5sISZcdzuISy7t-NHozt8Cbt6G3WNQfC9ekAgLL5wXOoGx8qgyyuHBItiDzKPXD_rdY16qGcpU59HclAfgdz0CYflEv6_IBPhrmODpCwRiODvtm3R_SuzKm4cWbKdedHMN6T1TAuRYJEf5UkNLOEUe_-Mk3gHF7KJHXmcm3Bkxh1Rw5z7RXq0fdCMU8Myoh4Qf9VVgnEOsTD-Ia_6f8PBWg-NlnjE46jTCrR2G5FLcm0fB9-qs9bUC1K8L_Y4LZ9N6eXJxppxRIo26mzhUeJ9A%3D%3D&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQmxMoAXoECDUQAw'}],
#   'genitori': 'Aldo Platini, Anna Platini',
#   'genitori_links': [{'text': 'Aldo Platini',
#     'link': 'https://www.google.com/search?sca_esv=dc08701f44d53b0e&hl=it&gl=it&q=Aldo+Platini&si=ACC90nxYhNno81_TzuVO0e1EieRzQwXK2DrwuMdAl3IyVcpNQ4FyHQ0IG7ls3z9f0-A7XqNVR7rMHHkXB-pKywLzoMGuQOnD6AWi2_pIZRb_7EOcdsUJAHeYX2LDhemq0xafPhoe0-m3nsZ0JMI0Av0hPL3UUapoVIYZPCE9nP4dvr_Whdg9MLNIqFrTGk4GHbYgS5c1iV2vGykdtyBIn_GQC4238BQUlNh2WSrlGs-_1Jrsad6k4le_-QkLeWiQHks1K5WwRwaRntgaHrnG3l2olAhpy8gDIw%3D%3D&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQmxMoAHoECDgQAg'},
#    {'text': 'Anna Platini',
#     'link': 'https://www.google.com/search?sca_esv=dc08701f44d53b0e&hl=it&gl=it&q=michel+platini+anna+platini&si=ACC90nxYhNno81_TzuVO0e1EieRzQwXK2DrwuMdAl3IyVcpNQ2sbeh9QiHfYnWZCdVK9AExCDxVpZ-7rnUNdCQlEffVrsc_aV_6N88Eq1z7RNlibLrdO2b1Q4_y_Uam5hGCTP3S3RSnLsrf3WlkoH7YmzVeYHnfQUspklCkik26sxmPHMU0TGFmyeo75Nm31KZfr-fDmyUu1TKr59RCl98hdjdFQk3QzEaf53f7zSM0_vHj6kY4WmU5Lu9pvjC6bLnZoMyLFhZlPyuxS_PMWERrdBZq2KIbTSg4shiSZLsEuLPZcysrJ55c%3D&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQmxMoAXoECDgQAw'}],
#   'fratelli_e_sorelle': 'Martine Platini',
#   'fratelli_e_sorelle_links': [{'text': 'Martine Platini',
#     'link': 'https://www.google.com/search?sca_esv=dc08701f44d53b0e&hl=it&gl=it&q=michel+platini+martine+platini&si=ACC90nwLLwns5sISZcdzuISy7t-NHozt8Cbt6G3WNQfC9ekAgOnnQrMlFkAN4sxdbXX_4waROnRrmi4G-JqQORL5g5TywUxtH8oF-aFCH7h4roEQf8SPjgf2pc_cC77T8Y3QxOfPv1dQrG9aLgCaHs3aVngYcLiCewPSZdg8UPYhi431rn23di-qS6hu23VYCtq15JWmQDpgoTnkN1oNoqPXP7NkFd623rf5DK1wr6m1X3KnF5Ma_2M%3D&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQmxMoAHoECDsQAg'}],
#   'profili': [{'name': 'YouTube',
#     'link': 'https://www.youtube.com/channel/UCsSClQ7XXRWuN0VQElQhvgQ',
#     'image': 'https://serpapi.com/searches/667739eaf218019f4cf6fa1a/images/0ac927d909d3ea54d689d1dce752cc10454d8d5393cb5d1e92906abf8a04a844.png'},
#    {'name': 'Instagram',
#     'link': 'https://www.instagram.com/michelplatiniofficiel/',
#     'image': 'https://serpapi.com/searches/667739eaf218019f4cf6fa1a/images/0ac927d909d3ea54d689d1dce752cc10454d8d5393cb5d1ef354b5448da5647f.png'}],
#   'brani': [{'name': 'Hier encore',
#     'extensions': 'Les Enfoirés au Grand Rex · 1994',
#     'link': 'https://www.google.com/search?sca_esv=dc08701f44d53b0e&hl=it&gl=it&q=les+enfoir%C3%A9s+hier+encore&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tFP1zcsNM1KKilLNtcS9C0tzkx2LCrJLC4JyQ_Oz0tfxCqZk1qskJqXlp9ZdHhlsUJGZmoRkJucX5QKADafiSlKAAAA&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQri56BAg9EAM',
#     'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=les+enfoir%C3%A9s+hier+encore&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tFP1zcsNM1KKilLNtcS9C0tzkx2LCrJLC4JyQ_Oz0tfxCqZk1qskJqXlp9ZdHhlsUJGZmoRkJucX5QKADafiSlKAAAA'},
#    {'name': "Jusqu'au dernier",
#     'extensions': 'Les Enfoirés 2024, On a 35 ans ! · 2024',
#     'link': 'https://www.google.com/search?sca_esv=dc08701f44d53b0e&hl=it&gl=it&q=les+enfoir%C3%A9s+jusqu%27au+dernier&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tVP1zc0rDTKyMrIMkjSEvQtLc5MdiwqySwuCckPzs9LX8Qql5NarJCal5afWXR4ZbFCVmlxYal6YqlCSmpRXmZqEQAFTZYRUAAAAA&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQri56BAg9EAU',
#     'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=les+enfoir%C3%A9s+jusqu%27au+dernier&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tVP1zc0rDTKyMrIMkjSEvQtLc5MdiwqySwuCckPzs9LX8Qql5NarJCal5afWXR4ZbFCVmlxYal6YqlCSmpRXmZqEQAFTZYRUAAAAA'},
#    {'name': 'La chanson des restos',
#     'extensions': "Tournée d'Enfoirés · 1989",
#     'link': 'https://www.google.com/search?sca_esv=dc08701f44d53b0e&hl=it&gl=it&q=les+enfoir%C3%A9s+la+chanson+des+restos&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tFP1zcsNjAtMbQ0qdQS9C0tzkx2LCrJLC4JyQ_Oz0tfxKqck1qskJqXlp9ZdHhlsUJOokJyRmJecX6eQgpQoii1uCS_GAC9YiMSVAAAAA&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQri56BAg9EAc',
#     'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=les+enfoir%C3%A9s+la+chanson+des+restos&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tFP1zcsNjAtMbQ0qdQS9C0tzkx2LCrJLC4JyQ_Oz0tfxKqck1qskJqXlp9ZdHhlsUJOokJyRmJecX6eQgpQoii1uCS_GAC9YiMSVAAAAA',
#     'image': 'https://serpapi.com/searches/667739eaf218019f4cf6fa1a/images/0ac927d909d3ea54d689d1dce752cc1048d894ab1a70a8bcf1cffa321d5cf86a.jpeg'},
#    {'name': 'Rêvons',
#     'extensions': '2023 Enfoirés un jour, toujours · 2023',
#     'link': 'https://www.google.com/search?sca_esv=dc08701f44d53b0e&hl=it&gl=it&q=les+enfoir%C3%A9s+r%C3%AAvons&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tVP1zc0LE4pT89NL0nTEvQtLc5MdiwqySwuCckPzs9LX8QqmpNarJCal5afWXR4ZbFC0eFVZfl5xQClALwHRwAAAA&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQri56BAg9EAo',
#     'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=les+enfoir%C3%A9s+r%C3%AAvons&stick=H4sIAAAAAAAAAONgFuLUz9U3MExLKc5V4tVP1zc0LE4pT89NL0nTEvQtLc5MdiwqySwuCckPzs9LX8QqmpNarJCal5afWXR4ZbFC0eFVZfl5xQClALwHRwAAAA'}],
#   'altre_ricerche_degli_utenti': [{'name': 'Didier Deschamps (In risalto)',
#     'extensions': ['In risalto'],
#     'link': 'https://www.google.com/search?sca_esv=dc08701f44d53b0e&hl=it&gl=it&si=ACC90nzx_D3_zUKRnpAjmO0UBLNxnt7EyN4YYdru6U3bxLI-L67xjEDocyAf8t0i5Zuq4jVW3BhUU4d7icga1YHvfx6lVi3mLO7-UDWRcIhfCxzlhLpR-lcUblLbx70Ss-oxUEC8CU4lSYNeFR1JADZiLGWVeaDmCMoVF_Cfb19NJ3q0Tcoqya6jG80uSr2byT7DpopQkADIQy0yS-IUUO_aLiFNsZ4zQu8qY7_7QMusFJqS100AT13lmXLJEW3C_lvMXF32sJQwzJT2KDDMciK7dAwkrrufj0YBfCM1sCzaLIynFjKsOiI%3D&q=Didier+Deschamps&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQs9oBKAB6BAg8EAI',
#     'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Didier+Deschamps&si=ACC90nzx_D3_zUKRnpAjmO0UBLNxnt7EyN4YYdru6U3bxLI-L67xjEDocyAf8t0i5Zuq4jVW3BhUU4d7icga1YHvfx6lVi3mLO7-UDWRcIhfCxzlhLpR-lcUblLbx70Ss-oxUEC8CU4lSYNeFR1JADZiLGWVeaDmCMoVF_Cfb19NJ3q0Tcoqya6jG80uSr2byT7DpopQkADIQy0yS-IUUO_aLiFNsZ4zQu8qY7_7QMusFJqS100AT13lmXLJEW3C_lvMXF32sJQwzJT2KDDMciK7dAwkrrufj0YBfCM1sCzaLIynFjKsOiI%3D',
#     'image': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQftUERC4w57ouWMyvF3exk1OvlWr_gUlmeoshjNMvb0YsEAXOypz0H&s=0'},
#    {'name': 'Paolo Maldini (In risalto)',
#     'extensions': ['In risalto'],
#     'link': 'https://www.google.com/search?sca_esv=dc08701f44d53b0e&hl=it&gl=it&si=ACC90nzx_D3_zUKRnpAjmO0UBLNxnt7EyN4YYdru6U3bxLI-L2BDxChwMyOkoMbAaXkElHHa_-NLwSNPPkq2kI_R3QNaGesuAU9urDDxP6laUGcrKfRM6K8UUxapqUH27RZ1Ryu-hqS6GnoT5hsttSG5xeNoLQi-sqhmfAyAsllo8dpwIifcgaPbg4Q3w50QtwiamAVeOXAwvsHrxaJKB1mG9nAM2nG3U9tnhLGuJwT4S0KqnwukSSP_S4x-_JnlIEERM9EuUfiKjP2lCWo3c3zbQMMovdq3Ji_grFxWCsvcgLBmFGDwtio%3D&q=Paolo+Maldini&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQs9oBKAF6BAg8EAM',
#     'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Paolo+Maldini&si=ACC90nzx_D3_zUKRnpAjmO0UBLNxnt7EyN4YYdru6U3bxLI-L2BDxChwMyOkoMbAaXkElHHa_-NLwSNPPkq2kI_R3QNaGesuAU9urDDxP6laUGcrKfRM6K8UUxapqUH27RZ1Ryu-hqS6GnoT5hsttSG5xeNoLQi-sqhmfAyAsllo8dpwIifcgaPbg4Q3w50QtwiamAVeOXAwvsHrxaJKB1mG9nAM2nG3U9tnhLGuJwT4S0KqnwukSSP_S4x-_JnlIEERM9EuUfiKjP2lCWo3c3zbQMMovdq3Ji_grFxWCsvcgLBmFGDwtio%3D',
#     'image': 'https://encrypted-tbn0.gstatic.com/licensed-image?q=tbn:ANd9GcTO1O2ebSiITpsG1EmY0iPwgQJuShBEAnCIqRoCuqsxWYz0r-fPTfqj6T_kEvLpZHWLB3KtMI_v&s=19'},
#    {'name': 'Sepp Blatter',
#     'link': 'https://www.google.com/search?sca_esv=dc08701f44d53b0e&hl=it&gl=it&si=ACC90nyvvWro6QmnyY1IfSdgk5wwjB1r8BGd_IWRjXqmKPQqm1xJY05G-_pJvUE639mUNh3pMH_KteVul5RRkXiITWKV9Oc7h43ju57hNB2q4Sl7Hj7lJ8dDWhIBfDvaJCvR4V1VUibDnvVXdmR9fMymdy_WRwRpkrAKjMd7b7UXJmSPYh3dYnskA1Rx5BRmJY_8RlL3zbY8kPrj34ip_aNaBXwQmol0h-3SyA2N96gDf3fB1Jms45izPCejqyeqTo6MnH-TCQZ0Q2g9IlIM7nWuntFnQ5u-aTxCympE9MTGj9vShOdJMM8%3D&q=Sepp+Blatter&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQs9oBKAJ6BAg8EAQ',
#     'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Sepp+Blatter&si=ACC90nyvvWro6QmnyY1IfSdgk5wwjB1r8BGd_IWRjXqmKPQqm1xJY05G-_pJvUE639mUNh3pMH_KteVul5RRkXiITWKV9Oc7h43ju57hNB2q4Sl7Hj7lJ8dDWhIBfDvaJCvR4V1VUibDnvVXdmR9fMymdy_WRwRpkrAKjMd7b7UXJmSPYh3dYnskA1Rx5BRmJY_8RlL3zbY8kPrj34ip_aNaBXwQmol0h-3SyA2N96gDf3fB1Jms45izPCejqyeqTo6MnH-TCQZ0Q2g9IlIM7nWuntFnQ5u-aTxCympE9MTGj9vShOdJMM8%3D',
#     'image': 'https://encrypted-tbn0.gstatic.com/licensed-image?q=tbn:ANd9GcRxPUHN8lLISOrREFBVTr9aRoiopKl9doq_KnM_Qmw9_MjXszwHtOdZpjAUhICfSPz9pBmur_47&s=19'},
#    {'name': 'Zinédine Zidane',
#     'link': 'https://www.google.com/search?sca_esv=dc08701f44d53b0e&hl=it&gl=it&si=ACC90nyvvWro6QmnyY1IfSdgk5wwjB1r8BGd_IWRjXqmKPQqm0bl7qGBWzuqnzPJjsycQVF1S2sXIxC2FlWoFu8b9RmNI525cX7Ap4o3G85ddzpCcUt9QON3bw6mJfk8IEv4edkJvDwFg1dXPCTnCHTR1zK7uIm0MTF0Af8IbWEjq8Cs8ja5u4RgeCTrqAVHdMJ1MmLQN07s9GmB13zuQLvLRHHssmlUkVMXMmUDzxjBAv7HRphumVm4P-x4biMHE6aIVbz1B2xFw94z1XTAIguRTrJsgeZYhjU1D8VMOI1qZTzWqUZdaUk%3D&q=Zin%C3%A9dine+Zidane&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQs9oBKAN6BAg8EAU',
#     'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Zin%C3%A9dine+Zidane&si=ACC90nyvvWro6QmnyY1IfSdgk5wwjB1r8BGd_IWRjXqmKPQqm0bl7qGBWzuqnzPJjsycQVF1S2sXIxC2FlWoFu8b9RmNI525cX7Ap4o3G85ddzpCcUt9QON3bw6mJfk8IEv4edkJvDwFg1dXPCTnCHTR1zK7uIm0MTF0Af8IbWEjq8Cs8ja5u4RgeCTrqAVHdMJ1MmLQN07s9GmB13zuQLvLRHHssmlUkVMXMmUDzxjBAv7HRphumVm4P-x4biMHE6aIVbz1B2xFw94z1XTAIguRTrJsgeZYhjU1D8VMOI1qZTzWqUZdaUk%3D',
#     'image': 'https://encrypted-tbn0.gstatic.com/licensed-image?q=tbn:ANd9GcROOWQivABZQOCQFe0ubA5ZwPRmh26vyRByI7JwsvH0TMSBvDzpMV7dDWsYGWau9KpqMHqjKh4&s=19'}]},
#  'inline_images': [{'link': 'https://www.google.com/search?q=Platini+UEFA+President&sca_esv=dc08701f44d53b0e&hl=it&gl=it&udm=2&source=iu&ictx=1&vet=1&fir=4rxwlz3p4KhOaM%252CKpfdVq9MS8QSSM%252C%252Fm%252F01fdsm%253BhjLM6fEeLy-V6M%252CieaJyGDvUWYwpM%252C_%253BGMvF_77REYhzbM%252CwzCz615SibumMM%252C_%253B2NQdMqhFloD6IM%252C_wrGAXM0Kb9igM%252C_%253BoZ3mClihPNsi7M%252Cbol1rGQnANZ-1M%252C_%253BYPylfLOoc5YEvM%252CeD-qJUQ2G4iMpM%252C_%253BswCs-zzm6TnLJM%252C4NFd32esJBL_sM%252C_&usg=AI4_-kRshmXWvSmosx7KlxuF-WSHIaJV9g&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQ_B16BAhBEAE#vhid=4rxwlz3p4KhOaM&vssid=mosaic',
#    'source': 'http://t2.gstatic.com/licensed-image?q=tbn:ANd9GcSVXrwWc-BoMdHE4E5rfWvmo1re6XhDNobZE1mBlh7oYYwPA6bKKIb0u2scI-s1lXpQ',
#    'thumbnail': 'https://serpapi.com/searches/667739eaf218019f4cf6fa1a/images/77be93c92544e25bae569aab3f6284984f9f92c2b2c61fce370f52ec8b2c1637.jpeg',
#    'original': 'http://t2.gstatic.com/licensed-image?q=tbn:ANd9GcSVXrwWc-BoMdHE4E5rfWvmo1re6XhDNobZE1mBlh7oYYwPA6bKKIb0u2scI-s1lXpQ',
#    'title': 'Credit: Keystone-France',
#    'source_name': 't2.gstatic.com'},
#   {'link': 'https://www.google.com/search?q=Platini+UEFA+President&sca_esv=dc08701f44d53b0e&hl=it&gl=it&udm=2&source=iu&ictx=1&vet=1&fir=hjLM6fEeLy-V6M%252CieaJyGDvUWYwpM%252C_%253BGMvF_77REYhzbM%252CwzCz615SibumMM%252C_%253B2NQdMqhFloD6IM%252C_wrGAXM0Kb9igM%252C_%253BoZ3mClihPNsi7M%252Cbol1rGQnANZ-1M%252C_%253BYPylfLOoc5YEvM%252CeD-qJUQ2G4iMpM%252C_%253BswCs-zzm6TnLJM%252C4NFd32esJBL_sM%252C_&usg=AI4_-kRAI8QLyI6avYpeUzCnWkiK6fnEOA&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQ_h16BAhIEAE#vhid=hjLM6fEeLy-V6M&vssid=mosaic',
#    'source': 'https://it.uefa.com/news-media/news/021f-0f8ec44048fa-51ead69ffd7b-1000--michel-platini-rieletto-presidente-uefa/',
#    'thumbnail': 'https://serpapi.com/searches/667739eaf218019f4cf6fa1a/images/77be93c92544e25b5e5d6d34516994475ed9aec69f83f8fb508d1d59ab82b412.jpeg',
#    'original': 'https://editorial.uefa.com/resources/021f-0f8a8cf626d8-379ed8e99808-1000/michel_platini_uefa_xxxix_ordinary_congress_.jpeg',
#    'title': 'Michel Platini rieletto Presidente UEFA | UEFA.com',
#    'source_name': 'UEFA.com'},
#   {'link': 'https://www.google.com/search?q=Platini+UEFA+President&sca_esv=dc08701f44d53b0e&hl=it&gl=it&udm=2&source=iu&ictx=1&vet=1&fir=hjLM6fEeLy-V6M%252CieaJyGDvUWYwpM%252C_%253BGMvF_77REYhzbM%252CwzCz615SibumMM%252C_%253B2NQdMqhFloD6IM%252C_wrGAXM0Kb9igM%252C_%253BoZ3mClihPNsi7M%252Cbol1rGQnANZ-1M%252C_%253BYPylfLOoc5YEvM%252CeD-qJUQ2G4iMpM%252C_%253BswCs-zzm6TnLJM%252C4NFd32esJBL_sM%252C_&usg=AI4_-kRAI8QLyI6avYpeUzCnWkiK6fnEOA&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQ_h16BAhFEAE#vhid=GMvF_77REYhzbM&vssid=mosaic',
#    'source': 'https://www.dailymail.co.uk/sport/football/article-3009342/We-love-FIFA-roars-Michel-Platini-UEFA-president-elected-unopposed.html',
#    'thumbnail': 'https://serpapi.com/searches/667739eaf218019f4cf6fa1a/images/77be93c92544e25ba8741917aa55e352fef110d7c6185dd02480a2f80a7c8a86.jpeg',
#    'original': 'https://i.dailymail.co.uk/i/pix/2015/03/24/12/26F2F94900000578-0-image-a-15_1427201099046.jpg',
#    'title': "We love FIFA!' roars Michel Platini after UEFA president is ...",
#    'source_name': 'Daily Mail'},
#   {'link': 'https://www.google.com/search?q=Platini+UEFA+President&sca_esv=dc08701f44d53b0e&hl=it&gl=it&udm=2&source=iu&ictx=1&vet=1&fir=hjLM6fEeLy-V6M%252CieaJyGDvUWYwpM%252C_%253BGMvF_77REYhzbM%252CwzCz615SibumMM%252C_%253B2NQdMqhFloD6IM%252C_wrGAXM0Kb9igM%252C_%253BoZ3mClihPNsi7M%252Cbol1rGQnANZ-1M%252C_%253BYPylfLOoc5YEvM%252CeD-qJUQ2G4iMpM%252C_%253BswCs-zzm6TnLJM%252C4NFd32esJBL_sM%252C_&usg=AI4_-kRAI8QLyI6avYpeUzCnWkiK6fnEOA&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQ_h16BAhEEAE#vhid=2NQdMqhFloD6IM&vssid=mosaic',
#    'source': 'https://it.uefa.com/news-media/news/0226-0f8aa4c5a18e-0edbc146e133-1000--dichiarazione-del-presidente-uefa-michel-platini/',
#    'thumbnail': 'https://serpapi.com/searches/667739eaf218019f4cf6fa1a/images/77be93c92544e25b05ba0678113c00cd021574e88a52e2016965ac8f6e298d68.jpeg',
#    'original': 'https://editorial.uefa.com/resources/0226-0f8aa4c092ef-bb40283eed24-1000/michel_platini_uefa_president_from_26_january_2007_to_14_september_2016.jpeg',
#    'title': 'Dichiarazione del Presidente UEFA Michel Platini | UEFA.com',
#    'source_name': 'UEFA.com'},
#   {'link': 'https://www.google.com/search?q=Platini+UEFA+President&sca_esv=dc08701f44d53b0e&hl=it&gl=it&udm=2&source=iu&ictx=1&vet=1&fir=hjLM6fEeLy-V6M%252CieaJyGDvUWYwpM%252C_%253BGMvF_77REYhzbM%252CwzCz615SibumMM%252C_%253B2NQdMqhFloD6IM%252C_wrGAXM0Kb9igM%252C_%253BoZ3mClihPNsi7M%252Cbol1rGQnANZ-1M%252C_%253BYPylfLOoc5YEvM%252CeD-qJUQ2G4iMpM%252C_%253BswCs-zzm6TnLJM%252C4NFd32esJBL_sM%252C_&usg=AI4_-kRAI8QLyI6avYpeUzCnWkiK6fnEOA&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQ_h16BAhDEAE#vhid=oZ3mClihPNsi7M&vssid=mosaic',
#    'source': 'https://www.skysports.com/football/news/30964/9551138/champions-league-uefa-president-michel-platini-happy-with-2022-dates-switch',
#    'thumbnail': 'https://serpapi.com/searches/667739eaf218019f4cf6fa1a/images/77be93c92544e25b46324ccdc1ed4b430759eb004d8821d53ffa773cfa427084.jpeg',
#    'original': 'https://e0.365dm.com/14/08/2048x1152/michel-platini-football-champions-league_3197215.jpg?20140829163816',
#    'title': 'UEFA president Michel Platini happy to move 2022 Champions ...',
#    'source_name': 'Sky Sports'},
#   {'link': 'https://www.google.com/search?q=Platini+UEFA+President&sca_esv=dc08701f44d53b0e&hl=it&gl=it&udm=2&source=iu&ictx=1&vet=1&fir=hjLM6fEeLy-V6M%252CieaJyGDvUWYwpM%252C_%253BGMvF_77REYhzbM%252CwzCz615SibumMM%252C_%253B2NQdMqhFloD6IM%252C_wrGAXM0Kb9igM%252C_%253BoZ3mClihPNsi7M%252Cbol1rGQnANZ-1M%252C_%253BYPylfLOoc5YEvM%252CeD-qJUQ2G4iMpM%252C_%253BswCs-zzm6TnLJM%252C4NFd32esJBL_sM%252C_&usg=AI4_-kRAI8QLyI6avYpeUzCnWkiK6fnEOA&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQ_h16BAhCEAE#vhid=YPylfLOoc5YEvM&vssid=mosaic',
#    'source': 'https://www.britannica.com/biography/Michel-Platini',
#    'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRNEQxl3eDkwuH-2IARQBxUbdJmMzWGe0cd3c_5GedrRg&s',
#    'original': 'https://cdn.britannica.com/30/171930-050-43B12C8F/Michel-Platini-2009.jpg',
#    'title': 'Michel Platini | Biography, FIFA, & Facts | Britannica',
#    'source_name': 'Britannica'},
#   {'link': 'https://www.google.com/search?q=Platini+UEFA+President&sca_esv=dc08701f44d53b0e&hl=it&gl=it&udm=2&source=iu&ictx=1&vet=1&fir=hjLM6fEeLy-V6M%252CieaJyGDvUWYwpM%252C_%253BGMvF_77REYhzbM%252CwzCz615SibumMM%252C_%253B2NQdMqhFloD6IM%252C_wrGAXM0Kb9igM%252C_%253BoZ3mClihPNsi7M%252Cbol1rGQnANZ-1M%252C_%253BYPylfLOoc5YEvM%252CeD-qJUQ2G4iMpM%252C_%253BswCs-zzm6TnLJM%252C4NFd32esJBL_sM%252C_&usg=AI4_-kRAI8QLyI6avYpeUzCnWkiK6fnEOA&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQ_h16BAhGEAE#vhid=swCs-zzm6TnLJM&vssid=mosaic',
#    'source': 'https://www.skysports.com/football/news/11095/12648027/sepp-blatter-and-michel-platini-former-heads-of-fifa-and-uefa-cleared-of-corruption-charges-by-swiss-court',
#    'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS6ypOrmIAGNzF5wnCSOBCwVRzCErONJIf9BRiPk03XkA&s',
#    'original': 'https://e0.365dm.com/22/07/2048x1152/skysports-sepp-blatter-michael-platini_5827625.jpg?20220708094231',
#    'title': 'Sepp Blatter and Michel Platini: Former heads of FIFA and ...',
#    'source_name': 'Sky Sports'}],
#  'related_questions': [{'question': "Perché non c'è Platini su FIFA?",
#    'snippet': 'Platini e Blatter sono stati assolti in primo grado nel luglio 2022. I due ex dirigenti del calcio mondiale erano accusati per un versamento di due milioni di franchi a Platini da parte di Blatter ritenuto una tangente.',
#    'title': "Scandalo FIFA: ricusata la Corte d'appello - RSI",
#    'date': '4 apr 2024',
#    'link': 'https://www.rsi.ch/info/svizzera/Scandalo-FIFA-ricusata-la-Corte-d%E2%80%99appello--2114854.html#:~:text=Platini%20e%20Blatter%20sono%20stati,di%20Blatter%20ritenuto%20una%20tangente.',
#    'displayed_link': 'https://www.rsi.ch › info › svizzera',
#    'source_logo': 'https://serpapi.com/searches/667739eaf218019f4cf6fa1a/images/13ffa00535a8b94be7e4b0918974b69ddb07e25aa148467d53e3f98a363d8eb9.png',
#    'next_page_token': 'eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210UE1WWkJNRmR1Um1oT1RUZHlSbE5MYkhGR1UwVlRRblU1ZUZkdVlWbzNRV3B0TmtKa1JUQkliM1pUWTJFd1JIQmFaRnAyVmxGcFVXZHlja3hwVldkSFdVNXBSMnBrRWhjMmFtd3pXblI2ZVVOd2FVNXdPRFJRYmpWdFVYRkJheG9pUVVaWWNrVmpiekYwUkVWRWJYSkpaRUpIVlhaS01GUmpkSGRVYVhGbVR6QTRkdyIsImZjdiI6IjMiLCJlaSI6IjZqbDNadHp5Q3BpTnA4NFBuNW1RcUFrIiwicWMiOiJDaFp3YkdGMGFXNXBJSFZsWm1FZ2NISmxjMmxrWlc1MEVBZDlWVnN0UHciLCJxdWVzdGlvbiI6IlBlcmNow6kgbm9uIGMnw6ggUGxhdGluaSBzdSBGSUZBPyIsImxrIjoiR2g1d1pYSmphTU9wSUhCc1lYUnBibWtnYm05dUlNT29JSE4xSUdacFptRSIsImJzIjoiYy1QeTRGSU1TQzFLemppOFVpRXZQMDhoV2Yzd0NvV0FuTVNTekx4TWhlSlNCVGRQTjBkN2liZldSZ3BTY2dWUWRRVlFhWkI2b0dxZ3FyVE10RVF1Tnk1NW1FbHdBekpCQ29veVN6S0xFa3Z5N1NXTzJHSXpCMVVWbHgyWHRITi1jU0xFNk9UazFPTGlmSVZFbUpIMkVqY2RqR1NrcEpJeFZVQ05nLXRQUzFUSUwwcFU4TTFNemtqTlFlaF9GUVBYRDFXUkMxRUIxU19BQ0FBIiwiaWQiOiJmY18xIn0=',
#    'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google_related_questions&google_domain=google.com&next_page_token=eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210UE1WWkJNRmR1Um1oT1RUZHlSbE5MYkhGR1UwVlRRblU1ZUZkdVlWbzNRV3B0TmtKa1JUQkliM1pUWTJFd1JIQmFaRnAyVmxGcFVXZHlja3hwVldkSFdVNXBSMnBrRWhjMmFtd3pXblI2ZVVOd2FVNXdPRFJRYmpWdFVYRkJheG9pUVVaWWNrVmpiekYwUkVWRWJYSkpaRUpIVlhaS01GUmpkSGRVYVhGbVR6QTRkdyIsImZjdiI6IjMiLCJlaSI6IjZqbDNadHp5Q3BpTnA4NFBuNW1RcUFrIiwicWMiOiJDaFp3YkdGMGFXNXBJSFZsWm1FZ2NISmxjMmxrWlc1MEVBZDlWVnN0UHciLCJxdWVzdGlvbiI6IlBlcmNow6kgbm9uIGMnw6ggUGxhdGluaSBzdSBGSUZBPyIsImxrIjoiR2g1d1pYSmphTU9wSUhCc1lYUnBibWtnYm05dUlNT29JSE4xSUdacFptRSIsImJzIjoiYy1QeTRGSU1TQzFLemppOFVpRXZQMDhoV2Yzd0NvV0FuTVNTekx4TWhlSlNCVGRQTjBkN2liZldSZ3BTY2dWUWRRVlFhWkI2b0dxZ3FyVE10RVF1Tnk1NW1FbHdBekpCQ29veVN6S0xFa3Z5N1NXTzJHSXpCMVVWbHgyWHRITi1jU0xFNk9UazFPTGlmSVZFbUpIMkVqY2RqR1NrcEpJeFZVQ05nLXRQUzFUSUwwcFU4TTFNemtqTlFlaF9GUVBYRDFXUkMxRUIxU19BQ0FBIiwiaWQiOiJmY18xIn0%3D'},
#   {'question': 'Perché Platini si è ritirato?',
#    'snippet': '1986-1987: il ritiro dal calcio giocato Era da tanti anni che giocavo a pallone ed ero veramente stanco, cotto. Ho passato tutto il 1986 infortunato e avevo dei problemi alle caviglie per cui ho deciso di smettere».',
#    'title': 'Michel Platini - Wikipedia',
#    'link': 'https://it.wikipedia.org/wiki/Michel_Platini#:~:text=1986%2D1987%3A%20il%20ritiro%20dal%20calcio%20giocato,-Rimasto%20tra%20i&text=Era%20da%20tanti%20anni%20che,cui%20ho%20deciso%20di%20smettere%C2%BB.',
#    'displayed_link': 'https://it.wikipedia.org › wiki › Michel_Platini',
#    'next_page_token': 'eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210UE1WWkJNRmR1Um1oT1RUZHlSbE5MYkhGR1UwVlRRblU1ZUZkdVlWbzNRV3B0TmtKa1JUQkliM1pUWTJFd1JIQmFaRnAyVmxGcFVXZHlja3hwVldkSFdVNXBSMnBrRWhjMmFtd3pXblI2ZVVOd2FVNXdPRFJRYmpWdFVYRkJheG9pUVVaWWNrVmpiekYwUkVWRWJYSkpaRUpIVlhaS01GUmpkSGRVYVhGbVR6QTRkdyIsImZjdiI6IjMiLCJlaSI6IjZqbDNadHp5Q3BpTnA4NFBuNW1RcUFrIiwicWMiOiJDaFp3YkdGMGFXNXBJSFZsWm1FZ2NISmxjMmxrWlc1MEVBZDlWVnN0UHciLCJxdWVzdGlvbiI6IlBlcmNow6kgUGxhdGluaSBzaSDDqCByaXRpcmF0bz8iLCJsayI6IkdoNXdaWEpqYU1PcElIQnNZWFJwYm1rZ2Mya2d3NmdnY21sMGFYSmhkRzgiLCJicyI6ImMtUHk0RklNU0MxS3pqaThVaUV2UDA4aFdmM3dDb1dBbk1TU3pMeE1oZUpTQlRkUE4wZDdpYmZXUmdwU2NnVlFkUVZRYVpCNm9HcWdxclRNdEVRdU55NTVtRWx3QXpKQkNvb3lTektMRWt2eTdTV08yR0l6QjFVVmx4Mlh0SE4tY1NMRTZPVGsxT0xpZklWRW1KSDJFamNkakdTa3BKSXhWVUNOZy10UFMxVElMMHBVOE0xTXprak5RZWhfRlFQWEQxV1JDMUVCMVNfQUNBQSIsImlkIjoiZmNfMSJ9',
#    'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google_related_questions&google_domain=google.com&next_page_token=eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210UE1WWkJNRmR1Um1oT1RUZHlSbE5MYkhGR1UwVlRRblU1ZUZkdVlWbzNRV3B0TmtKa1JUQkliM1pUWTJFd1JIQmFaRnAyVmxGcFVXZHlja3hwVldkSFdVNXBSMnBrRWhjMmFtd3pXblI2ZVVOd2FVNXdPRFJRYmpWdFVYRkJheG9pUVVaWWNrVmpiekYwUkVWRWJYSkpaRUpIVlhaS01GUmpkSGRVYVhGbVR6QTRkdyIsImZjdiI6IjMiLCJlaSI6IjZqbDNadHp5Q3BpTnA4NFBuNW1RcUFrIiwicWMiOiJDaFp3YkdGMGFXNXBJSFZsWm1FZ2NISmxjMmxrWlc1MEVBZDlWVnN0UHciLCJxdWVzdGlvbiI6IlBlcmNow6kgUGxhdGluaSBzaSDDqCByaXRpcmF0bz8iLCJsayI6IkdoNXdaWEpqYU1PcElIQnNZWFJwYm1rZ2Mya2d3NmdnY21sMGFYSmhkRzgiLCJicyI6ImMtUHk0RklNU0MxS3pqaThVaUV2UDA4aFdmM3dDb1dBbk1TU3pMeE1oZUpTQlRkUE4wZDdpYmZXUmdwU2NnVlFkUVZRYVpCNm9HcWdxclRNdEVRdU55NTVtRWx3QXpKQkNvb3lTektMRWt2eTdTV08yR0l6QjFVVmx4Mlh0SE4tY1NMRTZPVGsxT0xpZklWRW1KSDJFamNkakdTa3BKSXhWVUNOZy10UFMxVElMMHBVOE0xTXprak5RZWhfRlFQWEQxV1JDMUVCMVNfQUNBQSIsImlkIjoiZmNfMSJ9'},
#   {'question': 'Cosa è successo a Platini?',
#    'snippet': "Infatti «complicità nella cattiva gestione e appropriazione indebita» sono i capi di imputazione con i quali Michel Platini è entrato come «sospettato» nell'inchiesta della Procura svizzera su Sepp Blatter per avergli pagato 2 milioni di franchi svizzeri (1,8 milioni in euro) nel 2011 senza traccia scritta.",
#    'title': 'Michel Platini ora è indagato per appropriazione indebita nella ...',
#    'date': '26 giu 2020',
#    'link': 'https://www.corriere.it/sport/20_giugno_26/michel-platini-ora-indagato-appropriazione-indebita-consulenza-blatter-52735750-b7d5-11ea-b7f2-bfb2b67ec0ad.shtml#:~:text=Infatti%20%C2%ABcomplicit%C3%A0%20nella%20cattiva%20gestione,nel%202011%20senza%20traccia%20scritta.',
#    'displayed_link': 'https://www.corriere.it › sport › 20_giugno_26 › michel-...',
#    'source_logo': 'https://serpapi.com/searches/667739eaf218019f4cf6fa1a/images/13ffa00535a8b94be7e4b0918974b69d504c3d574af8ca86cdc11074f231cd20.png',
#    'next_page_token': 'eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210UE1WWkJNRmR1Um1oT1RUZHlSbE5MYkhGR1UwVlRRblU1ZUZkdVlWbzNRV3B0TmtKa1JUQkliM1pUWTJFd1JIQmFaRnAyVmxGcFVXZHlja3hwVldkSFdVNXBSMnBrRWhjMmFtd3pXblI2ZVVOd2FVNXdPRFJRYmpWdFVYRkJheG9pUVVaWWNrVmpiekYwUkVWRWJYSkpaRUpIVlhaS01GUmpkSGRVYVhGbVR6QTRkdyIsImZjdiI6IjMiLCJlaSI6IjZqbDNadHp5Q3BpTnA4NFBuNW1RcUFrIiwicWMiOiJDaFp3YkdGMGFXNXBJSFZsWm1FZ2NISmxjMmxrWlc1MEVBZDlWVnN0UHciLCJxdWVzdGlvbiI6IkNvc2Egw6ggc3VjY2Vzc28gYSBQbGF0aW5pPyIsImxrIjoiR2hwamIzTmhJTU9vSUhOMVkyTmxjM052SUdFZ2NHeGhkR2x1YVEiLCJicyI6ImMtUHk0RklNU0MxS3pqaThVaUV2UDA4aFdmM3dDb1dBbk1TU3pMeE1oZUpTQlRkUE4wZDdpYmZXUmdwU2NnVlFkUVZRYVpCNm9HcWdxclRNdEVRdU55NTVtRWx3QXpKQkNvb3lTektMRWt2eTdTV08yR0l6QjFVVmx4Mlh0SE4tY1NMRTZPVGsxT0xpZklWRW1KSDJFamNkakdTa3BKSXhWVUNOZy10UFMxVElMMHBVOE0xTXprak5RZWhfRlFQWEQxV1JDMUVCMVNfQUNBQSIsImlkIjoiZmNfMSJ9',
#    'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google_related_questions&google_domain=google.com&next_page_token=eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210UE1WWkJNRmR1Um1oT1RUZHlSbE5MYkhGR1UwVlRRblU1ZUZkdVlWbzNRV3B0TmtKa1JUQkliM1pUWTJFd1JIQmFaRnAyVmxGcFVXZHlja3hwVldkSFdVNXBSMnBrRWhjMmFtd3pXblI2ZVVOd2FVNXdPRFJRYmpWdFVYRkJheG9pUVVaWWNrVmpiekYwUkVWRWJYSkpaRUpIVlhaS01GUmpkSGRVYVhGbVR6QTRkdyIsImZjdiI6IjMiLCJlaSI6IjZqbDNadHp5Q3BpTnA4NFBuNW1RcUFrIiwicWMiOiJDaFp3YkdGMGFXNXBJSFZsWm1FZ2NISmxjMmxrWlc1MEVBZDlWVnN0UHciLCJxdWVzdGlvbiI6IkNvc2Egw6ggc3VjY2Vzc28gYSBQbGF0aW5pPyIsImxrIjoiR2hwamIzTmhJTU9vSUhOMVkyTmxjM052SUdFZ2NHeGhkR2x1YVEiLCJicyI6ImMtUHk0RklNU0MxS3pqaThVaUV2UDA4aFdmM3dDb1dBbk1TU3pMeE1oZUpTQlRkUE4wZDdpYmZXUmdwU2NnVlFkUVZRYVpCNm9HcWdxclRNdEVRdU55NTVtRWx3QXpKQkNvb3lTektMRWt2eTdTV08yR0l6QjFVVmx4Mlh0SE4tY1NMRTZPVGsxT0xpZklWRW1KSDJFamNkakdTa3BKSXhWVUNOZy10UFMxVElMMHBVOE0xTXprak5RZWhfRlFQWEQxV1JDMUVCMVNfQUNBQSIsImlkIjoiZmNfMSJ9'},
#   {'question': 'Cosa fa ora Michel Platini?',
#    'snippet': "Nel 1982 si trasferisce alla Juventus, dove resta per cinque anni, trionfando in Italia e in Europa. Tre volte Pallone d'Oro tra il 1983 e il 1985, vince un Europeo con la nazionale francese, con cui arriva terzo al Mondiale messicano del 1986. Attualmente è Presidente dell'UEFA.",
#    'title': 'Michel Platini | FIGC',
#    'link': "https://www.figc.it/it/museo-del-calcio/hall-of-fame/le-stelle/michel-platini/#:~:text=Nel%201982%20si%20trasferisce%20alla,Attualmente%20%C3%A8%20Presidente%20dell'UEFA.",
#    'displayed_link': 'https://www.figc.it › hall-of-fame › le-stelle › michel-plat...',
#    'source_logo': 'https://serpapi.com/searches/667739eaf218019f4cf6fa1a/images/13ffa00535a8b94be7e4b0918974b69d809b25bbce98724308f75c991baec9ae.png',
#    'next_page_token': 'eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210UE1WWkJNRmR1Um1oT1RUZHlSbE5MYkhGR1UwVlRRblU1ZUZkdVlWbzNRV3B0TmtKa1JUQkliM1pUWTJFd1JIQmFaRnAyVmxGcFVXZHlja3hwVldkSFdVNXBSMnBrRWhjMmFtd3pXblI2ZVVOd2FVNXdPRFJRYmpWdFVYRkJheG9pUVVaWWNrVmpiekYwUkVWRWJYSkpaRUpIVlhaS01GUmpkSGRVYVhGbVR6QTRkdyIsImZjdiI6IjMiLCJlaSI6IjZqbDNadHp5Q3BpTnA4NFBuNW1RcUFrIiwicWMiOiJDaFp3YkdGMGFXNXBJSFZsWm1FZ2NISmxjMmxrWlc1MEVBZDlWVnN0UHciLCJxdWVzdGlvbiI6IkNvc2EgZmEgb3JhIE1pY2hlbCBQbGF0aW5pPyIsImxrIjoiR2hwamIzTmhJR1poSUc5eVlTQnRhV05vWld3Z2NHeGhkR2x1YVEiLCJicyI6ImMtUHk0RklNU0MxS3pqaThVaUV2UDA4aFdmM3dDb1dBbk1TU3pMeE1oZUpTQlRkUE4wZDdpYmZXUmdwU2NnVlFkUVZRYVpCNm9HcWdxclRNdEVRdU55NTVtRWx3QXpKQkNvb3lTektMRWt2eTdTV08yR0l6QjFVVmx4Mlh0SE4tY1NMRTZPVGsxT0xpZklWRW1KSDJFamNkakdTa3BKSXhWVUNOZy10UFMxVElMMHBVOE0xTXprak5RZWhfRlFQWEQxV1JDMUVCMVNfQUNBQSIsImlkIjoiZmNfMSJ9',
#    'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google_related_questions&google_domain=google.com&next_page_token=eyJvbnMiOiIxMDA0MSIsImZjIjoiRW9zQkNreEJTa2M1U210UE1WWkJNRmR1Um1oT1RUZHlSbE5MYkhGR1UwVlRRblU1ZUZkdVlWbzNRV3B0TmtKa1JUQkliM1pUWTJFd1JIQmFaRnAyVmxGcFVXZHlja3hwVldkSFdVNXBSMnBrRWhjMmFtd3pXblI2ZVVOd2FVNXdPRFJRYmpWdFVYRkJheG9pUVVaWWNrVmpiekYwUkVWRWJYSkpaRUpIVlhaS01GUmpkSGRVYVhGbVR6QTRkdyIsImZjdiI6IjMiLCJlaSI6IjZqbDNadHp5Q3BpTnA4NFBuNW1RcUFrIiwicWMiOiJDaFp3YkdGMGFXNXBJSFZsWm1FZ2NISmxjMmxrWlc1MEVBZDlWVnN0UHciLCJxdWVzdGlvbiI6IkNvc2EgZmEgb3JhIE1pY2hlbCBQbGF0aW5pPyIsImxrIjoiR2hwamIzTmhJR1poSUc5eVlTQnRhV05vWld3Z2NHeGhkR2x1YVEiLCJicyI6ImMtUHk0RklNU0MxS3pqaThVaUV2UDA4aFdmM3dDb1dBbk1TU3pMeE1oZUpTQlRkUE4wZDdpYmZXUmdwU2NnVlFkUVZRYVpCNm9HcWdxclRNdEVRdU55NTVtRWx3QXpKQkNvb3lTektMRWt2eTdTV08yR0l6QjFVVmx4Mlh0SE4tY1NMRTZPVGsxT0xpZklWRW1KSDJFamNkakdTa3BKSXhWVUNOZy10UFMxVElMMHBVOE0xTXprak5RZWhfRlFQWEQxV1JDMUVCMVNfQUNBQSIsImlkIjoiZmNfMSJ9'}],
#  'organic_results': [{'position': 1,
#    'title': 'Michel Platini',
#    'link': 'https://it.wikipedia.org/wiki/Michel_Platini',
#    'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://it.wikipedia.org/wiki/Michel_Platini&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQFnoECA8QAQ',
#    'displayed_link': 'https://it.wikipedia.org › wiki › Michel_Platini',
#    'favicon': 'https://serpapi.com/searches/667739eaf218019f4cf6fa1a/images/0b3c34d018d50fabd0dcecb5b9ce4e91aefbf60e804e9d182689d8aeaefaa5a2.png',
#    'snippet': 'Terminò la carriera a 31 anni, dopo aver conquistato numerosi trofei tra cui un campionato francese, due campionati italiani e, ad eccezione della Coppa UEFA, ...',
#    'snippet_highlighted_words': ['UEFA'],
#    'source': 'Wikipedia'},
#   {'position': 2,
#    'title': 'I presidenti UEFA del passato',
#    'link': 'https://it.uefa.com/news-media/news/0280-17b41c3ba7b5-686eb5eee802-1000--i-presidenti-uefa-del-passato/',
#    'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://it.uefa.com/news-media/news/0280-17b41c3ba7b5-686eb5eee802-1000--i-presidenti-uefa-del-passato/&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQFnoECBIQAQ',
#    'displayed_link': 'https://it.uefa.com › news-media › news › 0280-17b41c...',
#    'favicon': 'https://serpapi.com/searches/667739eaf218019f4cf6fa1a/images/0b3c34d018d50fabd0dcecb5b9ce4e9132b3495992e39cdb424002ffb8f76565.png',
#    'date': '5 apr 2023',
#    'snippet': 'Michel Platini è stato membro del Comitato Sviluppo Tecnico UEFA ed è entrato nel Comitato Esecutivo UEFA nel 2002. È stato eletto come sesto ...',
#    'snippet_highlighted_words': ['Platini', 'UEFA', 'UEFA'],
#    'missing': ['President'],
#    'must_include': {'word': 'President',
#     'link': 'https://www.google.com/search?sca_esv=dc08701f44d53b0e&hl=it&gl=it&q=Platini+UEFA+%22President%22&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQ5t4CegQIKRAB'},
#    'source': 'UEFA.com'},
#   {'position': 3,
#    'title': "Platini rieletto presidente dell'Uefa per la 3a volta - Eurosport",
#    'link': 'https://www.eurosport.it/calcio/champions-league/2014-2015/modello-champions_sto4652230/story.shtml',
#    'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.eurosport.it/calcio/champions-league/2014-2015/modello-champions_sto4652230/story.shtml&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQFnoECA4QAQ',
#    'displayed_link': 'https://www.eurosport.it › champions-league › story',
#    'favicon': 'https://serpapi.com/searches/667739eaf218019f4cf6fa1a/images/0b3c34d018d50fabd0dcecb5b9ce4e915fc8ac06d52d315b2f43a709fbf96679.png',
#    'date': '24 mar 2015',
#    'snippet': "Michel Platini è stato rieletto all'unanimità per la terza volta presidente della Fifa. L'ex campione della Juventus, in carica dal 2007, ...",
#    'snippet_highlighted_words': ['Platini', 'Fifa'],
#    'source': 'eurosport.it'},
#   {'position': 4,
#    'title': 'Michel Platini awarded Facchetti honour',
#    'link': 'https://it.uefa.com/news-media/news/0254-0d7d3467d66e-b5bf3434918c-1000--michel-platini-awarded-facchetti-honour/',
#    'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://it.uefa.com/news-media/news/0254-0d7d3467d66e-b5bf3434918c-1000--michel-platini-awarded-facchetti-honour/&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQFnoECAwQAQ',
#    'displayed_link': 'https://it.uefa.com › news › 02...',
#    'favicon': 'https://serpapi.com/searches/667739eaf218019f4cf6fa1a/images/0b3c34d018d50fabd0dcecb5b9ce4e911cee94cf956f2c3e87e3a24f80f55883.png',
#    'snippet': 'UEFA President Michel Platini has received the Giacinto Facchetti International Prize for his distinguished services to the game as both a player and ...',
#    'snippet_highlighted_words': ['UEFA President', 'Platini'],
#    'source': 'UEFA.com'},
#   {'position': 5,
#    'title': 'Michel Platini | Biography, FIFA, & Facts',
#    'link': 'https://www.britannica.com/biography/Michel-Platini',
#    'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.britannica.com/biography/Michel-Platini&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQFnoECA0QAQ',
#    'displayed_link': 'https://www.britannica.com › M...',
#    'favicon': 'https://serpapi.com/searches/667739eaf218019f4cf6fa1a/images/0b3c34d018d50fabd0dcecb5b9ce4e91f86ecc9642688d2aa9566b875f9f82e4.png',
#    'date': '6 giorni fa',
#    'snippet': 'Michel Platini (born June 21, 1955, Joeuf, France) is a French professional football (soccer) player and administrator who was named the ...',
#    'snippet_highlighted_words': ['Platini'],
#    'source': 'Britannica'},
#   {'position': 6,
#    'title': 'Michel Platini',
#    'link': 'https://en.wikipedia.org/wiki/Michel_Platini',
#    'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://en.wikipedia.org/wiki/Michel_Platini&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQFnoECF4QAQ',
#    'displayed_link': 'https://en.wikipedia.org › wiki',
#    'favicon': 'https://serpapi.com/searches/667739eaf218019f4cf6fa1a/images/0b3c34d018d50fabd0dcecb5b9ce4e91f377c179917947e7ca62f8a003dc0ff1.png',
#    'snippet': 'In 2007, he became the first former player to be elected as the president of the Union of European Football Associations (UEFA). He also held the positions of ...',
#    'snippet_highlighted_words': ['president', 'UEFA'],
#    'source': 'Wikipedia'},
#   {'position': 7,
#    'title': '5.320 foto e immagini di Michel Platini President',
#    'link': 'https://www.gettyimages.it/immagine/michel-platini-president',
#    'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.gettyimages.it/immagine/michel-platini-president&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQFnoECGMQAQ',
#    'displayed_link': 'https://www.gettyimages.it › immagine › michel-platini-...',
#    'favicon': 'https://serpapi.com/searches/667739eaf218019f4cf6fa1a/images/0b3c34d018d50fabd0dcecb5b9ce4e913991a6198fc20ee67d0c6b06d1a8b0de.png',
#    'snippet': 'Scopri foto stock e immagini editoriali di attualità di Michel Platini President su Getty Images. Scegli tra 5.320 immagini premium su Michel Platini ...',
#    'snippet_highlighted_words': ['Platini President', 'Platini'],
#    'source': 'Getty Images'},
#   {'position': 8,
#    'title': 'Previous UEFA presidents',
#    'link': 'https://www.uefa.com/news-media/news/0280-17b4199e535e-64d8a4443b2c-1000--previous-uefa-presidents/',
#    'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.uefa.com/news-media/news/0280-17b4199e535e-64d8a4443b2c-1000--previous-uefa-presidents/&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQFnoECF8QAQ',
#    'displayed_link': 'https://www.uefa.com › news',
#    'favicon': 'https://serpapi.com/searches/667739eaf218019f4cf6fa1a/images/0b3c34d018d50fabd0dcecb5b9ce4e91aa9641237c0a20f6278ca5621a1b5a66.png',
#    'date': '5 apr 2023',
#    'snippet': 'Michel Platini resigned as UEFA president on 14 September 2016. Lennart Johansson (Sweden). Term of office: 19 April 1990 – 26 January 2007',
#    'snippet_highlighted_words': ['Platini', 'UEFA president'],
#    'source': 'UEFA.com'},
#   {'position': 9,
#    'title': 'Sepp Blatter and Michel Platini: Former heads of FIFA and ...',
#    'link': 'https://www.skysports.com/football/news/11095/12648027/sepp-blatter-and-michel-platini-former-heads-of-fifa-and-uefa-cleared-of-corruption-charges-by-swiss-court',
#    'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.skysports.com/football/news/11095/12648027/sepp-blatter-and-michel-platini-former-heads-of-fifa-and-uefa-cleared-of-corruption-charges-by-swiss-court&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQFnoECGsQAQ',
#    'displayed_link': 'https://www.skysports.com › se...',
#    'favicon': 'https://serpapi.com/searches/667739eaf218019f4cf6fa1a/images/0b3c34d018d50fabd0dcecb5b9ce4e9164de3b76a27b2c97c21eda352522b1e0.png',
#    'date': '8 lug 2022',
#    'snippet': 'Sepp Blatter and Michel Platini: Former heads of FIFA and UEFA cleared of corruption charges by Swiss court ... Former FIFA president Sepp Blatter ...',
#    'snippet_highlighted_words': ['Platini', 'FIFA', 'UEFA', 'FIFA president'],
#    'source': 'Sky Sports'},
#   {'position': 10,
#    'title': 'Michel Platini resigns as Uefa president after ban reduced ...',
#    'link': 'https://www.theguardian.com/football/2016/may/09/michel-platini-cas-appeal-ban-football',
#    'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.theguardian.com/football/2016/may/09/michel-platini-cas-appeal-ban-football&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQFnoECGIQAQ',
#    'displayed_link': 'https://www.theguardian.com › ...',
#    'favicon': 'https://serpapi.com/searches/667739eaf218019f4cf6fa1a/images/0b3c34d018d50fabd0dcecb5b9ce4e9166efc52c23276405149cda8656714eb7.png',
#    'date': '9 mag 2016',
#    'snippet': "Michel Platini will finally resign from his post as Uefa president after losing his appeal against a ban from all football over a 'disloyal ...",
#    'snippet_highlighted_words': ['Platini', 'Uefa president'],
#    'source': 'The Guardian'}],
#  'related_searches': [{'block_position': 1,
#    'query': 'platini cosa fa adesso',
#    'link': 'https://www.google.com/search?sca_esv=dc08701f44d53b0e&hl=it&gl=it&q=Platini+cosa+fa+adesso&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQ1QJ6BAhdEAE',
#    'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Platini+cosa+fa+adesso'},
#   {'block_position': 1,
#    'query': 'dove vive platini oggi',
#    'link': 'https://www.google.com/search?sca_esv=dc08701f44d53b0e&hl=it&gl=it&q=Dove+vive+Platini+oggi&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQ1QJ6BAhcEAE',
#    'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Dove+vive+Platini+oggi'},
#   {'block_position': 1,
#    'query': 'platini assist in carriera',
#    'link': 'https://www.google.com/search?sca_esv=dc08701f44d53b0e&hl=it&gl=it&q=Platini+assist+in+carriera&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQ1QJ6BAhbEAE',
#    'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Platini+assist+in+carriera'},
#   {'block_position': 1,
#    'query': 'platini squadre',
#    'link': 'https://www.google.com/search?sca_esv=dc08701f44d53b0e&hl=it&gl=it&q=Platini+squadre&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQ1QJ6BAhaEAE',
#    'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Platini+squadre'},
#   {'block_position': 1,
#    'query': 'platini altezza',
#    'link': 'https://www.google.com/search?sca_esv=dc08701f44d53b0e&hl=it&gl=it&q=Platini+altezza&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQ1QJ6BAhZEAE',
#    'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Platini+altezza'},
#   {'block_position': 1,
#    'query': 'platini milan',
#    'link': 'https://www.google.com/search?sca_esv=dc08701f44d53b0e&hl=it&gl=it&q=Platini+Milan&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQ1QJ6BAhXEAE',
#    'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Platini+Milan'},
#   {'block_position': 1,
#    'query': 'platini origini italiane',
#    'link': 'https://www.google.com/search?sca_esv=dc08701f44d53b0e&hl=it&gl=it&q=Platini+origini+italiane&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQ1QJ6BAhYEAE',
#    'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Platini+origini+italiane'},
#   {'block_position': 1,
#    'query': 'platini transfermarkt',
#    'link': 'https://www.google.com/search?sca_esv=dc08701f44d53b0e&hl=it&gl=it&q=Platini+transfermarkt&sa=X&ved=2ahUKEwjcj9b7i_CGAxWYxskDHZ8MBJUQ1QJ6BAhWEAE',
#    'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Platini+transfermarkt'}],
#  'dmca_messages': {'title': 'Avvisi relativi ai risultati filtrati',
#   'messages': [{'content': "Alcuni risultati possono essere stati rimossi nell'ambito della normativa europea sulla protezione dei dati. Ulteriori informazioni",
#     'highlighted_words': [{'link': 'https://www.google.com/policies/faq',
#       'text': 'Ulteriori informazioni'}]}]},
#  'pagination': {'current': 1,
#   'next': 'https://www.google.com/search?q=Platini+UEFA+Presidency&oq=Platini+UEFA+Presidency&hl=it&gl=it&start=10&sourceid=chrome&ie=UTF-8',
#   'other_pages': {'2': 'https://www.google.com/search?q=Platini+UEFA+Presidency&oq=Platini+UEFA+Presidency&hl=it&gl=it&start=10&sourceid=chrome&ie=UTF-8',
#    '3': 'https://www.google.com/search?q=Platini+UEFA+Presidency&oq=Platini+UEFA+Presidency&hl=it&gl=it&start=20&sourceid=chrome&ie=UTF-8',
#    '4': 'https://www.google.com/search?q=Platini+UEFA+Presidency&oq=Platini+UEFA+Presidency&hl=it&gl=it&start=30&sourceid=chrome&ie=UTF-8',
#    '5': 'https://www.google.com/search?q=Platini+UEFA+Presidency&oq=Platini+UEFA+Presidency&hl=it&gl=it&start=40&sourceid=chrome&ie=UTF-8'}},
#  'serpapi_pagination': {'current': 1,
#   'next_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Platini+UEFA+Presidency&start=10',
#   'next': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Platini+UEFA+Presidency&start=10',
#   'other_pages': {'2': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Platini+UEFA+Presidency&start=10',
#    '3': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Platini+UEFA+Presidency&start=20',
#    '4': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Platini+UEFA+Presidency&start=30',
#    '5': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=it&google_domain=google.com&hl=it&q=Platini+UEFA+Presidency&start=40'}}}

### Add paragraph-related docs to Vectorstore

In [ ]:
def organic_results_to_splitted_docs(web_search:dict) -> list:
    urls = [res["link"] for res in web_search["results"]["organic_results"]]
    docs = [list(text_getter(url))[0] for url in urls]
    docs = [d for d in docs if isinstance(d, dict)]
    docs = [Document(page_content=d["text"], metadata={'title':d["title"], 'url': d["url"], 'query': web_search["generated_query"]}) for d in docs if d]
    splitted_docs = split_docs(docs)
    return splitted_docs

In [ ]:
# organic_results_to_splitted_docs(outline_with_searches.h2_titles[0].web_search)

In [ ]:
def fill_vectorstore(outline_with_searches:OutlineGenerator, vectorstore: Chroma):
    outline_copy = outline_with_searches  # copy outline so i keep the empty version
    total_titles = len(outline_copy.h2_titles)
    
    for index, h2 in enumerate(outline_copy.h2_titles):
        print(f"[+] Adding docs for ## {h2.title} ({index + 1}/{total_titles})...")
        splitted_docs = organic_results_to_splitted_docs(h2.web_search)
        vectorstore.add_documents(splitted_docs)

        if h2.h3_titles:
            for h3 in h2.h3_titles:
                print(f"[+] Adding docs for ### {h3.title}...")
                splitted_docs = organic_results_to_splitted_docs(h3.web_search)
                vectorstore.add_documents(splitted_docs)

In [ ]:
len(vectorstore.get()["ids"])

In [ ]:
if len(vectorstore.get()["ids"]) < 5:  # new empty vectorstore
    fill_vectorstore(outline_with_searches, vectorstore)

In [ ]:
len(vectorstore.get()["ids"])

In [ ]:
# organic_results = sample_search["organic_results"]
# urls = [res["link"] for res in organic_results]
# urls

In [ ]:
# docs = [get_text_from_html(url, generated_query.query) for url in urls]
# docs = [Document(page_content=d["text"], metadata={'url': d["url"], 'query': d["query"]}) for d in docs if d]
# len(docs)

In [ ]:
# docs

In [ ]:
# splitted_docs = split_docs(docs)
# splitted_docs

In [ ]:
# vectorstore.add_documents(u)

In [ ]:
# found = vectorstore.similarity_search(generated_query.query, k=6)
# found

### Fill outline

In [ ]:
def write_paragraph(title, outline, vectorstore, n_sources):
    sources = vectorstore.similarity_search(title, k=n_sources)
    print(f"sources l: {[len(d.page_content) for d in sources]}")
    context = "\n\n-----\n\n".join([d.page_content for d in sources])
    content = use_component(
        component_cls=ParagraphWriter, 
        input_dict={"outline": outline.print_formatted_outline(), "context": context, "p_title": title, "style": style_instructions}
    )
    print(f"len input dict: {len(outline.print_formatted_outline()) + len(context) + len(title) + len(style_instructions)}")
    return content, sources

In [ ]:
from lorem_text import lorem

def fill_outline_paragraphs(outline:OutlineGenerator, vectorstore:Chroma, n_sources:int) -> OutlineGenerator:
    outline_copy = outline  # copy outline so i keep the empty version
    for h2 in outline_copy.h2_titles:
        print(h2.title)
        h2_content, h2_sources = write_paragraph(title=h2.title, outline=outline_copy, vectorstore=vectorstore, n_sources=n_sources)
        # h2_content = ParagraphWriter(paragraph=lorem.paragraph())
        h2.content = h2_content
        h2.sources = h2_sources
        # print("\n\n^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
        # print(outline_copy.print_formatted_outline())

        if h2.h3_titles:
            for h3 in h2.h3_titles:
                print(h3.title)
                h3_content, h3_sources = write_paragraph(title=h3.title, outline=outline_copy, vectorstore=vectorstore, n_sources=n_sources)
                # h3_content = ParagraphWriter(paragraph=lorem.paragraph())
                h3.content = h3_content
                h3.sources = h3_sources
    return outline_copy

# filled_outline = fill_outline_paragraphs(outline, vectorstore=vectorstore, n_sources=4)
# filled_outline

In [ ]:
filled_outline = OutlineGenerator(h2_titles=[H2Title(title='Platini vs Maradona', content=ParagraphWriter(paragraph="In the 1980s, Michel Platini and Diego Maradona were fierce rivals in the Italian league. Platini, aged 65, praised Maradona as the 'greatest football lover' and a 'child king' known for his on-field brilliance. Maradona's impact at Napoli challenged Juventus' dominance, securing two Serie A titles and the 1989 UEFA Cup. While Maradona won more national team titles, Platini excelled in club competitions, winning more international titles. Their rivalry symbolized a golden era in Italian football, showcasing two of the sport's greatest talents."), h3_titles=None, web_search=None, sources=[Document(page_content='BRUSSELS (AP) — France great Michel Platini, one of Diego Maradona’s fiercest rivals in the Italian league during the 1980s, calls him “the greatest football lover there was.”\n\nIn a message to The Associated Press on Thursday, Platini added Maradona was “a child king. He was a bit excessive in all things, but the most important thing is that he was excessive on the pitch and it was beautiful.”\n\nMaradona died on Wednesday from a heart attack, aged 60.\n\nPlatini, who is 65, and Maradona played against each other at club level at a time when the Italian league was arguably the best in the world. Maradona arrived at Napoli in 1984, two years after Platini signed with Juventus.\n\nMaradona, who scored 115 goals for Napoli in 259 matches, helped the club challenge Juventus’ supremacy, winning its first two Serie A titles and the 1989 UEFA Cup.', metadata={'query': 'Comparison between Platini and Maradona', 'title': 'Platini to AP: Maradona was the ‘greatest football lover’', 'url': 'https://apnews.com/platini-to-ap-maradona-was-the-greatest-football-lover-be8c3ad6a57cffbd405a3475ac6951af'}), Document(page_content='BRUSSELS (AP) — France great Michel Platini, one of Diego Maradona’s fiercest rivals in the Italian league during the 1980s, calls him “the greatest football lover there was.”\n\nIn a message to The Associated Press on Thursday, Platini added Maradona was “a child king. He was a bit excessive in all things, but the most important thing is that he was excessive on the pitch and it was beautiful.”\n\nMaradona died on Wednesday from a heart attack, aged 60.\n\nPlatini, who is 65, and Maradona played against each other at club level at a time when the Italian league was arguably the best in the world. Maradona arrived at Napoli in 1984, two years after Platini signed with Juventus.\n\nMaradona, who scored 115 goals for Napoli in 259 matches, helped the club challenge Juventus’ supremacy, winning its first two Serie A titles and the 1989 UEFA Cup.', metadata={'query': 'Comparison between Platini and Maradona', 'title': 'Platini to AP: Maradona was the ‘greatest football lover’', 'url': 'https://apnews.com/platini-to-ap-maradona-was-the-greatest-football-lover-be8c3ad6a57cffbd405a3475ac6951af'}), Document(page_content="Diego Maradona vs. Michel Platini: Titles Won Comparison\n\nWho won the most tournaments, Michel Platini or Diego Maradona? Here you have the details, head to head, of the titles won by each of these players with their national teams and the international and domestic titles won with their clubs.\n\nWith his national team, Diego Maradona won 1 title more than Michel Platini, 3 against 2. Also in FIFA World Cups won Diego Maradona has the advantage over Michel Platini 1 to 0.\n\nAt club international level, Michel Platini won 3 titles more than Diego Maradona, 4 against 1.\n\nWhen you consider domestic leagues, tournaments and cups, Diego Maradona surpasses Michel Platini in titles won 8 to 5.\n\nOf course, these stats only tell a part of the story since they don't take into consideration the value of each tournament or how relevant the players were for their teams.\n\nTournaments with the National Team:\n\nDiego Maradona\n\nMichel Platini 3\n\nnational team\n\ntitles 2\n\nnational team\n\ntitles", metadata={'query': 'Comparison between Platini and Maradona', 'title': 'Diego Maradona vs. Michel Platini: Titles Won Comparison', 'url': 'https://www.aworldofsoccer.com/players_comparison/diego_maradona_vs_michel_platini.htm'}), Document(page_content="Diego Maradona vs. Michel Platini: Titles Won Comparison\n\nWho won the most tournaments, Michel Platini or Diego Maradona? Here you have the details, head to head, of the titles won by each of these players with their national teams and the international and domestic titles won with their clubs.\n\nWith his national team, Diego Maradona won 1 title more than Michel Platini, 3 against 2. Also in FIFA World Cups won Diego Maradona has the advantage over Michel Platini 1 to 0.\n\nAt club international level, Michel Platini won 3 titles more than Diego Maradona, 4 against 1.\n\nWhen you consider domestic leagues, tournaments and cups, Diego Maradona surpasses Michel Platini in titles won 8 to 5.\n\nOf course, these stats only tell a part of the story since they don't take into consideration the value of each tournament or how relevant the players were for their teams.\n\nTournaments with the National Team:\n\nDiego Maradona\n\nMichel Platini 3\n\nnational team\n\ntitles 2\n\nnational team\n\ntitles", metadata={'query': 'Comparison between Platini and Maradona', 'title': 'Diego Maradona vs. Michel Platini: Titles Won Comparison', 'url': 'https://www.aworldofsoccer.com/players_comparison/diego_maradona_vs_michel_platini.htm'})]), H2Title(title='Platini Quotes', content=ParagraphWriter(paragraph='Michel Platini, a football legend, has left behind a legacy of inspiring and memorable quotes throughout his career. Some of his notable quotes include:\n- "Football is a game of deceit. You have to be like a fox in the box."\n- "The ball is like a woman, she loves to be caressed."\n- "I prefer to win 5-4 than 1-0."\n- "Football is a team game, but you can\'t win without individual brilliance."\nPlatini\'s words reflect his passion for the sport and his unique perspective on the game.'), h3_titles=None, web_search=None, sources=[Document(page_content='Platini is considered to be one of the greatest players of all time, and is regarded as one of the finest passers in football history, as well as one of the best ever penalty kick and free kick specialists to have played the game.[8][12][23][24][25][26] A quick, versatile, elegant, and intelligent offensive midfield playmaker, with a unique ability to read the game and bend the ball from set pieces, he was renowned in particular for his ball control, technical ability, dribbling skills, creativity, range of passing, and vision, despite his lack of notable physical or athletic attributes.[8][23][27][28][29][30][31]', metadata={'query': 'Platini UEFA Presidency', 'title': 'Michel Platini', 'url': 'https://en.wikipedia.org/wiki/Michel_Platini'}), Document(page_content='Platini is considered to be one of the greatest players of all time, and is regarded as one of the finest passers in football history, as well as one of the best ever penalty kick and free kick specialists to have played the game.[8][12][23][24][25][26] A quick, versatile, elegant, and intelligent offensive midfield playmaker, with a unique ability to read the game and bend the ball from set pieces, he was renowned in particular for his ball control, technical ability, dribbling skills, creativity, range of passing, and vision, despite his lack of notable physical or athletic attributes.[8][23][27][28][29][30][31]', metadata={'query': 'Michel Platini career highlights', 'title': 'Michel Platini', 'url': 'https://en.wikipedia.org/wiki/Michel_Platini'}), Document(page_content='Platini is considered to be one of the greatest players of all time, and is regarded as one of the finest passers in football history, as well as one of the best ever penalty kick and free kick specialists to have played the game.[8][12][23][24][25][26] A quick, versatile, elegant, and intelligent offensive midfield playmaker, with a unique ability to read the game and bend the ball from set pieces, he was renowned in particular for his ball control, technical ability, dribbling skills, creativity, range of passing, and vision, despite his lack of notable physical or athletic attributes.[8][23][27][28][29][30][31]', metadata={'query': 'Platini career highlights', 'title': 'Michel Platini', 'url': 'https://en.wikipedia.org/wiki/Michel_Platini'}), Document(page_content="French association football player, manager and administrator\n\nMichel François Platini (French pronunciation: [miʃɛl fʁɑ̃swa platini]; born 21 June 1955) is a French football administrator and former player and manager. Regarded as one of the greatest footballers of all time, Platini won the Ballon d'Or three times in a row, in 1983, 1984 and 1985,[5] and came seventh in the FIFA Player of the Century vote.[6] In recognition of his achievements, he was named a Chevalier of the Légion d'honneur in 1985 and became an Officier in 1998. As the president of UEFA in 2015 he was banned from involvement in football under FIFA's organisation, over ethics violations. The ban lasted until 2023.[7]", metadata={'query': 'Platini UEFA Presidency', 'title': 'Michel Platini', 'url': 'https://en.wikipedia.org/wiki/Michel_Platini'})]), H2Title(title='Platini Career Highlights', content=ParagraphWriter(paragraph="Michel Platini's career highlights are a testament to his exceptional talent and success in the world of football. Some key points include:\n- Starting at Nancy, he scored 98 goals and won the 1978 French Cup.\n- At Saint-Étienne, he secured the 1981 French league title and reached the 1976 European Cup final.\n- Joining Juventus in 1982, he achieved unprecedented success with three Ballon d’Ors, two Serie A titles, and a European Cup victory.\n- Captaining France in Euro 1984, he scored a record nine goals and led his team to victory, earning the Best Player and Top Scorer awards."), h3_titles=[H3Title(title='Bonini Luiz-Koopmeiners Juve Chiesa Platini presidente', content=ParagraphWriter(paragraph="The Juventus team is making strategic moves in the transfer market, focusing on strengthening the midfield with top talents like Douglas Luiz and Koopmeiners. The potential acquisition of these players could elevate the team's performance significantly, as highlighted by former player Bonini. Amidst uncertainties surrounding Rabiot and Chiesa's future, the club's attention shifts to potential leadership changes, with the intriguing prospect of Michel Platini stepping into a presidential role. Platini's vision for the club and his outspoken views on the football landscape add an exciting dimension to Juventus' future prospects."), web_search=None, sources=[Document(page_content="Podcast Bonini a RBN: “Luiz-Koopmeiners top. Juve tieni Chiesa. Platini presidente”\n\nvedi letture\n\nLa Juve prosegue sulla strada del rafforzamento, soprattutto a centrocampo. Douglas Luiz sembra in dirittura d'arrivo dopo aver sostituito McKennie con Berrenechea come seconda contropartita tecnica insieme a Iling Junior, oltre a versare cash nelle case nelle casse dell'Aston Villa, per una cifra intorno ai 25 milioni. Giuntoli ora darà l'assalto a Koopmeiners, priorità per il nuovo tecnico Thiago Motta. L'Atalanta lo valuta 60 milioni, Huijsen potrebbe rientrare nell'operazione. Resta il doppio nodo legato a Rabiot e Chiesa. Il contratto del francese scade il 30 giugno, quello dell'attaccante nel 2025: le possibilità di addio sono concrete per entrambi. Intanto Platini tuona contro Uefa e Fifa, il sogno di molti tifosi bianconeri è di vederlo nella dirigenza. In ESCLUSIVA a Fuori di Juve Massimo Bonini, ex centrocampista della Vecchia Signora.", metadata={'query': 'Bonini Luiz-Koopmeiners Juve Chiesa Platini presidente', 'title': 'Bonini a RBN: “Luiz-Koopmeiners top. Juve tieni Chiesa. Platini presidente”', 'url': 'https://www.bianconeranews.it/podcast/bonini-a-rbn-luiz-koopmeiners-top-juve-tieni-chiesa-platini-presidente-75484'}), Document(page_content="Platini: “John Elkann ci tiene e la Juve tornerà vincente. Io Presidente...”\n\nvedi letture\n\nPlatini parla della Juve e, convinto che tornerà a vincere con John Elkann. L'ex fuoriclasse si esprime inoltre sull'ipotesi di diventare presidente\n\nMichel Platini torna a parlare di Juventus, John Elkann, Superlega, Infantino e Ceferin. L'ex fuoriclasse bianconero si è concesso ai microfoni di Tuttosport con una lunga intervista.\n\nIn primis l'ex fuoriclasse bianconero risponde ai tifosi convinti che John Elkann abbia poca passione: “Sbagliano. Magari non è a livello di suo nonno, l'Avvocato, di Umberto, ma ci tiene. Si capisce che vive con grande serietà l’impegno centenario della sua famiglia nella Juventus. E poi ha grandissime responsabilità nella gestione delle aziende. Forse non riesce a comunicarlo bene, ma alla Juventus ci tiene, è vicino alla società”.", metadata={'query': 'Bonini Luiz-Koopmeiners Juve Chiesa Platini presidente', 'title': 'Platini: “John Elkann ci tiene e la Juve tornerà vincente. Io Presidente...”', 'url': 'https://www.bianconeranews.it/primo-piano/platini-john-elkann-ci-tiene-e-la-juve-tornera-vincente-io-presidente-74178'}), Document(page_content="La Juve continua a lavorare per mettere a segno i primi colpi di mercato. Il centrocampo è il reparto su cui è concentrato Giuntoli. Douglas Luiz si avvicina sempre di più: all'Aston Villa andranno Iling Junior, Barrenechea e 25 milioni. Poi sarà la volta di Koopmeiners, tra le prime richieste di Thiago Motta: l'Atalanta chiede 60 milioni, la chiave per arrivare all'olandese potrebbe essere Huijsen. La permanenza di Rabiot e Chiesa resta in forte dubbio. Il francese è in scadenza il 30 giugno, su di lui pare ci sia il Real Madrid. Contratto fino al 2025 invece per Chiesa, accordo per il rinnovo lontano, cessione probabile. Intanto arriva la profezia di Platini, secondo il quale tra qualche anno Uefa e Fifa potrebbero scomparire. In ESCLUSIVA a Fuori di Juve Massimo Bonini, ex centrocampista della Vecchia Signora ai tempi de Le Roi.", metadata={'query': 'Bonini Luiz-Koopmeiners Juve Chiesa Platini presidente', 'title': 'Bonini a RBN: ‚ÄúLuiz e Koopmeiners tanta roba. Motta scelta giustissima. Platini? Presidente della Juve ‚Äù', 'url': 'https://www.tuttojuve.com/podcast-tj/bonini-a-rbn-luiz-e-koopmeiners-tanta-roba-motta-scelta-giustissima-platini-presidente-della-juve-692047'}), Document(page_content="In caso di doppio colpo Douglas Luiz-Koopmeiners, secondo Bonini la Juve farebbe un salto di qualità importante: “La mediana detta i tempi e comanda il gioco di una squadra. Si tratta di due giocatori fantastici, fermo restando che dipende anche dall'organizzazione di gioco. Ritengo la Juve abbia diversi giovani interessanti, ma se può investire su elementi già pronti, va bene. Molto dipenderà da Thiago Motta. A Bologna la sua squadra ha mostrato qualità, idee ben precise. Douglas Luiz e Koopmeiners comunque sono tanta roba, sono bravi con e senza palla, abili ad attaccare la profondità, sono completi, due top player. La scelta di Motta è giustissima. Ha dimostrato di essere un ottimo allenatore. A Bologna malgrado le sostituzioni, la musica non cambiava. Questa è la bravura di un tecnico, evidentemente riesce ad entrare nella testa dei suoi ragazzi, ognuno sa cosa fare e così è tutto più semplice. Rabiot? Fa parte della Nazionale francese, nelle ultime stagioni ha dimostrato il suo", metadata={'query': 'Bonini Luiz-Koopmeiners Juve Chiesa Platini presidente', 'title': 'Bonini a RBN: ‚ÄúLuiz e Koopmeiners tanta roba. Motta scelta giustissima. Platini? Presidente della Juve ‚Äù', 'url': 'https://www.tuttojuve.com/podcast-tj/bonini-a-rbn-luiz-e-koopmeiners-tanta-roba-motta-scelta-giustissima-platini-presidente-della-juve-692047'})]), H3Title(title='Michel Platini career highlights', content=ParagraphWriter(paragraph="Michel Platini's career highlights shine brightly with remarkable achievements. Starting at Nancy, he scored 98 goals and clinched the 1978 French Cup. His move to Saint-Étienne brought the 1981 French league title and a 1976 European Cup final appearance. Joining Juventus in 1982, Platini soared with three Ballon d’Ors, two Serie A titles, and a European Cup triumph. Leading France in Euro 1984, he set a record with nine goals, securing victory and earning Best Player and Top Scorer accolades."), web_search=None, sources=[Document(page_content='MICHEAL PLATINI Football Career Achievements and Contributions Mondebaron · Follow 3 min read · Jan 18, 2024 -- Listen Share\n\nMichel Platini, a French football legend, stands among the greatest players in history. His remarkable career includes three consecutive Ballon d’Or wins and leading France to victory in the 1984 European Championship. Beyond playing, he made significant contributions as a coach and football administrator. However, controversies marred his later years, particularly during his involvement with FIFA.\n\nEarly Life\n\nBorn on June 21, 1955, in Jœuf, Lorraine, France, Platini inherited a passion for football from his father, Aldo, a former footballer and AS Nancy director. Raised in a modest family with Italian roots, Platini joined AS Jœuf at six, showcasing talent and creativity. Inspired by Pelé, he dreamed of representing the French national team.\n\nPlaying Career\n\nA. Clubs and Transfer Market', metadata={'query': 'Platini career highlights', 'title': 'MICHEAL PLATINI Football Career Achievements and Contributions', 'url': 'https://medium.com/@mondebaron/michel-platini-a-french-football-legend-stands-among-the-greatest-players-in-history-8b2f89a4abe6'}), Document(page_content='MICHEAL PLATINI Football Career Achievements and Contributions Mondebaron · Follow 3 min read · Jan 18, 2024 -- Listen Share\n\nMichel Platini, a French football legend, stands among the greatest players in history. His remarkable career includes three consecutive Ballon d’Or wins and leading France to victory in the 1984 European Championship. Beyond playing, he made significant contributions as a coach and football administrator. However, controversies marred his later years, particularly during his involvement with FIFA.\n\nEarly Life\n\nBorn on June 21, 1955, in Jœuf, Lorraine, France, Platini inherited a passion for football from his father, Aldo, a former footballer and AS Nancy director. Raised in a modest family with Italian roots, Platini joined AS Jœuf at six, showcasing talent and creativity. Inspired by Pelé, he dreamed of representing the French national team.\n\nPlaying Career\n\nA. Clubs and Transfer Market', metadata={'query': 'Michel Platini career highlights', 'title': 'MICHEAL PLATINI Football Career Achievements and Contributions', 'url': 'https://medium.com/@mondebaron/michel-platini-a-french-football-legend-stands-among-the-greatest-players-in-history-8b2f89a4abe6'}), Document(page_content='Michel Platini (born June 21, 1955, Joeuf, France) is a French professional football (soccer) player and administrator who was named the European Footballer of the Year three times (1983–85) and served as president of the Union of European Football Associations (UEFA; 2007–16).\n\nPlatini made his French first-division debut with AS Nancy in 1973. A prolific attacking midfielder, he led the club to a French Cup championship in 1978 and then joined Saint-Étienne the following year. He helped his new team win a league title in 1981, and in 1982 he transferred to Juventus of Italy. Platini emerged as an international superstar with Juventus, leading the team to a European Cup final in 1983 (a loss) and a European Cup championship in 1985. Juventus also captured two league championships (1984, 1986), a UEFA Super Cup (1984), and a European Cup Winners’ Cup (1984) during Platini’s tenure with the club, which lasted until 1987.\n\nBritannica Quiz I Am the Greatest (Athlete)', metadata={'query': 'Platini career highlights', 'title': 'Michel Platini | Biography, FIFA, & Facts', 'url': 'https://www.britannica.com/biography/Michel-Platini'}), Document(page_content='Michel Platini (born June 21, 1955, Joeuf, France) is a French professional football (soccer) player and administrator who was named the European Footballer of the Year three times (1983–85) and served as president of the Union of European Football Associations (UEFA; 2007–16).\n\nPlatini made his French first-division debut with AS Nancy in 1973. A prolific attacking midfielder, he led the club to a French Cup championship in 1978 and then joined Saint-Étienne the following year. He helped his new team win a league title in 1981, and in 1982 he transferred to Juventus of Italy. Platini emerged as an international superstar with Juventus, leading the team to a European Cup final in 1983 (a loss) and a European Cup championship in 1985. Juventus also captured two league championships (1984, 1986), a UEFA Super Cup (1984), and a European Cup Winners’ Cup (1984) during Platini’s tenure with the club, which lasted until 1987.\n\nBritannica Quiz I Am the Greatest (Athlete)', metadata={'query': 'Michel Platini career highlights', 'title': 'Michel Platini | Biography, FIFA, & Facts', 'url': 'https://www.britannica.com/biography/Michel-Platini'})])], web_search=None, sources=[Document(page_content='Throughout his remarkable and prolific professional soccer player career spanning 15 years, Platini won three consecutive European Footballer of the Year awards (1983-1985) as well as two prestigious World Footballer of the Year awards in 1984 and 1985, among numerous other distinctions.\n\nManager of Les Bleus and UEFA President', metadata={'query': 'Michel Platini career highlights', 'title': 'Michel Platini biography and career accomplishments', 'url': 'https://www.footballhistory.org/player/michel-platini.html'}), Document(page_content='Throughout his remarkable and prolific professional soccer player career spanning 15 years, Platini won three consecutive European Footballer of the Year awards (1983-1985) as well as two prestigious World Footballer of the Year awards in 1984 and 1985, among numerous other distinctions.\n\nManager of Les Bleus and UEFA President', metadata={'query': 'Platini career highlights', 'title': 'Michel Platini biography and career accomplishments', 'url': 'https://www.footballhistory.org/player/michel-platini.html'}), Document(page_content='Playing Career\n\nA. Clubs and Transfer Market\n\n1. Nancy (1972–1979): Platini’s professional journey began at Nancy in 1972, yielding 98 goals in 181 appearances and a 1978 French Cup victory.\n\n2. Saint-Étienne (1979–1982): Moving to Saint-Étienne in 1979, Platini won the 1981 French league title and reached the 1976 European Cup final, scoring 58 goals in 104 games.\n\n3. Juventus (1982–1987): Joining Juventus in 1982 for a record fee, he achieved unparalleled success — three Ballon d’Ors, two Serie A titles, a European Cup, and other honors, scoring 68 goals in 147 matches.\n\nB. Euro 1984 Triumph\n\nCaptaining France in Euro 1984, Platini scored a record nine goals, leading to a historic victory. Notable moments include a semi-final goal against Portugal and the opening goal in the final against Spain. He earned the tournament’s Best Player and Top Scorer awards, becoming the first to score two hat-tricks in a single European Championship.\n\nC. Trophies and Achievements', metadata={'query': 'Platini career highlights', 'title': 'MICHEAL PLATINI Football Career Achievements and Contributions', 'url': 'https://medium.com/@mondebaron/michel-platini-a-french-football-legend-stands-among-the-greatest-players-in-history-8b2f89a4abe6'}), Document(page_content='Playing Career\n\nA. Clubs and Transfer Market\n\n1. Nancy (1972–1979): Platini’s professional journey began at Nancy in 1972, yielding 98 goals in 181 appearances and a 1978 French Cup victory.\n\n2. Saint-Étienne (1979–1982): Moving to Saint-Étienne in 1979, Platini won the 1981 French league title and reached the 1976 European Cup final, scoring 58 goals in 104 games.\n\n3. Juventus (1982–1987): Joining Juventus in 1982 for a record fee, he achieved unparalleled success — three Ballon d’Ors, two Serie A titles, a European Cup, and other honors, scoring 68 goals in 147 matches.\n\nB. Euro 1984 Triumph\n\nCaptaining France in Euro 1984, Platini scored a record nine goals, leading to a historic victory. Notable moments include a semi-final goal against Portugal and the opening goal in the final against Spain. He earned the tournament’s Best Player and Top Scorer awards, becoming the first to score two hat-tricks in a single European Championship.\n\nC. Trophies and Achievements', metadata={'query': 'Michel Platini career highlights', 'title': 'MICHEAL PLATINI Football Career Achievements and Contributions', 'url': 'https://medium.com/@mondebaron/michel-platini-a-french-football-legend-stands-among-the-greatest-players-in-history-8b2f89a4abe6'})]), H2Title(title='Platini Controversy', content=ParagraphWriter(paragraph="Michel Platini's career was marred by controversy when he accepted a £1.6m payment from Blatter, leading to an eight-year ban from football activities. Despite challenging the accusations, he faced scrutiny and a shortened four-year ban. Additionally, Platini was found guilty of ethics violations alongside Blatter, further tarnishing his reputation. His involvement in the 2015 Greek football scandal and mention in the Panama Papers added to the cloud of controversy surrounding his name. Platini's claim of a fixed 1998 FIFA World Cup draw also raised eyebrows, highlighting a turbulent chapter in his career."), h3_titles=None, web_search=None, sources=[Document(page_content="However, soon after his decision to run for presidency, Platini was himself under the spotlight after it was discovered that he had accepted a mysterious £1.6m payment from Blatter, supposedly for his work after the 1998 World Cup.\n\nMICHAEL BUHOLZER/GettyImages\n\nFIFA led their own investigation into the matter, and both Blatter and Platini were ultimately found guilty of breaching ethics regulations. The pair were handed eight-year global bans from any football activities, but Platini's ban was ultimately shortened to four years.\n\nAway from the headlines, he has continued to challenge the accusations against him, although he now finds himself back under pressure once again.", metadata={'query': 'Platini UEFA Presidency', 'title': "Michel Platini: The Former UEFA President's Controversial Career in Football Politics", 'url': 'https://www.si.com/soccer/2019/06/18/michel-platini-former-uefa-presidents-controversial-career-football-politics'}), Document(page_content="Platini then forced his way into everybody’s long-term consciousness when he won the UEFA presidency, probably on the back of his publicity in Jose's advert. Since then he has been almost constantly annoying everybody by employing a method of arguing some good policies very poorly.\n\nIt's not surprising then that most of us don't know and probably don't care about Platini's playing days, but as I hope to demonstrate, that's a real shame.\n\nFor as far back as I can remember, France has always been a footballing power. One of my earliest memories is their absolute domination of Brazil in a World Cup final in 1998. The list of France legends runs deep with big names which hasn't yet been exhausted: Deschamps, Cantona, Blanc, Desailly, Petit, Zidane, Henry, Pires, Vieira, Ribéry, Benzema...", metadata={'query': 'Platini career highlights', 'title': "Legends Revisited: Michel Platini - France's Greatest Player?", 'url': 'https://bleacherreport.com/articles/262769-legends-revisited-the-life-and-times-of-michel-platini'}), Document(page_content="Platini then forced his way into everybody’s long-term consciousness when he won the UEFA presidency, probably on the back of his publicity in Jose's advert. Since then he has been almost constantly annoying everybody by employing a method of arguing some good policies very poorly.\n\nIt's not surprising then that most of us don't know and probably don't care about Platini's playing days, but as I hope to demonstrate, that's a real shame.\n\nFor as far back as I can remember, France has always been a footballing power. One of my earliest memories is their absolute domination of Brazil in a World Cup final in 1998. The list of France legends runs deep with big names which hasn't yet been exhausted: Deschamps, Cantona, Blanc, Desailly, Petit, Zidane, Henry, Pires, Vieira, Ribéry, Benzema...", metadata={'query': 'Michel Platini career highlights', 'title': "Legends Revisited: Michel Platini - France's Greatest Player?", 'url': 'https://bleacherreport.com/articles/262769-legends-revisited-the-life-and-times-of-michel-platini'}), Document(page_content='On 8 October 2015, Platini was provisionally suspended until 6 January 2016 from any football-related activity.[51] On 21 December, Platini and Blatter were both found guilty of ethics violations and barred from the sport until 2023.[7] The committee said Platini "did not show commitment to an ethical attitude" and lacked respect for laws and regulations of the organization.[52][53] Platini boycotted the hearing, and said he planned to appeal the decision, declaring himself "at peace with my conscience."[54][55][56][57] He appealed to Swiss courts, and to the European Court of Human Rights, but the courts rejected his appeals.[58]\n\nPlatini was also involved in the Greek public polemica regarding the 2015 Greek football scandal.[59][60][61]\n\nIn April 2016, Platini has been named in the Panama Papers.[62]\n\nOn 18 May 2018, Platini said that the 1998 FIFA World Cup draw was fixed to ensure France and Brazil could not face each other until the final if both teams won their groups.[63]', metadata={'query': 'Platini career highlights', 'title': 'Michel Platini', 'url': 'https://en.wikipedia.org/wiki/Michel_Platini'})]), H2Title(title='Platini UEFA Presidency', content=ParagraphWriter(paragraph="Michel Platini's UEFA presidency marked a significant milestone in his football career. Elected in 2007 after defeating Lennart Johansson, Platini focused on promoting solidarity and universality within UEFA. His leadership brought about positive changes, emphasizing the importance of unity and inclusivity in European football governance. Platini's tenure saw advancements in technical development and a commitment to fostering a strong football community. Through his presidency, Platini left a lasting impact on UEFA, shaping the organization's values and direction for the future."), h3_titles=None, web_search=None, sources=[Document(page_content='Platini was, along with Fernand Sastre, head of the organizing committee for the 1998 FIFA World Cup, held in France. He served on the UEFA Technical Development Committee from 1988 to 1990. He has been a member of the UEFA Executive Committee and European member of the FIFA Executive Committee since 2002. He continued to climb the ranks of UEFA and FIFA football administration and in 2006, became a chairman of the FIFA Technical and Development Committee, while also being vice-president of the French Football Federation.\n\nUEFA presidency\n\nPlatini confirmed that he would run for the UEFA presidency in July 2006. In the election in Düsseldorf on 26 January 2007, he defeated Lennart Johansson, who held the post for the previous 16 years, by 27 votes to 23.[40] Platini based his speech on virtues of solidarity and universality.\n\nMichel Platini visiting the construction site of the Stadion Miejski, 2009', metadata={'query': 'Platini UEFA Presidency', 'title': 'Michel Platini', 'url': 'https://en.wikipedia.org/wiki/Michel_Platini'}), Document(page_content='Platini was, along with Fernand Sastre, head of the organizing committee for the 1998 FIFA World Cup, held in France. He served on the UEFA Technical Development Committee from 1988 to 1990. He has been a member of the UEFA Executive Committee and European member of the FIFA Executive Committee since 2002. He continued to climb the ranks of UEFA and FIFA football administration and in 2006, became a chairman of the FIFA Technical and Development Committee, while also being vice-president of the French Football Federation.\n\nUEFA presidency\n\nPlatini confirmed that he would run for the UEFA presidency in July 2006. In the election in Düsseldorf on 26 January 2007, he defeated Lennart Johansson, who held the post for the previous 16 years, by 27 votes to 23.[40] Platini based his speech on virtues of solidarity and universality.\n\nMichel Platini visiting the construction site of the Stadion Miejski, 2009', metadata={'query': 'Michel Platini career highlights', 'title': 'Michel Platini', 'url': 'https://en.wikipedia.org/wiki/Michel_Platini'}), Document(page_content='Platini was, along with Fernand Sastre, head of the organizing committee for the 1998 FIFA World Cup, held in France. He served on the UEFA Technical Development Committee from 1988 to 1990. He has been a member of the UEFA Executive Committee and European member of the FIFA Executive Committee since 2002. He continued to climb the ranks of UEFA and FIFA football administration and in 2006, became a chairman of the FIFA Technical and Development Committee, while also being vice-president of the French Football Federation.\n\nUEFA presidency\n\nPlatini confirmed that he would run for the UEFA presidency in July 2006. In the election in Düsseldorf on 26 January 2007, he defeated Lennart Johansson, who held the post for the previous 16 years, by 27 votes to 23.[40] Platini based his speech on virtues of solidarity and universality.\n\nMichel Platini visiting the construction site of the Stadion Miejski, 2009', metadata={'query': 'Platini career highlights', 'title': 'Michel Platini', 'url': 'https://en.wikipedia.org/wiki/Michel_Platini'}), Document(page_content='Shortly after leaving the pitch for the final time, Platini turned to coaching, becoming the French national coach in 1988. He resigned that post in 1992 and then served as copresident of the organizing committee for the 1998 World Cup, which was hosted by France. Platini held a number of administrative positions in both the Fédération Internationale de Football Association (FIFA) and UEFA before he was elected UEFA president in 2007.', metadata={'query': 'Platini career highlights', 'title': 'Michel Platini | Biography, FIFA, & Facts', 'url': 'https://www.britannica.com/biography/Michel-Platini'})])])


In [ ]:
print(filled_outline.print_formatted_outline())

### Auto links

In [ ]:
# from typing import Dict
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# def calculate_cosine_similarity(phrase1: str, phrases2: List[str]) -> Dict[str, float]:
#     # Combine phrase1 and phrases2 into a single list
#     all_phrases = [phrase1] + phrases2
    
#     # Vectorize using TF-IDF
#     vectorizer = TfidfVectorizer().fit_transform(all_phrases)
#     vectors = vectorizer.toarray()
    
#     # Get vectors for phrase1 and phrases2
#     phrase1_vector = vectors[0]
#     phrases2_vectors = vectors[1:]
    
#     # Calculate cosine similarity between phrase1 and each phrase in phrases2
#     similarities = {}
#     for i, phrase2 in enumerate(phrases2):
#         similarity_score = cosine_similarity([phrase1_vector], [phrases2_vectors[i]])[0][0]
#         similarities[phrase2] = similarity_score
    
#     return similarities


# def calculate_cosine_similarity_strings(str1: str, str2: str) -> float:
#     # Tokenize and vectorize the input strings
#     vectorizer = TfidfVectorizer(stop_words='english')
#     vectors = vectorizer.fit_transform([str1, str2])

#     # Calculate cosine similarity
#     cosine_sim = cosine_similarity(vectors[0], vectors[1]).flatten()[0]
    
#     return cosine_sim

In [ ]:
from typing import Dict
import spacy


nlp = spacy.load("en_core_web_lg")

# Function to extract key phrases using spaCy
def extract_key_phrases(text: str) -> List[str]:
    doc = nlp(text)
    return [chunk.text for chunk in doc.noun_chunks]

In [ ]:
# # Automatically define the mapping of key phrases to source URLs
# def map_phrases_to_sources(paragraph: str, sources: List[Document], similarity_threshold=0.7) -> Dict[str, str]:
#     paragraph_phrases = extract_key_phrases(paragraph)
    
#     phrases_and_urls = []
#     for phrase in paragraph_phrases:
#         # print("Phrase :", phrase)
#         phrase_to_sources_sim = []
#         for idx, source in enumerate(sources):
#             # print("source ", idx)
#             source_phrases = extract_key_phrases(source.page_content)
#             source_content = source.page_content

#             similarity_score = calculate_cosine_similarity_strings(phrase, source_content)
#             # print(similarity_score)
#             phrase_to_sources_sim.append({"title": source.metadata["title"], "url": source.metadata["url"], "similarity": similarity_score})
#         best_phrase_match = max(phrase_to_sources_sim, key=lambda x: x['similarity'])
#         if best_phrase_match["similarity"] > similarity_threshold:
#             phrases_and_urls.append({"par_phrase": phrase, "url": best_phrase_match["url"], "best_match": best_phrase_match})
            
#     [print(p) for p in phrases_and_urls]
#     return phrases_and_urls

# # h2par = outline.h2_titles[0]
# # mapping = map_phrases_to_sources(paragraph=h2par.content.paragraph, sources=h2par.sources, similarity_threshold=0.3)

In [ ]:
from typing import Any


def create_links_in_paragraph(paragraph: str, matches: List[Dict[str, Any]]) -> str:
    for match in matches:
        par_phrase = f" {match['par_phrase']} "
        url = match['best_match']['url']
        markdown_link = f"[{par_phrase.strip()}]({url})"
        if markdown_link in paragraph:
            continue
        else:
            if url not in paragraph:
                paragraph = paragraph.replace(par_phrase.strip(), f" {markdown_link} ", 1)

    return paragraph

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

def get_most_similar_element(element:str, list_of_elements:list):

    # Initialize the model
    model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

    # List of sentences (corpus)


    # Generate embeddings for the corpus
    corpus_embeddings = model.encode(list_of_elements, convert_to_tensor=True)

    # Generate embedding for the query
    query_embedding = model.encode(element, convert_to_tensor=True)

    # Compute cosine similarity between the query embedding and corpus embeddings
    cosine_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)

    # Get the most similar sentence
    top_score, top_idx = torch.max(cosine_scores, dim=1)

    # Print the most similar sentence and its score
    most_similar_sentence = list_of_elements[top_idx.item()]
    most_similar_score = top_score.item()

    if most_similar_score > 0.50:
        if len(most_similar_sentence) > 5:
            print(element)
            print(f"Most similar sentence: '{most_similar_sentence.strip()}' {type(most_similar_sentence)}")
            print(f"(Score: {most_similar_score:.4f})\n-------\n")
    return most_similar_sentence, most_similar_score


In [ ]:
def map_phrases_to_sources(paragraph: str, sources: List[Document], similarity_threshold=0.5):
    paragraph_sentences = paragraph.split(".")

    all_sources_sentences = []
    for s in sources:
        sentences = s.page_content.split(".")
        all_sources_sentences.extend(sentences)
    
    mapping= []
    for s in paragraph_sentences:
        if len(s) > 2:
            most_similar_source_sentence, score = get_most_similar_element(s, all_sources_sentences)
            if score >= similarity_threshold:
                mapping.append({"par_sent": s, "most_similar": most_similar_source_sentence, "score": score})
    
    return mapping

In [ ]:
# def add_link_to_outline(outline: OutlineGenerator, similarity_threshold=0.6) -> OutlineGenerator:
#     outline_copy = outline  # copy outline so i keep the empty version
#     for h2 in outline_copy.h2_titles:
#         matches = map_phrases_to_sources(h2.content.paragraph, h2.sources, similarity_threshold=similarity_threshold)
#         h2.content.paragraph = create_links_in_paragraph(h2.content.paragraph, matches)

#         if h2.h3_titles:
#             for h3 in h2.h3_titles:
#                 matches = map_phrases_to_sources(h3.content.paragraph, h3.sources, similarity_threshold=similarity_threshold)
#                 h3.content.paragraph = create_links_in_paragraph(h3.content.paragraph, matches)
#     return outline_copy

#### HF test

In [ ]:
def get_source_by_string(string:str, sources:List[Document]) -> Document:
    for s in sources:
        if string.lower() in s.page_content.lower():
            return s

In [ ]:
def add_link_to_outline(outline: OutlineGenerator, similarity_threshold=0.5) -> OutlineGenerator:
    outline_copy = outline  # copy outline so i keep the empty version
    for h2 in outline_copy.h2_titles:
        mapping = map_phrases_to_sources(h2.content.paragraph, h2.sources, similarity_threshold=similarity_threshold)
        matches = []
        for m in mapping:
            par_sent_phrases = extract_key_phrases(m["par_sent"])
            most_similar_source_phrase = m["most_similar"]
            noun_phrase_for_link, score = get_most_similar_element(most_similar_source_phrase, par_sent_phrases)
            source = get_source_by_string(most_similar_source_phrase, h2.sources)
            if source: # come fa a non esserci source? #TODO
                matches.append({"par_phrase": noun_phrase_for_link, "best_match": {"url": source.metadata["url"]}})
        h2.content.paragraph = create_links_in_paragraph(h2.content.paragraph, matches)

        if h2.h3_titles:
            for h3 in h2.h3_titles:
                mapping = map_phrases_to_sources(h3.content.paragraph, h3.sources, similarity_threshold=similarity_threshold)
                matches = []
                for m in mapping:
                    par_sent_phrases = extract_key_phrases(m["par_sent"])
                    most_similar_source_phrase = m["most_similar"]
                    noun_phrase_for_link, score = get_most_similar_element(most_similar_source_phrase, par_sent_phrases)
                    source = get_source_by_string(most_similar_source_phrase, h3.sources)
                    if source:
                        matches.append({"par_phrase": noun_phrase_for_link, "best_match": {"url": source.metadata["url"]}})
                h3.content.paragraph = create_links_in_paragraph(h3.content.paragraph, matches)
    return outline_copy

In [ ]:
outline_wlink2 = add_link_to_outline(filled_outline)

In [ ]:
outline_wlink2 = OutlineGenerator(h2_titles=[H2Title(title='Platini vs Maradona', content=ParagraphWriter(paragraph="In the 1980s,  [Michel Platini](https://www.aworldofsoccer.com/players_comparison/diego_maradona_vs_michel_platini.htm)  and Diego  [Maradona](https://apnews.com/platini-to-ap-maradona-was-the-greatest-football-lover-be8c3ad6a57cffbd405a3475ac6951af)  were fierce rivals in the Italian league. Platini, aged 65, praised Maradona as the 'greatest football lover' and a 'child king' known for his on-field brilliance. Maradona's impact at Napoli challenged Juventus' dominance, securing two Serie A titles and the 1989 UEFA Cup. While Maradona won more national team titles, Platini excelled in club competitions, winning more international titles. Their rivalry symbolized a golden era in Italian football, showcasing two of the sport's greatest talents."), h3_titles=None, web_search=None, sources=[Document(page_content='BRUSSELS (AP) — France great Michel Platini, one of Diego Maradona’s fiercest rivals in the Italian league during the 1980s, calls him “the greatest football lover there was.”\n\nIn a message to The Associated Press on Thursday, Platini added Maradona was “a child king. He was a bit excessive in all things, but the most important thing is that he was excessive on the pitch and it was beautiful.”\n\nMaradona died on Wednesday from a heart attack, aged 60.\n\nPlatini, who is 65, and Maradona played against each other at club level at a time when the Italian league was arguably the best in the world. Maradona arrived at Napoli in 1984, two years after Platini signed with Juventus.\n\nMaradona, who scored 115 goals for Napoli in 259 matches, helped the club challenge Juventus’ supremacy, winning its first two Serie A titles and the 1989 UEFA Cup.', metadata={'query': 'Comparison between Platini and Maradona', 'title': 'Platini to AP: Maradona was the ‘greatest football lover’', 'url': 'https://apnews.com/platini-to-ap-maradona-was-the-greatest-football-lover-be8c3ad6a57cffbd405a3475ac6951af'}), Document(page_content='BRUSSELS (AP) — France great Michel Platini, one of Diego Maradona’s fiercest rivals in the Italian league during the 1980s, calls him “the greatest football lover there was.”\n\nIn a message to The Associated Press on Thursday, Platini added Maradona was “a child king. He was a bit excessive in all things, but the most important thing is that he was excessive on the pitch and it was beautiful.”\n\nMaradona died on Wednesday from a heart attack, aged 60.\n\nPlatini, who is 65, and Maradona played against each other at club level at a time when the Italian league was arguably the best in the world. Maradona arrived at Napoli in 1984, two years after Platini signed with Juventus.\n\nMaradona, who scored 115 goals for Napoli in 259 matches, helped the club challenge Juventus’ supremacy, winning its first two Serie A titles and the 1989 UEFA Cup.', metadata={'query': 'Comparison between Platini and Maradona', 'title': 'Platini to AP: Maradona was the ‘greatest football lover’', 'url': 'https://apnews.com/platini-to-ap-maradona-was-the-greatest-football-lover-be8c3ad6a57cffbd405a3475ac6951af'}), Document(page_content="Diego Maradona vs. Michel Platini: Titles Won Comparison\n\nWho won the most tournaments, Michel Platini or Diego Maradona? Here you have the details, head to head, of the titles won by each of these players with their national teams and the international and domestic titles won with their clubs.\n\nWith his national team, Diego Maradona won 1 title more than Michel Platini, 3 against 2. Also in FIFA World Cups won Diego Maradona has the advantage over Michel Platini 1 to 0.\n\nAt club international level, Michel Platini won 3 titles more than Diego Maradona, 4 against 1.\n\nWhen you consider domestic leagues, tournaments and cups, Diego Maradona surpasses Michel Platini in titles won 8 to 5.\n\nOf course, these stats only tell a part of the story since they don't take into consideration the value of each tournament or how relevant the players were for their teams.\n\nTournaments with the National Team:\n\nDiego Maradona\n\nMichel Platini 3\n\nnational team\n\ntitles 2\n\nnational team\n\ntitles", metadata={'query': 'Comparison between Platini and Maradona', 'title': 'Diego Maradona vs. Michel Platini: Titles Won Comparison', 'url': 'https://www.aworldofsoccer.com/players_comparison/diego_maradona_vs_michel_platini.htm'}), Document(page_content="Diego Maradona vs. Michel Platini: Titles Won Comparison\n\nWho won the most tournaments, Michel Platini or Diego Maradona? Here you have the details, head to head, of the titles won by each of these players with their national teams and the international and domestic titles won with their clubs.\n\nWith his national team, Diego Maradona won 1 title more than Michel Platini, 3 against 2. Also in FIFA World Cups won Diego Maradona has the advantage over Michel Platini 1 to 0.\n\nAt club international level, Michel Platini won 3 titles more than Diego Maradona, 4 against 1.\n\nWhen you consider domestic leagues, tournaments and cups, Diego Maradona surpasses Michel Platini in titles won 8 to 5.\n\nOf course, these stats only tell a part of the story since they don't take into consideration the value of each tournament or how relevant the players were for their teams.\n\nTournaments with the National Team:\n\nDiego Maradona\n\nMichel Platini 3\n\nnational team\n\ntitles 2\n\nnational team\n\ntitles", metadata={'query': 'Comparison between Platini and Maradona', 'title': 'Diego Maradona vs. Michel Platini: Titles Won Comparison', 'url': 'https://www.aworldofsoccer.com/players_comparison/diego_maradona_vs_michel_platini.htm'})]), H2Title(title='Platini Quotes', content=ParagraphWriter(paragraph=' [Michel Platini](https://en.wikipedia.org/wiki/Michel_Platini) , a football legend, has left behind a legacy of inspiring and memorable quotes throughout his career. Some of his notable quotes include:\n- "Football is a game of deceit. You have to be like a fox in the box."\n- "The ball is like a woman, she loves to be caressed."\n- "I prefer to win 5-4 than 1-0."\n- "Football is a team game, but you can\'t win without individual brilliance."\nPlatini\'s words reflect his passion for the sport and his unique perspective on the game.'), h3_titles=None, web_search=None, sources=[Document(page_content='Platini is considered to be one of the greatest players of all time, and is regarded as one of the finest passers in football history, as well as one of the best ever penalty kick and free kick specialists to have played the game.[8][12][23][24][25][26] A quick, versatile, elegant, and intelligent offensive midfield playmaker, with a unique ability to read the game and bend the ball from set pieces, he was renowned in particular for his ball control, technical ability, dribbling skills, creativity, range of passing, and vision, despite his lack of notable physical or athletic attributes.[8][23][27][28][29][30][31]', metadata={'query': 'Platini UEFA Presidency', 'title': 'Michel Platini', 'url': 'https://en.wikipedia.org/wiki/Michel_Platini'}), Document(page_content='Platini is considered to be one of the greatest players of all time, and is regarded as one of the finest passers in football history, as well as one of the best ever penalty kick and free kick specialists to have played the game.[8][12][23][24][25][26] A quick, versatile, elegant, and intelligent offensive midfield playmaker, with a unique ability to read the game and bend the ball from set pieces, he was renowned in particular for his ball control, technical ability, dribbling skills, creativity, range of passing, and vision, despite his lack of notable physical or athletic attributes.[8][23][27][28][29][30][31]', metadata={'query': 'Michel Platini career highlights', 'title': 'Michel Platini', 'url': 'https://en.wikipedia.org/wiki/Michel_Platini'}), Document(page_content='Platini is considered to be one of the greatest players of all time, and is regarded as one of the finest passers in football history, as well as one of the best ever penalty kick and free kick specialists to have played the game.[8][12][23][24][25][26] A quick, versatile, elegant, and intelligent offensive midfield playmaker, with a unique ability to read the game and bend the ball from set pieces, he was renowned in particular for his ball control, technical ability, dribbling skills, creativity, range of passing, and vision, despite his lack of notable physical or athletic attributes.[8][23][27][28][29][30][31]', metadata={'query': 'Platini career highlights', 'title': 'Michel Platini', 'url': 'https://en.wikipedia.org/wiki/Michel_Platini'}), Document(page_content="French association football player, manager and administrator\n\nMichel François Platini (French pronunciation: [miʃɛl fʁɑ̃swa platini]; born 21 June 1955) is a French football administrator and former player and manager. Regarded as one of the greatest footballers of all time, Platini won the Ballon d'Or three times in a row, in 1983, 1984 and 1985,[5] and came seventh in the FIFA Player of the Century vote.[6] In recognition of his achievements, he was named a Chevalier of the Légion d'honneur in 1985 and became an Officier in 1998. As the president of UEFA in 2015 he was banned from involvement in football under FIFA's organisation, over ethics violations. The ban lasted until 2023.[7]", metadata={'query': 'Platini UEFA Presidency', 'title': 'Michel Platini', 'url': 'https://en.wikipedia.org/wiki/Michel_Platini'})]), H2Title(title='Platini Career Highlights', content=ParagraphWriter(paragraph=" [Michel Platini's career highlights](https://www.footballhistory.org/player/michel-platini.html)  are a testament to his exceptional talent and success in the world of football. Some key points include:\n- Starting at Nancy, he scored 98 goals and won  [the 1978 French Cup](https://medium.com/@mondebaron/michel-platini-a-french-football-legend-stands-among-the-greatest-players-in-history-8b2f89a4abe6) .\n- At Saint-Étienne, he secured the 1981 French league title and reached the 1976 European Cup final.\n- Joining Juventus in 1982, he achieved unprecedented success with three Ballon d’Ors, two Serie A titles, and a European Cup victory.\n- Captaining France in Euro 1984, he scored a record nine goals and led his team to victory, earning the Best Player and Top Scorer awards."), h3_titles=[H3Title(title='Bonini Luiz-Koopmeiners Juve Chiesa Platini presidente', content=ParagraphWriter(paragraph="The Juventus team is making strategic moves in the transfer market, focusing on strengthening the midfield with top talents like Douglas Luiz and Koopmeiners. The potential acquisition of these players could elevate  [the team's performance](https://www.tuttojuve.com/podcast-tj/bonini-a-rbn-luiz-e-koopmeiners-tanta-roba-motta-scelta-giustissima-platini-presidente-della-juve-692047)  significantly, as highlighted by former player Bonini. Amidst uncertainties surrounding Rabiot and Chiesa's future, the club's attention shifts to potential leadership changes, with the intriguing prospect of  [Michel Platini](https://www.bianconeranews.it/primo-piano/platini-john-elkann-ci-tiene-e-la-juve-tornera-vincente-io-presidente-74178)  stepping into a presidential role. Platini's vision for the club and his outspoken views on the football landscape add an exciting dimension to Juventus' future prospects."), web_search=None, sources=[Document(page_content="Podcast Bonini a RBN: “Luiz-Koopmeiners top. Juve tieni Chiesa. Platini presidente”\n\nvedi letture\n\nLa Juve prosegue sulla strada del rafforzamento, soprattutto a centrocampo. Douglas Luiz sembra in dirittura d'arrivo dopo aver sostituito McKennie con Berrenechea come seconda contropartita tecnica insieme a Iling Junior, oltre a versare cash nelle case nelle casse dell'Aston Villa, per una cifra intorno ai 25 milioni. Giuntoli ora darà l'assalto a Koopmeiners, priorità per il nuovo tecnico Thiago Motta. L'Atalanta lo valuta 60 milioni, Huijsen potrebbe rientrare nell'operazione. Resta il doppio nodo legato a Rabiot e Chiesa. Il contratto del francese scade il 30 giugno, quello dell'attaccante nel 2025: le possibilità di addio sono concrete per entrambi. Intanto Platini tuona contro Uefa e Fifa, il sogno di molti tifosi bianconeri è di vederlo nella dirigenza. In ESCLUSIVA a Fuori di Juve Massimo Bonini, ex centrocampista della Vecchia Signora.", metadata={'query': 'Bonini Luiz-Koopmeiners Juve Chiesa Platini presidente', 'title': 'Bonini a RBN: “Luiz-Koopmeiners top. Juve tieni Chiesa. Platini presidente”', 'url': 'https://www.bianconeranews.it/podcast/bonini-a-rbn-luiz-koopmeiners-top-juve-tieni-chiesa-platini-presidente-75484'}), Document(page_content="Platini: “John Elkann ci tiene e la Juve tornerà vincente. Io Presidente...”\n\nvedi letture\n\nPlatini parla della Juve e, convinto che tornerà a vincere con John Elkann. L'ex fuoriclasse si esprime inoltre sull'ipotesi di diventare presidente\n\nMichel Platini torna a parlare di Juventus, John Elkann, Superlega, Infantino e Ceferin. L'ex fuoriclasse bianconero si è concesso ai microfoni di Tuttosport con una lunga intervista.\n\nIn primis l'ex fuoriclasse bianconero risponde ai tifosi convinti che John Elkann abbia poca passione: “Sbagliano. Magari non è a livello di suo nonno, l'Avvocato, di Umberto, ma ci tiene. Si capisce che vive con grande serietà l’impegno centenario della sua famiglia nella Juventus. E poi ha grandissime responsabilità nella gestione delle aziende. Forse non riesce a comunicarlo bene, ma alla Juventus ci tiene, è vicino alla società”.", metadata={'query': 'Bonini Luiz-Koopmeiners Juve Chiesa Platini presidente', 'title': 'Platini: “John Elkann ci tiene e la Juve tornerà vincente. Io Presidente...”', 'url': 'https://www.bianconeranews.it/primo-piano/platini-john-elkann-ci-tiene-e-la-juve-tornera-vincente-io-presidente-74178'}), Document(page_content="La Juve continua a lavorare per mettere a segno i primi colpi di mercato. Il centrocampo è il reparto su cui è concentrato Giuntoli. Douglas Luiz si avvicina sempre di più: all'Aston Villa andranno Iling Junior, Barrenechea e 25 milioni. Poi sarà la volta di Koopmeiners, tra le prime richieste di Thiago Motta: l'Atalanta chiede 60 milioni, la chiave per arrivare all'olandese potrebbe essere Huijsen. La permanenza di Rabiot e Chiesa resta in forte dubbio. Il francese è in scadenza il 30 giugno, su di lui pare ci sia il Real Madrid. Contratto fino al 2025 invece per Chiesa, accordo per il rinnovo lontano, cessione probabile. Intanto arriva la profezia di Platini, secondo il quale tra qualche anno Uefa e Fifa potrebbero scomparire. In ESCLUSIVA a Fuori di Juve Massimo Bonini, ex centrocampista della Vecchia Signora ai tempi de Le Roi.", metadata={'query': 'Bonini Luiz-Koopmeiners Juve Chiesa Platini presidente', 'title': 'Bonini a RBN: ‚ÄúLuiz e Koopmeiners tanta roba. Motta scelta giustissima. Platini? Presidente della Juve ‚Äù', 'url': 'https://www.tuttojuve.com/podcast-tj/bonini-a-rbn-luiz-e-koopmeiners-tanta-roba-motta-scelta-giustissima-platini-presidente-della-juve-692047'}), Document(page_content="In caso di doppio colpo Douglas Luiz-Koopmeiners, secondo Bonini la Juve farebbe un salto di qualità importante: “La mediana detta i tempi e comanda il gioco di una squadra. Si tratta di due giocatori fantastici, fermo restando che dipende anche dall'organizzazione di gioco. Ritengo la Juve abbia diversi giovani interessanti, ma se può investire su elementi già pronti, va bene. Molto dipenderà da Thiago Motta. A Bologna la sua squadra ha mostrato qualità, idee ben precise. Douglas Luiz e Koopmeiners comunque sono tanta roba, sono bravi con e senza palla, abili ad attaccare la profondità, sono completi, due top player. La scelta di Motta è giustissima. Ha dimostrato di essere un ottimo allenatore. A Bologna malgrado le sostituzioni, la musica non cambiava. Questa è la bravura di un tecnico, evidentemente riesce ad entrare nella testa dei suoi ragazzi, ognuno sa cosa fare e così è tutto più semplice. Rabiot? Fa parte della Nazionale francese, nelle ultime stagioni ha dimostrato il suo", metadata={'query': 'Bonini Luiz-Koopmeiners Juve Chiesa Platini presidente', 'title': 'Bonini a RBN: ‚ÄúLuiz e Koopmeiners tanta roba. Motta scelta giustissima. Platini? Presidente della Juve ‚Äù', 'url': 'https://www.tuttojuve.com/podcast-tj/bonini-a-rbn-luiz-e-koopmeiners-tanta-roba-motta-scelta-giustissima-platini-presidente-della-juve-692047'})]), H3Title(title='Michel Platini career highlights', content=ParagraphWriter(paragraph=" [Michel Platini's career highlights](https://medium.com/@mondebaron/michel-platini-a-french-football-legend-stands-among-the-greatest-players-in-history-8b2f89a4abe6)  shine brightly with remarkable achievements. Starting at Nancy, he scored 98 goals and clinched  [the 1978 French Cup](https://www.britannica.com/biography/Michel-Platini) . His move to Saint-Étienne brought the 1981 French league title and a 1976 European Cup final appearance. Joining Juventus in 1982, Platini soared with three Ballon d’Ors, two Serie A titles, and a European Cup triumph. Leading France in Euro 1984, he set a record with nine goals, securing victory and earning Best Player and Top Scorer accolades."), web_search=None, sources=[Document(page_content='MICHEAL PLATINI Football Career Achievements and Contributions Mondebaron · Follow 3 min read · Jan 18, 2024 -- Listen Share\n\nMichel Platini, a French football legend, stands among the greatest players in history. His remarkable career includes three consecutive Ballon d’Or wins and leading France to victory in the 1984 European Championship. Beyond playing, he made significant contributions as a coach and football administrator. However, controversies marred his later years, particularly during his involvement with FIFA.\n\nEarly Life\n\nBorn on June 21, 1955, in Jœuf, Lorraine, France, Platini inherited a passion for football from his father, Aldo, a former footballer and AS Nancy director. Raised in a modest family with Italian roots, Platini joined AS Jœuf at six, showcasing talent and creativity. Inspired by Pelé, he dreamed of representing the French national team.\n\nPlaying Career\n\nA. Clubs and Transfer Market', metadata={'query': 'Platini career highlights', 'title': 'MICHEAL PLATINI Football Career Achievements and Contributions', 'url': 'https://medium.com/@mondebaron/michel-platini-a-french-football-legend-stands-among-the-greatest-players-in-history-8b2f89a4abe6'}), Document(page_content='MICHEAL PLATINI Football Career Achievements and Contributions Mondebaron · Follow 3 min read · Jan 18, 2024 -- Listen Share\n\nMichel Platini, a French football legend, stands among the greatest players in history. His remarkable career includes three consecutive Ballon d’Or wins and leading France to victory in the 1984 European Championship. Beyond playing, he made significant contributions as a coach and football administrator. However, controversies marred his later years, particularly during his involvement with FIFA.\n\nEarly Life\n\nBorn on June 21, 1955, in Jœuf, Lorraine, France, Platini inherited a passion for football from his father, Aldo, a former footballer and AS Nancy director. Raised in a modest family with Italian roots, Platini joined AS Jœuf at six, showcasing talent and creativity. Inspired by Pelé, he dreamed of representing the French national team.\n\nPlaying Career\n\nA. Clubs and Transfer Market', metadata={'query': 'Michel Platini career highlights', 'title': 'MICHEAL PLATINI Football Career Achievements and Contributions', 'url': 'https://medium.com/@mondebaron/michel-platini-a-french-football-legend-stands-among-the-greatest-players-in-history-8b2f89a4abe6'}), Document(page_content='Michel Platini (born June 21, 1955, Joeuf, France) is a French professional football (soccer) player and administrator who was named the European Footballer of the Year three times (1983–85) and served as president of the Union of European Football Associations (UEFA; 2007–16).\n\nPlatini made his French first-division debut with AS Nancy in 1973. A prolific attacking midfielder, he led the club to a French Cup championship in 1978 and then joined Saint-Étienne the following year. He helped his new team win a league title in 1981, and in 1982 he transferred to Juventus of Italy. Platini emerged as an international superstar with Juventus, leading the team to a European Cup final in 1983 (a loss) and a European Cup championship in 1985. Juventus also captured two league championships (1984, 1986), a UEFA Super Cup (1984), and a European Cup Winners’ Cup (1984) during Platini’s tenure with the club, which lasted until 1987.\n\nBritannica Quiz I Am the Greatest (Athlete)', metadata={'query': 'Platini career highlights', 'title': 'Michel Platini | Biography, FIFA, & Facts', 'url': 'https://www.britannica.com/biography/Michel-Platini'}), Document(page_content='Michel Platini (born June 21, 1955, Joeuf, France) is a French professional football (soccer) player and administrator who was named the European Footballer of the Year three times (1983–85) and served as president of the Union of European Football Associations (UEFA; 2007–16).\n\nPlatini made his French first-division debut with AS Nancy in 1973. A prolific attacking midfielder, he led the club to a French Cup championship in 1978 and then joined Saint-Étienne the following year. He helped his new team win a league title in 1981, and in 1982 he transferred to Juventus of Italy. Platini emerged as an international superstar with Juventus, leading the team to a European Cup final in 1983 (a loss) and a European Cup championship in 1985. Juventus also captured two league championships (1984, 1986), a UEFA Super Cup (1984), and a European Cup Winners’ Cup (1984) during Platini’s tenure with the club, which lasted until 1987.\n\nBritannica Quiz I Am the Greatest (Athlete)', metadata={'query': 'Michel Platini career highlights', 'title': 'Michel Platini | Biography, FIFA, & Facts', 'url': 'https://www.britannica.com/biography/Michel-Platini'})])], web_search=None, sources=[Document(page_content='Throughout his remarkable and prolific professional soccer player career spanning 15 years, Platini won three consecutive European Footballer of the Year awards (1983-1985) as well as two prestigious World Footballer of the Year awards in 1984 and 1985, among numerous other distinctions.\n\nManager of Les Bleus and UEFA President', metadata={'query': 'Michel Platini career highlights', 'title': 'Michel Platini biography and career accomplishments', 'url': 'https://www.footballhistory.org/player/michel-platini.html'}), Document(page_content='Throughout his remarkable and prolific professional soccer player career spanning 15 years, Platini won three consecutive European Footballer of the Year awards (1983-1985) as well as two prestigious World Footballer of the Year awards in 1984 and 1985, among numerous other distinctions.\n\nManager of Les Bleus and UEFA President', metadata={'query': 'Platini career highlights', 'title': 'Michel Platini biography and career accomplishments', 'url': 'https://www.footballhistory.org/player/michel-platini.html'}), Document(page_content='Playing Career\n\nA. Clubs and Transfer Market\n\n1. Nancy (1972–1979): Platini’s professional journey began at Nancy in 1972, yielding 98 goals in 181 appearances and a 1978 French Cup victory.\n\n2. Saint-Étienne (1979–1982): Moving to Saint-Étienne in 1979, Platini won the 1981 French league title and reached the 1976 European Cup final, scoring 58 goals in 104 games.\n\n3. Juventus (1982–1987): Joining Juventus in 1982 for a record fee, he achieved unparalleled success — three Ballon d’Ors, two Serie A titles, a European Cup, and other honors, scoring 68 goals in 147 matches.\n\nB. Euro 1984 Triumph\n\nCaptaining France in Euro 1984, Platini scored a record nine goals, leading to a historic victory. Notable moments include a semi-final goal against Portugal and the opening goal in the final against Spain. He earned the tournament’s Best Player and Top Scorer awards, becoming the first to score two hat-tricks in a single European Championship.\n\nC. Trophies and Achievements', metadata={'query': 'Platini career highlights', 'title': 'MICHEAL PLATINI Football Career Achievements and Contributions', 'url': 'https://medium.com/@mondebaron/michel-platini-a-french-football-legend-stands-among-the-greatest-players-in-history-8b2f89a4abe6'}), Document(page_content='Playing Career\n\nA. Clubs and Transfer Market\n\n1. Nancy (1972–1979): Platini’s professional journey began at Nancy in 1972, yielding 98 goals in 181 appearances and a 1978 French Cup victory.\n\n2. Saint-Étienne (1979–1982): Moving to Saint-Étienne in 1979, Platini won the 1981 French league title and reached the 1976 European Cup final, scoring 58 goals in 104 games.\n\n3. Juventus (1982–1987): Joining Juventus in 1982 for a record fee, he achieved unparalleled success — three Ballon d’Ors, two Serie A titles, a European Cup, and other honors, scoring 68 goals in 147 matches.\n\nB. Euro 1984 Triumph\n\nCaptaining France in Euro 1984, Platini scored a record nine goals, leading to a historic victory. Notable moments include a semi-final goal against Portugal and the opening goal in the final against Spain. He earned the tournament’s Best Player and Top Scorer awards, becoming the first to score two hat-tricks in a single European Championship.\n\nC. Trophies and Achievements', metadata={'query': 'Michel Platini career highlights', 'title': 'MICHEAL PLATINI Football Career Achievements and Contributions', 'url': 'https://medium.com/@mondebaron/michel-platini-a-french-football-legend-stands-among-the-greatest-players-in-history-8b2f89a4abe6'})]), H2Title(title='Platini Controversy', content=ParagraphWriter(paragraph=" [Michel Platini's career](https://www.si.com/soccer/2019/06/18/michel-platini-former-uefa-presidents-controversial-career-football-politics)  was marred by controversy when he accepted a £1.6m payment from Blatter, leading to an eight-year ban from football activities. Despite challenging the accusations, he faced scrutiny and a shortened four-year ban. Additionally, Platini was found guilty of  [ethics violations](https://en.wikipedia.org/wiki/Michel_Platini)  alongside Blatter, further tarnishing his reputation. His involvement in the 2015 Greek football scandal and mention in the Panama Papers added to the cloud of controversy surrounding his name.  [Platini's claim](https://bleacherreport.com/articles/262769-legends-revisited-the-life-and-times-of-michel-platini)  of a fixed 1998 FIFA World Cup draw also raised eyebrows, highlighting a turbulent chapter in his career."), h3_titles=None, web_search=None, sources=[Document(page_content="However, soon after his decision to run for presidency, Platini was himself under the spotlight after it was discovered that he had accepted a mysterious £1.6m payment from Blatter, supposedly for his work after the 1998 World Cup.\n\nMICHAEL BUHOLZER/GettyImages\n\nFIFA led their own investigation into the matter, and both Blatter and Platini were ultimately found guilty of breaching ethics regulations. The pair were handed eight-year global bans from any football activities, but Platini's ban was ultimately shortened to four years.\n\nAway from the headlines, he has continued to challenge the accusations against him, although he now finds himself back under pressure once again.", metadata={'query': 'Platini UEFA Presidency', 'title': "Michel Platini: The Former UEFA President's Controversial Career in Football Politics", 'url': 'https://www.si.com/soccer/2019/06/18/michel-platini-former-uefa-presidents-controversial-career-football-politics'}), Document(page_content="Platini then forced his way into everybody’s long-term consciousness when he won the UEFA presidency, probably on the back of his publicity in Jose's advert. Since then he has been almost constantly annoying everybody by employing a method of arguing some good policies very poorly.\n\nIt's not surprising then that most of us don't know and probably don't care about Platini's playing days, but as I hope to demonstrate, that's a real shame.\n\nFor as far back as I can remember, France has always been a footballing power. One of my earliest memories is their absolute domination of Brazil in a World Cup final in 1998. The list of France legends runs deep with big names which hasn't yet been exhausted: Deschamps, Cantona, Blanc, Desailly, Petit, Zidane, Henry, Pires, Vieira, Ribéry, Benzema...", metadata={'query': 'Platini career highlights', 'title': "Legends Revisited: Michel Platini - France's Greatest Player?", 'url': 'https://bleacherreport.com/articles/262769-legends-revisited-the-life-and-times-of-michel-platini'}), Document(page_content="Platini then forced his way into everybody’s long-term consciousness when he won the UEFA presidency, probably on the back of his publicity in Jose's advert. Since then he has been almost constantly annoying everybody by employing a method of arguing some good policies very poorly.\n\nIt's not surprising then that most of us don't know and probably don't care about Platini's playing days, but as I hope to demonstrate, that's a real shame.\n\nFor as far back as I can remember, France has always been a footballing power. One of my earliest memories is their absolute domination of Brazil in a World Cup final in 1998. The list of France legends runs deep with big names which hasn't yet been exhausted: Deschamps, Cantona, Blanc, Desailly, Petit, Zidane, Henry, Pires, Vieira, Ribéry, Benzema...", metadata={'query': 'Michel Platini career highlights', 'title': "Legends Revisited: Michel Platini - France's Greatest Player?", 'url': 'https://bleacherreport.com/articles/262769-legends-revisited-the-life-and-times-of-michel-platini'}), Document(page_content='On 8 October 2015, Platini was provisionally suspended until 6 January 2016 from any football-related activity.[51] On 21 December, Platini and Blatter were both found guilty of ethics violations and barred from the sport until 2023.[7] The committee said Platini "did not show commitment to an ethical attitude" and lacked respect for laws and regulations of the organization.[52][53] Platini boycotted the hearing, and said he planned to appeal the decision, declaring himself "at peace with my conscience."[54][55][56][57] He appealed to Swiss courts, and to the European Court of Human Rights, but the courts rejected his appeals.[58]\n\nPlatini was also involved in the Greek public polemica regarding the 2015 Greek football scandal.[59][60][61]\n\nIn April 2016, Platini has been named in the Panama Papers.[62]\n\nOn 18 May 2018, Platini said that the 1998 FIFA World Cup draw was fixed to ensure France and Brazil could not face each other until the final if both teams won their groups.[63]', metadata={'query': 'Platini career highlights', 'title': 'Michel Platini', 'url': 'https://en.wikipedia.org/wiki/Michel_Platini'})]), H2Title(title='Platini UEFA Presidency', content=ParagraphWriter(paragraph=" [Michel Platini's  [UEFA](https://www.britannica.com/biography/Michel-Platini)  presidency](https://en.wikipedia.org/wiki/Michel_Platini)  marked a significant milestone in his football career. Elected in 2007 after defeating Lennart Johansson, Platini focused on promoting solidarity and universality within UEFA. His leadership brought about positive changes, emphasizing the importance of unity and inclusivity in European football governance. Platini's tenure saw advancements in technical development and a commitment to fostering a strong football community. Through his presidency, Platini left a lasting impact on UEFA, shaping the organization's values and direction for the future."), h3_titles=None, web_search=None, sources=[Document(page_content='Platini was, along with Fernand Sastre, head of the organizing committee for the 1998 FIFA World Cup, held in France. He served on the UEFA Technical Development Committee from 1988 to 1990. He has been a member of the UEFA Executive Committee and European member of the FIFA Executive Committee since 2002. He continued to climb the ranks of UEFA and FIFA football administration and in 2006, became a chairman of the FIFA Technical and Development Committee, while also being vice-president of the French Football Federation.\n\nUEFA presidency\n\nPlatini confirmed that he would run for the UEFA presidency in July 2006. In the election in Düsseldorf on 26 January 2007, he defeated Lennart Johansson, who held the post for the previous 16 years, by 27 votes to 23.[40] Platini based his speech on virtues of solidarity and universality.\n\nMichel Platini visiting the construction site of the Stadion Miejski, 2009', metadata={'query': 'Platini UEFA Presidency', 'title': 'Michel Platini', 'url': 'https://en.wikipedia.org/wiki/Michel_Platini'}), Document(page_content='Platini was, along with Fernand Sastre, head of the organizing committee for the 1998 FIFA World Cup, held in France. He served on the UEFA Technical Development Committee from 1988 to 1990. He has been a member of the UEFA Executive Committee and European member of the FIFA Executive Committee since 2002. He continued to climb the ranks of UEFA and FIFA football administration and in 2006, became a chairman of the FIFA Technical and Development Committee, while also being vice-president of the French Football Federation.\n\nUEFA presidency\n\nPlatini confirmed that he would run for the UEFA presidency in July 2006. In the election in Düsseldorf on 26 January 2007, he defeated Lennart Johansson, who held the post for the previous 16 years, by 27 votes to 23.[40] Platini based his speech on virtues of solidarity and universality.\n\nMichel Platini visiting the construction site of the Stadion Miejski, 2009', metadata={'query': 'Michel Platini career highlights', 'title': 'Michel Platini', 'url': 'https://en.wikipedia.org/wiki/Michel_Platini'}), Document(page_content='Platini was, along with Fernand Sastre, head of the organizing committee for the 1998 FIFA World Cup, held in France. He served on the UEFA Technical Development Committee from 1988 to 1990. He has been a member of the UEFA Executive Committee and European member of the FIFA Executive Committee since 2002. He continued to climb the ranks of UEFA and FIFA football administration and in 2006, became a chairman of the FIFA Technical and Development Committee, while also being vice-president of the French Football Federation.\n\nUEFA presidency\n\nPlatini confirmed that he would run for the UEFA presidency in July 2006. In the election in Düsseldorf on 26 January 2007, he defeated Lennart Johansson, who held the post for the previous 16 years, by 27 votes to 23.[40] Platini based his speech on virtues of solidarity and universality.\n\nMichel Platini visiting the construction site of the Stadion Miejski, 2009', metadata={'query': 'Platini career highlights', 'title': 'Michel Platini', 'url': 'https://en.wikipedia.org/wiki/Michel_Platini'}), Document(page_content='Shortly after leaving the pitch for the final time, Platini turned to coaching, becoming the French national coach in 1988. He resigned that post in 1992 and then served as copresident of the organizing committee for the 1998 World Cup, which was hosted by France. Platini held a number of administrative positions in both the Fédération Internationale de Football Association (FIFA) and UEFA before he was elected UEFA president in 2007.', metadata={'query': 'Platini career highlights', 'title': 'Michel Platini | Biography, FIFA, & Facts', 'url': 'https://www.britannica.com/biography/Michel-Platini'})])])
outline_wlink2

In [ ]:
print(outline_wlink2.print_formatted_outline())

### Auto bold

In [ ]:
def contains_noun(text):
    # Process the text with spaCy
    doc = nlp(text)
    # Check for nouns
    for token in doc:
        if token.pos_ in ["NOUN", "PROPN"]:  # NOUN for common nouns, PROPN for proper nouns
            return True
    return False


In [ ]:
para = filled_outline.h2_titles[0].content.paragraph

In [ ]:
import yake

def create_bolds_in_paragraph(paragraph: str, percent_bold_words_per_para:int=4, language="en") -> str:
    n_keywords = int((len(paragraph.split()) * percent_bold_words_per_para) / 100)
    max_ngram_size = 3
    deduplication_threshold = 0.1
    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=n_keywords, features=None)

    para_keywords = custom_kw_extractor.extract_keywords(paragraph)
    for kw in para_keywords:
        word = kw[0]
        if contains_noun(word):
            bold_word = f"**{word}**"
            paragraph = paragraph.replace(word.strip(), bold_word, 1)

    return paragraph

In [ ]:
def add_bolds_to_outline(outline: OutlineGenerator, percent_bold_words_per_para=10) -> OutlineGenerator:
    outline_copy = outline  # copy outline so i keep the empty version
    for h2 in outline_copy.h2_titles:
        h2.content.paragraph = create_bolds_in_paragraph(h2.content.paragraph, percent_bold_words_per_para)

        if h2.h3_titles:
            for h3 in h2.h3_titles:
                h3.content.paragraph = create_bolds_in_paragraph(h3.content.paragraph, percent_bold_words_per_para)
    return outline_copy

In [ ]:
outline_w_bolds = add_bolds_to_outline(outline_wlink2, percent_bold_words_per_para=50)
print(outline_w_bolds.print_formatted_outline())

### Write intro and conclusion

In [ ]:
class IntroductionWriter(BaseModel):
    """Component responsible for writing the article introduction."""

    introduction:str = Field(description="The written content of the introduction based on a provided outline and primary keyword")

    @classmethod
    def get_system_prompt(cls) -> str:
        return """You are IntroductionWriter, a specialized model designed to craft compelling and informative introductions based on a provided outline and primary keyword. Your primary objective is to create introductions that engage the reader, provide a clear overview, and set the context for the topic being discussed. Below are your guidelines and responsibilities:

    Engagement:
        Begin with a captivating opening sentence to grab the reader's attention.
        Use engaging language to maintain reader interest throughout the introduction.

    Clarity and Precision:
        Clearly state the main topic and its significance.
        Provide a concise and accurate summary of what will be covered, ensuring the reader understands the scope and importance of the topic.

    Contextualization:
        Offer necessary background information to set the context.
        Highlight key points, relevant data, or significant issues related to the topic.

    Structure:
        Follow a logical structure, starting from a broad introduction and narrowing down to the specifics.
        Ensure smooth transitions between sentences and ideas to maintain coherence.
        
    Search Engine Optimization:
        Very important: clearly mention the primary keyword of the article"""
    
    @classmethod
    def get_user_prompt(cls):
        return """Overall article outline: {outline}\n\nPrimary keyword:\n{p_k}\n\n{style}"""

In [ ]:
# intro = use_component(IntroductionWriter, {"outline": filled_outline.print_formatted_outline(), "p_k": pk.primary_keyword.keyword, "style": style_instructions})
# intro

In [ ]:
intro = IntroductionWriter(introduction="Michel Platini, a renowned football icon, has left an indelible mark on the sport through his exceptional skills and impactful contributions. Contrasted with the likes of Maradona, Platini's career highlights and controversies have shaped his legacy in football history. From his memorable quotes reflecting humility to his controversial moments and UEFA presidency, Platini's journey is a mix of triumphs and challenges. Let's explore the intriguing tale of Michel Platini, a figure who transcended the boundaries of the beautiful game.")
intro

In [ ]:
class ConclusionWriter(BaseModel):
    """Component responsible for writing the article conclusion and keypoints."""

    conclusion:str = Field(description="The written content of the conclusion based on a provided article outline")
    keypoints:List[str] = Field(description="A list of keypoint from the article")

    @classmethod
    def get_system_prompt(cls) -> str:
        return """You are ConclusionWriter, a specialized model designed to craft compelling and conclusive endings and keypoints for the provided article outline. Your primary objective is to create conclusions that effectively summarize the content, reinforce the key points, and leave a lasting impression on the reader. Below are your guidelines and responsibilities:

    Summary:
        Concisely summarize the main points discussed in the content.
        Ensure that all significant aspects of the topic are briefly recapped.

    Reinforcement:
        Reinforce the importance of the topic and the key arguments presented.
        Highlight any important takeaways or lessons learned.

    Closure:
        Provide a sense of closure to the discussion, ensuring the reader feels the topic has been comprehensively covered.
        End with a strong closing sentence that leaves a lasting impression.

    Call to Action (if applicable):
        If relevant, include a call to action, encouraging the reader to take specific steps or further explore the topic.
        Ensure the call to action is clear, relevant, and motivating."""
    
    @classmethod
    def get_user_prompt(cls):
        return """Overall article outline: {outline}\n\n{style}"""

In [ ]:
# conclusion = use_component(ConclusionWriter, {"outline": filled_outline.print_formatted_outline(), "style": style_instructions})
# conclusion

In [ ]:
conclusion = ConclusionWriter(conclusion="The comparison between Platini and Maradona highlights the unique strengths and achievements of both football legends. Platini's success at Juventus and Euro 84 victory contrast with Maradona's impact at Napoli and World Cup triumphs. Despite controversies, Platini's career highlights and UEFA presidency underscore his lasting influence on football governance. His impactful quotes and leadership in advocating for positive changes in the sport reflect his profound impact beyond the field. The ongoing debate between these icons continues to captivate fans and showcase their remarkable contributions to the beautiful game.", keypoints=['Platini and Maradona have distinct strengths and achievements in their football careers.', "Platini's career highlights include Ballon d’Or wins, Euro 84 victory, and UEFA presidency with positive changes in football governance.", "Controversies marred Platini's career, including ethics violations and involvement in scandals, impacting his once illustrious reputation.", "Platini's impactful quotes and leadership in advocating for positive changes in football showcase his profound impact beyond playing.", 'The comparison between Platini and Maradona continues to spark debate and admiration for their remarkable contributions to the sport.'])
conclusion

## Assemble Article

In [ ]:
def stitch_h2_paragraphs(outline: OutlineGenerator) -> str:
    article_body = ""

    for h2 in outline.h2_titles:
        # Add H2 title
        article_body += f"## {h2.title}\n"
        # Add H2 content
        article_body += f"{h2.content.paragraph}\n\n"
        # If there are H3 titles, add them
        if h2.h3_titles:
            for h3 in h2.h3_titles:
                # Add H3 title
                article_body += f"### {h3.title}\n"
                # Add H3 content
                article_body += f"{h3.content.paragraph}\n\n"

    return article_body.strip()

In [ ]:
article = stitch_h2_paragraphs(outline_w_bolds)
article

## Generate Title

In [ ]:
class ArticleTitleGenerator(BaseModel):
    """Component responsible for creating compelling, catchy, and SEO-efficient titles for articles based on the article outline and primary, secondary and longtail keywords provided."""

    title:str = Field(description="The generated article title")

    @classmethod
    def get_system_prompt(cls) -> str:
        return """You are ArticleTitleGenerator, a specialized component designed to create compelling, catchy, and SEO-efficient titles for articles based on the article outline and primary, secondary and longtail keywords provided. Your primary objective is to generate titles that attract readers' attention, accurately represent the article's content, and optimize for search engine visibility. Below are your guidelines and responsibilities:

    Relevance and Accuracy:
        Ensure the title accurately reflects the main content and message of the article.
        Avoid clickbait; the title should set appropriate expectations for the reader.

    SEO Efficiency:
        Incorporate relevant keywords naturally to enhance search engine optimization.

    Catchiness and Engagement:
        Craft titles that are engaging and intriguing to encourage clicks and shares.
        Use powerful and emotive language to draw in the reader.

    Length and Format:
        Keep the title concise, ideally between 50-60 characters, ensuring it is fully visible in search engine results.
        Use title case format, capitalizing the first and last words, and all major words in between.

    Clarity and Readability:
        Ensure the title is easy to read and understand at a glance.
        Avoid jargon or complex language that might confuse readers.

    Uniqueness and Originality:
        Create unique titles that stand out from similar articles on the same topic.
        Avoid duplicating titles from other sources or previous articles."""
    
    @classmethod
    def get_user_prompt(cls):
        return """Overall article outline: {outline}


        Primary keyword: {p_k}
        Secondary keywords: {sec_k}
        """

In [ ]:
# article_title = use_component(ArticleTitleGenerator, {"outline": filled_outline, "p_k": pk.primary_keyword.keyword, "sec_k": "; ".join(sec_keys_list)})
# article_title

In [ ]:
article_title = ArticleTitleGenerator(title='Platini vs Maradona - A Legendary Football Debate')


In [ ]:
# Add Key points
nl = "\n"
keypoints = f"## Key points\n{nl.join(f'- {c}' for c in conclusion.keypoints)}\n\n"

article_intro_concl = "# " + article_title.title + "\n\n" + keypoints + "\n\n" + intro.introduction + "\n\n" + article + "\n\n# Conclusion\n" + conclusion.conclusion
print(article_intro_concl)